In [223]:
#import requests
#FOURSQUARE_API_URL="https://api.foursquare.com/v2/venues/search?client_id=R2DI0ES2KM4GH4BJFR1AY102DHGGAF4WPBGTITH5BCRE0HV5&client_secret=HYUY25VXAZXK4XHSQCBOVQLR50APA4DCIIDJMFH0EPQ2ICC4&v=20120609"

#body = {
#    "near": "Tallinn"
#}
#response = requests.get(FOURSQUARE_API_URL, params=body)
#response.json().keys()
#venues = response.json()["response"]["venues"]
#print(response.json()["meta"])
#print(response.json().keys())
#print(response.json()["response"].keys())
# print(f'In total {len(venues)} venues')
# for venue in venues:
#     print(venue["name"])

## Downloading needed dependencies

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install -c conda-forge geopandas --yes
#import geopandas as gpd

import os
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0


print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\marim\Anaconda3

  added / updated specs:
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    boost-cpp-1.70.0           |       h6a4c333_2        31.8 MB  conda-forge
    click-plugins-1.1.1        |             py_0           9 KB  conda-forge
    cligj-0.5.0                |             py_0           8 KB  conda-forge
    expat-2.2.5                |    he025d50_1004          88 KB  conda-for

ModuleNotFoundError: No module named 'ibm_boto3'

## Download Data

In [51]:
os.listdir('..')

# @hidden_cell
# The following code accesses a file in  IBM Cloud Object Storage. It includes credentials.
# You might want to remove those credentials before you share the notebook.
client_5f435218b6c9416ca726546492b81114 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RnRZYkxxCfvsASjyS5D3cth58fYEUNeeobuYrhgWQV5G',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_5f435218b6c9416ca726546492b81114.get_object(Bucket='capstone-donotdelete-pr-gnnla3gmg2gh4k',Key='RL0731_26122019015000571.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_districts = pd.read_csv(body)    


body = client_5f435218b6c9416ca726546492b81114.get_object(Bucket='capstone-donotdelete-pr-gnnla3gmg2gh4k',Key='District.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_district_area = pd.read_csv(body)
    
body = client_5f435218b6c9416ca726546492b81114.get_object(Bucket='capstone-donotdelete-pr-gnnla3gmg2gh4k',Key='sihtnumbrid.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )
df_postalCode =  pd.read_csv(body)



UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 8: invalid continuation byte

## Data Shaping

In [45]:
# delete extra collummns and flags
df_districts.drop(['DIM1', 'DIM2','Flag Codes','Flags'], axis=1, inplace =True)
df_districts.head(2)

,Place of residence,Indicator,Value
0,..Haabersti city district,Number of families,12257.0
1,..Haabersti city district,Number of family members,33133.0


In [46]:
# rename column'Place of residence' as District
df_districts=df_districts.rename(columns={'Place of residence': 'District'})
df_districts.head(2)

,District,Indicator,Value
0,..Haabersti city district,Number of families,12257.0
1,..Haabersti city district,Number of family members,33133.0


In [47]:
# remove extra charachers and text from column 'District'
df_districts['District'] = df_districts['District'].map(lambda x: x.lstrip('.'))
df_districts['District'] = df_districts['District'].str.replace('city district','')
df_districts['District'] = df_districts['District'].str.strip()


df_district_area['District'] = df_district_area['District'].map(lambda x: x.lstrip('.'))
df_district_area['District'] = df_district_area['District'].str.replace('city district','')
df_district_area['District']=df_district_area['District'].str.strip()
df_districts.head(2)

,District,Indicator,Value
0,Haabersti,Number of families,12257.0
1,Haabersti,Number of family members,33133.0


In [48]:
df_districts

,District,Indicator,Value
0,Haabersti,Number of families,12257.00
1,Haabersti,Number of family members,33133.00
2,Haabersti,Average size of family,2.70
3,Haabersti,Population,41694.00
4,Haabersti,Share of population living in families,79.50
5,Haabersti,Number of families with children aged under 18,5234.00
6,Kesklinna,Number of families,12049.00
7,Kesklinna,Number of family members,31279.00
8,Kesklinna,Average size of family,2.60
9,Kesklinna,Population,46494.00


In [6]:
# set place of residence and indicator as indexes
#df_districts.set_index(['Place of residence', 'Indicator'],inplace=True)

In [7]:
# set District as index
#df_district_area.set_index(['District'],inplace=True)


In [8]:
#Calculate Share of families with children by deviding'Number of families with children aged under 18' by  and 'Number of families'
#print(df_districts.index.levels[0])
#for region in df_districts.index.levels[0]:
#    print(region)
#    df_districts.loc[(region, 'Newrow'), 'Value'] = 0
    


# for index, row in df_districts.iterrows():
#     print(index)
#     df_districts.loc[(index[0],'Share of families with children')]=(df_districts.loc[(index[0], 'Number of families with children aged under 18')]/df_districts.loc[(index[0], 'Number of families')] *100)

In [9]:
# df_districts.set_index('Newrow',append=True,inplace=True)
#df_districts.reset_index(inplace=True)
#df_districts.sort_values(by=['Place of residence', 'Indicator'])
# df_districts.order()
# df_districts.set_index(['Place of residence', 'Indicator'],inplace=True)

## Find out Tallinn cooordinate

In [9]:
address = 'Tallinn, Estonia'

geolocator = Nominatim(user_agent="tln_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of Tallinn are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_Tallinn = folium.Map(location=[latitude, longitude], zoom_start=12)


The geograpical coordinate of Tallinn are 59.4250354, 24.74386926051089.


## Add districts layer on the map

In [10]:
districts_geo = {
"type": "FeatureCollection",
"name": "Tallinn's Districts",
"crs": { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } },
"features": [
{ "type": "Feature", "properties": { "ONIMI": "Tallinn", "OKOOD": "0784", "MNIMI": "Harju maakond", "MKOOD": "0037", "TYYP": "4", "District": "Kristiine", "AKOOD": "0339", "TYYP_2": "6", "ONIMI_2": "Tallinn", "OKOOD_2": "0784", "MNIMI_2": "Harju maakond", "MKOOD_2": "0037" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 24.67369560922279, 59.418964368157212 ], [ 24.673925743905183, 59.419308884820701 ], [ 24.674976497450388, 59.420205035781137 ], [ 24.676183712263416, 59.421234038456113 ], [ 24.679902263601697, 59.424432688664247 ], [ 24.68395770429937, 59.423046494505471 ], [ 24.685212197650255, 59.424350217236878 ], [ 24.686243380923241, 59.424003459361145 ], [ 24.687076254480047, 59.423712756060887 ], [ 24.687657369791676, 59.424183335259798 ], [ 24.687979912518564, 59.424550145752299 ], [ 24.688251512378447, 59.424827633780168 ], [ 24.688489874893968, 59.425064091778509 ], [ 24.688891323753822, 59.425149425074487 ], [ 24.689076999325792, 59.425262637576395 ], [ 24.689301542620441, 59.425512272511952 ], [ 24.689319077342667, 59.425551857949344 ], [ 24.690085548259713, 59.425688777146782 ], [ 24.692025952758836, 59.425725359538333 ], [ 24.692374783866452, 59.425725769442295 ], [ 24.692743141145389, 59.425733436609327 ], [ 24.693097495426827, 59.425745575087838 ], [ 24.693972001999573, 59.425762063126889 ], [ 24.694505184134407, 59.425766073539407 ], [ 24.696147266686751, 59.427554996162335 ], [ 24.69719767221472, 59.427417030149719 ], [ 24.698437053969617, 59.427121049658773 ], [ 24.698752227444597, 59.427465072219256 ], [ 24.698861926534768, 59.427634330226233 ], [ 24.699116836626164, 59.42814115114782 ], [ 24.69983812180055, 59.429465160831846 ], [ 24.70067889514884, 59.429404135182175 ], [ 24.701644244694517, 59.429335079327089 ], [ 24.703748658960073, 59.429402393581064 ], [ 24.703762249318046, 59.429360758526087 ], [ 24.704288578806995, 59.429405610514955 ], [ 24.705468325764883, 59.42972811286397 ], [ 24.705753080548128, 59.429831704581943 ], [ 24.70617352384058, 59.429927566069026 ], [ 24.706555555545108, 59.430014745377846 ], [ 24.706774545357874, 59.430064827251677 ], [ 24.706889051531761, 59.430097875349581 ], [ 24.707133444205017, 59.4301749300596 ], [ 24.709570685375031, 59.430951502872347 ], [ 24.709647638427676, 59.430889866576024 ], [ 24.711516281983997, 59.431467220725985 ], [ 24.713511034838444, 59.432173938546732 ], [ 24.71514631248435, 59.432691194748493 ], [ 24.716445976850874, 59.432881707761034 ], [ 24.716789618311289, 59.432983967920755 ], [ 24.717143273611541, 59.433093084760038 ], [ 24.717499763439026, 59.433202813584494 ], [ 24.717933897575282, 59.433334470259403 ], [ 24.718189899372362, 59.433299231705426 ], [ 24.718672851120008, 59.433397495394843 ], [ 24.720434553130186, 59.433719649242853 ], [ 24.721432025040258, 59.433899378509231 ], [ 24.722005063466995, 59.434015359271427 ], [ 24.722077410369828, 59.434028337929725 ], [ 24.723244348548192, 59.434237914413394 ], [ 24.723703896372204, 59.434319771055797 ], [ 24.723625047650685, 59.43403312720141 ], [ 24.7234079019339, 59.433173627223567 ], [ 24.723333533970681, 59.432807065156688 ], [ 24.723292759354312, 59.432479098147809 ], [ 24.723345204014507, 59.431976559211833 ], [ 24.72343272191457, 59.431549411942093 ], [ 24.724038705685977, 59.430410966380002 ], [ 24.724907320373518, 59.429246115963622 ], [ 24.72533164777353, 59.428675996597605 ], [ 24.725962805617343, 59.42792349370432 ], [ 24.726431475227116, 59.427368924945824 ], [ 24.727289962397087, 59.426397921687837 ], [ 24.728586314112892, 59.425027804101703 ], [ 24.728678328451814, 59.42444362236629 ], [ 24.72886877125784, 59.424212581986694 ], [ 24.729673510031603, 59.423049575297348 ], [ 24.729826965628224, 59.422778252895327 ], [ 24.73083671979613, 59.420925039766324 ], [ 24.732029472320765, 59.418336134715119 ], [ 24.733655851552619, 59.414828800408493 ], [ 24.733988234910196, 59.414109157448607 ], [ 24.734191065466355, 59.413466988424894 ], [ 24.7342561538495, 59.413151986272737 ], [ 24.734281869286644, 59.412528938274079 ], [ 24.734261277030548, 59.412176890793141 ], [ 24.734214386166734, 59.4115596338331 ], [ 24.734166936274327, 59.41148215892153 ], [ 24.734014485346609, 59.41129593152732 ], [ 24.733948941290347, 59.411171249376139 ], [ 24.733889060730235, 59.411072478750206 ], [ 24.733653459622793, 59.410663011101732 ], [ 24.733414041989779, 59.410297012324421 ], [ 24.733180551660872, 59.409985469667227 ], [ 24.732813979242035, 59.409552492637083 ], [ 24.733000080405141, 59.40951016238494 ], [ 24.732073292279015, 59.408456320596649 ], [ 24.731538012081348, 59.407839893522691 ], [ 24.731431096675976, 59.407863019906067 ], [ 24.731251460184627, 59.407647406436062 ], [ 24.731170485037637, 59.407664912538024 ], [ 24.730708927867898, 59.407110551392606 ], [ 24.730789627639432, 59.407080210159982 ], [ 24.730665483860097, 59.406989155985798 ], [ 24.730558158976251, 59.406877630790213 ], [ 24.730203017451064, 59.40648210621913 ], [ 24.73031491800722, 59.406453477233057 ], [ 24.728933609600833, 59.404933542205733 ], [ 24.727799830145319, 59.403787905892408 ], [ 24.727100818417593, 59.403081699705481 ], [ 24.726966053480535, 59.402945367468313 ], [ 24.726518571732356, 59.402434092397506 ], [ 24.727738163068665, 59.402594321177141 ], [ 24.729322883903809, 59.40280305475828 ], [ 24.73015071020135, 59.402902504709836 ], [ 24.730094382528165, 59.4028370167322 ], [ 24.729570583189595, 59.402175273763568 ], [ 24.729202556970652, 59.401519397895214 ], [ 24.728764395937279, 59.400967054023859 ], [ 24.725347051730203, 59.394146889755973 ], [ 24.725486300481762, 59.393782825952556 ], [ 24.723764003296793, 59.393913863073557 ], [ 24.721562052949356, 59.394461614848062 ], [ 24.72007480237799, 59.395066461067913 ], [ 24.71886142487822, 59.394257637036254 ], [ 24.718384196450526, 59.393961403589138 ], [ 24.717326761059642, 59.393417429942161 ], [ 24.717078994494216, 59.393302350699358 ], [ 24.716446623589782, 59.393639558510294 ], [ 24.71578447830559, 59.393992814338795 ], [ 24.714613685883169, 59.394617689597901 ], [ 24.714223295289127, 59.394845668960642 ], [ 24.713621067228424, 59.39476581203219 ], [ 24.713238644865736, 59.394734761794666 ], [ 24.712532661952682, 59.394701424238256 ], [ 24.710401880949817, 59.394525513694198 ], [ 24.709865677000572, 59.394505248609555 ], [ 24.70933892787853, 59.394491034631969 ], [ 24.70902838939859, 59.394454557398035 ], [ 24.708793006214169, 59.394027179714598 ], [ 24.708377921026813, 59.394016130361621 ], [ 24.707250656802323, 59.393986020497408 ], [ 24.705437202947166, 59.393925202010763 ], [ 24.704778472624355, 59.393843363607424 ], [ 24.703806656711581, 59.393703142450789 ], [ 24.702437432517733, 59.393503366214745 ], [ 24.70193962025342, 59.393405931970229 ], [ 24.701927872395615, 59.393922793759053 ], [ 24.702592092712106, 59.395189206108881 ], [ 24.702442715751598, 59.395243595976318 ], [ 24.702584801665775, 59.395382696762894 ], [ 24.703318755852415, 59.395115482036765 ], [ 24.703750815578168, 59.39539495503513 ], [ 24.704691458313327, 59.396113005925237 ], [ 24.70471645271428, 59.396104433738742 ], [ 24.705365462413639, 59.396583461804958 ], [ 24.705539466543414, 59.396758654822193 ], [ 24.705811027730096, 59.397208644895464 ], [ 24.705865629451718, 59.397440583628537 ], [ 24.705952766501714, 59.398106020440075 ], [ 24.706316012076844, 59.400957816665581 ], [ 24.706281965123711, 59.4009970488326 ], [ 24.70632876077039, 59.401268257890692 ], [ 24.706347159319186, 59.402005697990809 ], [ 24.706423840054629, 59.402291737801811 ], [ 24.707190863607831, 59.402962496764808 ], [ 24.706774915218183, 59.403161953739982 ], [ 24.707192522699437, 59.40353655782701 ], [ 24.707701286123427, 59.404138323266977 ], [ 24.708261929642219, 59.4046683516477 ], [ 24.708543730121214, 59.404842960063846 ], [ 24.709368697758691, 59.405407916111322 ], [ 24.709795104019161, 59.405779591881732 ], [ 24.710230933487267, 59.406239996451809 ], [ 24.710525449626072, 59.40654245211423 ], [ 24.710045831784345, 59.406661650766495 ], [ 24.709838553643664, 59.406634402441355 ], [ 24.709648482720521, 59.407062995078917 ], [ 24.709324346664523, 59.407734685018568 ], [ 24.708908045247043, 59.40862428963505 ], [ 24.707621347384414, 59.408928618773352 ], [ 24.707272922824963, 59.409705821536178 ], [ 24.707354437866812, 59.409790214419118 ], [ 24.707275220989015, 59.409808414400651 ], [ 24.707096109884418, 59.409883795032698 ], [ 24.706030577131884, 59.410105499535604 ], [ 24.706235911763173, 59.410361763319607 ], [ 24.705166523299578, 59.410602422478732 ], [ 24.701648361082427, 59.411385786224095 ], [ 24.699018587962229, 59.411898714453386 ], [ 24.69741337064195, 59.412267651169806 ], [ 24.69054146572763, 59.413907868007442 ], [ 24.69081406790389, 59.414255544836877 ], [ 24.689117112782647, 59.41468760830022 ], [ 24.689107397267136, 59.41467742563362 ], [ 24.684518338124736, 59.415855970588872 ], [ 24.684223277585385, 59.415941168628748 ], [ 24.682760503790018, 59.416316544488808 ], [ 24.681045010572866, 59.416756498196833 ], [ 24.680645857132085, 59.416904346010142 ], [ 24.679625991658341, 59.416926493079814 ], [ 24.679451131601851, 59.416937090888034 ], [ 24.67855035230961, 59.416916154611343 ], [ 24.677632197517607, 59.417306976235842 ], [ 24.67682657351553, 59.417644608768349 ], [ 24.67369560922279, 59.418964368157212 ] ] ] ] } },
{ "type": "Feature", "properties": { "ONIMI": "Tallinn", "OKOOD": "0784", "MNIMI": "Harju maakond", "MKOOD": "0037", "TYYP": "4", "District": "Mustamäe", "AKOOD": "0482", "TYYP_2": "6", "ONIMI_2": "Tallinn", "OKOOD_2": "0784", "MNIMI_2": "Harju maakond", "MKOOD_2": "0037" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 24.639199515335264, 59.395191478943708 ], [ 24.639891649732203, 59.395325637740505 ], [ 24.640230667109527, 59.395429734486939 ], [ 24.640739293362572, 59.39561451346178 ], [ 24.641666393710523, 59.395944020076719 ], [ 24.642439643895788, 59.396318976219085 ], [ 24.643225393244133, 59.396908412056717 ], [ 24.643936764662943, 59.397556825806625 ], [ 24.644459367584545, 59.398340543253518 ], [ 24.644690676570857, 59.398723706026587 ], [ 24.644844876660834, 59.399433103744968 ], [ 24.644829591802228, 59.399864243882256 ], [ 24.64469582258242, 59.400173521396184 ], [ 24.644122280855701, 59.401341876309843 ], [ 24.643932126561751, 59.401780066106632 ], [ 24.643831368993432, 59.402225897123131 ], [ 24.643821508047647, 59.40267469431484 ], [ 24.643866078222963, 59.402919362535307 ], [ 24.644010460626262, 59.403509327348807 ], [ 24.644150720161804, 59.403852091860792 ], [ 24.644295568591975, 59.40408316271423 ], [ 24.645213693600166, 59.405061439534649 ], [ 24.646068435803635, 59.405729514809821 ], [ 24.646533575869029, 59.406072296641234 ], [ 24.647251864948537, 59.406486722981505 ], [ 24.648608734213802, 59.407120712325785 ], [ 24.649361472294544, 59.407445907884465 ], [ 24.649718715673334, 59.407454198165581 ], [ 24.651443085820972, 59.408195680136458 ], [ 24.655285095176165, 59.409899781100734 ], [ 24.660283305002977, 59.412127513203863 ], [ 24.660950290643378, 59.412469051847928 ], [ 24.661600345003801, 59.412763095425177 ], [ 24.662489083699455, 59.413132326606323 ], [ 24.663021403739553, 59.413320311971503 ], [ 24.66806932721278, 59.415552886049092 ], [ 24.669260711834067, 59.416080231660025 ], [ 24.669993040891786, 59.41636833982276 ], [ 24.671318548942867, 59.417229188815718 ], [ 24.672782485780107, 59.418184470573713 ], [ 24.6728226159135, 59.418218825940606 ], [ 24.67369560922279, 59.418964368157212 ], [ 24.67682657351553, 59.417644608768349 ], [ 24.677632197517607, 59.417306976235842 ], [ 24.67855035230961, 59.416916154611343 ], [ 24.679451131601851, 59.416937090888034 ], [ 24.679625991658341, 59.416926493079814 ], [ 24.680645857132085, 59.416904346010142 ], [ 24.681045010572866, 59.416756498196833 ], [ 24.682760503790018, 59.416316544488808 ], [ 24.684223277585385, 59.415941168628748 ], [ 24.684518338124736, 59.415855970588872 ], [ 24.689107397267136, 59.41467742563362 ], [ 24.689117112782647, 59.41468760830022 ], [ 24.69081406790389, 59.414255544836877 ], [ 24.69054146572763, 59.413907868007442 ], [ 24.69741337064195, 59.412267651169806 ], [ 24.699018587962229, 59.411898714453386 ], [ 24.701648361082427, 59.411385786224095 ], [ 24.705166523299578, 59.410602422478732 ], [ 24.706235911763173, 59.410361763319607 ], [ 24.706030577131884, 59.410105499535604 ], [ 24.707096109884418, 59.409883795032698 ], [ 24.707275220989015, 59.409808414400651 ], [ 24.707354437866812, 59.409790214419118 ], [ 24.707272922824963, 59.409705821536178 ], [ 24.707621347384414, 59.408928618773352 ], [ 24.708908045247043, 59.40862428963505 ], [ 24.709324346664523, 59.407734685018568 ], [ 24.709648482720521, 59.407062995078917 ], [ 24.709838553643664, 59.406634402441355 ], [ 24.710045831784345, 59.406661650766495 ], [ 24.710525449626072, 59.40654245211423 ], [ 24.710230933487267, 59.406239996451809 ], [ 24.709795104019161, 59.405779591881732 ], [ 24.709368697758691, 59.405407916111322 ], [ 24.708543730121214, 59.404842960063846 ], [ 24.708261929642219, 59.4046683516477 ], [ 24.707701286123427, 59.404138323266977 ], [ 24.707192522699437, 59.40353655782701 ], [ 24.706774915218183, 59.403161953739982 ], [ 24.707190863607831, 59.402962496764808 ], [ 24.706423840054629, 59.402291737801811 ], [ 24.706347159319186, 59.402005697990809 ], [ 24.70632876077039, 59.401268257890692 ], [ 24.706281965123711, 59.4009970488326 ], [ 24.706316012076844, 59.400957816665581 ], [ 24.705952766501714, 59.398106020440075 ], [ 24.705865629451718, 59.397440583628537 ], [ 24.705811027730096, 59.397208644895464 ], [ 24.705539466543414, 59.396758654822193 ], [ 24.705365462413639, 59.396583461804958 ], [ 24.70471645271428, 59.396104433738742 ], [ 24.704691458313327, 59.396113005925237 ], [ 24.703750815578168, 59.39539495503513 ], [ 24.703318755852415, 59.395115482036765 ], [ 24.702584801665775, 59.395382696762894 ], [ 24.702442715751598, 59.395243595976318 ], [ 24.702592092712106, 59.395189206108881 ], [ 24.701927872395615, 59.393922793759053 ], [ 24.70193962025342, 59.393405931970229 ], [ 24.701556145765412, 59.393330866778228 ], [ 24.700416826341701, 59.393131728120757 ], [ 24.699880228155106, 59.393047778477204 ], [ 24.699321840230983, 59.392991232244057 ], [ 24.698519767439876, 59.392964434925339 ], [ 24.697822493488811, 59.392932678004705 ], [ 24.697110021856279, 59.392863295068445 ], [ 24.696941317971877, 59.392832859928831 ], [ 24.696699138866538, 59.392772471999848 ], [ 24.696473697395568, 59.392712892674041 ], [ 24.696072308052582, 59.392552177276166 ], [ 24.696184619970648, 59.393080859410148 ], [ 24.692129982488609, 59.39278465950413 ], [ 24.692023546201412, 59.392786925238212 ], [ 24.68970263491563, 59.393068603332715 ], [ 24.689600468298146, 59.393073178693136 ], [ 24.688940469778057, 59.392916849652508 ], [ 24.687583824518924, 59.392461192311025 ], [ 24.687328482708402, 59.392366654179476 ], [ 24.685044526441235, 59.392112317739539 ], [ 24.684732403137858, 59.39207336785897 ], [ 24.684441518585121, 59.391961145267196 ], [ 24.684178846618327, 59.391851468318876 ], [ 24.681194507736041, 59.391006259529497 ], [ 24.678925710869954, 59.391517734222766 ], [ 24.678665688681324, 59.391398606973162 ], [ 24.676575059149915, 59.391907235976625 ], [ 24.675154658274327, 59.392250359986285 ], [ 24.674710578005875, 59.392382623903423 ], [ 24.674402375070308, 59.392515266781714 ], [ 24.674178794414935, 59.392637331293791 ], [ 24.673407921596311, 59.392539842541055 ], [ 24.663917009544917, 59.391223530721994 ], [ 24.662704225323061, 59.393294858302021 ], [ 24.661280653614828, 59.393128861147225 ], [ 24.66062474742354, 59.393195893098515 ], [ 24.660047560093389, 59.393256242924451 ], [ 24.658878571920525, 59.393367851796242 ], [ 24.659163323420888, 59.394124697152641 ], [ 24.658981442237351, 59.394130545882213 ], [ 24.658630828525304, 59.394294962233772 ], [ 24.657268896985158, 59.394406804524792 ], [ 24.656244886079019, 59.394059218991309 ], [ 24.655663084525507, 59.393916696382739 ], [ 24.655920719093128, 59.393629948846424 ], [ 24.655680127746106, 59.393584558882509 ], [ 24.652897224108706, 59.39285269989508 ], [ 24.652172075348016, 59.393516542815618 ], [ 24.651818952519829, 59.393485976947531 ], [ 24.651812506059976, 59.393498666206881 ], [ 24.651677511353324, 59.393461812123775 ], [ 24.648637046363493, 59.392962471952259 ], [ 24.64337709215663, 59.393471980168563 ], [ 24.641278617891906, 59.393680796276819 ], [ 24.639505797019535, 59.394975803729281 ], [ 24.639199515335264, 59.395191478943708 ] ] ] ] } },
{ "type": "Feature", "properties": { "ONIMI": "Tallinn", "OKOOD": "0784", "MNIMI": "Harju maakond", "MKOOD": "0037", "TYYP": "4", "District": "Põhja-Tallinna", "AKOOD": "0614", "TYYP_2": "6", "ONIMI_2": "Tallinn", "OKOOD_2": "0784", "MNIMI_2": "Harju maakond", "MKOOD_2": "0037" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 24.751838582277717, 59.45090431705745 ], [ 24.751947763034082, 59.450898496354213 ], [ 24.752144507761145, 59.450897983602211 ], [ 24.752225238920211, 59.450904443720361 ], [ 24.752329401749328, 59.45093334651753 ], [ 24.752399902207785, 59.450947575989453 ], [ 24.752449163462668, 59.450943650063728 ], [ 24.752502951067129, 59.450920308469371 ], [ 24.752554632465525, 59.450886197799235 ], [ 24.752874992164465, 59.450776475660049 ], [ 24.753106196616649, 59.450655718332129 ], [ 24.753130742511065, 59.450627938564104 ], [ 24.753107420416548, 59.45060967851699 ], [ 24.7530492751, 59.450616851204138 ], [ 24.753002272856783, 59.450636850796606 ], [ 24.752936657685741, 59.450635475373815 ], [ 24.752871127601878, 59.450637959438176 ], [ 24.752813651193382, 59.450664284599881 ], [ 24.752756165731729, 59.450707800279019 ], [ 24.752745364860438, 59.45074409219167 ], [ 24.752725642554726, 59.450764366683487 ], [ 24.75266660560936, 59.45077749588436 ], [ 24.752596860760924, 59.450778343191367 ], [ 24.752542676732329, 59.450758078004299 ], [ 24.752474113023801, 59.450746916574602 ], [ 24.752438577990574, 59.450758475145697 ], [ 24.752419649808992, 59.45077719207567 ], [ 24.752428661056459, 59.450797293426284 ], [ 24.752439878524221, 59.450814303147119 ], [ 24.752435633882197, 59.45083690393546 ], [ 24.752412243653961, 59.450852351879703 ], [ 24.752367223296115, 59.450859108180566 ], [ 24.752318725858999, 59.450859286406818 ], [ 24.752262543594664, 59.450852354023979 ], [ 24.752178466152952, 59.450827636359328 ], [ 24.75209727874153, 59.450810056616838 ], [ 24.752023796654598, 59.450802118758354 ], [ 24.751940557828405, 59.450795457389695 ], [ 24.751853394557585, 59.450798728705706 ], [ 24.751805350982451, 59.450809909656193 ], [ 24.751766879604567, 59.450829851138352 ], [ 24.751738121615457, 59.450855356629532 ], [ 24.751717912446239, 59.450881576356394 ], [ 24.751694342357386, 59.45091530187171 ], [ 24.751669642662819, 59.450944967357316 ], [ 24.75162341403129, 59.450971335530163 ], [ 24.751581136772632, 59.450988991643648 ], [ 24.751539537689897, 59.450999812563694 ], [ 24.751484108509541, 59.451007076779 ], [ 24.751445029407872, 59.451027461491009 ], [ 24.751429655918507, 59.451064335924436 ], [ 24.751406374553682, 59.451092772428147 ], [ 24.751369805196219, 59.45111424694182 ], [ 24.751314684926282, 59.451133960080526 ], [ 24.751230100451817, 59.451159128527834 ], [ 24.751149534124366, 59.451168196642634 ], [ 24.751067771534526, 59.451188609168057 ], [ 24.750994788401201, 59.451211404293097 ], [ 24.750973305871511, 59.451227056541185 ], [ 24.750976089966059, 59.451239814590309 ], [ 24.751009270879809, 59.451251842796623 ], [ 24.751088738999872, 59.451257763232313 ], [ 24.751185429559719, 59.451252337543536 ], [ 24.751322337621655, 59.451212543825186 ], [ 24.751494077948561, 59.451121222182529 ], [ 24.751644645718301, 59.451006789271105 ], [ 24.751725897025825, 59.450955975028506 ], [ 24.751772579939932, 59.450922216356581 ], [ 24.751838582277717, 59.45090431705745 ] ] ], [ [ [ 24.73505670681741, 59.454694410571534 ], [ 24.734968911155686, 59.454592637602445 ], [ 24.734936600368094, 59.454599622196881 ], [ 24.734807778279535, 59.454627468378881 ], [ 24.734884577120773, 59.454731986829351 ], [ 24.735029785258636, 59.45470028791447 ], [ 24.73505670681741, 59.454694410571534 ] ] ], [ [ [ 24.735192565126599, 59.45484923699189 ], [ 24.735119118964565, 59.45476045415149 ], [ 24.735080040430315, 59.454769227001144 ], [ 24.734938817463178, 59.454800948622506 ], [ 24.735004227419559, 59.454890763830853 ], [ 24.735192565126599, 59.45484923699189 ] ] ], [ [ [ 24.65963717744274, 59.455078544513739 ], [ 24.659518969724999, 59.455021012725666 ], [ 24.658059930875506, 59.455740278909495 ], [ 24.65817493851856, 59.45580377048978 ], [ 24.658791632225437, 59.455497916385724 ], [ 24.658821620448755, 59.455483044798434 ], [ 24.65963717744274, 59.455078544513739 ] ] ], [ [ [ 24.735288497731105, 59.455683410535357 ], [ 24.73526841920609, 59.455679563856336 ], [ 24.735211879425762, 59.455684071496428 ], [ 24.735137608257084, 59.455696290301283 ], [ 24.735032770373763, 59.455710555726895 ], [ 24.734890112985152, 59.455737258305014 ], [ 24.734670093115586, 59.455750541059082 ], [ 24.734229987115356, 59.455801109560227 ], [ 24.733808977471696, 59.455876345951026 ], [ 24.733654331395332, 59.455916336646176 ], [ 24.733582986967136, 59.455929409011574 ], [ 24.73345186818819, 59.455932706325996 ], [ 24.733259078732253, 59.455943842294417 ], [ 24.733042400694938, 59.455975712440029 ], [ 24.732953982104323, 59.45599758686491 ], [ 24.73291202432409, 59.456020181252974 ], [ 24.732909245120489, 59.456035286561644 ], [ 24.732926963041272, 59.45604499056882 ], [ 24.732970358902229, 59.45604913877105 ], [ 24.733103348392074, 59.456036037939406 ], [ 24.733327949077946, 59.456023288876786 ], [ 24.73373730309952, 59.455965964655128 ], [ 24.733973155901737, 59.455944327718157 ], [ 24.734356228116283, 59.455876034797413 ], [ 24.734903716397294, 59.455795001439753 ], [ 24.735138577667367, 59.45576589033243 ], [ 24.735266974882606, 59.455744329932948 ], [ 24.735298396122491, 59.455724648180166 ], [ 24.73530277189699, 59.455697756453787 ], [ 24.735288497731105, 59.455683410535357 ] ] ], [ [ [ 24.727928299823741, 59.457065345730214 ], [ 24.728240030146107, 59.456963003331047 ], [ 24.728814251909064, 59.456764890292412 ], [ 24.728995867944562, 59.456705860170104 ], [ 24.730171567580214, 59.456323719797801 ], [ 24.730510857454469, 59.456207116783716 ], [ 24.730470254923564, 59.456162090322174 ], [ 24.729898616358504, 59.456353921707951 ], [ 24.729117260974871, 59.456618545340469 ], [ 24.72847446458908, 59.456832254398684 ], [ 24.727999464686597, 59.456989054559052 ], [ 24.727941288312643, 59.456949196278707 ], [ 24.727825119865834, 59.456987306774778 ], [ 24.72781433494448, 59.45699838075857 ], [ 24.727809479970386, 59.457013829544877 ], [ 24.727866945924138, 59.457068341778829 ], [ 24.727901343569631, 59.457070566486621 ], [ 24.727928299823741, 59.457065345730214 ] ] ], [ [ [ 24.724807781315231, 59.458101600671746 ], [ 24.725122568110589, 59.45799616099864 ], [ 24.725633226450608, 59.457831571792681 ], [ 24.726109592537441, 59.457660015125839 ], [ 24.727062269431915, 59.457334850209577 ], [ 24.727025004777605, 59.457289912025971 ], [ 24.726993447369054, 59.457275328459794 ], [ 24.726945234952236, 59.457271097140008 ], [ 24.726793412222928, 59.457325929512947 ], [ 24.726836121653037, 59.457363234463351 ], [ 24.725999072372272, 59.457644681522439 ], [ 24.725245841616385, 59.457894584524972 ], [ 24.724849615421679, 59.458017986063524 ], [ 24.724838767603092, 59.458011205500128 ], [ 24.724803364758014, 59.457989075218649 ], [ 24.724669831500485, 59.458024414833247 ], [ 24.724662131044312, 59.458033819872085 ], [ 24.724664067794027, 59.458055281464112 ], [ 24.724698668583283, 59.458093628096407 ], [ 24.724751036371579, 59.458106661472257 ], [ 24.724807781315231, 59.458101600671746 ] ] ], [ [ [ 24.725347463715877, 59.472784688213771 ], [ 24.725294719047355, 59.472750840274735 ], [ 24.724827214447302, 59.472948536883919 ], [ 24.724895418565069, 59.472989283818762 ], [ 24.725347463715877, 59.472784688213771 ] ] ], [ [ [ 24.679752011765995, 59.436795589880113 ], [ 24.679931356814986, 59.436891908579554 ], [ 24.680085890485518, 59.436992699903939 ], [ 24.680294561828486, 59.437142978210815 ], [ 24.680466837425037, 59.437307133736759 ], [ 24.680648038548242, 59.437476036459906 ], [ 24.680853255008198, 59.437664033883571 ], [ 24.680956437351782, 59.437843384465019 ], [ 24.68104862845821, 59.438030933670291 ], [ 24.6811354189641, 59.438199480133576 ], [ 24.681152490239565, 59.438287569951875 ], [ 24.681190102393003, 59.438428264689662 ], [ 24.681184165546284, 59.438522963664049 ], [ 24.681178473485364, 59.438629941491349 ], [ 24.681199482344699, 59.438705129361161 ], [ 24.68123140181849, 59.43877596094709 ], [ 24.6813210733166, 59.438897059481263 ], [ 24.68145931791101, 59.439059702109617 ], [ 24.681549639698776, 59.439202718210993 ], [ 24.681657268921313, 59.439295366184531 ], [ 24.681775009991316, 59.439446929637654 ], [ 24.68183722775294, 59.439572514654898 ], [ 24.681834143763158, 59.439648473467038 ], [ 24.68184899094847, 59.439699177760971 ], [ 24.681896763634978, 59.439779164106909 ], [ 24.681972664473964, 59.439848511018958 ], [ 24.682079101195114, 59.439959836341238 ], [ 24.682301126450042, 59.440299980140622 ], [ 24.682416501764649, 59.440484553574954 ], [ 24.682543053969351, 59.440759177016396 ], [ 24.682637906301331, 59.441034090307127 ], [ 24.682686165420183, 59.441176156620244 ], [ 24.682711446498768, 59.44133081962773 ], [ 24.682762468074777, 59.441442629110021 ], [ 24.682832726136791, 59.441562714085798 ], [ 24.682956959548953, 59.441728919641541 ], [ 24.683126240966928, 59.44192752168685 ], [ 24.683207853616789, 59.442037098689681 ], [ 24.683270649552828, 59.442133810924147 ], [ 24.683314971369235, 59.442226866722308 ], [ 24.683398623410653, 59.442360786774714 ], [ 24.683569949119828, 59.442625149811569 ], [ 24.683693004924077, 59.442812285411634 ], [ 24.683821228778779, 59.443035632939903 ], [ 24.683911232426752, 59.443183747714066 ], [ 24.683926181553783, 59.44324459485302 ], [ 24.683936458104082, 59.443303599102762 ], [ 24.683973747500584, 59.443366583369773 ], [ 24.6840009067704, 59.443464144565901 ], [ 24.684066366241073, 59.443623203587642 ], [ 24.684092752494873, 59.443718228370656 ], [ 24.684123701666657, 59.44384041084367 ], [ 24.684128319581284, 59.443935449937221 ], [ 24.684137843042208, 59.444107321906657 ], [ 24.684145956181094, 59.444180610286189 ], [ 24.684152745768738, 59.44428638317509 ], [ 24.684190724866493, 59.444430369359736 ], [ 24.684204681334961, 59.444497047465617 ], [ 24.684230491262934, 59.44460372692015 ], [ 24.68424126690077, 59.444699039201531 ], [ 24.684243875475907, 59.444787024052097 ], [ 24.684258398111872, 59.444901311515643 ], [ 24.684277619723005, 59.444996741486364 ], [ 24.684291330112949, 59.44508279252355 ], [ 24.68431901517857, 59.445239040397546 ], [ 24.684332359028119, 59.445390371735705 ], [ 24.684321913845711, 59.44555969922753 ], [ 24.684324960026927, 59.445724064324281 ], [ 24.684309765944562, 59.445805938747313 ], [ 24.684283303536901, 59.445913123098208 ], [ 24.684273279071245, 59.44602955778727 ], [ 24.684240561554795, 59.44612545502553 ], [ 24.684193188079703, 59.446234202189842 ], [ 24.684104495777103, 59.44648119826229 ], [ 24.68403555331977, 59.44666959967725 ], [ 24.683997550041319, 59.446793863631392 ], [ 24.683950584735101, 59.446967976681968 ], [ 24.683908965913862, 59.447117717207817 ], [ 24.683818406249276, 59.447522604358916 ], [ 24.683799393205319, 59.447587290203444 ], [ 24.683786352893762, 59.447697600381254 ], [ 24.683781827231229, 59.447729544911681 ], [ 24.683777210713625, 59.447762234976665 ], [ 24.683772423292343, 59.447796074941898 ], [ 24.683771050318033, 59.447872787707773 ], [ 24.683754972024172, 59.447938786848397 ], [ 24.683746458899698, 59.448003148869546 ], [ 24.68373016906158, 59.448073556645419 ], [ 24.683703788187326, 59.448137436895983 ], [ 24.683692455934182, 59.448164892504963 ], [ 24.683636760891552, 59.448276474292676 ], [ 24.683596938665367, 59.448376079594688 ], [ 24.683578790714481, 59.448408283310677 ], [ 24.683557194442031, 59.448469023788071 ], [ 24.683545278925187, 59.448554552461978 ], [ 24.683518105764509, 59.448637018469576 ], [ 24.683516082270746, 59.448648662728431 ], [ 24.683501046167898, 59.448735069343577 ], [ 24.683481229885878, 59.448816060896853 ], [ 24.683427608806845, 59.448952084232609 ], [ 24.68340626974177, 59.449010731771786 ], [ 24.683379304029589, 59.449063070970617 ], [ 24.683346704452568, 59.449126363988498 ], [ 24.683303918032351, 59.449194216122748 ], [ 24.683259481038423, 59.449248306599792 ], [ 24.683191879137834, 59.44930505759811 ], [ 24.683144849467446, 59.449354700077265 ], [ 24.68309594808963, 59.449390097303315 ], [ 24.682913268806796, 59.449483432282314 ], [ 24.682750619536769, 59.449545208964118 ], [ 24.682630547758606, 59.449586827645625 ], [ 24.682434832688301, 59.449674610102818 ], [ 24.682256959719108, 59.449749247791189 ], [ 24.68211906997815, 59.449800042656825 ], [ 24.681903845053196, 59.449861776317547 ], [ 24.681696107475194, 59.449932115398056 ], [ 24.681606800278715, 59.44997212867861 ], [ 24.68148438803609, 59.450006155173703 ], [ 24.681353864319373, 59.450046291729024 ], [ 24.681241087780805, 59.450074648768044 ], [ 24.681205266704985, 59.450082742443151 ], [ 24.681169919420661, 59.450090725938701 ], [ 24.681057451093025, 59.450142510300651 ], [ 24.681040540520435, 59.450150290722164 ], [ 24.680871269532222, 59.450199487063507 ], [ 24.680673308873239, 59.450259917598558 ], [ 24.680455468666057, 59.450291536709017 ], [ 24.680287588857805, 59.450317717935143 ], [ 24.680105552680669, 59.450339510634116 ], [ 24.679843182360049, 59.450377543529832 ], [ 24.679695523054907, 59.450385109780193 ], [ 24.679582241627081, 59.45039441952666 ], [ 24.679379541514919, 59.450403768453 ], [ 24.679288362393041, 59.450403933444157 ], [ 24.679212026569338, 59.45040758557365 ], [ 24.679148484090007, 59.450405615155347 ], [ 24.679089961183987, 59.45039541416903 ], [ 24.679044281828929, 59.450382112844373 ], [ 24.679008487598658, 59.450367656424163 ], [ 24.678959957432262, 59.450351955044788 ], [ 24.67892482543806, 59.450345089437967 ], [ 24.678845291562027, 59.450340400572912 ], [ 24.678785839917392, 59.450330473537811 ], [ 24.678666824215341, 59.450289874920202 ], [ 24.678637237066201, 59.450279021966843 ], [ 24.678599721707158, 59.450267545575308 ], [ 24.678544968258873, 59.450258043063002 ], [ 24.67851910379575, 59.450255312732878 ], [ 24.678444968076015, 59.450255218799526 ], [ 24.678350834254125, 59.450263710741822 ], [ 24.678231465040085, 59.45027007940358 ], [ 24.678091819888309, 59.450278150171997 ], [ 24.677950180719321, 59.45028524362214 ], [ 24.677779469424802, 59.450286104101139 ], [ 24.677618285133718, 59.450279312129332 ], [ 24.677482072492527, 59.450277346650353 ], [ 24.677343726928221, 59.450262950358059 ], [ 24.6771867044583, 59.450241980280111 ], [ 24.677105128676434, 59.450224087231952 ], [ 24.677046761570729, 59.450216362097045 ], [ 24.677018550769883, 59.450201094173018 ], [ 24.676980532283835, 59.450190885616905 ], [ 24.676889420966919, 59.450184657226153 ], [ 24.676858771955413, 59.450184240201601 ], [ 24.676798920149682, 59.450183434607595 ], [ 24.676660962556529, 59.4501823569585 ], [ 24.67658699734022, 59.450184630940505 ], [ 24.676532493104649, 59.450176993458506 ], [ 24.676472217096848, 59.4501672311969 ], [ 24.676360115561614, 59.450133749395114 ], [ 24.676294907813475, 59.450126884902247 ], [ 24.676239118563643, 59.450122009728368 ], [ 24.676183717328058, 59.45012159393476 ], [ 24.676132319368453, 59.450123024714088 ], [ 24.676075897670117, 59.450118260437151 ], [ 24.676024648960837, 59.450106746078227 ], [ 24.675993188845457, 59.450100973954648 ], [ 24.675972929512987, 59.450102621898523 ], [ 24.675860511422762, 59.450108342331674 ], [ 24.675768959985835, 59.450116379323475 ], [ 24.675707130915708, 59.450116319462509 ], [ 24.675660266051807, 59.450120536497145 ], [ 24.675590806662086, 59.45012421409708 ], [ 24.675458317704663, 59.450121509300374 ], [ 24.675306544474495, 59.450106461592839 ], [ 24.675200953186998, 59.450100799980831 ], [ 24.675132500131308, 59.450102766617732 ], [ 24.675067932421488, 59.450111320122545 ], [ 24.675024646591194, 59.450121793267726 ], [ 24.674973561034268, 59.450138141184901 ], [ 24.674896512071651, 59.450157440768088 ], [ 24.674864045335266, 59.450163172620165 ], [ 24.674818015875598, 59.450157034998327 ], [ 24.674785745843135, 59.450147738913209 ], [ 24.674728810781527, 59.450138398598817 ], [ 24.674700517646855, 59.450139432199606 ], [ 24.67466242539178, 59.450149546488724 ], [ 24.674623068577411, 59.45015901194737 ], [ 24.674573859286646, 59.450165933595144 ], [ 24.674511776070993, 59.450173540531857 ], [ 24.674449837637585, 59.450178660166344 ], [ 24.674399760903391, 59.450178261244851 ], [ 24.674337109737635, 59.450174964378967 ], [ 24.674269957387395, 59.45016959896347 ], [ 24.674091561526087, 59.450171427570268 ], [ 24.673942298110337, 59.450181697550001 ], [ 24.673830056919897, 59.450210709742286 ], [ 24.673683028451862, 59.450140348725519 ], [ 24.672902974341461, 59.450529406778209 ], [ 24.673772773578534, 59.450979044567198 ], [ 24.673494500566857, 59.451122262581961 ], [ 24.672965428187666, 59.450855496496331 ], [ 24.672719956232125, 59.450734344730698 ], [ 24.672606906013698, 59.450678557636017 ], [ 24.671662238844146, 59.451155880979542 ], [ 24.671432190963145, 59.451272116257044 ], [ 24.671420791690132, 59.451278781162962 ], [ 24.671249543098828, 59.451379034226434 ], [ 24.67119941213431, 59.451420815585813 ], [ 24.671762557843824, 59.451734111414453 ], [ 24.671226847620471, 59.451992344052819 ], [ 24.670841617190305, 59.451773480983469 ], [ 24.670660441332583, 59.451838873648711 ], [ 24.66994847567625, 59.452095825836331 ], [ 24.66962825979904, 59.45181956414919 ], [ 24.669379467841622, 59.451604330003995 ], [ 24.669371038552434, 59.451597038921818 ], [ 24.669377549215469, 59.451536126291174 ], [ 24.669382867880078, 59.451486359721777 ], [ 24.669390527085874, 59.451414785983431 ], [ 24.669576748289625, 59.451031210422684 ], [ 24.669785082920249, 59.450590583258524 ], [ 24.670035215038411, 59.450059321351347 ], [ 24.669612026790546, 59.450013340383414 ], [ 24.66929370563777, 59.450670172051247 ], [ 24.669253737699265, 59.450671048353946 ], [ 24.669208132092287, 59.450690122002115 ], [ 24.669186832868416, 59.450726908747768 ], [ 24.669199146732936, 59.450767806600624 ], [ 24.669218293138659, 59.450802897763701 ], [ 24.66922491295345, 59.45081502748085 ], [ 24.669224865157016, 59.450843178463664 ], [ 24.669224749065766, 59.450896634937919 ], [ 24.669191341629812, 59.451091077052837 ], [ 24.669118290458556, 59.451240909131947 ], [ 24.669023326920588, 59.451389093019877 ], [ 24.668936212484834, 59.451504193973136 ], [ 24.668867296907187, 59.451604229433919 ], [ 24.668825061419199, 59.451684730894605 ], [ 24.66881369706088, 59.451788639282952 ], [ 24.668797990724141, 59.451853593160337 ], [ 24.668779927735596, 59.451935955715214 ], [ 24.668761454669365, 59.452008185726129 ], [ 24.668756792632756, 59.452093334813071 ], [ 24.668757412496088, 59.452132191514387 ], [ 24.668757515970686, 59.45213837588981 ], [ 24.668729922968289, 59.452172555352156 ], [ 24.668691490087021, 59.452206897548933 ], [ 24.668631924704059, 59.452235296739033 ], [ 24.668580395040063, 59.452270701777799 ], [ 24.668518969130034, 59.452330160530003 ], [ 24.668436431755001, 59.452395570156405 ], [ 24.66839942503989, 59.45243794808858 ], [ 24.668387327430942, 59.45249726421418 ], [ 24.668368437040019, 59.452568948690981 ], [ 24.66837409757898, 59.452610436894709 ], [ 24.668407157913361, 59.452697916994531 ], [ 24.668442961285763, 59.452737268423398 ], [ 24.668474107596928, 59.452762981040038 ], [ 24.668486145082824, 59.452790666727445 ], [ 24.668490314113519, 59.452806390584016 ], [ 24.668492465780322, 59.452814530447029 ], [ 24.668513313535446, 59.452832934473093 ], [ 24.668518139678934, 59.452837191836494 ], [ 24.668559726715202, 59.45285404537708 ], [ 24.668650716955291, 59.452893430863298 ], [ 24.668781939436144, 59.452974999710783 ], [ 24.668807095710356, 59.452997322559781 ], [ 24.668840900832695, 59.453027330374773 ], [ 24.668867654345892, 59.45305108136413 ], [ 24.668920916780699, 59.453088791064381 ], [ 24.668973975543746, 59.453102597010108 ], [ 24.668998838965013, 59.453109059603221 ], [ 24.669060157476846, 59.453111584829685 ], [ 24.669098938216276, 59.453119260383041 ], [ 24.66915531487016, 59.45313952155621 ], [ 24.66920403992993, 59.453172397831807 ], [ 24.669242651252961, 59.453218206930735 ], [ 24.669297451861759, 59.453355055548137 ], [ 24.669326231335749, 59.453419173033879 ], [ 24.669344553646599, 59.453452616636952 ], [ 24.669360088473876, 59.453487770982512 ], [ 24.669349201907419, 59.453504459988032 ], [ 24.669303388091464, 59.453543687219977 ], [ 24.669136633150426, 59.453664506915779 ], [ 24.668932918876507, 59.453830226635027 ], [ 24.668693504169468, 59.45403136945972 ], [ 24.668651293351157, 59.454134015982696 ], [ 24.668646053980179, 59.454151888106885 ], [ 24.668628867017997, 59.454166508490395 ], [ 24.668444265509102, 59.454230703500748 ], [ 24.668310177356304, 59.454291212953009 ], [ 24.66812666611672, 59.454437331633784 ], [ 24.66811517808604, 59.454450361113707 ], [ 24.668105104475103, 59.454461785580818 ], [ 24.667919183675018, 59.454552413706402 ], [ 24.667652670711522, 59.454683000992162 ], [ 24.667541565521958, 59.454637657523328 ], [ 24.667553707150692, 59.454630989210145 ], [ 24.667496333782601, 59.45460852415632 ], [ 24.667457972376109, 59.454593503126787 ], [ 24.667088391153616, 59.454823973597613 ], [ 24.666263554635115, 59.455351762568192 ], [ 24.665794610193107, 59.455648009648236 ], [ 24.665557565835627, 59.455797761317939 ], [ 24.665297977925551, 59.455691047759238 ], [ 24.664956644457572, 59.455550715929107 ], [ 24.663970180622563, 59.455153188372435 ], [ 24.663428468000596, 59.454934885999428 ], [ 24.663125738479412, 59.454810468094223 ], [ 24.662663330063001, 59.454620416785069 ], [ 24.661657931464894, 59.454216566947188 ], [ 24.661587525747496, 59.454085440002054 ], [ 24.661985595002044, 59.453711341671934 ], [ 24.662010353603247, 59.453687887032558 ], [ 24.662942307237422, 59.452804928100214 ], [ 24.662966081303036, 59.452807115457922 ], [ 24.663033842415437, 59.452751038592801 ], [ 24.663017635920259, 59.452722709140204 ], [ 24.662835581504417, 59.452669370514023 ], [ 24.662759509290339, 59.452676045845038 ], [ 24.662700614392193, 59.45273747292115 ], [ 24.662728832245296, 59.452752420680056 ], [ 24.662517477188157, 59.452952746796363 ], [ 24.661894255265917, 59.453574494222373 ], [ 24.661536217338615, 59.453890680623822 ], [ 24.661386815778002, 59.454035945282627 ], [ 24.66139817094145, 59.454071462612397 ], [ 24.661353745732505, 59.454098229482554 ], [ 24.661125826742097, 59.454203385449688 ], [ 24.661088932715039, 59.454181111087017 ], [ 24.660769309916073, 59.454341923751912 ], [ 24.66059371185154, 59.45443028143039 ], [ 24.660487600498442, 59.45448671080743 ], [ 24.660255934472101, 59.454609909052067 ], [ 24.660246952359937, 59.454628679280802 ], [ 24.66026140594332, 59.454643023289883 ], [ 24.660367949500976, 59.454690047410075 ], [ 24.660417076671212, 59.454688427742092 ], [ 24.66062058280583, 59.454581564231511 ], [ 24.660589410898361, 59.454564332827132 ], [ 24.66090144951141, 59.45440324449094 ], [ 24.661211826993178, 59.45424302548259 ], [ 24.661314709550901, 59.454361395410025 ], [ 24.661480980222997, 59.454499797004686 ], [ 24.661600406337822, 59.454594654677919 ], [ 24.661799476301528, 59.454666697598761 ], [ 24.662160585949387, 59.454767782159827 ], [ 24.662522262839026, 59.454905263062209 ], [ 24.662948687834227, 59.455086294894492 ], [ 24.663372040957707, 59.455271434159883 ], [ 24.663701267942528, 59.455396283551849 ], [ 24.664129277607707, 59.455568398805255 ], [ 24.664818760948812, 59.45585611399494 ], [ 24.664860492596599, 59.455879395219242 ], [ 24.664866880382444, 59.45593125679779 ], [ 24.664861571935202, 59.456006372969426 ], [ 24.664862299555914, 59.456079061923624 ], [ 24.664856279186377, 59.456153840571567 ], [ 24.66475878168163, 59.456242348216584 ], [ 24.664609597766226, 59.456377785922129 ], [ 24.664578565276713, 59.456405958461346 ], [ 24.664517279581602, 59.45646158145685 ], [ 24.664272874759657, 59.456674976103272 ], [ 24.664472880114701, 59.456738895442136 ], [ 24.663164652299056, 59.457688577895361 ], [ 24.663152649041105, 59.457684311583009 ], [ 24.662576000054546, 59.457479404118594 ], [ 24.66227194649144, 59.45737136426785 ], [ 24.661233427853656, 59.456991250284844 ], [ 24.660067143602806, 59.45657457821995 ], [ 24.659912669043511, 59.456677254428506 ], [ 24.660908725079629, 59.457049449964039 ], [ 24.66200012454393, 59.457449552037154 ], [ 24.662395234315692, 59.457599208137303 ], [ 24.662706695159955, 59.457717174483506 ], [ 24.662958558482256, 59.457812567581321 ], [ 24.66297169953387, 59.457820032834711 ], [ 24.663021784876857, 59.457848461075017 ], [ 24.662933743268731, 59.457901345392884 ], [ 24.662983806162625, 59.457924980252805 ], [ 24.662312131546251, 59.458323095324161 ], [ 24.662257725787629, 59.458343763623013 ], [ 24.662243931774665, 59.458349003474822 ], [ 24.662125796825176, 59.458415162266185 ], [ 24.661987843994563, 59.458492407874566 ], [ 24.661976418331296, 59.458512483098936 ], [ 24.661969255809922, 59.45852505939542 ], [ 24.661878941025979, 59.458683725693319 ], [ 24.661598021907793, 59.458835413753562 ], [ 24.66127707568689, 59.458974636093231 ], [ 24.660975633185299, 59.459093041807776 ], [ 24.66087287129907, 59.459134643132359 ], [ 24.660790779229384, 59.459160880505003 ], [ 24.660410345897979, 59.45925636877412 ], [ 24.660021209800966, 59.459336908606062 ], [ 24.659863982998093, 59.459375388775264 ], [ 24.65980158712533, 59.459391571843057 ], [ 24.659715323000629, 59.459416438053879 ], [ 24.659740304886604, 59.45943618716101 ], [ 24.65981100666502, 59.459492089481003 ], [ 24.659558404277984, 59.459573147048438 ], [ 24.659494020787118, 59.459531189532584 ], [ 24.659179971554131, 59.459628352693514 ], [ 24.657601126389288, 59.460116821158536 ], [ 24.655615618562528, 59.460739232139694 ], [ 24.655131830182121, 59.460892732105236 ], [ 24.653582294627931, 59.461379820970357 ], [ 24.652509203181509, 59.461712012000227 ], [ 24.651998421270115, 59.461304920950788 ], [ 24.651875911870146, 59.461207269403346 ], [ 24.651695529788402, 59.461053584902146 ], [ 24.651441792925159, 59.460837399818566 ], [ 24.650695994076795, 59.460226628737615 ], [ 24.65029040298748, 59.45989218865914 ], [ 24.650284401167006, 59.459887245251515 ], [ 24.650330715926138, 59.459862546520604 ], [ 24.650509776445105, 59.459767031551472 ], [ 24.650635564789255, 59.459701303361207 ], [ 24.651347629316991, 59.459329251847805 ], [ 24.651686831783, 59.459150116575522 ], [ 24.652488637455217, 59.458726681669653 ], [ 24.654339021780139, 59.457764496904879 ], [ 24.655427875017946, 59.45720069591286 ], [ 24.655710130858136, 59.457054543911028 ], [ 24.656223553963645, 59.456783452849734 ], [ 24.656255851289238, 59.456797780223546 ], [ 24.65649557324052, 59.45666927088471 ], [ 24.656382234494146, 59.456612780273268 ], [ 24.653269664357946, 59.458238533743483 ], [ 24.652906954910172, 59.458428714232163 ], [ 24.652806748548407, 59.458480790209471 ], [ 24.651504893131538, 59.459157299871542 ], [ 24.650677065457433, 59.459596135251992 ], [ 24.65055750027021, 59.459549751144721 ], [ 24.650529696505707, 59.4595389639085 ], [ 24.650405749019011, 59.459527268864221 ], [ 24.65036741913589, 59.459529531642119 ], [ 24.650283026335547, 59.459534508249831 ], [ 24.650116918691641, 59.45955915173316 ], [ 24.650037808499864, 59.459588733863086 ], [ 24.650013574597001, 59.459597794085525 ], [ 24.649956388756454, 59.45962298618263 ], [ 24.649874457247872, 59.459659081173903 ], [ 24.64976078625682, 59.459722217486267 ], [ 24.64969025589453, 59.45979681055767 ], [ 24.649664941750792, 59.459883496538033 ], [ 24.649693402728332, 59.459971452381389 ], [ 24.649747201090815, 59.46005641077852 ], [ 24.649819053998407, 59.460107806451362 ], [ 24.649892386182284, 59.460170200076966 ], [ 24.649900263092778, 59.460229182243729 ], [ 24.649899250664387, 59.460281610464634 ], [ 24.649930403519559, 59.460318233864584 ], [ 24.650020068852264, 59.460385385234645 ], [ 24.650302140912601, 59.460511245508975 ], [ 24.650600057790601, 59.460679082230214 ], [ 24.650833861806493, 59.460888877298302 ], [ 24.651024618993421, 59.461091191694933 ], [ 24.651155831068138, 59.461254159487716 ], [ 24.651251925052133, 59.461390882637389 ], [ 24.651362324350028, 59.461472248263753 ], [ 24.65161377216117, 59.461587682704518 ], [ 24.651834590234625, 59.461752172573348 ], [ 24.65206016400753, 59.461987338059309 ], [ 24.651941979869072, 59.462033209621687 ], [ 24.651744406893272, 59.462095236599737 ], [ 24.650127607110807, 59.462598819459515 ], [ 24.649490246210043, 59.46279415062142 ], [ 24.649486530507172, 59.462802786441777 ], [ 24.649818652301363, 59.463073049493822 ], [ 24.650860064901252, 59.463923018276063 ], [ 24.651671925593053, 59.464590004693278 ], [ 24.651691677720414, 59.464608398749718 ], [ 24.651586500263534, 59.464854786951676 ], [ 24.650531083846868, 59.465188273734839 ], [ 24.648154855652699, 59.465935700201349 ], [ 24.647718708548489, 59.466067914708887 ], [ 24.647537793589461, 59.466005518979998 ], [ 24.647504036778869, 59.465481184584164 ], [ 24.64744699082647, 59.464621600181523 ], [ 24.647470074768723, 59.464620499353082 ], [ 24.647415138108215, 59.463797618437582 ], [ 24.647402313528243, 59.463498941365188 ], [ 24.647373807598779, 59.463500535602016 ], [ 24.647363650967598, 59.46325510335393 ], [ 24.647341625397438, 59.462822729535219 ], [ 24.647318267378758, 59.462511662561504 ], [ 24.647622072513499, 59.462047032761056 ], [ 24.647721224794314, 59.462062065939413 ], [ 24.647717875353219, 59.462081660303255 ], [ 24.647816180229196, 59.462095710152994 ], [ 24.647871855743727, 59.462012932782869 ], [ 24.647774372493732, 59.461997559380535 ], [ 24.647757975613601, 59.462015458473147 ], [ 24.647661522443006, 59.461999532353467 ], [ 24.647816511488251, 59.46174511057432 ], [ 24.647908776838516, 59.461752278081491 ], [ 24.648025034743913, 59.461541511007063 ], [ 24.647779094157222, 59.461453555903006 ], [ 24.647721036163425, 59.461496237639167 ], [ 24.647686468243133, 59.461479396938962 ], [ 24.647621204732054, 59.46147878412841 ], [ 24.647547147375416, 59.461479435312022 ], [ 24.647497034264678, 59.461500282788705 ], [ 24.647463297721906, 59.461521696097329 ], [ 24.647429746877165, 59.461532444316717 ], [ 24.647387902406155, 59.461538287180545 ], [ 24.647352427237937, 59.461547069987724 ], [ 24.647326183642644, 59.461565582924855 ], [ 24.64731501856874, 59.461592468743582 ], [ 24.647322240994438, 59.461625520909749 ], [ 24.647286380170996, 59.461674430868207 ], [ 24.647161582893276, 59.461792232941029 ], [ 24.647156495696116, 59.46183832568606 ], [ 24.647138620445457, 59.461865702243813 ], [ 24.647032014785797, 59.461942167502855 ], [ 24.646928961279002, 59.46202742124806 ], [ 24.646834486503344, 59.462112174994395 ], [ 24.646768407680732, 59.46219985031609 ], [ 24.646720290516278, 59.462274008506888 ], [ 24.646674926465725, 59.462367829804769 ], [ 24.64663605898798, 59.462510424804236 ], [ 24.646631095669804, 59.462668320051186 ], [ 24.646664166298578, 59.462924510570879 ], [ 24.646645999273229, 59.463102272886509 ], [ 24.646671117645422, 59.463427433422986 ], [ 24.646697054225083, 59.463577896992831 ], [ 24.646699621477882, 59.463753457005119 ], [ 24.646712142100792, 59.463914444083365 ], [ 24.646737345510722, 59.464146274776539 ], [ 24.646761904907063, 59.464320820190956 ], [ 24.646779511697357, 59.464646906100974 ], [ 24.646772339561725, 59.464751500271532 ], [ 24.646782997903713, 59.465000341030262 ], [ 24.646780393463164, 59.465150531640539 ], [ 24.646786398224542, 59.465271972958796 ], [ 24.646792235683499, 59.465327796454666 ], [ 24.646810006114361, 59.465383211299709 ], [ 24.646844489685282, 59.465405654006915 ], [ 24.646872576471551, 59.465451664636205 ], [ 24.646883208260142, 59.465525929371644 ], [ 24.646916002495569, 59.465663046891606 ], [ 24.646923657470101, 59.465784049206754 ], [ 24.646982905149812, 59.465960048959445 ], [ 24.647010320767023, 59.466091591468967 ], [ 24.647016378440505, 59.466264162933072 ], [ 24.647021458574997, 59.466381434633043 ], [ 24.647011195498244, 59.46644669968336 ], [ 24.646998625546491, 59.466460980252108 ], [ 24.646986546313023, 59.466503013915592 ], [ 24.64693702010328, 59.466580365624189 ], [ 24.646838611905114, 59.466716044497659 ], [ 24.646787878435475, 59.466833976083088 ], [ 24.646712673104936, 59.467036263684804 ], [ 24.646705993041031, 59.467054213659381 ], [ 24.646706142044767, 59.467069526925243 ], [ 24.646707064882282, 59.467161065268201 ], [ 24.646685945211416, 59.467232424757313 ], [ 24.646681674241449, 59.467354266482786 ], [ 24.646642413532369, 59.467488505933986 ], [ 24.646587109940739, 59.467618551230579 ], [ 24.646533907629209, 59.467777607370046 ], [ 24.646482936932436, 59.467894328104812 ], [ 24.646417887920165, 59.46802025598484 ], [ 24.646332359084727, 59.468140575169322 ], [ 24.646248313469879, 59.468280572583993 ], [ 24.646239600213931, 59.468321593240411 ], [ 24.646217849623866, 59.4684240055795 ], [ 24.646165111291836, 59.468565797332609 ], [ 24.646117758021131, 59.468704151569696 ], [ 24.646087221021094, 59.468776670163237 ], [ 24.646034438237251, 59.468839746740585 ], [ 24.645994410219235, 59.468900112702435 ], [ 24.645971393472859, 59.468989739602456 ], [ 24.645943317109122, 59.469069274725804 ], [ 24.645938157058868, 59.469189029146058 ], [ 24.645950618747353, 59.469279873524762 ], [ 24.645994786815542, 59.469409341172955 ], [ 24.646056637764982, 59.469473965357089 ], [ 24.646108006657901, 59.469519727319494 ], [ 24.646242030103473, 59.469583818088175 ], [ 24.646475583843639, 59.469678012657099 ], [ 24.646696639483839, 59.469786558789089 ], [ 24.646898677889691, 59.469869830179448 ], [ 24.64697151516485, 59.469894122273502 ], [ 24.647038281593353, 59.469894072817581 ], [ 24.647059094448124, 59.469893019135512 ], [ 24.64708499802682, 59.469886931548537 ], [ 24.647108558759658, 59.469881017036109 ], [ 24.64713818935417, 59.469878914678922 ], [ 24.647166772827461, 59.46987926805226 ], [ 24.647195237441803, 59.469882656074191 ], [ 24.647240835828079, 59.469888186140288 ], [ 24.647278092015949, 59.469884493365115 ], [ 24.647298749664792, 59.469880837210077 ], [ 24.647320508797897, 59.469873872274881 ], [ 24.64734495512927, 59.469872800703563 ], [ 24.647388643693397, 59.469871571785134 ], [ 24.647414491352155, 59.469867189950712 ], [ 24.647444998214183, 59.469864786994528 ], [ 24.647492801875057, 59.469862891517309 ], [ 24.647513489805768, 59.469865482853294 ], [ 24.647531098746562, 59.469875934810034 ], [ 24.647538176938117, 59.469891052441447 ], [ 24.647539288273869, 59.469921737780453 ], [ 24.647542951570774, 59.469956764205172 ], [ 24.647502032550445, 59.470045411088847 ], [ 24.647480527824722, 59.470099268257627 ], [ 24.647480872537329, 59.470111860652239 ], [ 24.647495602539482, 59.47012118673431 ], [ 24.647508499392778, 59.470122218518831 ], [ 24.647521813399738, 59.470120133392939 ], [ 24.647558703565956, 59.470088767659718 ], [ 24.647604646958882, 59.470038084800549 ], [ 24.647632780425202, 59.469997049448494 ], [ 24.647651648076906, 59.46995390524701 ], [ 24.64766442670085, 59.469910593477017 ], [ 24.647677097620662, 59.469871833341294 ], [ 24.647704332545111, 59.469818782794285 ], [ 24.647838962343187, 59.469755671356268 ], [ 24.648005096251197, 59.469703697391893 ], [ 24.648173344021565, 59.46962279937074 ], [ 24.648318239016977, 59.469520705676032 ], [ 24.648387746280783, 59.469455113273064 ], [ 24.648458764538614, 59.469353005785976 ], [ 24.648539328267571, 59.46925568964631 ], [ 24.648626631368167, 59.46916866331555 ], [ 24.648720965296683, 59.469106332704804 ], [ 24.649058347014794, 59.468928587816507 ], [ 24.649271718869585, 59.468819396456816 ], [ 24.64942933414126, 59.468752076801842 ], [ 24.649490441385439, 59.468715223195019 ], [ 24.64954605854156, 59.468667956887138 ], [ 24.649628162657351, 59.468592508293 ], [ 24.649877664330639, 59.46844577743677 ], [ 24.650082812957617, 59.468327442248409 ], [ 24.650208690511061, 59.468238770343163 ], [ 24.650405310112458, 59.468151544554225 ], [ 24.650692258247027, 59.468038406076857 ], [ 24.650980280514624, 59.467927119801814 ], [ 24.651482147822772, 59.467777550276317 ], [ 24.651848063830052, 59.467658229001408 ], [ 24.652133212415031, 59.467576002505687 ], [ 24.652265146822288, 59.467537953313425 ], [ 24.652336964562448, 59.467519106171331 ], [ 24.652726502194614, 59.467416864587129 ], [ 24.653111662533455, 59.467343171031054 ], [ 24.653555302775469, 59.467275496890913 ], [ 24.654043102391199, 59.467221318303253 ], [ 24.654622148788299, 59.467175616408852 ], [ 24.655122706030348, 59.467148506590618 ], [ 24.655519308866857, 59.467108043145693 ], [ 24.6557636054557, 59.467088883736558 ], [ 24.655799410704404, 59.467086075451299 ], [ 24.655907756721586, 59.467080168321743 ], [ 24.656007133986691, 59.467074745595056 ], [ 24.656390913089755, 59.467059118661091 ], [ 24.656776888673058, 59.467050409570412 ], [ 24.65702827354075, 59.467026598534602 ], [ 24.657787446480327, 59.466955738922934 ], [ 24.658124729208421, 59.466929172127251 ], [ 24.658271909370473, 59.466925753123114 ], [ 24.658566124866429, 59.466930611812465 ], [ 24.658979457212642, 59.466933851000903 ], [ 24.659252310693223, 59.466937692894028 ], [ 24.659427137977843, 59.466940148608103 ], [ 24.659511048333968, 59.466940357114709 ], [ 24.659562043377647, 59.466945945676486 ], [ 24.659587360462016, 59.466954463415576 ], [ 24.659608299846379, 59.466969196891185 ], [ 24.659623534059293, 59.466983519064712 ], [ 24.659635598879834, 59.466992650689001 ], [ 24.65965504805634, 59.466996898006656 ], [ 24.659749420309971, 59.466999953431781 ], [ 24.660038757634126, 59.467006987633205 ], [ 24.660335535976849, 59.466997476008174 ], [ 24.660603615411016, 59.466987919053715 ], [ 24.660949608304456, 59.466955807701666 ], [ 24.661537703395176, 59.46685982400929 ], [ 24.661558438138908, 59.466852125754095 ], [ 24.661636981343182, 59.466822961573477 ], [ 24.661720274311481, 59.466755847480407 ], [ 24.661725305150114, 59.466750588880323 ], [ 24.661820163449896, 59.46665137028868 ], [ 24.66191456119569, 59.466555717634613 ], [ 24.662007034447178, 59.466429069487425 ], [ 24.662067107593842, 59.466308885583388 ], [ 24.662077613314214, 59.466273985746675 ], [ 24.66208352966909, 59.466254369140231 ], [ 24.662072757573167, 59.466185501113486 ], [ 24.662053809765254, 59.466078864819501 ], [ 24.662051628024933, 59.466048220774809 ], [ 24.6620495417264, 59.466018859900551 ], [ 24.662070700723369, 59.465884832078956 ], [ 24.662063591997704, 59.465816591952482 ], [ 24.66206242239376, 59.46580539506094 ], [ 24.662057406892686, 59.465796883531979 ], [ 24.662004203563573, 59.465706460433083 ], [ 24.661997448466852, 59.465657356584117 ], [ 24.661997997836185, 59.465648395201214 ], [ 24.662002373173546, 59.465577512127119 ], [ 24.662004660942106, 59.465481747488198 ], [ 24.662025489811938, 59.46538982145178 ], [ 24.662056937366977, 59.46534068727204 ], [ 24.662074569550512, 59.465313121391318 ], [ 24.662140362785046, 59.465270627868456 ], [ 24.66217129496615, 59.465261316348382 ], [ 24.662198106544093, 59.465253255316391 ], [ 24.662427281856196, 59.465196484732417 ], [ 24.662853630667154, 59.465090859077222 ], [ 24.663055473695337, 59.465040858545521 ], [ 24.663075484190646, 59.465035901539537 ], [ 24.66326408791203, 59.465033471357984 ], [ 24.663505056546544, 59.46503644150809 ], [ 24.663661589265562, 59.465035518702315 ], [ 24.663827897904564, 59.465021234200115 ], [ 24.663993485619255, 59.465003443276046 ], [ 24.664184227815721, 59.464955920608055 ], [ 24.664470604551742, 59.464858804636876 ], [ 24.664617316620408, 59.464784717194306 ], [ 24.664764609540185, 59.464694298247473 ], [ 24.664807362808634, 59.464668050495327 ], [ 24.664849473030287, 59.46463867314494 ], [ 24.66494249848963, 59.464573796828823 ], [ 24.665032593515274, 59.464529626258788 ], [ 24.66523264615811, 59.464429712593201 ], [ 24.665584114317827, 59.464259321826141 ], [ 24.665829774159743, 59.464152220029447 ], [ 24.665987079300979, 59.46410159615052 ], [ 24.666293111371949, 59.464011827362704 ], [ 24.666587990307736, 59.463901028259265 ], [ 24.666788190247814, 59.463807269361233 ], [ 24.666918229740467, 59.463729762763542 ], [ 24.666990701678522, 59.463704593673754 ], [ 24.66711314256176, 59.463690455179972 ], [ 24.667219304962671, 59.463670923268779 ], [ 24.667259319807208, 59.463659033150826 ], [ 24.667267513825188, 59.463653255621146 ], [ 24.667285763717175, 59.463640407480433 ], [ 24.667299806589625, 59.463611736956473 ], [ 24.667310058733545, 59.463599369161408 ], [ 24.667381959349246, 59.463512524586477 ], [ 24.667496891831789, 59.46344455522744 ], [ 24.667592017062145, 59.463400895973272 ], [ 24.667769461440994, 59.463359503554962 ], [ 24.668182577431786, 59.463272724422197 ], [ 24.668390226367784, 59.463212129678332 ], [ 24.668544427323784, 59.463154705195734 ], [ 24.668718371564253, 59.463116623649874 ], [ 24.668901568516326, 59.463089033430244 ], [ 24.669068885657669, 59.463078202091147 ], [ 24.669297553468745, 59.463080523861763 ], [ 24.669476115204684, 59.463075613308582 ], [ 24.669677099043479, 59.463073712429626 ], [ 24.669888484594118, 59.463077009678933 ], [ 24.670052320958526, 59.463083842771731 ], [ 24.67029061516406, 59.463099471050256 ], [ 24.670500094620877, 59.463101789588897 ], [ 24.670721101248663, 59.463113618179818 ], [ 24.670905441405861, 59.463122742608547 ], [ 24.67114274534131, 59.463135510856162 ], [ 24.671348303056995, 59.463145558918015 ], [ 24.671516393091999, 59.463155214105726 ], [ 24.671664881581219, 59.463170767866742 ], [ 24.671852591782983, 59.463185313615888 ], [ 24.672069088397333, 59.463205403461217 ], [ 24.67232043497285, 59.463227272062312 ], [ 24.67277395713695, 59.463280800988038 ], [ 24.673006705586825, 59.463305734483185 ], [ 24.673263608621625, 59.463336522585365 ], [ 24.673484748169479, 59.463367475009044 ], [ 24.673675414075493, 59.463392281272426 ], [ 24.673939563730524, 59.463427366715948 ], [ 24.674167224799575, 59.463454478570107 ], [ 24.674421942426669, 59.463480769349573 ], [ 24.674706942047319, 59.463519516730891 ], [ 24.67496502133849, 59.463560573716364 ], [ 24.675140921238629, 59.463590417555238 ], [ 24.675330087545962, 59.463618640226152 ], [ 24.67555098639485, 59.463643477365792 ], [ 24.67581525332686, 59.463683396853483 ], [ 24.676012318695157, 59.463712341053267 ], [ 24.676282778721319, 59.463743430769192 ], [ 24.676485012399368, 59.463774152040337 ], [ 24.676760581047272, 59.463808446133136 ], [ 24.676999562291922, 59.463844067638284 ], [ 24.677308949842136, 59.463879641064167 ], [ 24.677877603468335, 59.463943727634053 ], [ 24.678114904883508, 59.46397902363185 ], [ 24.678314917461218, 59.464003434007033 ], [ 24.678531697415924, 59.464036429151065 ], [ 24.678630426702679, 59.464054915283974 ], [ 24.678734379859968, 59.464062916694935 ], [ 24.678885216473642, 59.464065802222642 ], [ 24.678932024721583, 59.464061342083994 ], [ 24.67912737247012, 59.464047139953863 ], [ 24.679243023915692, 59.464029676886632 ], [ 24.679365374485783, 59.464012071464076 ], [ 24.679489418803815, 59.463994502077306 ], [ 24.679598639271926, 59.463980043110105 ], [ 24.679656158189683, 59.463976730345188 ], [ 24.679732206687259, 59.463975727715052 ], [ 24.679990328063329, 59.463994503723029 ], [ 24.68026635602104, 59.464021481181781 ], [ 24.68053759811945, 59.464054317572803 ], [ 24.680658765109353, 59.464067605564374 ], [ 24.680794462965792, 59.464073708908479 ], [ 24.680873278944048, 59.464074387903345 ], [ 24.681012269993683, 59.464073167050714 ], [ 24.68123828980853, 59.464078506940837 ], [ 24.68145965608489, 59.464086626325454 ], [ 24.681685549042637, 59.464109452538764 ], [ 24.681849579741904, 59.464120336398871 ], [ 24.682059139947629, 59.464126325568216 ], [ 24.682241485154925, 59.464130498393779 ], [ 24.68247362437711, 59.464146360981744 ], [ 24.682694810815235, 59.464170143410577 ], [ 24.682865984918752, 59.46418983096445 ], [ 24.683042620529605, 59.464210971131273 ], [ 24.683219478208251, 59.464222926838872 ], [ 24.683415224480484, 59.464230430704305 ], [ 24.683609963245726, 59.464239582225289 ], [ 24.683817751893095, 59.464259213264789 ], [ 24.684037364014735, 59.464264195616387 ], [ 24.684139270371851, 59.464265892839364 ], [ 24.684218473241405, 59.464262968206654 ], [ 24.68427086839888, 59.464260083977209 ], [ 24.684315197227644, 59.464243695904884 ], [ 24.684342328119143, 59.464239542478545 ], [ 24.684408609070804, 59.464238686615126 ], [ 24.684659724573191, 59.464253513243378 ], [ 24.684771108624698, 59.464269604108452 ], [ 24.685008267240196, 59.464307850762324 ], [ 24.685151479936504, 59.464335023560515 ], [ 24.685337782923444, 59.46436055382479 ], [ 24.685829576105682, 59.464440275082069 ], [ 24.686067946698607, 59.464485174178641 ], [ 24.686367068470762, 59.464554136946496 ], [ 24.686709348424699, 59.464644058972837 ], [ 24.686943078673963, 59.46470977948912 ], [ 24.687136166156169, 59.46475412275003 ], [ 24.687280109238369, 59.464783228389358 ], [ 24.687408299042708, 59.46479728147024 ], [ 24.687586105445156, 59.464820204789625 ], [ 24.687836466426113, 59.464874328944745 ], [ 24.688120662821717, 59.464943617000451 ], [ 24.688293386423311, 59.464976817203215 ], [ 24.688548095526553, 59.465018170513659 ], [ 24.689180566631308, 59.465130250040652 ], [ 24.689434036192033, 59.465177362140352 ], [ 24.689716514075485, 59.46523550681016 ], [ 24.689886264934056, 59.465281171060546 ], [ 24.690174498764456, 59.465355236070508 ], [ 24.690316009624031, 59.465414701088726 ], [ 24.690483199662832, 59.465547316510857 ], [ 24.690653673496762, 59.46568699706166 ], [ 24.69085603624832, 59.465849239024934 ], [ 24.690974073489361, 59.465957470661394 ], [ 24.691128582073031, 59.466074164398684 ], [ 24.691309295643038, 59.466225379211707 ], [ 24.691420797853315, 59.466305170904725 ], [ 24.691628872225976, 59.466445999421857 ], [ 24.69185209387755, 59.466583929442237 ], [ 24.69194405423606, 59.466658455259747 ], [ 24.691971036876893, 59.466695665285137 ], [ 24.691985937584455, 59.466732023125246 ], [ 24.691995489880011, 59.46676818462953 ], [ 24.692011810998174, 59.466802389597689 ], [ 24.692038009779758, 59.466837467290013 ], [ 24.692067520458476, 59.466870597614616 ], [ 24.692085764202467, 59.466910887597294 ], [ 24.692086475325301, 59.466932912470675 ], [ 24.692089044750446, 59.467011794425744 ], [ 24.692087861920928, 59.467028272720512 ], [ 24.692082851192325, 59.467097804940273 ], [ 24.692071851753234, 59.467189504928903 ], [ 24.692075350267043, 59.467214163263854 ], [ 24.692084586313879, 59.467254724956142 ], [ 24.692125433110789, 59.467401520425355 ], [ 24.69212188620455, 59.467447553158863 ], [ 24.692120878325586, 59.467485709047324 ], [ 24.692128371400582, 59.467519466638159 ], [ 24.692129509290108, 59.467550358156267 ], [ 24.692134988129638, 59.467642231921623 ], [ 24.692142853220393, 59.467692594284784 ], [ 24.692146948061144, 59.46774665685578 ], [ 24.692157123905151, 59.467828335448921 ], [ 24.692177868004624, 59.467922310353025 ], [ 24.69219227167336, 59.468005447875882 ], [ 24.692208580733205, 59.468120783459646 ], [ 24.692200106295903, 59.468244794926584 ], [ 24.692219459375352, 59.468337134396471 ], [ 24.692248506118442, 59.468422986721507 ], [ 24.69226869089054, 59.468497180087411 ], [ 24.692283503842543, 59.468575252627581 ], [ 24.692293361691295, 59.468656878996683 ], [ 24.692291739809981, 59.468713458090512 ], [ 24.692293869470213, 59.468801067559674 ], [ 24.692296167743123, 59.468859591936841 ], [ 24.69230104208032, 59.468912061526368 ], [ 24.692324680195814, 59.468953858116322 ], [ 24.692357688668682, 59.468992355949865 ], [ 24.692399856916726, 59.469032825393946 ], [ 24.692422036041862, 59.469061820013778 ], [ 24.692473243049921, 59.469084378505734 ], [ 24.692524457723493, 59.469111658632102 ], [ 24.692575522259219, 59.469132404558891 ], [ 24.692635105820788, 59.469149622776712 ], [ 24.692750940372402, 59.469176293930495 ], [ 24.692852674368261, 59.469196557994962 ], [ 24.693018765249743, 59.469231932359484 ], [ 24.693247889603068, 59.469270215558993 ], [ 24.69344638432251, 59.469299449511716 ], [ 24.693520880430647, 59.469312630192285 ], [ 24.693614654338624, 59.469337836781563 ], [ 24.693722716436916, 59.469369108103457 ], [ 24.693832626776338, 59.469393233206432 ], [ 24.693935021479852, 59.469414695827851 ], [ 24.694225392741437, 59.469487618757469 ], [ 24.694349966420965, 59.469524071870588 ], [ 24.694429876837294, 59.469544090622747 ], [ 24.694565260071876, 59.469567335855096 ], [ 24.694770574086483, 59.469605598263819 ], [ 24.694901583498336, 59.469639261117855 ], [ 24.695003531248549, 59.469664692931133 ], [ 24.695196520088089, 59.469718746683526 ], [ 24.695461447417671, 59.469785005812582 ], [ 24.695554810202491, 59.469807215013617 ], [ 24.695782620036091, 59.469862888458579 ], [ 24.695916365271621, 59.470008142025684 ], [ 24.695993812680967, 59.470127005280979 ], [ 24.69608331909636, 59.470230795920386 ], [ 24.696206280709486, 59.470386896013181 ], [ 24.696321755514912, 59.470519238556491 ], [ 24.69641536113819, 59.470644003590024 ], [ 24.696536499148234, 59.470784071806456 ], [ 24.696709401891525, 59.470995302133822 ], [ 24.696757942474534, 59.471054605332569 ], [ 24.696823057897962, 59.471156738037386 ], [ 24.696874355331964, 59.471261187911715 ], [ 24.696960375877854, 59.471439340595914 ], [ 24.696997267041308, 59.471563067538682 ], [ 24.69705089230948, 59.471782306791532 ], [ 24.697096857761345, 59.472015159135864 ], [ 24.697144294228156, 59.47227076832494 ], [ 24.697179010642454, 59.472461543933676 ], [ 24.697177267641266, 59.472643077947239 ], [ 24.697167289057703, 59.472730625632224 ], [ 24.697153139260443, 59.472987072115167 ], [ 24.697107570325663, 59.473164116167048 ], [ 24.697057494726891, 59.473239730522927 ], [ 24.697016570834766, 59.473276266225014 ], [ 24.696988821292717, 59.473306556359113 ], [ 24.6969741236807, 59.473314237245972 ], [ 24.696952740696315, 59.473325418443196 ], [ 24.69689115985577, 59.473339585918758 ], [ 24.696804429278085, 59.473347216695139 ], [ 24.696600513839311, 59.473356220744165 ], [ 24.69631118821561, 59.473366681031685 ], [ 24.695943847436244, 59.473383943910932 ], [ 24.69565151297309, 59.473405199522688 ], [ 24.695610471224633, 59.473413530939915 ], [ 24.69559642586054, 59.47342812918329 ], [ 24.695594838336479, 59.473445453410228 ], [ 24.695623763834213, 59.473457212716575 ], [ 24.695663441731316, 59.473462461103274 ], [ 24.696082642532584, 59.473494960718973 ], [ 24.696396432515723, 59.473507540294221 ], [ 24.69648551700099, 59.473512356866642 ], [ 24.696685886395358, 59.473523183087856 ], [ 24.696783554494452, 59.473528465760957 ], [ 24.697094043454513, 59.473547829479955 ], [ 24.697132635761527, 59.47355605441988 ], [ 24.69715613089928, 59.473564493839994 ], [ 24.697172923288246, 59.473586810615245 ], [ 24.69717849001767, 59.473603684104518 ], [ 24.697206514084083, 59.473688732760074 ], [ 24.697223289900116, 59.473739595189123 ], [ 24.697230053181588, 59.473818490619117 ], [ 24.697243165934726, 59.473857577171884 ], [ 24.697258814924766, 59.473904226554048 ], [ 24.697298299901092, 59.473945758868076 ], [ 24.697303761599851, 59.47395921282326 ], [ 24.697324569228705, 59.474010520691678 ], [ 24.697340899562583, 59.474099141012708 ], [ 24.697349749800317, 59.474124740325536 ], [ 24.697331453556117, 59.474185123914332 ], [ 24.697294421093218, 59.47427141431092 ], [ 24.697259587017754, 59.474359102390565 ], [ 24.697201946548343, 59.474478149618086 ], [ 24.697105974205144, 59.474668305615403 ], [ 24.697040571195839, 59.474755455087724 ], [ 24.696986764679309, 59.474816098175125 ], [ 24.696902860813616, 59.474886307814842 ], [ 24.696867306536014, 59.474896468755873 ], [ 24.696725818231471, 59.474923562895995 ], [ 24.696566298756174, 59.474950653415071 ], [ 24.696343867316646, 59.474981316612556 ], [ 24.696136451149556, 59.475012061722772 ], [ 24.696023186058127, 59.475011268404984 ], [ 24.69598665644612, 59.475006497211346 ], [ 24.695900832830397, 59.475003449412341 ], [ 24.695866763067873, 59.475009428130626 ], [ 24.695851260460788, 59.475020075405297 ], [ 24.695835987840116, 59.475035945838769 ], [ 24.695802363487797, 59.475065503687262 ], [ 24.695761205537885, 59.475088135454172 ], [ 24.695707651059358, 59.475109351491241 ], [ 24.695629149096796, 59.475122754867975 ], [ 24.695534388420242, 59.47512789575223 ], [ 24.695393538398129, 59.475137058836211 ], [ 24.69532120340623, 59.47513189284485 ], [ 24.695233332240736, 59.475121880601051 ], [ 24.695185405510163, 59.475124045355905 ], [ 24.695123450404292, 59.475131292999258 ], [ 24.695031120029665, 59.475155280516212 ], [ 24.694914331165851, 59.475188535062216 ], [ 24.694805825208981, 59.475220480150242 ], [ 24.694708203858667, 59.475265383839663 ], [ 24.694674365252219, 59.475278407648709 ], [ 24.694422754480961, 59.475375253099124 ], [ 24.694272203794288, 59.475433190979352 ], [ 24.694205563384333, 59.47546080380225 ], [ 24.694170479608353, 59.475482020681099 ], [ 24.694129554731724, 59.475500575329782 ], [ 24.694091528815445, 59.475512463053398 ], [ 24.69403796687304, 59.475524719807666 ], [ 24.693975205453334, 59.475534008834657 ], [ 24.693919217782302, 59.475550183123502 ], [ 24.693861650933389, 59.47556158118045 ], [ 24.693804497681274, 59.475582043385081 ], [ 24.693763388726254, 59.475596765885179 ], [ 24.693680722036696, 59.475611949416169 ], [ 24.693544087461113, 59.475640271286942 ], [ 24.693432178080567, 59.475660276320248 ], [ 24.693319670955404, 59.475680392120289 ], [ 24.693162264824448, 59.475709334494297 ], [ 24.692994728607605, 59.475744227543373 ], [ 24.692935290746259, 59.475750401679392 ], [ 24.692892420121378, 59.475754854977112 ], [ 24.692817699754489, 59.475759987308422 ], [ 24.692742472164152, 59.475760993034974 ], [ 24.692674411720411, 59.475758720515387 ], [ 24.692638754483582, 59.475756959941606 ], [ 24.692504072881132, 59.475750306679927 ], [ 24.692445264554578, 59.47573707889515 ], [ 24.692415559050293, 59.475730394763083 ], [ 24.69231087162057, 59.475674203591218 ], [ 24.692197970169445, 59.475612283523468 ], [ 24.692155426965506, 59.475587641762246 ], [ 24.692108830018618, 59.475574511293651 ], [ 24.692065973223322, 59.475572680643864 ], [ 24.692025873281626, 59.475577163998615 ], [ 24.691968107528339, 59.475589226536158 ], [ 24.691903403940952, 59.475609790376545 ], [ 24.691839592768833, 59.475636956264935 ], [ 24.69178987835717, 59.475651938742388 ], [ 24.691753387294838, 59.475658647221067 ], [ 24.691704583361325, 59.475662844015005 ], [ 24.69165483402557, 59.475658697530825 ], [ 24.691584049474596, 59.475656932453461 ], [ 24.691534806673932, 59.475660368477406 ], [ 24.691485716268446, 59.475666101688113 ], [ 24.691416417925382, 59.475669768532974 ], [ 24.691374835940035, 59.475669035091556 ], [ 24.691333649526243, 59.475666926145912 ], [ 24.691261427479557, 59.475666487990907 ], [ 24.691209805816708, 59.475667458830529 ], [ 24.691149851652622, 59.475670798268574 ], [ 24.691101746147655, 59.475678124003593 ], [ 24.691051101319264, 59.475689870532442 ], [ 24.691005669682795, 59.475703780002 ], [ 24.690968654128181, 59.475719449637694 ], [ 24.690935265073062, 59.475741698042206 ], [ 24.690874244620815, 59.475776039070347 ], [ 24.690833202024539, 59.475789754923142 ], [ 24.690747776999899, 59.475803855916155 ], [ 24.690695053925303, 59.475811439085895 ], [ 24.690619678945495, 59.475815631057856 ], [ 24.69053735628982, 59.475822624164579 ], [ 24.690445618624544, 59.475822961024761 ], [ 24.690346382473336, 59.475829396277483 ], [ 24.690254244665777, 59.47583001337366 ], [ 24.690203897587804, 59.47583208126354 ], [ 24.690159262599963, 59.4758400437725 ], [ 24.690030779637272, 59.475866829192952 ], [ 24.689976754542581, 59.475877219581037 ], [ 24.689936287759043, 59.475888023726803 ], [ 24.689907521998038, 59.475896387834162 ], [ 24.689899892367411, 59.475904731095078 ], [ 24.689893745209773, 59.475915714687936 ], [ 24.689901640182295, 59.475930969460549 ], [ 24.689915102252392, 59.475944947354733 ], [ 24.689922462111237, 59.47595380462684 ], [ 24.689922632357472, 59.475961353037292 ], [ 24.689905870167465, 59.475972643525459 ], [ 24.689867372981922, 59.475977898793772 ], [ 24.689830603636508, 59.475981349694344 ], [ 24.689776229050253, 59.475989300198314 ], [ 24.689728200576994, 59.475994362926961 ], [ 24.689590060145733, 59.476030471240335 ], [ 24.689524018378485, 59.476061758929134 ], [ 24.689492181287367, 59.476073559083225 ], [ 24.689366350245997, 59.476097188175245 ], [ 24.689335152005874, 59.47610568157031 ], [ 24.689315465475804, 59.476116331982062 ], [ 24.689300840206172, 59.476124245002076 ], [ 24.689276450919362, 59.476144901948295 ], [ 24.689251945259052, 59.476170281176771 ], [ 24.689221640266602, 59.476189811061964 ], [ 24.689158589662018, 59.47622334502168 ], [ 24.689136947279774, 59.476240244349377 ], [ 24.68909896010906, 59.476272525191746 ], [ 24.689064884326573, 59.476310656258597 ], [ 24.689068084052067, 59.4763354328338 ], [ 24.689072619294787, 59.476354699788061 ], [ 24.689082648439413, 59.476380419113283 ], [ 24.689083344701064, 59.476410531881278 ], [ 24.689070938847976, 59.476446233723948 ], [ 24.689028254296172, 59.476479194383479 ], [ 24.688991002205992, 59.476512072778306 ], [ 24.688929686418668, 59.476547680118671 ], [ 24.688900262136997, 59.47655929696068 ], [ 24.688854771483264, 59.476566886474203 ], [ 24.688803155100917, 59.476586545561894 ], [ 24.688742693600314, 59.476604177249641 ], [ 24.688704389601405, 59.476622545936429 ], [ 24.688669143304374, 59.476642056617287 ], [ 24.688633079394247, 59.476666517650735 ], [ 24.688609601609855, 59.476691289950722 ], [ 24.688589180056582, 59.476715382016934 ], [ 24.688570166801941, 59.476742814985784 ], [ 24.688541752390567, 59.476794004178267 ], [ 24.688496596852058, 59.476843584219473 ], [ 24.688445652892828, 59.476896605567568 ], [ 24.688430732085212, 59.476912634849853 ], [ 24.688401927032615, 59.476943601831202 ], [ 24.688371516413394, 59.476995564412547 ], [ 24.688362102362817, 59.477013441036661 ], [ 24.688335424733538, 59.47706416330297 ], [ 24.688287944719981, 59.477117498630051 ], [ 24.688227189364415, 59.477182734358799 ], [ 24.688150538803445, 59.477272577013878 ], [ 24.688087960822688, 59.477321825937757 ], [ 24.688049504576547, 59.477370329561502 ], [ 24.688023360149408, 59.477410779775525 ], [ 24.687989381367078, 59.477448586872029 ], [ 24.687955010487812, 59.477469573914561 ], [ 24.687902259582064, 59.477498690879628 ], [ 24.687870710262619, 59.477519609317717 ], [ 24.687844512268562, 59.477544287940702 ], [ 24.687827841971295, 59.477558459148895 ], [ 24.68781897294857, 59.477574510641006 ], [ 24.687801549531972, 59.477588084344561 ], [ 24.687753451264211, 59.47761432242585 ], [ 24.687722966793281, 59.477635558428183 ], [ 24.687676189716782, 59.477678333403013 ], [ 24.687608738292415, 59.477721341859301 ], [ 24.687571385724411, 59.477758125153201 ], [ 24.687509915156127, 59.477805500882823 ], [ 24.687475538017139, 59.47782620955298 ], [ 24.687437759915966, 59.477854144130092 ], [ 24.687392667114803, 59.477883337646247 ], [ 24.687316881431357, 59.477928705352468 ], [ 24.687286895485357, 59.477944058998531 ], [ 24.687259258268057, 59.477961267520008 ], [ 24.687232356036677, 59.477986937106181 ], [ 24.687215641116598, 59.478013810305534 ], [ 24.687208581036366, 59.478044358453097 ], [ 24.687198250880598, 59.478064977609463 ], [ 24.687178034951515, 59.478089732628419 ], [ 24.687153788831882, 59.478117632493756 ], [ 24.687134916088965, 59.478131985641838 ], [ 24.68710923642799, 59.478156167694848 ], [ 24.687089438769604, 59.478190310006028 ], [ 24.687069891975852, 59.478230779484605 ], [ 24.68703008345366, 59.478279720702524 ], [ 24.687004914639317, 59.478312634268235 ], [ 24.686974248362343, 59.478341545995789 ], [ 24.68696546907168, 59.478349814273344 ], [ 24.686927202793196, 59.478379842776171 ], [ 24.686891586792356, 59.47841969577388 ], [ 24.686865618803672, 59.47844535143328 ], [ 24.686828981826512, 59.478504617062036 ], [ 24.686803989250844, 59.478527018110476 ], [ 24.686795774015796, 59.478534385780996 ], [ 24.68678394159895, 59.478540874636771 ], [ 24.686758758272816, 59.478554677121899 ], [ 24.686714559405232, 59.478586280429184 ], [ 24.686701122537752, 59.47859809175943 ], [ 24.686678860651313, 59.478617659919138 ], [ 24.686642099911047, 59.478644493812048 ], [ 24.686600878325063, 59.478682015081297 ], [ 24.686590121083562, 59.478707744070086 ], [ 24.68655120843863, 59.478786365953944 ], [ 24.686542239104973, 59.478816771392751 ], [ 24.686517299190331, 59.478859351419139 ], [ 24.686480090783064, 59.478893772787984 ], [ 24.686476765077185, 59.478899490220648 ], [ 24.686464434484545, 59.478920694224335 ], [ 24.68645340468164, 59.478943408492974 ], [ 24.686447939143324, 59.478952395557592 ], [ 24.686438135116862, 59.478968505669933 ], [ 24.686423183846301, 59.479003286897296 ], [ 24.686416914359786, 59.479035464603626 ], [ 24.686414260243232, 59.479049131771049 ], [ 24.68639299833799, 59.479087733956405 ], [ 24.686366952494168, 59.479114826161165 ], [ 24.686322272780327, 59.479166440581004 ], [ 24.686298332723936, 59.479185065889119 ], [ 24.68627812795366, 59.479200790274135 ], [ 24.686225480227645, 59.479257176779484 ], [ 24.686204086395868, 59.479285725868799 ], [ 24.686196221681655, 59.479299132898632 ], [ 24.686189738584094, 59.479310171902121 ], [ 24.68616567728105, 59.47934120342083 ], [ 24.686131040289695, 59.479371921931616 ], [ 24.686105400862157, 59.479394676286176 ], [ 24.686080807961876, 59.479425602835192 ], [ 24.686049340487493, 59.479468288536268 ], [ 24.686030844727153, 59.479508474087503 ], [ 24.686004999856102, 59.479557880892472 ], [ 24.685994610273152, 59.479608078005668 ], [ 24.685978896537957, 59.479650609906813 ], [ 24.685959541910197, 59.479728833589562 ], [ 24.685955544254021, 59.479745021168831 ], [ 24.685940005230826, 59.479804149808913 ], [ 24.685918182767793, 59.479867583975206 ], [ 24.685909797079177, 59.479891954070446 ], [ 24.685908248085195, 59.479900920732952 ], [ 24.685903487878058, 59.479928332972868 ], [ 24.685893140177658, 59.479957802980934 ], [ 24.685884554603568, 59.479986276399856 ], [ 24.685879169536982, 59.479998404811809 ], [ 24.685868383733332, 59.480022760458411 ], [ 24.68585013197595, 59.480054919661619 ], [ 24.685835846764547, 59.480083386817597 ], [ 24.685778959871651, 59.480196733460438 ], [ 24.685761703281063, 59.480254713023669 ], [ 24.685748865692325, 59.480285856628129 ], [ 24.68573384784505, 59.480317388585348 ], [ 24.685715747691834, 59.480343056926529 ], [ 24.685682336258765, 59.480364585909754 ], [ 24.685640588406251, 59.480388366467174 ], [ 24.685608293662153, 59.480399611468208 ], [ 24.68556876311327, 59.480419583389597 ], [ 24.685536036455883, 59.480443586302847 ], [ 24.685485974589973, 59.480490066860391 ], [ 24.68547502852822, 59.480516128846261 ], [ 24.685470608501436, 59.480542928887246 ], [ 24.685487150047734, 59.48057756420279 ], [ 24.685509767597434, 59.480607221845084 ], [ 24.685536439290658, 59.480653348334862 ], [ 24.685545378691113, 59.480723265118478 ], [ 24.685538043084797, 59.480787629242919 ], [ 24.685513936217163, 59.480904306148311 ], [ 24.685509394531028, 59.480961941278608 ], [ 24.685500577079019, 59.481003494701042 ], [ 24.685484043457183, 59.481028993278251 ], [ 24.685458389115269, 59.48105460210428 ], [ 24.685426229396356, 59.481070846291907 ], [ 24.685396393119852, 59.481088559525013 ], [ 24.685361047008815, 59.481126265242779 ], [ 24.685340066022118, 59.481158770674035 ], [ 24.685325941464122, 59.481175073694523 ], [ 24.685302933461394, 59.481205937968959 ], [ 24.685289149413045, 59.481269069885307 ], [ 24.685275013684674, 59.481339295101648 ], [ 24.685237127823036, 59.481444742057121 ], [ 24.685233372600777, 59.481542444397206 ], [ 24.68521139710445, 59.481609801937161 ], [ 24.68513480287055, 59.481780673606536 ], [ 24.685101619681319, 59.481844417946391 ], [ 24.685052976566702, 59.481897533574177 ], [ 24.6850120777802, 59.481918015101165 ], [ 24.684989181242429, 59.481931616825186 ], [ 24.684971817429002, 59.481952721128195 ], [ 24.684970010080843, 59.481961141543088 ], [ 24.684965186821934, 59.481983706712249 ], [ 24.684970416423827, 59.482005995245359 ], [ 24.684980954733415, 59.482030778610437 ], [ 24.684982101797907, 59.482052756218565 ], [ 24.68497193920593, 59.482095034694197 ], [ 24.684938214716826, 59.482121295729314 ], [ 24.684966446365031, 59.482165995910549 ], [ 24.684997460120176, 59.482196328132531 ], [ 24.685018034770025, 59.482214730888401 ], [ 24.685042567316358, 59.482227233439957 ], [ 24.685072815987873, 59.482237910979386 ], [ 24.68509228621555, 59.482252280015231 ], [ 24.685107304345305, 59.482271663140907 ], [ 24.685116979079051, 59.482292977056396 ], [ 24.685115600348645, 59.48231483313689 ], [ 24.685107674864245, 59.482334936899711 ], [ 24.685097637818714, 59.482348310245769 ], [ 24.685090858692568, 59.482359593081611 ], [ 24.685082816682048, 59.482379167831617 ], [ 24.685070900819696, 59.482402676482018 ], [ 24.685067366037718, 59.482437559755773 ], [ 24.685071499557374, 59.482455347749564 ], [ 24.685073913479645, 59.482465712096392 ], [ 24.685092000302983, 59.482506093375385 ], [ 24.68515432568114, 59.482570122406216 ], [ 24.685244397661325, 59.482696825090095 ], [ 24.685297418982667, 59.482793998927512 ], [ 24.685339916574502, 59.48290226858947 ], [ 24.685378037618324, 59.48302545307115 ], [ 24.685399232360673, 59.483152872682233 ], [ 24.685426814665295, 59.483256534074378 ], [ 24.685425077079252, 59.483297413313437 ], [ 24.685433394831705, 59.483335484484677 ], [ 24.685454491529917, 59.483379826667885 ], [ 24.685483343312811, 59.48346936133801 ], [ 24.685537645822681, 59.483578386412155 ], [ 24.685590592766708, 59.483647158712117 ], [ 24.685650939426786, 59.483745173997931 ], [ 24.68569542155835, 59.483804555920905 ], [ 24.685745680622269, 59.483865119619495 ], [ 24.685898540385988, 59.484073101033424 ], [ 24.685980316698497, 59.484157270583417 ], [ 24.686038387361016, 59.484237658617459 ], [ 24.686101593747434, 59.484314537005567 ], [ 24.686174539107991, 59.484392154621432 ], [ 24.686295736338479, 59.484506764750982 ], [ 24.686403606728362, 59.484596632978757 ], [ 24.686502931822815, 59.484665235296823 ], [ 24.686570779739338, 59.484708705537386 ], [ 24.686627374186536, 59.484751579051959 ], [ 24.686735010428752, 59.484853404979539 ], [ 24.68691448121638, 59.484979245908114 ], [ 24.687051785387588, 59.485053035897465 ], [ 24.687174140432475, 59.48511972241328 ], [ 24.687286136823793, 59.485178536519378 ], [ 24.687433654050057, 59.485275638818116 ], [ 24.687543682095392, 59.485340944029453 ], [ 24.687626924653401, 59.485393310003595 ], [ 24.687746300615743, 59.48544779445502 ], [ 24.687809064562281, 59.485468283703973 ], [ 24.687867279300686, 59.48547846465555 ], [ 24.687924001112425, 59.48547937162747 ], [ 24.688043797256785, 59.485489895503697 ], [ 24.688153914234309, 59.485498270528922 ], [ 24.688203000246329, 59.485503840145398 ], [ 24.688251164284321, 59.485515321119045 ], [ 24.688313205115204, 59.485540185478833 ], [ 24.688364077786581, 59.485556454722605 ], [ 24.688503121280963, 59.485607720970094 ], [ 24.688523871093398, 59.485615368364414 ], [ 24.68856792819194, 59.485634958528905 ], [ 24.688607044800126, 59.485644978555527 ], [ 24.688797165462823, 59.485664892012231 ], [ 24.688873990362044, 59.485664993402587 ], [ 24.688978334806166, 59.485666261547493 ], [ 24.689045203098612, 59.485664466947682 ], [ 24.689141029409129, 59.485650303915179 ], [ 24.689191858048694, 59.485639903768607 ], [ 24.689270963312591, 59.485608745669708 ], [ 24.689371251860834, 59.485563015577952 ], [ 24.689455716087213, 59.485508867356813 ], [ 24.689504181091657, 59.48546099340561 ], [ 24.689564900638764, 59.485390757624081 ], [ 24.689686447148432, 59.48527920782734 ], [ 24.689825007804544, 59.485177371293517 ], [ 24.689886329261931, 59.485129977505395 ], [ 24.690021480352211, 59.485069037773037 ], [ 24.690094178777802, 59.485041440468528 ], [ 24.690129644396212, 59.485020168963473 ], [ 24.690184593826327, 59.484968391917342 ], [ 24.690254401370652, 59.484927021678729 ], [ 24.690335865469521, 59.484874764666159 ], [ 24.690431471418613, 59.484756536735325 ], [ 24.69051493679989, 59.484641882129253 ], [ 24.69062844689596, 59.484522438214412 ], [ 24.690716141585671, 59.484426719781951 ], [ 24.690817055147161, 59.484358086092953 ], [ 24.690915886918432, 59.484295603189267 ], [ 24.69099268746109, 59.484245856682996 ], [ 24.691060843873021, 59.484218058209891 ], [ 24.691098178691654, 59.484208984740377 ], [ 24.691117163088244, 59.484203544197101 ], [ 24.691129395221513, 59.484196693816365 ], [ 24.691153550112432, 59.484175562090265 ], [ 24.691189267097439, 59.484146739689209 ], [ 24.691200733478126, 59.484123870174791 ], [ 24.691206439073159, 59.484072745748293 ], [ 24.691207844456628, 59.484038456938727 ], [ 24.691189912563352, 59.483979117196313 ], [ 24.691166306238944, 59.48394978886968 ], [ 24.691152703637677, 59.483923828171577 ], [ 24.691145692038198, 59.483903515082766 ], [ 24.691145126150786, 59.483881678102364 ], [ 24.691151491724323, 59.483852677410773 ], [ 24.691155230109281, 59.483842954165659 ], [ 24.69115707383277, 59.483838142047539 ], [ 24.691162857499403, 59.483823102940718 ], [ 24.691201150363366, 59.483748971354558 ], [ 24.691228199002442, 59.483646829021389 ], [ 24.691261025480486, 59.48359992501603 ], [ 24.69131382853497, 59.483516444543405 ], [ 24.691353722859443, 59.483429064090075 ], [ 24.691383559391891, 59.483346098960304 ], [ 24.691429453101481, 59.483231712478528 ], [ 24.691466240533313, 59.483146053814053 ], [ 24.69149890284362, 59.483062355703652 ], [ 24.691555007299652, 59.482938095263904 ], [ 24.691613625145422, 59.482823803537876 ], [ 24.691655756756401, 59.482729346676877 ], [ 24.691719340800365, 59.482583251834079 ], [ 24.691777558151379, 59.482440560289803 ], [ 24.691830173818893, 59.482314046925502 ], [ 24.691860237289184, 59.482202346489075 ], [ 24.691874022896037, 59.482177292396429 ], [ 24.691934593324099, 59.482067164334062 ], [ 24.692027148406286, 59.481967963401928 ], [ 24.692034817046409, 59.481958022010581 ], [ 24.69210809790891, 59.481862979575574 ], [ 24.692203097013966, 59.481779923484332 ], [ 24.692320260616238, 59.481713456322829 ], [ 24.692504790994604, 59.481652784205892 ], [ 24.692646302329788, 59.481592257024872 ], [ 24.692851521986068, 59.48150857652233 ], [ 24.692925156580714, 59.481480640513553 ], [ 24.69304735659011, 59.481415519631732 ], [ 24.693185335910748, 59.481341051866714 ], [ 24.693320825728232, 59.481276964972068 ], [ 24.693376346150419, 59.481247741560452 ], [ 24.693439554223982, 59.481206072084447 ], [ 24.693524358207629, 59.481153858222193 ], [ 24.69363798054734, 59.481097444272677 ], [ 24.693833082587531, 59.48102932692526 ], [ 24.694000128211563, 59.48096213791991 ], [ 24.69412392842348, 59.480927294514061 ], [ 24.694222066335193, 59.480879193072113 ], [ 24.694281141733416, 59.480849573534201 ], [ 24.694356789739885, 59.480828789402494 ], [ 24.694433571640356, 59.480821239689426 ], [ 24.694654993945445, 59.480810998024786 ], [ 24.694944670305954, 59.480810306394623 ], [ 24.695253470349829, 59.480815464755125 ], [ 24.695550640162747, 59.480827461053543 ], [ 24.695904468537218, 59.480843672872069 ], [ 24.696140200647321, 59.480862805305556 ], [ 24.696322082797625, 59.480878711875441 ], [ 24.69642338830835, 59.480889998978554 ], [ 24.696732831882009, 59.480910204186074 ], [ 24.69675659391946, 59.480911595661865 ], [ 24.69707516391043, 59.480930225823862 ], [ 24.697220135828207, 59.480944602787126 ], [ 24.697418736586798, 59.480964306296919 ], [ 24.697674815982438, 59.480998103803223 ], [ 24.69786198638452, 59.481026610307744 ], [ 24.698044933711017, 59.481057409977502 ], [ 24.698246906240083, 59.481098683045751 ], [ 24.698454254629283, 59.481138769352256 ], [ 24.69866851922481, 59.481193432543833 ], [ 24.698867359932844, 59.481254029865525 ], [ 24.699002121380268, 59.481311690160247 ], [ 24.699035353035168, 59.481336872796419 ], [ 24.699063088080898, 59.48138316155292 ], [ 24.699088144415168, 59.481397750283264 ], [ 24.699132457758214, 59.481400531456821 ], [ 24.699157878108728, 59.481406186556541 ], [ 24.699180179545436, 59.481415502677415 ], [ 24.699195839307684, 59.48142879472816 ], [ 24.699209831280697, 59.481438199789622 ], [ 24.699230961091867, 59.4814842003805 ], [ 24.699252318541696, 59.481499113898323 ], [ 24.699278481190234, 59.481507404150129 ], [ 24.699313099554558, 59.481509823757136 ], [ 24.699354817422012, 59.481507358352019 ], [ 24.699424842893848, 59.48149816295372 ], [ 24.699462100046748, 59.481495021003717 ], [ 24.699552051268597, 59.48148512738107 ], [ 24.699673026297109, 59.481463282839449 ], [ 24.699730643838386, 59.481453102859327 ], [ 24.699807374497919, 59.481434437297096 ], [ 24.699928496845537, 59.481419790947157 ], [ 24.700047435531246, 59.481426008654317 ], [ 24.700159184754941, 59.481418099401111 ], [ 24.700277423063689, 59.481397067748119 ], [ 24.700376253517955, 59.481382198264605 ], [ 24.70046910275185, 59.481368231208876 ], [ 24.700549562084177, 59.481348136080797 ], [ 24.700587367971771, 59.481348527665347 ], [ 24.70065116770748, 59.481358430930953 ], [ 24.700707135750637, 59.481378797586153 ], [ 24.700719814373429, 59.481385238223595 ], [ 24.70072949040382, 59.481398418269627 ], [ 24.700738238379245, 59.481416244134941 ], [ 24.700763319049031, 59.481431981422666 ], [ 24.70078615026544, 59.481435280155104 ], [ 24.700822159442485, 59.481434963096241 ], [ 24.700846156851558, 59.481429745829196 ], [ 24.700863161119099, 59.481419116949709 ], [ 24.700869072307125, 59.481407936626098 ], [ 24.700863131065244, 59.481394458464443 ], [ 24.70083754234204, 59.481369451096221 ], [ 24.70079903985885, 59.481327411988055 ], [ 24.700788073668292, 59.481312416559433 ], [ 24.700785191040751, 59.481300080115858 ], [ 24.700792340481929, 59.481292483843227 ], [ 24.700852477292688, 59.481269247018538 ], [ 24.700892814658982, 59.481246842516597 ], [ 24.70097347845384, 59.481139664298759 ], [ 24.700978253863664, 59.481133328379919 ], [ 24.701014251383942, 59.481104115737466 ], [ 24.701034433480057, 59.481081549047772 ], [ 24.701075272698294, 59.481068127369014 ], [ 24.701117657235407, 59.481062928978922 ], [ 24.701156421234153, 59.481049850442602 ], [ 24.701192758248137, 59.48102686568653 ], [ 24.701227679383628, 59.481000243960644 ], [ 24.701290467778026, 59.480930054317412 ], [ 24.701303043657237, 59.480920337611174 ], [ 24.701323210369754, 59.480904763707869 ], [ 24.701359442925728, 59.480856106453423 ], [ 24.701379602789764, 59.480829042560593 ], [ 24.70144025696905, 59.480761503303896 ], [ 24.701507504292167, 59.480728856795572 ], [ 24.701580126354141, 59.480689000392267 ], [ 24.701634845617114, 59.480641797634192 ], [ 24.701672318218332, 59.480607998896986 ], [ 24.701714277716651, 59.480588170752391 ], [ 24.701760006873315, 59.480560306447501 ], [ 24.701819291886679, 59.480524919412723 ], [ 24.701858925867811, 59.480502706852519 ], [ 24.701899532807172, 59.480478900246304 ], [ 24.701971509990734, 59.480442054204623 ], [ 24.702033953127668, 59.480410097261526 ], [ 24.702059071795048, 59.480395098303283 ], [ 24.702076333898486, 59.48037474625125 ], [ 24.702095802158393, 59.480356132881845 ], [ 24.70212182299521, 59.480339531276044 ], [ 24.702150162864744, 59.480325008858074 ], [ 24.702178693623139, 59.480315476394793 ], [ 24.702205372571729, 59.48031713860432 ], [ 24.702224589283137, 59.480323176239132 ], [ 24.702240448797017, 59.480336709241577 ], [ 24.702254885536757, 59.480359989407546 ], [ 24.702271860211425, 59.480381406879005 ], [ 24.7022979508492, 59.480401635767471 ], [ 24.702320563331707, 59.480408050270498 ], [ 24.702356049399018, 59.480410661937668 ], [ 24.702381578449152, 59.480407034056043 ], [ 24.702400267572695, 59.480396010000433 ], [ 24.702412155057853, 59.480352877048865 ], [ 24.702464542373942, 59.48027390926471 ], [ 24.702527121334079, 59.480199734505938 ], [ 24.702562635429928, 59.480166008773757 ], [ 24.70262790893096, 59.480118165865584 ], [ 24.702790447601409, 59.48002596289556 ], [ 24.70285838136067, 59.479991651347731 ], [ 24.702932823404797, 59.479948642665562 ], [ 24.702959840092088, 59.479923884843203 ], [ 24.702960584938431, 59.479910362141077 ], [ 24.702956468473719, 59.479893840249339 ], [ 24.702953585018587, 59.479866934855799 ], [ 24.702962721877714, 59.479835934890346 ], [ 24.702991450998358, 59.479812936316314 ], [ 24.703024356839325, 59.479794825655269 ], [ 24.703053736682538, 59.479777721211306 ], [ 24.703103618164604, 59.479771647371905 ], [ 24.703148942445235, 59.479767913775227 ], [ 24.703228091791114, 59.479749583367443 ], [ 24.703291400185105, 59.479742603019972 ], [ 24.703360142904199, 59.479730468023625 ], [ 24.703403316894445, 59.479728568056132 ], [ 24.703471828211825, 59.479715482717921 ], [ 24.703536587573826, 59.479697578986034 ], [ 24.703590820323065, 59.479690314787092 ], [ 24.703624548814808, 59.479690727249867 ], [ 24.703666752320377, 59.479697387058224 ], [ 24.703686641732663, 59.479704390361782 ], [ 24.703688898537006, 59.479726550476308 ], [ 24.703727041806843, 59.479778464935215 ], [ 24.703751340504681, 59.479798200418792 ], [ 24.703811870164049, 59.479825031486385 ], [ 24.703850257308151, 59.479840849769445 ], [ 24.703863620151697, 59.479861640011521 ], [ 24.70387989996382, 59.479896382242011 ], [ 24.703888080877807, 59.479917451489371 ], [ 24.703862056215282, 59.47996289523158 ], [ 24.703858530415701, 59.479975885214607 ], [ 24.703857462753891, 59.479992560426901 ], [ 24.703859429465851, 59.480010916014507 ], [ 24.703869621866879, 59.480024281593423 ], [ 24.703891074525917, 59.48003434651222 ], [ 24.703915117005657, 59.480036516162201 ], [ 24.703955571355962, 59.480039621515068 ], [ 24.704030835387364, 59.480025395717263 ], [ 24.704092296803147, 59.48001782341904 ], [ 24.704129831824741, 59.480023969416123 ], [ 24.704188641276225, 59.480044364312576 ], [ 24.704277648608649, 59.480082174032248 ], [ 24.704364890248613, 59.480136159949424 ], [ 24.704416038228239, 59.480165778717399 ], [ 24.704482543045948, 59.480199067625605 ], [ 24.70454469979423, 59.480221742484765 ], [ 24.704586032711049, 59.480233200148774 ], [ 24.704631157334035, 59.480236011041036 ], [ 24.704682216180874, 59.480233099049137 ], [ 24.704745519492882, 59.480224134210459 ], [ 24.704815307372375, 59.480221570887338 ], [ 24.704856265164189, 59.480229385979854 ], [ 24.704891803991156, 59.480248217367404 ], [ 24.704895978132313, 59.48026321287422 ], [ 24.704888544556784, 59.480270667272229 ], [ 24.704878189845882, 59.480281072628713 ], [ 24.704841680478996, 59.480307515330296 ], [ 24.704809323477686, 59.480347903386416 ], [ 24.704824257632275, 59.480366988557641 ], [ 24.704838368974638, 59.480377846599986 ], [ 24.704863756913724, 59.48038619375086 ], [ 24.704890891659442, 59.480386847604123 ], [ 24.704923071409013, 59.480380589441069 ], [ 24.704953322141307, 59.48037149600728 ], [ 24.705120545045691, 59.480367029149789 ], [ 24.705250484456446, 59.480398497283709 ], [ 24.70530422001481, 59.480426046232296 ], [ 24.705332925930392, 59.480492463008943 ], [ 24.705385946648132, 59.480530661974782 ], [ 24.705416383500602, 59.480570687297046 ], [ 24.705422605451133, 59.480578885534776 ], [ 24.705505541027811, 59.480666834196825 ], [ 24.705563867691129, 59.480698774952394 ], [ 24.705645721391814, 59.480703660027395 ], [ 24.705770430616646, 59.480702283401484 ], [ 24.705803452669649, 59.480708587731471 ], [ 24.70582370176394, 59.480712455962795 ], [ 24.705902314956678, 59.4807321426565 ], [ 24.7059485282624, 59.480754273854352 ], [ 24.706026200737227, 59.480783067765771 ], [ 24.706072181661391, 59.480799078135902 ], [ 24.706160893969216, 59.480817319225217 ], [ 24.706240141609808, 59.48082756793319 ], [ 24.706276225179689, 59.480835139398508 ], [ 24.706305719689905, 59.480851740685651 ], [ 24.706346409305354, 59.480884053848747 ], [ 24.706382379853633, 59.480910880659536 ], [ 24.706445284903104, 59.48097042642037 ], [ 24.706535898391362, 59.481080451600327 ], [ 24.70666063410907, 59.481208094406959 ], [ 24.706754404397003, 59.48128048166258 ], [ 24.706778900426794, 59.481302791812467 ], [ 24.706797922543078, 59.481341962390175 ], [ 24.706806354078704, 59.481378433925386 ], [ 24.706811178216668, 59.481422204804289 ], [ 24.706810114244632, 59.481450064838164 ], [ 24.706814784352549, 59.481482140056777 ], [ 24.706823856144759, 59.481510331749668 ], [ 24.706837567792935, 59.481545284863799 ], [ 24.706856701555331, 59.481584724129192 ], [ 24.706878482227619, 59.481625863728226 ], [ 24.706881705364989, 59.481648566173561 ], [ 24.706877390432972, 59.48167622819615 ], [ 24.706879785904633, 59.481704123541434 ], [ 24.70689582096858, 59.48173111991845 ], [ 24.706925790971514, 59.48176465732373 ], [ 24.706958158246572, 59.481798065173408 ], [ 24.707008056830251, 59.481864242272501 ], [ 24.707074729093868, 59.481929171426664 ], [ 24.707102576053799, 59.481970942722256 ], [ 24.707117683893021, 59.482022153877558 ], [ 24.707125379675773, 59.482046089049746 ], [ 24.707077151786176, 59.482090359843767 ], [ 24.707049966807759, 59.48211545153071 ], [ 24.707035439256458, 59.482154406991825 ], [ 24.707031958910793, 59.482183168658153 ], [ 24.707043243073947, 59.482209104322443 ], [ 24.707068901465828, 59.482235519454754 ], [ 24.707129775079572, 59.482306709415212 ], [ 24.707172196345503, 59.482394353634874 ], [ 24.707212900525629, 59.482536216600167 ], [ 24.707221005348551, 59.482610526120602 ], [ 24.707224915238609, 59.482669750558856 ], [ 24.707245420630478, 59.482714864572195 ], [ 24.707353293110387, 59.482939472343723 ], [ 24.707511485731864, 59.483217184746707 ], [ 24.707584402262992, 59.483361792110038 ], [ 24.707611368381883, 59.483419716868369 ], [ 24.707613272765077, 59.483440065520497 ], [ 24.70760571844087, 59.4834859494255 ], [ 24.707591696375786, 59.48360798938517 ], [ 24.707574666760635, 59.483731374008023 ], [ 24.707543548762629, 59.483821297738906 ], [ 24.707491849430625, 59.483887184903395 ], [ 24.707474752781341, 59.483898425534129 ], [ 24.707414913384415, 59.483925801957994 ], [ 24.70739671065947, 59.483943278253328 ], [ 24.707398939836125, 59.483969082917412 ], [ 24.707427217554216, 59.484001740589243 ], [ 24.707454829330985, 59.484022705353816 ], [ 24.707470754819873, 59.484048598113361 ], [ 24.707468835199709, 59.484076022863988 ], [ 24.707455306103263, 59.484139317445027 ], [ 24.707441969318968, 59.48415997227508 ], [ 24.707433857842034, 59.484180706789815 ], [ 24.707442141674878, 59.484200626244075 ], [ 24.707480549343199, 59.484304772831656 ], [ 24.707514743511787, 59.484341034250967 ], [ 24.707524738712053, 59.484391563601413 ], [ 24.707513591456994, 59.484469297679716 ], [ 24.707463206061632, 59.48453732316073 ], [ 24.707410281565554, 59.484681214257733 ], [ 24.707401151029437, 59.484793245945639 ], [ 24.707390293996205, 59.484888401962515 ], [ 24.707380287651684, 59.484955645258722 ], [ 24.707365097675403, 59.485019720699484 ], [ 24.707340974000683, 59.48510574688342 ], [ 24.707295216308815, 59.485223396850003 ], [ 24.707268605328533, 59.4852524531019 ], [ 24.707199287094522, 59.485386288465662 ], [ 24.707173658614089, 59.485412520784379 ], [ 24.707063417651987, 59.485498874036566 ], [ 24.70702067556838, 59.485549210396172 ], [ 24.706967039238794, 59.485598447168009 ], [ 24.706942909226992, 59.485616951557823 ], [ 24.706900917140558, 59.485640470852175 ], [ 24.706852617752869, 59.485661106557089 ], [ 24.706819879248012, 59.485682359012046 ], [ 24.706804434309149, 59.485711193733351 ], [ 24.70680209910223, 59.485747552362227 ], [ 24.706805582164851, 59.485782784665105 ], [ 24.706864081970529, 59.4858651901529 ], [ 24.706882412902587, 59.485948385438149 ], [ 24.706881668139594, 59.485990803697554 ], [ 24.706883276624438, 59.486030956224432 ], [ 24.706845711565805, 59.486132736378536 ], [ 24.706803020082489, 59.486194939394515 ], [ 24.706734986451071, 59.486261470264409 ], [ 24.706534048593241, 59.486363326839815 ], [ 24.706376791016293, 59.486444769824587 ], [ 24.706375484090493, 59.486449839605996 ], [ 24.706385210879418, 59.486458476817084 ], [ 24.706404635532699, 59.486461568423991 ], [ 24.706425492122634, 59.486459661389027 ], [ 24.70644647929031, 59.486457241987182 ], [ 24.706531835208668, 59.486420923346451 ], [ 24.706617908139847, 59.486384295603898 ], [ 24.706741032911115, 59.486333546591709 ], [ 24.70685691991638, 59.486273357005913 ], [ 24.706987718428394, 59.48623184839176 ], [ 24.707144272505097, 59.486201727447792 ], [ 24.707258637270549, 59.486174579419 ], [ 24.707300124098907, 59.486164976402769 ], [ 24.707335535815663, 59.486150073879458 ], [ 24.707358539992551, 59.486132589813344 ], [ 24.707381533537959, 59.486114594135813 ], [ 24.707434392087762, 59.48605932912762 ], [ 24.707601647285614, 59.485979357217403 ], [ 24.707785134809836, 59.485925590329124 ], [ 24.708034850816819, 59.485815435883183 ], [ 24.708101021051938, 59.485769829719104 ], [ 24.708143049674849, 59.485725995872642 ], [ 24.708204465337463, 59.485660505158265 ], [ 24.708287377837681, 59.485549181293266 ], [ 24.708403841505042, 59.485415029254668 ], [ 24.708423960996271, 59.485372560744167 ], [ 24.708422583797798, 59.485313035537359 ], [ 24.708452340566563, 59.485235793877237 ], [ 24.708468093628653, 59.485210826194844 ], [ 24.708497320767883, 59.485182024936044 ], [ 24.708552351252543, 59.48512527561472 ], [ 24.708579509075935, 59.485107311198092 ], [ 24.708611141619528, 59.485083021178184 ], [ 24.708627776070966, 59.485065660869537 ], [ 24.708658643053798, 59.485007066499051 ], [ 24.708677974075119, 59.484936011799185 ], [ 24.708768149551993, 59.484827862161623 ], [ 24.708804360409697, 59.484744490618546 ], [ 24.70885523225089, 59.4846711119663 ], [ 24.708906468264377, 59.484601681025168 ], [ 24.708962756190669, 59.484519745464333 ], [ 24.709039741398147, 59.484434260561592 ], [ 24.709190244202734, 59.484352922391594 ], [ 24.709311254083385, 59.484280773123039 ], [ 24.70938017547893, 59.484231058147053 ], [ 24.709444184249467, 59.484148784597124 ], [ 24.709469230603755, 59.484104871088668 ], [ 24.709478612291662, 59.484088420410103 ], [ 24.709511946504815, 59.483989040924399 ], [ 24.709552167207963, 59.483927200315989 ], [ 24.709605947722316, 59.483852109110458 ], [ 24.709671714471867, 59.483791630074442 ], [ 24.709774178246658, 59.483727317968544 ], [ 24.709826815238127, 59.483671945380877 ], [ 24.709895057916683, 59.483634657349363 ], [ 24.709945289987822, 59.483605356661911 ], [ 24.709977326187751, 59.483563263560143 ], [ 24.710011700804461, 59.483505493730057 ], [ 24.710051448762702, 59.483433727399323 ], [ 24.710109235605447, 59.483314656084609 ], [ 24.710126191651387, 59.483222949807157 ], [ 24.710153117576024, 59.483137150279148 ], [ 24.710163779682986, 59.483114418017287 ], [ 24.710184427603792, 59.483089028764709 ], [ 24.710198821715142, 59.483065010701651 ], [ 24.710208743114187, 59.483032129912793 ], [ 24.710205551757134, 59.482967829905839 ], [ 24.710188516456508, 59.482928541399872 ], [ 24.710163916952137, 59.482906510711082 ], [ 24.710131062095176, 59.482875314505371 ], [ 24.710119418305528, 59.482853420496454 ], [ 24.710106616278352, 59.482817421539274 ], [ 24.710089676979699, 59.482749667764793 ], [ 24.710045883769066, 59.482595632070861 ], [ 24.71001539199834, 59.482559252598868 ], [ 24.709994247984003, 59.482509061687836 ], [ 24.709966739524731, 59.482475269349003 ], [ 24.709946797766602, 59.482456561804483 ], [ 24.709934883345092, 59.482442003101262 ], [ 24.70993456268641, 59.482426592036987 ], [ 24.70994117299951, 59.482407588929419 ], [ 24.709982428043389, 59.482339611561436 ], [ 24.71006524268498, 59.48224375353297 ], [ 24.710178645735144, 59.482164364310961 ], [ 24.710241782249852, 59.482095568823318 ], [ 24.710290656295381, 59.482052864271346 ], [ 24.710327062057974, 59.482023251891128 ], [ 24.710374944842542, 59.48199820958606 ], [ 24.710426268497883, 59.481968076954168 ], [ 24.710553311219552, 59.48189635171822 ], [ 24.710822012624778, 59.481757445013635 ], [ 24.710947580077928, 59.481690368230836 ], [ 24.711038273347686, 59.481636612134537 ], [ 24.711100558209122, 59.481588045048213 ], [ 24.711235974523852, 59.481503356660461 ], [ 24.711358065141695, 59.481429565784218 ], [ 24.711466401941703, 59.481366441377354 ], [ 24.711544862468156, 59.481331188833572 ], [ 24.711580790232667, 59.481313436740884 ], [ 24.71162766891419, 59.481275877041611 ], [ 24.711670374325763, 59.481237899996628 ], [ 24.711744149408382, 59.481179387347623 ], [ 24.71180332098043, 59.48113646503721 ], [ 24.711854505847818, 59.481090497870952 ], [ 24.711891126692692, 59.481051009624942 ], [ 24.711970395819822, 59.480991138642352 ], [ 24.712016676512203, 59.48095033249863 ], [ 24.712071432019453, 59.480891159348765 ], [ 24.712105416918391, 59.480855599066402 ], [ 24.71212898804783, 59.480826674859863 ], [ 24.712151462559202, 59.480784946976492 ], [ 24.712200851304182, 59.480723989481298 ], [ 24.712228322798975, 59.480686651073846 ], [ 24.712244936004861, 59.480644793253433 ], [ 24.712266286243953, 59.480599938588675 ], [ 24.712311193141574, 59.480525486508562 ], [ 24.712334421720342, 59.480480172852658 ], [ 24.712332790041948, 59.480449347073623 ], [ 24.712333395854465, 59.480414972428662 ], [ 24.712332771080607, 59.480381663669455 ], [ 24.712323739513305, 59.48032263750676 ], [ 24.712304134691774, 59.48027189102222 ], [ 24.712290626839586, 59.480222431155944 ], [ 24.712269458646048, 59.480190615768876 ], [ 24.712251465812145, 59.480118182093626 ], [ 24.712255629803042, 59.480060682471816 ], [ 24.712253153406238, 59.48001981644196 ], [ 24.712242828460631, 59.479975743281997 ], [ 24.712241176466872, 59.479963382459609 ], [ 24.712242392361897, 59.479943860776203 ], [ 24.712253852955797, 59.479926527899359 ], [ 24.712290025260508, 59.47988507606366 ], [ 24.712306885382972, 59.47984830661445 ], [ 24.712312544035576, 59.479816768197161 ], [ 24.712310758488726, 59.479781104856642 ], [ 24.712299814310224, 59.479758085157037 ], [ 24.712297815998877, 59.479735896833866 ], [ 24.712303832689251, 59.479701223649563 ], [ 24.712293273761489, 59.479645939984884 ], [ 24.712279670589314, 59.479592746357696 ], [ 24.712277533161981, 59.479557129797691 ], [ 24.712289559659265, 59.479530565916875 ], [ 24.71230548842653, 59.4795049773548 ], [ 24.712327932918697, 59.479479613909561 ], [ 24.712357352051072, 59.479448566498156 ], [ 24.712383576117258, 59.479409852376591 ], [ 24.712399942287885, 59.479378175305534 ], [ 24.712398326935482, 59.479358273943127 ], [ 24.712395832228243, 59.479333440203831 ], [ 24.712398983686633, 59.479302678332957 ], [ 24.712405702504491, 59.479274599141718 ], [ 24.712416506456318, 59.479262171014227 ], [ 24.71247360783677, 59.479198891634766 ], [ 24.712510130827006, 59.479145552811133 ], [ 24.712542019268415, 59.479110255053484 ], [ 24.712583577052325, 59.47906312773199 ], [ 24.712614174175709, 59.479015853175945 ], [ 24.712644700546036, 59.478965194816396 ], [ 24.712670475542613, 59.478889804672917 ], [ 24.712690595517174, 59.478852021131843 ], [ 24.712710048208205, 59.478814393796178 ], [ 24.712712656494364, 59.478770313573072 ], [ 24.712708315230742, 59.478695938987876 ], [ 24.71269384937073, 59.478398694250579 ], [ 24.71270413414463, 59.478327633745877 ], [ 24.712710441141326, 59.478200446102761 ], [ 24.712729858685975, 59.478133276947737 ], [ 24.712762384989119, 59.478059304468701 ], [ 24.712806923906523, 59.477968292278391 ], [ 24.712838197566942, 59.477898563502698 ], [ 24.712905708907069, 59.477788971166412 ], [ 24.712942746913289, 59.477728295511312 ], [ 24.713029534800427, 59.477657171323621 ], [ 24.713193903474611, 59.477534541568389 ], [ 24.713339436611676, 59.477434311395939 ], [ 24.713509638787446, 59.477352816372466 ], [ 24.713686227515154, 59.477278925641272 ], [ 24.713935812919974, 59.477198499395193 ], [ 24.71422914486558, 59.47712353582498 ], [ 24.714510514806225, 59.47704528814824 ], [ 24.714796941671075, 59.476974355309622 ], [ 24.715246245472944, 59.476866775486904 ], [ 24.715554245451138, 59.476784799406154 ], [ 24.715759090592623, 59.476738597920885 ], [ 24.715961643415035, 59.476702579062653 ], [ 24.716235174090841, 59.476640393475691 ], [ 24.716501195052214, 59.476585285792197 ], [ 24.716700772600845, 59.476538734437113 ], [ 24.716924747007948, 59.476481089663153 ], [ 24.717112492268097, 59.476439943105227 ], [ 24.717293056674016, 59.476406375702489 ], [ 24.717475829711884, 59.476381368696956 ], [ 24.717649361921815, 59.476356582295082 ], [ 24.717890222996825, 59.47632851165023 ], [ 24.718282036405679, 59.476280085093464 ], [ 24.718821088720556, 59.476208457976831 ], [ 24.719002404964677, 59.47617553940777 ], [ 24.719251068257982, 59.476132693079798 ], [ 24.719516663097025, 59.476077034003573 ], [ 24.719731403450698, 59.476042028954893 ], [ 24.719951611693272, 59.475989408496091 ], [ 24.720097322161632, 59.47595066851143 ], [ 24.720223316350264, 59.475913203037997 ], [ 24.720439668354974, 59.475809202690407 ], [ 24.720512027984309, 59.475766805631594 ], [ 24.720575416749924, 59.475727374966766 ], [ 24.72065979481653, 59.475682650023629 ], [ 24.720762677152244, 59.475631091391655 ], [ 24.720899962664472, 59.475565305116589 ], [ 24.720998058420381, 59.475516034577943 ], [ 24.721132597240921, 59.475462246866016 ], [ 24.721271829271352, 59.475399304110098 ], [ 24.721384346763504, 59.475355439087629 ], [ 24.721434730581493, 59.475333520736427 ], [ 24.721474240981387, 59.475300503929454 ], [ 24.721570553554606, 59.47524624273003 ], [ 24.721618275755596, 59.475205263547387 ], [ 24.721706424128179, 59.475126208612053 ], [ 24.721767440195013, 59.475074815521261 ], [ 24.721847605453075, 59.475012904662663 ], [ 24.721899705579368, 59.474975411259479 ], [ 24.721973745464197, 59.474924207069954 ], [ 24.721997262850408, 59.474898333309518 ], [ 24.722012830046705, 59.474877781343089 ], [ 24.722024642783186, 59.474840714991828 ], [ 24.722087675532489, 59.474751267751913 ], [ 24.722112636837981, 59.474720269381521 ], [ 24.722170009934537, 59.474666032499513 ], [ 24.722215118491047, 59.474625839390626 ], [ 24.722233605087059, 59.474587354051955 ], [ 24.722282174596625, 59.474546477686175 ], [ 24.722365032887527, 59.47449604181682 ], [ 24.722399526773412, 59.474468662592351 ], [ 24.722448413295567, 59.474423610291907 ], [ 24.722539098973002, 59.474371255295871 ], [ 24.722670899619352, 59.474308459446696 ], [ 24.722814237788182, 59.47425898435597 ], [ 24.72295713343193, 59.474225283471952 ], [ 24.723148538204992, 59.474166262562655 ], [ 24.723326133543512, 59.47410415736892 ], [ 24.723421509332159, 59.474063427679923 ], [ 24.723600083451334, 59.474008426229972 ], [ 24.723845790410568, 59.473930345049375 ], [ 24.724167250965873, 59.473833920943989 ], [ 24.724607071232622, 59.473712572682672 ], [ 24.724959313981966, 59.47361535802056 ], [ 24.725331260625829, 59.473518958581252 ], [ 24.725561993174395, 59.473468784111368 ], [ 24.725849997001621, 59.473412252620129 ], [ 24.726112193140558, 59.473365835694672 ], [ 24.72644226036504, 59.473304645713149 ], [ 24.726735154586233, 59.473246756776398 ], [ 24.726933206132102, 59.473203833636873 ], [ 24.727250802884697, 59.473152001200368 ], [ 24.727468423007284, 59.4731506927141 ], [ 24.727506930342837, 59.47315046225075 ], [ 24.72786026964404, 59.473148332222493 ], [ 24.72830515408463, 59.473148747559769 ], [ 24.728505534658527, 59.473148932962737 ], [ 24.728574053700196, 59.473143877449949 ], [ 24.728598555034857, 59.473142072374998 ], [ 24.728625270988754, 59.473135748780237 ], [ 24.728633983636055, 59.473126544447332 ], [ 24.728662122731915, 59.473096810872306 ], [ 24.728664257763253, 59.473029070303383 ], [ 24.727722160125143, 59.473026294275513 ], [ 24.726732942508875, 59.473028537891686 ], [ 24.726288071518848, 59.473023700676769 ], [ 24.725225422055544, 59.473028114915124 ], [ 24.724785471185161, 59.47302981646056 ], [ 24.724088146592358, 59.473032481689913 ], [ 24.723977473959572, 59.473007272754671 ], [ 24.723803807941795, 59.472995623232094 ], [ 24.723718462615977, 59.472990068493381 ], [ 24.723608511956368, 59.47298724301524 ], [ 24.7235707698139, 59.472980286832296 ], [ 24.723504129390452, 59.472949979441999 ], [ 24.723430514705097, 59.472895257937935 ], [ 24.723368776869588, 59.472837383256646 ], [ 24.723280655653493, 59.472773243992222 ], [ 24.723221458064007, 59.472716172177641 ], [ 24.723137013380942, 59.472595720161799 ], [ 24.723103903775403, 59.472571745661661 ], [ 24.723056060914399, 59.47254181966894 ], [ 24.723026618883853, 59.472513588059805 ], [ 24.722998000876654, 59.472475118559089 ], [ 24.722981711333354, 59.472439005127377 ], [ 24.722979628886034, 59.472404824540256 ], [ 24.722978777298646, 59.472338806047922 ], [ 24.722972980000694, 59.472294152145281 ], [ 24.722981604881383, 59.472264086966909 ], [ 24.722992005043199, 59.472252037138027 ], [ 24.722995583523982, 59.472203606506334 ], [ 24.7229802383881, 59.472187918718092 ], [ 24.722961487736274, 59.472172195745856 ], [ 24.722923628755279, 59.47214051836238 ], [ 24.722861607410998, 59.472096639571035 ], [ 24.722813257543084, 59.472071886812799 ], [ 24.722737374651853, 59.472019044497003 ], [ 24.722670853374051, 59.471940917485703 ], [ 24.722620717250074, 59.471885106306402 ], [ 24.722554408225644, 59.471837812823246 ], [ 24.722468580440605, 59.471790150525521 ], [ 24.722392400608349, 59.471755729700419 ], [ 24.722290852989143, 59.471700469433543 ], [ 24.722168895863831, 59.471647753574764 ], [ 24.722064832331931, 59.471588611057818 ], [ 24.721944727254265, 59.47152500516264 ], [ 24.721871556221974, 59.471467641225743 ], [ 24.721800151111061, 59.47142620111164 ], [ 24.721660002819835, 59.471314841273873 ], [ 24.721551846961308, 59.471263970289904 ], [ 24.721333577550162, 59.471163872519362 ], [ 24.721233173098781, 59.471108282028837 ], [ 24.721126146288949, 59.47105234169647 ], [ 24.721020670125725, 59.470998807511769 ], [ 24.720900593096612, 59.470941349516416 ], [ 24.720841977211034, 59.470912316469118 ], [ 24.720820171369034, 59.470887166477063 ], [ 24.72077487609387, 59.470840672806261 ], [ 24.72072021295368, 59.4707933057472 ], [ 24.72061223946055, 59.470727531780184 ], [ 24.720532506044986, 59.470654098888247 ], [ 24.72045003114232, 59.470576129775793 ], [ 24.72028019069084, 59.470442507116346 ], [ 24.720197726596339, 59.470381719075064 ], [ 24.720122424514102, 59.470339311919624 ], [ 24.720068563643547, 59.470297281307921 ], [ 24.71999080440985, 59.470236216089511 ], [ 24.719903307447591, 59.47016927045037 ], [ 24.719821834579538, 59.470097704792408 ], [ 24.71974045782828, 59.470034863853613 ], [ 24.719677618516236, 59.469964363320997 ], [ 24.719595510754079, 59.469840045270722 ], [ 24.719573154450888, 59.469787034581621 ], [ 24.719522680094006, 59.469703908147913 ], [ 24.719481396084685, 59.469618253974957 ], [ 24.719430685532878, 59.469498791345991 ], [ 24.719427606767717, 59.4694933234221 ], [ 24.719371066527135, 59.469392689674329 ], [ 24.719333378625734, 59.469278425215379 ], [ 24.719287710270159, 59.469202310141299 ], [ 24.719274870994823, 59.469134624505529 ], [ 24.719264429755178, 59.469102886213577 ], [ 24.719252551217849, 59.469076604586121 ], [ 24.719234877294515, 59.469059914706492 ], [ 24.719203490024629, 59.469048739498014 ], [ 24.719156947680347, 59.469036605680834 ], [ 24.719082953367163, 59.468990761718842 ], [ 24.718972185825663, 59.468873987387873 ], [ 24.71887671890202, 59.468760863796476 ], [ 24.71880306408633, 59.468643141837596 ], [ 24.718738399455688, 59.468576214524653 ], [ 24.718707589624756, 59.468537980444509 ], [ 24.718593286278054, 59.468417347158628 ], [ 24.718506608859641, 59.468306859388477 ], [ 24.718453193376668, 59.468220373336585 ], [ 24.718437045304302, 59.468161323219235 ], [ 24.718433168801635, 59.468107044621284 ], [ 24.718434758320619, 59.468064396890561 ], [ 24.718456620268284, 59.468022823572731 ], [ 24.718460805945021, 59.468002118564392 ], [ 24.718434451243493, 59.467958456190004 ], [ 24.718374279879232, 59.467881988953899 ], [ 24.718331682794272, 59.467846853132166 ], [ 24.71830606898596, 59.467811561904078 ], [ 24.718271165283536, 59.467743017939142 ], [ 24.718247030079187, 59.467699190826664 ], [ 24.718220504884926, 59.467624766367216 ], [ 24.718195760076643, 59.467572917443498 ], [ 24.718151883793116, 59.467475275161384 ], [ 24.718102552887963, 59.467395867222756 ], [ 24.718021937134552, 59.467292437636608 ], [ 24.71796202666367, 59.467204792855242 ], [ 24.717899886319181, 59.467115104538756 ], [ 24.717835681609753, 59.467020337675308 ], [ 24.71776883520177, 59.466904777293557 ], [ 24.717702364558917, 59.466783577493075 ], [ 24.717615768233184, 59.466636338258397 ], [ 24.717482894259387, 59.466397017076496 ], [ 24.717326783322573, 59.466111089777435 ], [ 24.717256906166291, 59.465991308629278 ], [ 24.71718901421411, 59.465881202445281 ], [ 24.717137248132005, 59.465802363923551 ], [ 24.717026492123342, 59.465703801465658 ], [ 24.716950112607517, 59.465635446856517 ], [ 24.716888956253101, 59.465571354097065 ], [ 24.716792647501343, 59.465460208377451 ], [ 24.716737898826999, 59.465392130969228 ], [ 24.716698236996589, 59.46531931261179 ], [ 24.716674884214274, 59.465276450412965 ], [ 24.716574310834499, 59.465101548596543 ], [ 24.716509561922603, 59.46499058115014 ], [ 24.716451712832082, 59.464812529528508 ], [ 24.716414189742256, 59.464697006478715 ], [ 24.716375992072148, 59.464583785104907 ], [ 24.716326232224482, 59.464484378798232 ], [ 24.716277288545292, 59.464386871085566 ], [ 24.716217961554744, 59.464272777061858 ], [ 24.716182496885107, 59.464180483305412 ], [ 24.716202930710661, 59.464142284305467 ], [ 24.716321864722666, 59.464104761795092 ], [ 24.716134777290033, 59.464031146641055 ], [ 24.715982618340682, 59.463971282213762 ], [ 24.715937736070106, 59.463942046705434 ], [ 24.715763534795641, 59.463888226365057 ], [ 24.715718216166994, 59.463857512004246 ], [ 24.71569800359315, 59.463845709751517 ], [ 24.715673337983343, 59.463827612111778 ], [ 24.715619913012997, 59.463808988412666 ], [ 24.715417306408924, 59.463729397275941 ], [ 24.715314936538547, 59.463691290504109 ], [ 24.715241274037247, 59.463647013492505 ], [ 24.715131982245055, 59.463582391195729 ], [ 24.715037893359845, 59.463524481651426 ], [ 24.715024117903091, 59.463516010615905 ], [ 24.715217442185612, 59.463424557638959 ], [ 24.714432887126847, 59.462974594352936 ], [ 24.712750696963074, 59.462010992644615 ], [ 24.711759345352114, 59.461442921848921 ], [ 24.710657533814018, 59.460810986977172 ], [ 24.709993081010786, 59.460423365828987 ], [ 24.70894268343185, 59.459826866796377 ], [ 24.708325431128205, 59.459476954620328 ], [ 24.707596545955642, 59.459058069957287 ], [ 24.707263781407995, 59.458870754124582 ], [ 24.707185878265634, 59.458826764699417 ], [ 24.707131599312135, 59.458801212188909 ], [ 24.705356225798255, 59.459073842720549 ], [ 24.703199459286047, 59.45941681534935 ], [ 24.702938140800462, 59.458976199130028 ], [ 24.703242606874856, 59.458879763096959 ], [ 24.703781070092518, 59.458712705134353 ], [ 24.705019065579709, 59.458322900305312 ], [ 24.70498548004452, 59.458301060252815 ], [ 24.705001375212039, 59.458210005717909 ], [ 24.704938297754481, 59.458186932062894 ], [ 24.704910046205072, 59.458176598464547 ], [ 24.704626334046424, 59.45809811594949 ], [ 24.703542229800888, 59.457847445695812 ], [ 24.702924996573934, 59.457704725578665 ], [ 24.703022081787108, 59.457599055005979 ], [ 24.703217612632745, 59.457386216236536 ], [ 24.703241068833933, 59.457360696069451 ], [ 24.702657140380239, 59.456662823745113 ], [ 24.702642280032418, 59.456645048408518 ], [ 24.702577019121666, 59.456567057068554 ], [ 24.702349023643571, 59.456512312094574 ], [ 24.701910861757483, 59.456407097626993 ], [ 24.701428193881092, 59.456285440013161 ], [ 24.70055052404166, 59.456073161513359 ], [ 24.700050423546774, 59.455952201989255 ], [ 24.700006901356645, 59.455941679184178 ], [ 24.700046688008086, 59.455897284659265 ], [ 24.70015061882021, 59.45578131047089 ], [ 24.700314628827268, 59.455587341788508 ], [ 24.700326204542069, 59.455579165354614 ], [ 24.700389280080017, 59.45553461999345 ], [ 24.700397300562202, 59.455537081846721 ], [ 24.700594956751292, 59.455597781447473 ], [ 24.700906055057906, 59.455693317188434 ], [ 24.700958004379597, 59.455659109024531 ], [ 24.70045338585269, 59.455495607725673 ], [ 24.700480625741946, 59.455473990722098 ], [ 24.700550096045152, 59.455418863699421 ], [ 24.700563507678783, 59.455408217875352 ], [ 24.700584207589728, 59.455414193986812 ], [ 24.700693099967058, 59.4554456526389 ], [ 24.70106249123862, 59.455552368116415 ], [ 24.701097580833146, 59.455548097017271 ], [ 24.701144409626373, 59.455508413327038 ], [ 24.701465640278741, 59.455238794840682 ], [ 24.701525885137574, 59.455232513569783 ], [ 24.701705622674648, 59.455286143539794 ], [ 24.702215511586129, 59.455438265473894 ], [ 24.703075481608305, 59.455700663304945 ], [ 24.703116929912436, 59.455670379258123 ], [ 24.703151413807607, 59.4556572508722 ], [ 24.703256770463593, 59.455694103313029 ], [ 24.703378366285818, 59.455736048515469 ], [ 24.703484897681133, 59.45576998606343 ], [ 24.703663999928249, 59.455794209208008 ], [ 24.703843230774087, 59.455834068780163 ], [ 24.703937145781545, 59.45586326186762 ], [ 24.704078480145487, 59.45590720150954 ], [ 24.704262917578131, 59.455955793920396 ], [ 24.704427013674639, 59.456002995726962 ], [ 24.704660466964498, 59.456070149589706 ], [ 24.704766298404497, 59.456099107801684 ], [ 24.70487762715851, 59.456129562553286 ], [ 24.705014333468593, 59.456169576223999 ], [ 24.70509528240159, 59.456193272336854 ], [ 24.705165769883983, 59.456216719184098 ], [ 24.705217956685189, 59.456248818713924 ], [ 24.705245889582002, 59.456265994173677 ], [ 24.705299069068218, 59.456306876517097 ], [ 24.705329887305655, 59.456330562552097 ], [ 24.705388308263146, 59.456363113381748 ], [ 24.705414423888374, 59.456377668371012 ], [ 24.705444933085818, 59.456411140602881 ], [ 24.705478815588606, 59.456460232122694 ], [ 24.705534053116395, 59.456514496541338 ], [ 24.705573634015916, 59.456543234344629 ], [ 24.70565193509081, 59.456581235154673 ], [ 24.705697157115605, 59.456606477714878 ], [ 24.705750623454218, 59.45665094900729 ], [ 24.705776028002024, 59.456674592122482 ], [ 24.705792687880425, 59.456735840455856 ], [ 24.705796472619554, 59.456770506027766 ], [ 24.705807802405829, 59.456793129376841 ], [ 24.705838951475581, 59.456808043315611 ], [ 24.705882761069539, 59.456821237713527 ], [ 24.705921298315314, 59.456840402898841 ], [ 24.70592740133371, 59.456857551543273 ], [ 24.705929704643385, 59.456886228633429 ], [ 24.705920486455611, 59.456907750211073 ], [ 24.705915453246796, 59.456925290655924 ], [ 24.705917955968062, 59.456931201847965 ], [ 24.705924796001838, 59.456947323203238 ], [ 24.705938659514008, 59.456961997591165 ], [ 24.705965149615434, 59.456978426588705 ], [ 24.706002016550389, 59.456992699439397 ], [ 24.706020884690581, 59.457002948435381 ], [ 24.706028928190058, 59.45701330929257 ], [ 24.70603339954145, 59.457027441537647 ], [ 24.706026802735753, 59.457040142964196 ], [ 24.706021829137249, 59.457058859036408 ], [ 24.706017001968654, 59.457076981863572 ], [ 24.706023420764151, 59.457108545337952 ], [ 24.706044236240274, 59.457145597200693 ], [ 24.706070111289712, 59.457171499856507 ], [ 24.706085743498154, 59.457183866723717 ], [ 24.706099565039747, 59.457194798048995 ], [ 24.706124792259704, 59.457224330739173 ], [ 24.706131854636357, 59.457252048744664 ], [ 24.706124704370648, 59.457262715437686 ], [ 24.7061178174534, 59.457269062942267 ], [ 24.706106119877496, 59.457279834793567 ], [ 24.706109337233638, 59.457302393868403 ], [ 24.706115435220465, 59.457313321698948 ], [ 24.706141860061081, 59.457360684603181 ], [ 24.706143641155016, 59.457399059283411 ], [ 24.70614023976826, 59.457419616124774 ], [ 24.70616130275188, 59.457449009543453 ], [ 24.70620974514463, 59.457478974347701 ], [ 24.706250011170589, 59.457519440776331 ], [ 24.706286487272475, 59.457544622292254 ], [ 24.706323927691841, 59.457556746556406 ], [ 24.706462703985316, 59.45758309413592 ], [ 24.706489860973573, 59.457588253736276 ], [ 24.706545465937563, 59.45760727336377 ], [ 24.706589309199821, 59.457613510432708 ], [ 24.706606163802398, 59.457615906583399 ], [ 24.706600611530586, 59.457633063837775 ], [ 24.706966009680468, 59.457686247473006 ], [ 24.707101335394693, 59.457706006071355 ], [ 24.70772661985723, 59.457797307962025 ], [ 24.708295749787439, 59.457880407868004 ], [ 24.708868091515932, 59.457963488048982 ], [ 24.709221299857546, 59.458015249889662 ], [ 24.709895987560877, 59.458115817811937 ], [ 24.710459229232864, 59.458198473356703 ], [ 24.71096208617455, 59.458272269204187 ], [ 24.711881054951768, 59.458409282548047 ], [ 24.712387862558494, 59.458485340688782 ], [ 24.7126239456605, 59.458518920512375 ], [ 24.713393755771346, 59.458634683261778 ], [ 24.714067976684586, 59.458731973925396 ], [ 24.714683953145695, 59.458822440623656 ], [ 24.714942181558861, 59.458859836959078 ], [ 24.715648398717917, 59.458962089600867 ], [ 24.715897519577091, 59.458996472491734 ], [ 24.715997299935289, 59.458839413039165 ], [ 24.716078581691278, 59.45869248097457 ], [ 24.716108455257885, 59.458639553722946 ], [ 24.716202275248545, 59.458473343334447 ], [ 24.716311799550695, 59.45827234365526 ], [ 24.71637212194841, 59.458171288407492 ], [ 24.716403489858795, 59.458144241688828 ], [ 24.716437927588782, 59.458123991569586 ], [ 24.716557648755412, 59.458053564833641 ], [ 24.716741985373677, 59.457947247843222 ], [ 24.716825735882431, 59.457943363153809 ], [ 24.718273457221187, 59.459069386462708 ], [ 24.718298858288328, 59.459072246207725 ], [ 24.718701656429939, 59.45893767075826 ], [ 24.718700801428287, 59.458926373783207 ], [ 24.717365478601558, 59.457881305290485 ], [ 24.719201788810679, 59.457273322100619 ], [ 24.721328104494532, 59.458931685314795 ], [ 24.72127269407801, 59.45906178248255 ], [ 24.721403626405802, 59.459173408890841 ], [ 24.721124517964935, 59.459266696294499 ], [ 24.720311188753843, 59.4595325457706 ], [ 24.720095218239155, 59.459607119509812 ], [ 24.720071528599146, 59.4595909473191 ], [ 24.720045208909664, 59.459572985183556 ], [ 24.719071913307225, 59.459890679456187 ], [ 24.718827284022829, 59.459977965835414 ], [ 24.719025014391651, 59.460018449050949 ], [ 24.719050749205543, 59.460023721558706 ], [ 24.720033966493389, 59.459693809867488 ], [ 24.720594327158089, 59.459500485969699 ], [ 24.721148550210124, 59.459316609768052 ], [ 24.721738403088139, 59.459129214664564 ], [ 24.722256537250018, 59.458956339304024 ], [ 24.722316338640912, 59.458935625451197 ], [ 24.72236583399653, 59.458918729300898 ], [ 24.722902521266079, 59.458741234365419 ], [ 24.723584307327734, 59.458509241934756 ], [ 24.723897190355139, 59.458410970122344 ], [ 24.723759008508882, 59.458305328517426 ], [ 24.723638544231726, 59.45834836011862 ], [ 24.723710636821252, 59.458404283913453 ], [ 24.723341340275098, 59.458526653201702 ], [ 24.722826932128207, 59.458695876925816 ], [ 24.722193958248173, 59.4589049317225 ], [ 24.722068797700867, 59.458946228165395 ], [ 24.721953557814054, 59.458867083830782 ], [ 24.721855836958071, 59.458799970922257 ], [ 24.72183243124994, 59.458783896229484 ], [ 24.721805566914874, 59.458765210310368 ], [ 24.721246215197922, 59.458330875315802 ], [ 24.720728172416635, 59.457928089466982 ], [ 24.720063610285816, 59.45741099433608 ], [ 24.719429943029159, 59.456922282010197 ], [ 24.719243852802713, 59.456774751751546 ], [ 24.719364609289485, 59.456262645695901 ], [ 24.719492231652097, 59.455715043982018 ], [ 24.719580426445336, 59.455333259381753 ], [ 24.71974981860528, 59.454597699409625 ], [ 24.719714898472326, 59.454587270742614 ], [ 24.719635548500982, 59.454563619712069 ], [ 24.719570517583566, 59.454544226173333 ], [ 24.719443170978479, 59.45450625900601 ], [ 24.719880838682897, 59.45417589572164 ], [ 24.720097405425133, 59.454010797705692 ], [ 24.720380953447233, 59.453794704496886 ], [ 24.720803565635375, 59.453507894440641 ], [ 24.721423594361681, 59.453305514138322 ], [ 24.722166848954924, 59.453059250567328 ], [ 24.723022829435301, 59.452774751712973 ], [ 24.724122516756466, 59.452409823058773 ], [ 24.724545317636736, 59.452268808257898 ], [ 24.725161010139978, 59.45206191858437 ], [ 24.725612596058138, 59.451913542516749 ], [ 24.726560143622585, 59.452622525377393 ], [ 24.726921186411012, 59.452506062679475 ], [ 24.726906880679788, 59.452485863218797 ], [ 24.726893404483739, 59.45247481542728 ], [ 24.726798683911749, 59.452423183315183 ], [ 24.726819138046764, 59.452414991428789 ], [ 24.726854436077808, 59.452391816297265 ], [ 24.72687514013047, 59.452370517020881 ], [ 24.72694690020117, 59.452337445829976 ], [ 24.727129669146635, 59.452437856101412 ], [ 24.727139345426149, 59.452434391542816 ], [ 24.727546618757337, 59.452291711264664 ], [ 24.72758468234619, 59.452278377093236 ], [ 24.727589755277283, 59.452283115711886 ], [ 24.727786004182796, 59.452465811995019 ], [ 24.727875124608254, 59.452547456573498 ], [ 24.727929372942452, 59.452601079964815 ], [ 24.727899954929779, 59.452613890698863 ], [ 24.728076348192815, 59.452789676458885 ], [ 24.728131849629804, 59.452844980462508 ], [ 24.728259285654335, 59.452958899734426 ], [ 24.728493165754973, 59.453167949027524 ], [ 24.728876695921301, 59.453529330729495 ], [ 24.729300258944434, 59.453914663993459 ], [ 24.72972903362664, 59.454314051483465 ], [ 24.730210443701175, 59.454756799597853 ], [ 24.730352990696847, 59.454715892552883 ], [ 24.73011079005072, 59.454490644830578 ], [ 24.729373826738289, 59.453811814491424 ], [ 24.729420567660416, 59.453797722603227 ], [ 24.729277963375718, 59.453675647795706 ], [ 24.729229534678353, 59.453684183414808 ], [ 24.72901201239366, 59.453481912097836 ], [ 24.72856837857411, 59.453073573847625 ], [ 24.728292714650269, 59.452822475350111 ], [ 24.728173825636706, 59.452713302430986 ], [ 24.72884628656487, 59.452497769267623 ], [ 24.72953635010321, 59.452273867573126 ], [ 24.729852923345309, 59.452171880075426 ], [ 24.730098259476108, 59.452363205935491 ], [ 24.730283980399609, 59.452508048231927 ], [ 24.730443117940695, 59.452631601258091 ], [ 24.730980368326829, 59.453048722817037 ], [ 24.730992614224181, 59.453058233027612 ], [ 24.731250807148577, 59.452975749381835 ], [ 24.731683615424469, 59.453309416713793 ], [ 24.732200351197722, 59.453711763863851 ], [ 24.732420030462674, 59.453880472030512 ], [ 24.732438077621875, 59.453881745137494 ], [ 24.733372773409062, 59.453574254453002 ], [ 24.733565934071528, 59.4535079721785 ], [ 24.733669007471565, 59.453472595477777 ], [ 24.733966044943692, 59.453370662045089 ], [ 24.734014737190055, 59.453353954390579 ], [ 24.734197043780341, 59.453291394146376 ], [ 24.734542367247936, 59.453172888560481 ], [ 24.734517789596488, 59.453145054124931 ], [ 24.734909811101193, 59.45302384551951 ], [ 24.734911358534443, 59.453000712848187 ], [ 24.734941081415784, 59.452980053180752 ], [ 24.734985709494548, 59.45297362822631 ], [ 24.73502477478214, 59.452974164323344 ], [ 24.735162905220292, 59.452976050702155 ], [ 24.735195250823715, 59.452976489595216 ], [ 24.735219520022348, 59.45310872492535 ], [ 24.737788942769019, 59.453025958106473 ], [ 24.737931112477163, 59.453021373673742 ], [ 24.738978255595828, 59.452988286740414 ], [ 24.739500539729701, 59.452977052622359 ], [ 24.739539123749033, 59.453514694725179 ], [ 24.739545644960206, 59.453640772172022 ], [ 24.739552109966585, 59.453765880441495 ], [ 24.739579317745417, 59.454181978528446 ], [ 24.739580421135603, 59.454198731881867 ], [ 24.73960496574054, 59.4545944138208 ], [ 24.739643232208305, 59.454594611927689 ], [ 24.739644146643055, 59.454609965969873 ], [ 24.738927418509252, 59.455004771410096 ], [ 24.73865171883692, 59.45515896653108 ], [ 24.738595433994107, 59.455137702035771 ], [ 24.737437721744524, 59.455142248062394 ], [ 24.737133538891069, 59.455135592948587 ], [ 24.737131222477434, 59.455107517754641 ], [ 24.737101809812998, 59.455096012575915 ], [ 24.737005742004882, 59.455096576934565 ], [ 24.736922015323916, 59.455097063219853 ], [ 24.736893328370613, 59.455107744384797 ], [ 24.736896597444598, 59.455200401743539 ], [ 24.736906881417038, 59.455208764386207 ], [ 24.737432878774577, 59.455216826822983 ], [ 24.738682359734931, 59.455214181112233 ], [ 24.739117866268217, 59.455000633107211 ], [ 24.739765091697194, 59.454635418173758 ], [ 24.739803981624455, 59.454644858759998 ], [ 24.740126555794102, 59.454462982905589 ], [ 24.739957727788145, 59.454390609325664 ], [ 24.739801727091621, 59.45438452843679 ], [ 24.739767150764742, 59.453908408506578 ], [ 24.739743888470286, 59.453485708305969 ], [ 24.739702950163576, 59.452889991033565 ], [ 24.739678287565219, 59.452531185924464 ], [ 24.739693644339876, 59.452468504947994 ], [ 24.739705016199434, 59.452422094228297 ], [ 24.739749758587216, 59.452324975077374 ], [ 24.739777517407077, 59.452230087739473 ], [ 24.739812057022895, 59.452198358516483 ], [ 24.739903463313869, 59.452162045962488 ], [ 24.739978542323399, 59.45211674957622 ], [ 24.740057638988834, 59.452082750237579 ], [ 24.740191475623813, 59.452047743114932 ], [ 24.740183808225424, 59.451866438198302 ], [ 24.740249774076787, 59.45181070802203 ], [ 24.740324016962465, 59.451761870313263 ], [ 24.740440507195739, 59.45171159223203 ], [ 24.740472277091659, 59.4516652014266 ], [ 24.740513190818426, 59.451629082461018 ], [ 24.74059128099281, 59.451600779694743 ], [ 24.740679811407595, 59.451570173998419 ], [ 24.740761212870162, 59.451540918931229 ], [ 24.74093413773786, 59.451505269324905 ], [ 24.741009470175676, 59.451484782601263 ], [ 24.741214481677673, 59.451432039960544 ], [ 24.741494186808097, 59.451362736421174 ], [ 24.741661495765051, 59.451313742116348 ], [ 24.741786479048709, 59.451274761600168 ], [ 24.741846825329514, 59.451255938679942 ], [ 24.74195488706788, 59.451214719337386 ], [ 24.742081777613215, 59.451161311148574 ], [ 24.742123357980088, 59.451143805667485 ], [ 24.742347031076982, 59.451047014958299 ], [ 24.742590326748356, 59.450930005345882 ], [ 24.742802290482118, 59.450826726823244 ], [ 24.743013000997674, 59.450713715250046 ], [ 24.7431733757934, 59.450625825954376 ], [ 24.743346285171967, 59.450508790050741 ], [ 24.743416451648073, 59.450427711114244 ], [ 24.743458894192202, 59.450381043863779 ], [ 24.743492239058579, 59.450333548224052 ], [ 24.743476455793481, 59.450287999528605 ], [ 24.743430404549972, 59.450262020398178 ], [ 24.743553711326843, 59.450108244011147 ], [ 24.743690465919112, 59.449943816966091 ], [ 24.743835989535, 59.449766009775033 ], [ 24.743958349291564, 59.449620721333787 ], [ 24.744016387206809, 59.449551963789986 ], [ 24.74401781993604, 59.449535995038097 ], [ 24.744046216392544, 59.449470214613555 ], [ 24.744058333382412, 59.449442138714701 ], [ 24.744084186771889, 59.449372117663493 ], [ 24.744126455549328, 59.449336187340407 ], [ 24.744211264570414, 59.449280842337359 ], [ 24.744311263495064, 59.449188795283845 ], [ 24.744441419604332, 59.44910939652361 ], [ 24.744513861955411, 59.449073124985411 ], [ 24.74455502925067, 59.449043313909229 ], [ 24.744570173487027, 59.449023120386165 ], [ 24.744608858098438, 59.448976473961629 ], [ 24.744674547804674, 59.448904620747953 ], [ 24.744702388072319, 59.448881330269089 ], [ 24.744740737395023, 59.448849237012766 ], [ 24.744750311836579, 59.448841229478994 ], [ 24.744812647330999, 59.448782932096684 ], [ 24.744837526236829, 59.448759667307975 ], [ 24.74488289488885, 59.448704275549183 ], [ 24.744953591022853, 59.448647008027002 ], [ 24.744961559514437, 59.44864054458364 ], [ 24.744988236287142, 59.448627781374995 ], [ 24.745035401630158, 59.448610324135842 ], [ 24.745314071523723, 59.448520030437948 ], [ 24.745593413869297, 59.448428305028692 ], [ 24.745923538011596, 59.448339523093885 ], [ 24.746191668647313, 59.448275391593434 ], [ 24.746411543248342, 59.448209450381526 ], [ 24.746646979964403, 59.448158672098643 ], [ 24.746959121511438, 59.448084594643959 ], [ 24.74716604222882, 59.448029407951871 ], [ 24.747360912549119, 59.447976219303932 ], [ 24.747454566962528, 59.447965373606792 ], [ 24.747615925534298, 59.447952527778966 ], [ 24.747768486788711, 59.447941275696436 ], [ 24.748175958797177, 59.447916187886911 ], [ 24.748319035317849, 59.447908956765048 ], [ 24.748510440527248, 59.447903695061854 ], [ 24.748718183802396, 59.447901473105254 ], [ 24.748837190268901, 59.447897968936772 ], [ 24.74893733919313, 59.44789390759955 ], [ 24.749002383388522, 59.447888789055703 ], [ 24.749044650801839, 59.447884464282076 ], [ 24.749141842409994, 59.447869351257523 ], [ 24.749164994163326, 59.447866364975447 ], [ 24.749366759039802, 59.448070551767415 ], [ 24.749412699089554, 59.448098621072091 ], [ 24.749534133300401, 59.44814466317024 ], [ 24.749524678814947, 59.448068908209791 ], [ 24.749480708050214, 59.448012488099465 ], [ 24.749377610908216, 59.447917543952315 ], [ 24.749297177193604, 59.447852776358545 ], [ 24.749239817950805, 59.447824080755289 ], [ 24.749212394372691, 59.447797073052946 ], [ 24.749147828969583, 59.447742834622353 ], [ 24.749139570838906, 59.447732028682054 ], [ 24.749069587457296, 59.447683135270161 ], [ 24.749108938486895, 59.447632659565151 ], [ 24.749146365964656, 59.447590938154278 ], [ 24.749204396671725, 59.447553022392619 ], [ 24.749288554985153, 59.447509571923639 ], [ 24.74940350999276, 59.447479761431531 ], [ 24.749513599805677, 59.44745559797591 ], [ 24.749556687253456, 59.447498306614222 ], [ 24.749593171522232, 59.447533323821197 ], [ 24.749633184402686, 59.447530717270368 ], [ 24.749851767343198, 59.447748524134091 ], [ 24.749940839893242, 59.447831752300402 ], [ 24.750052808998166, 59.447914805148017 ], [ 24.75005874101133, 59.447919214868207 ], [ 24.750097966687267, 59.4479048261705 ], [ 24.75007602535177, 59.447888272286129 ], [ 24.750083178737263, 59.447877656924305 ], [ 24.750055690928939, 59.447855784468807 ], [ 24.750026103779707, 59.447832236329518 ], [ 24.749972131598035, 59.447789221770563 ], [ 24.749956459988557, 59.447786833409921 ], [ 24.749683238204028, 59.447516527394988 ], [ 24.749691595704341, 59.447499729185871 ], [ 24.749698551468455, 59.447485730727955 ], [ 24.749746523057841, 59.447457800274869 ], [ 24.749836418669474, 59.447420376109235 ], [ 24.750140167013932, 59.447317927332207 ], [ 24.750173124051098, 59.447307910081847 ], [ 24.75020890038736, 59.447297032949955 ], [ 24.750258526429775, 59.447290682004599 ], [ 24.75032904111433, 59.447288287505053 ], [ 24.750390388895703, 59.447289903944224 ], [ 24.750456267034199, 59.447293918271022 ], [ 24.750489382396928, 59.447290309450096 ], [ 24.750514952101383, 59.447284266026827 ], [ 24.750591447327349, 59.447305338486217 ], [ 24.750614241781268, 59.447290127625472 ], [ 24.750667179470422, 59.447254789627124 ], [ 24.750666137983217, 59.447245100654087 ], [ 24.750645010457777, 59.447236693127699 ], [ 24.750106734503188, 59.447080736112703 ], [ 24.749476793062264, 59.44689493011861 ], [ 24.749341989130293, 59.446854771998652 ], [ 24.748801730336254, 59.446693856824197 ], [ 24.748500873208013, 59.446599156343169 ], [ 24.748285034693197, 59.44651635118128 ], [ 24.748250929219836, 59.446503708153266 ], [ 24.748027476519749, 59.446420900943089 ], [ 24.747723182958136, 59.446312349113725 ], [ 24.747500771128792, 59.446226240636335 ], [ 24.747336122941064, 59.446162488346594 ], [ 24.747477934078592, 59.446101790675634 ], [ 24.747567174706152, 59.446067513617812 ], [ 24.747681520230127, 59.446029341704943 ], [ 24.747791202045313, 59.445987506716797 ], [ 24.747816886939905, 59.445975475653562 ], [ 24.747862271584548, 59.445954212338819 ], [ 24.747967470721033, 59.445904916027693 ], [ 24.748044978855191, 59.44587022949338 ], [ 24.748125925468145, 59.44583398835465 ], [ 24.74819226673895, 59.445795711612043 ], [ 24.748240143215977, 59.445765906078122 ], [ 24.748263220359966, 59.445751528848724 ], [ 24.748301444879765, 59.445727729720808 ], [ 24.748572779455806, 59.445757840018622 ], [ 24.74889804514374, 59.445783450981338 ], [ 24.748895705386076, 59.445788491275529 ], [ 24.748763722102467, 59.446072180407249 ], [ 24.748713431874638, 59.446187288121621 ], [ 24.74864484388042, 59.446344241778391 ], [ 24.748883722905919, 59.44641099955134 ], [ 24.749373386148296, 59.446540648904886 ], [ 24.74944407571029, 59.446558343881108 ], [ 24.749626436534896, 59.446603985359474 ], [ 24.749708027650861, 59.446624409927296 ], [ 24.749872761429167, 59.446631533338987 ], [ 24.750123780733574, 59.446686594850853 ], [ 24.750296777136171, 59.446722360431153 ], [ 24.750457436225702, 59.446768654012409 ], [ 24.750527012889286, 59.44677897584868 ], [ 24.750607450853259, 59.446778167399565 ], [ 24.750723204805581, 59.446796098607138 ], [ 24.750813142164287, 59.446813629290517 ], [ 24.750858310951294, 59.446826684364119 ], [ 24.750881942346901, 59.446862393890349 ], [ 24.750894666118313, 59.446886271375277 ], [ 24.750906134124012, 59.446949249844749 ], [ 24.750894941330809, 59.446978173962449 ], [ 24.750892197780573, 59.446992902525857 ], [ 24.750889215880807, 59.447008817381274 ], [ 24.750902544377198, 59.447022511761958 ], [ 24.750925376838225, 59.447028288311337 ], [ 24.750936775248, 59.447031167709397 ], [ 24.751136116818987, 59.447085624237651 ], [ 24.751171551474396, 59.447095305588313 ], [ 24.751188705064372, 59.447100162940224 ], [ 24.751498394394716, 59.447187921760118 ], [ 24.751519044064661, 59.447198638906698 ], [ 24.751528213068834, 59.447212356956413 ], [ 24.751533365344471, 59.447227794532225 ], [ 24.751528878457243, 59.447325119323175 ], [ 24.751530801657726, 59.447358044112299 ], [ 24.751538625323978, 59.447372308442517 ], [ 24.751547524603243, 59.44738337988268 ], [ 24.751567048240791, 59.447394992146343 ], [ 24.751591292417302, 59.447403220161064 ], [ 24.751782299775105, 59.447416071071942 ], [ 24.751982239382958, 59.447423861677052 ], [ 24.752419739732851, 59.447449101530218 ], [ 24.752552617497866, 59.447456771625575 ], [ 24.752486523380988, 59.447906274887458 ], [ 24.752407601767207, 59.448434577531692 ], [ 24.752447733398352, 59.448435721741376 ], [ 24.752486277417553, 59.448436821147531 ], [ 24.752752963510826, 59.448442837907606 ], [ 24.7527265972202, 59.448630333674558 ], [ 24.752724602069854, 59.448642194405593 ], [ 24.752723138586731, 59.448650982044754 ], [ 24.753017640061145, 59.448662351064122 ], [ 24.753038410315209, 59.448540148313384 ], [ 24.753190944066521, 59.448545191923081 ], [ 24.753191135291168, 59.448567453209918 ], [ 24.753191197390315, 59.448574266220369 ], [ 24.753182641289929, 59.448577995616809 ], [ 24.753165232590295, 59.448585599731516 ], [ 24.753144890537072, 59.44860168570569 ], [ 24.753138532262295, 59.44861874200727 ], [ 24.753141821246341, 59.448628786157187 ], [ 24.753144806823009, 59.448637862551017 ], [ 24.75315788020654, 59.448649511459493 ], [ 24.753178231818229, 59.448660239028769 ], [ 24.753203425622605, 59.448665104009869 ], [ 24.753224299043328, 59.448667516103228 ], [ 24.753250648376373, 59.448668020736442 ], [ 24.753276496861396, 59.448665000359483 ], [ 24.753308106502988, 59.448654621989895 ], [ 24.753329053741155, 59.448641997561296 ], [ 24.753336525533484, 59.448632268899999 ], [ 24.753339268931803, 59.448619972994479 ], [ 24.753339476377906, 59.448610995041868 ], [ 24.753334637747102, 59.448600232640736 ], [ 24.753324118242912, 59.448590876172986 ], [ 24.753308086426355, 59.448581775647163 ], [ 24.753298908917685, 59.448577312817285 ], [ 24.753302033137995, 59.448550310791141 ], [ 24.753458528098356, 59.448556704883693 ], [ 24.753441353844362, 59.448651651631558 ], [ 24.753434847643323, 59.4486875689797 ], [ 24.753743902141434, 59.448700109886147 ], [ 24.753779256713962, 59.448480793675884 ], [ 24.753779291045767, 59.448480794910736 ], [ 24.753779291571817, 59.448480775522903 ], [ 24.753781340805425, 59.44831801499874 ], [ 24.753918519808987, 59.447328797801617 ], [ 24.755020159546792, 59.447361895887923 ], [ 24.755167583120659, 59.44650296086408 ], [ 24.755246805332117, 59.446003936422194 ], [ 24.754151413719349, 59.445964431177707 ], [ 24.754335450758354, 59.444639481895699 ], [ 24.753255097077556, 59.444600421913087 ], [ 24.753138299737287, 59.444403780193369 ], [ 24.753179072698924, 59.444110455128225 ], [ 24.753140051950091, 59.444027553262458 ], [ 24.753012349669866, 59.443943811914046 ], [ 24.753154265815606, 59.443528991221442 ], [ 24.749665082529383, 59.443858955586009 ], [ 24.748592982229404, 59.443945754710313 ], [ 24.746805115647486, 59.444140466772801 ], [ 24.746354124021874, 59.443990956663974 ], [ 24.746081321279991, 59.443921136943466 ], [ 24.745850115884004, 59.44385924996876 ], [ 24.745518703481363, 59.44376911440586 ], [ 24.745224079311789, 59.443579576538532 ], [ 24.74447615034801, 59.442413594251903 ], [ 24.744235685617401, 59.442298255243095 ], [ 24.744173956040182, 59.442272212339574 ], [ 24.743810836294841, 59.442101101648319 ], [ 24.74313522581533, 59.441800872228008 ], [ 24.7421306807605, 59.441408955361325 ], [ 24.74136201793052, 59.441115973834854 ], [ 24.741326513258421, 59.441079907526152 ], [ 24.741291249231232, 59.441063050151648 ], [ 24.741047867184058, 59.440966662791737 ], [ 24.739435473422841, 59.440354533028973 ], [ 24.739140676099449, 59.440259956904406 ], [ 24.738783358681829, 59.440094185977777 ], [ 24.738545149153577, 59.44003259490259 ], [ 24.737058921644824, 59.439562190100574 ], [ 24.736778187408515, 59.439448948101266 ], [ 24.736166647155347, 59.439262240046794 ], [ 24.736049916059358, 59.439194130453849 ], [ 24.735913809554475, 59.439101981512785 ], [ 24.735588246392183, 59.438960890205642 ], [ 24.735144724130773, 59.438759863433013 ], [ 24.734732613301361, 59.438552286417874 ], [ 24.734454978118048, 59.438361103819936 ], [ 24.73423957119843, 59.438203326360927 ], [ 24.734122778837463, 59.438123814911009 ], [ 24.73408653447968, 59.438077427501064 ], [ 24.733381770600683, 59.437248758458225 ], [ 24.732927043762604, 59.43673736972324 ], [ 24.732790733437071, 59.43659378176396 ], [ 24.732489174737779, 59.436485583818992 ], [ 24.732470444209095, 59.436228413539133 ], [ 24.732338163751354, 59.436124659587612 ], [ 24.732669707710901, 59.435042639143283 ], [ 24.732582610227592, 59.434983069069993 ], [ 24.73147969535157, 59.435043420434987 ], [ 24.730646351289359, 59.435093379976848 ], [ 24.730120623218021, 59.435098989182706 ], [ 24.729785349646342, 59.43509438454133 ], [ 24.728147000796643, 59.43492032844739 ], [ 24.727746474746823, 59.434888430981083 ], [ 24.726645718853927, 59.434810218256267 ], [ 24.726305740882651, 59.434783099029396 ], [ 24.725555961238953, 59.43467708766137 ], [ 24.724874690491436, 59.43457724795504 ], [ 24.723703896372204, 59.434319771055797 ], [ 24.723244348548192, 59.434237914413394 ], [ 24.722077410369828, 59.434028337929725 ], [ 24.722005063466995, 59.434015359271427 ], [ 24.721432025040258, 59.433899378509231 ], [ 24.720434553130186, 59.433719649242853 ], [ 24.718672851120008, 59.433397495394843 ], [ 24.718189899372362, 59.433299231705426 ], [ 24.717933897575282, 59.433334470259403 ], [ 24.717499763439026, 59.433202813584494 ], [ 24.717143273611541, 59.433093084760038 ], [ 24.716789618311289, 59.432983967920755 ], [ 24.716445976850874, 59.432881707761034 ], [ 24.71514631248435, 59.432691194748493 ], [ 24.713511034838444, 59.432173938546732 ], [ 24.711516281983997, 59.431467220725985 ], [ 24.709647638427676, 59.430889866576024 ], [ 24.709570685375031, 59.430951502872347 ], [ 24.707133444205017, 59.4301749300596 ], [ 24.706889051531761, 59.430097875349581 ], [ 24.706774545357874, 59.430064827251677 ], [ 24.706555555545108, 59.430014745377846 ], [ 24.70617352384058, 59.429927566069026 ], [ 24.705753080548128, 59.429831704581943 ], [ 24.705468325764883, 59.42972811286397 ], [ 24.704288578806995, 59.429405610514955 ], [ 24.703762249318046, 59.429360758526087 ], [ 24.703748658960073, 59.429402393581064 ], [ 24.701644244694517, 59.429335079327089 ], [ 24.70067889514884, 59.429404135182175 ], [ 24.69983812180055, 59.429465160831846 ], [ 24.697221653102332, 59.429654718520965 ], [ 24.696725028494381, 59.429683829077341 ], [ 24.696562842986388, 59.429688188217092 ], [ 24.695863795369455, 59.429727163576267 ], [ 24.693114966092615, 59.429890767776783 ], [ 24.693099618403743, 59.429959341140389 ], [ 24.691352496877631, 59.429995547345804 ], [ 24.691102404424768, 59.43000072041054 ], [ 24.690149043212777, 59.429993513137198 ], [ 24.690068370719857, 59.429994743914811 ], [ 24.688463874658488, 59.429947847420692 ], [ 24.688135365619029, 59.430285025572751 ], [ 24.687934360065274, 59.430330510878377 ], [ 24.687140800214141, 59.431164464316723 ], [ 24.686509995701488, 59.43186587535255 ], [ 24.68537302343578, 59.43294146546981 ], [ 24.682668606171397, 59.435704903596125 ], [ 24.679571909932431, 59.436743770852466 ], [ 24.679752011765995, 59.436795589880113 ] ] ] ] } },
{ "type": "Feature", "properties": { "ONIMI": "Tallinn", "OKOOD": "0784", "MNIMI": "Harju maakond", "MKOOD": "0037", "TYYP": "4", "District": "Kesklinna", "AKOOD": "0298", "TYYP_2": "6", "ONIMI_2": "Tallinn", "OKOOD_2": "0784", "MNIMI_2": "Harju maakond", "MKOOD_2": "0037" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 24.779768282065547, 59.591205812293452 ], [ 24.779815157344022, 59.591196416080983 ], [ 24.779867430521996, 59.591196987598998 ], [ 24.779901247444464, 59.591195603364838 ], [ 24.779952910352183, 59.591193494540931 ], [ 24.78005685624067, 59.591173860912505 ], [ 24.780150914369504, 59.591144564208939 ], [ 24.780238433602634, 59.591124255351012 ], [ 24.780332538195388, 59.591115513912513 ], [ 24.780546520182337, 59.591104844354483 ], [ 24.780816969684654, 59.591107808246711 ], [ 24.78092352476984, 59.591106236751742 ], [ 24.781042899963232, 59.591104023973301 ], [ 24.78114381661052, 59.591094362060844 ], [ 24.781236298068432, 59.59109358254338 ], [ 24.781302494251161, 59.591099142864401 ], [ 24.781315602174267, 59.591100241338182 ], [ 24.781397263468339, 59.591113178532126 ], [ 24.781466285938652, 59.591120005684523 ], [ 24.781521578239502, 59.591120226590917 ], [ 24.781575813232244, 59.591129133781145 ], [ 24.78162900521508, 59.591151969318041 ], [ 24.781661078053244, 59.591166366257575 ], [ 24.781697970225725, 59.591175546423031 ], [ 24.781735897779384, 59.591174316954408 ], [ 24.781776098339943, 59.591167230485439 ], [ 24.781856902483501, 59.591165200031966 ], [ 24.781906273128758, 59.591158876165572 ], [ 24.781984034515684, 59.591139234209621 ], [ 24.782063810286953, 59.591110963071472 ], [ 24.782121599051919, 59.591095801579804 ], [ 24.78218811700258, 59.59108147709965 ], [ 24.782241643076535, 59.591072642107015 ], [ 24.78229706531808, 59.591060043794634 ], [ 24.782372960347814, 59.591041633417596 ], [ 24.782446589095045, 59.59101090299373 ], [ 24.782487081898701, 59.590988833167671 ], [ 24.782522462527524, 59.590954630711003 ], [ 24.782535633594136, 59.590928467729661 ], [ 24.782548912927872, 59.590878543893815 ], [ 24.782634023747026, 59.590826749916978 ], [ 24.782704508716932, 59.59079637006441 ], [ 24.782760954225548, 59.590782876846212 ], [ 24.782807088276432, 59.59078210117886 ], [ 24.782830695622291, 59.590789420766619 ], [ 24.782847497246507, 59.590815630815058 ], [ 24.782864530350388, 59.590852664885432 ], [ 24.782896452766582, 59.590883587750973 ], [ 24.782925394782595, 59.590903469471058 ], [ 24.782955408039587, 59.590915284144842 ], [ 24.783000208035954, 59.590920377818136 ], [ 24.783072729773409, 59.590915415516555 ], [ 24.78314740970146, 59.590903465828305 ], [ 24.783195748726126, 59.590883889617061 ], [ 24.783239858975815, 59.59085903344377 ], [ 24.783271585165302, 59.590835300808138 ], [ 24.783312584632693, 59.590807590618716 ], [ 24.783337786914171, 59.590760405668448 ], [ 24.783369955556768, 59.590686717379782 ], [ 24.783366163482626, 59.590668114051233 ], [ 24.783354924142998, 59.590655138071114 ], [ 24.783314696995063, 59.590637971270951 ], [ 24.783284373472789, 59.59062190375748 ], [ 24.783223216683897, 59.590604528735426 ], [ 24.783160642505528, 59.590594019956065 ], [ 24.783105742193506, 59.59057926481902 ], [ 24.783058872320098, 59.590561140898686 ], [ 24.783005786504411, 59.590538305302182 ], [ 24.782953323924982, 59.590524083226242 ], [ 24.78290724871302, 59.590518171268691 ], [ 24.782878371044355, 59.590514913236184 ], [ 24.78280604249073, 59.590492083725984 ], [ 24.782762052721584, 59.590471402370227 ], [ 24.782732454153741, 59.590436049388593 ], [ 24.78272362859628, 59.590413328783264 ], [ 24.782709161517054, 59.590387778124011 ], [ 24.782684465108126, 59.590355430108481 ], [ 24.782669569846931, 59.590343579784346 ], [ 24.782621089764305, 59.590314693698964 ], [ 24.782568245585161, 59.590288481416785 ], [ 24.782504900815773, 59.590267528496888 ], [ 24.782419105414171, 59.590243342233563 ], [ 24.782362361600288, 59.590230752002576 ], [ 24.782290224304553, 59.590224661847706 ], [ 24.782261785128259, 59.590227370316271 ], [ 24.782241687318624, 59.590235612700063 ], [ 24.782224082470254, 59.590251416320825 ], [ 24.782204363672886, 59.590267672276966 ], [ 24.78218123147148, 59.590276372431752 ], [ 24.782145083593726, 59.59027719655549 ], [ 24.78209347457944, 59.59026772652372 ], [ 24.782002774390467, 59.59023654953868 ], [ 24.781906808443154, 59.590204245677263 ], [ 24.78181239108164, 59.590157669283315 ], [ 24.781742709442977, 59.590116691464502 ], [ 24.781666677603646, 59.590078237563532 ], [ 24.781632106775028, 59.590056010114772 ], [ 24.781571903557793, 59.590034598351075 ], [ 24.781483112998341, 59.589998652300991 ], [ 24.781383210086801, 59.589955976787813 ], [ 24.781235109304539, 59.589891216966294 ], [ 24.781155585962402, 59.589861786604956 ], [ 24.781126212965301, 59.589822284893344 ], [ 24.781125363307911, 59.589762166775344 ], [ 24.781173102108706, 59.589651736906454 ], [ 24.781209012081906, 59.589613609019132 ], [ 24.781228559776789, 59.589603772259899 ], [ 24.781270455141431, 59.589597331188656 ], [ 24.781322392869651, 59.589592652973586 ], [ 24.781361984755655, 59.589586889723783 ], [ 24.781378809317335, 59.589568712107685 ], [ 24.781386338775871, 59.589548165800132 ], [ 24.781387293801515, 59.589539650642244 ], [ 24.781405063192231, 59.589525614483875 ], [ 24.78144053040474, 59.589513690916029 ], [ 24.781467338306012, 59.589500104025326 ], [ 24.781483244667943, 59.589478179732588 ], [ 24.781481038746097, 59.589455366075597 ], [ 24.781465882778928, 59.589431363268197 ], [ 24.781446867900968, 59.589404286437897 ], [ 24.781440722621159, 59.589383372094986 ], [ 24.781454443710995, 59.589368040330122 ], [ 24.781479492842031, 59.589354957521152 ], [ 24.781550093152362, 59.589321211464174 ], [ 24.781559354289339, 59.589320564310832 ], [ 24.781587031292663, 59.589318614499383 ], [ 24.781641070805946, 59.589323761744836 ], [ 24.781687738710033, 59.589332435328764 ], [ 24.781723670876737, 59.589342294377104 ], [ 24.781747827709225, 59.589348928710528 ], [ 24.781793310516427, 59.58935225041936 ], [ 24.78183410402017, 59.589343329247818 ], [ 24.78187589009913, 59.589322912552127 ], [ 24.781896869202743, 59.589297583138496 ], [ 24.781902718082524, 59.589277872549019 ], [ 24.781892910634514, 59.589262455405539 ], [ 24.781887155430613, 59.58923618890897 ], [ 24.781914355082666, 59.58920269918363 ], [ 24.781947029288627, 59.589183269857926 ], [ 24.781981193188795, 59.589178588730597 ], [ 24.782023434293414, 59.589182593768321 ], [ 24.782063977168807, 59.589195118351704 ], [ 24.782126686418092, 59.589207009188399 ], [ 24.782174652363175, 59.589212856372058 ], [ 24.782230846580894, 59.589210863464452 ], [ 24.782281113715477, 59.589207469379623 ], [ 24.782314568292779, 59.589213510046406 ], [ 24.782370359005654, 59.589230145259229 ], [ 24.782463691086956, 59.589248093330752 ], [ 24.78251786793972, 59.58925920863615 ], [ 24.782575865748516, 59.589262509897907 ], [ 24.782623377819299, 59.589257857350013 ], [ 24.782672694952307, 59.589249334301527 ], [ 24.782744324068883, 59.589251890588869 ], [ 24.782788212875563, 59.589258255372741 ], [ 24.782878353529139, 59.58928827719204 ], [ 24.782971819167408, 59.589317390617133 ], [ 24.783022948194056, 59.58932836218424 ], [ 24.78305408942963, 59.589333868805497 ], [ 24.783085370939002, 59.589331628053721 ], [ 24.783112851698721, 59.589325002438926 ], [ 24.783129814710225, 59.589317496577067 ], [ 24.783188372755468, 59.589312832287938 ], [ 24.783226336344093, 59.589304035155124 ], [ 24.78324138948042, 59.589291235601515 ], [ 24.783231262168183, 59.589271179707296 ], [ 24.783208122757898, 59.589257232921376 ], [ 24.783189573401234, 59.589241203385079 ], [ 24.78317246772944, 59.589224061215326 ], [ 24.783152054515909, 59.58921395808224 ], [ 24.783130420377653, 59.589205415072051 ], [ 24.783120653745382, 59.589196406843399 ], [ 24.783117640589936, 59.589184710627208 ], [ 24.783125287325976, 59.589169306928824 ], [ 24.783133503146811, 59.589156323458269 ], [ 24.783148080667988, 59.589150546186879 ], [ 24.783161249034755, 59.589123548336815 ], [ 24.783161767724323, 59.589091490942728 ], [ 24.783147423536583, 59.589065885732637 ], [ 24.78311225668185, 59.589040834659315 ], [ 24.783087078262533, 59.589019817632362 ], [ 24.783046780046735, 59.589004087377482 ], [ 24.783000891995751, 59.588992375670813 ], [ 24.782970812513668, 59.588982993973367 ], [ 24.782947425820868, 59.588962136863948 ], [ 24.782942888028423, 59.588942649273179 ], [ 24.782950491681486, 59.588916914130905 ], [ 24.783025677466156, 59.588830242856609 ], [ 24.783129821440326, 59.588765014996476 ], [ 24.783185021189563, 59.588728325232459 ], [ 24.783233196208432, 59.588707143185722 ], [ 24.783286354579612, 59.588691739270828 ], [ 24.783319907186431, 59.588678991728294 ], [ 24.783367706578769, 59.588640738952492 ], [ 24.783396475717357, 59.588597850408014 ], [ 24.783412257428463, 59.588524474507537 ], [ 24.783410067372767, 59.588497791990783 ], [ 24.783400253826333, 59.588477518814067 ], [ 24.783385367350157, 59.588461413750515 ], [ 24.7833784433573, 59.588439014030591 ], [ 24.783374481254892, 59.588389192153286 ], [ 24.783370235454846, 59.588295540725717 ], [ 24.783388451197716, 59.58811726243453 ], [ 24.783389743506071, 59.588077301305319 ], [ 24.783392879809853, 59.58803516596786 ], [ 24.783444741174936, 59.587986557377768 ], [ 24.783531673987174, 59.587926628370333 ], [ 24.783663575829742, 59.587833983585661 ], [ 24.783816987705926, 59.587748401210675 ], [ 24.78399430119449, 59.587669965735074 ], [ 24.784155270991342, 59.587589032757506 ], [ 24.784275392400701, 59.587523763061135 ], [ 24.784367990361492, 59.587492380411938 ], [ 24.784487237453284, 59.587469052956621 ], [ 24.784527198010206, 59.587464821496432 ], [ 24.784547041398902, 59.587460224627833 ], [ 24.784556961273459, 59.587454394030551 ], [ 24.784561193364684, 59.587443714064371 ], [ 24.784567936886926, 59.587409151229096 ], [ 24.784587402603332, 59.58734597728224 ], [ 24.784628333539203, 59.587314757285796 ], [ 24.784655284385131, 59.587282175025592 ], [ 24.784692508747174, 59.587262933423766 ], [ 24.784764872407131, 59.587237657715605 ], [ 24.784826554443935, 59.587208980496015 ], [ 24.78486728290985, 59.587192015983874 ], [ 24.78494647133995, 59.587139564327515 ], [ 24.784989847580935, 59.587096974684037 ], [ 24.785045933442436, 59.587021420432798 ], [ 24.785071501265659, 59.58696268935315 ], [ 24.785137958143981, 59.586899344100587 ], [ 24.785222496419514, 59.586854786581945 ], [ 24.785288590343054, 59.586835543964227 ], [ 24.78541392363897, 59.586731724935134 ], [ 24.785563227385975, 59.586628086561959 ], [ 24.785707978355909, 59.586537356048126 ], [ 24.785933183664895, 59.586416193409427 ], [ 24.78608305212293, 59.586323331602145 ], [ 24.786196019724485, 59.586239916602153 ], [ 24.786263581911268, 59.586193143406611 ], [ 24.786398345651925, 59.586147568420785 ], [ 24.786455817404185, 59.586123735715979 ], [ 24.786482706421008, 59.586104671732919 ], [ 24.786500768090306, 59.586081243893133 ], [ 24.786500313044225, 59.586060798576725 ], [ 24.786496283115184, 59.586041146479943 ], [ 24.786481717650148, 59.586020569636588 ], [ 24.786459999300689, 59.585999146669224 ], [ 24.786439814931132, 59.58598512002898 ], [ 24.786335397822715, 59.585946011719287 ], [ 24.786261978742221, 59.585920865717348 ], [ 24.786186213499345, 59.585889934927671 ], [ 24.786097934847739, 59.585844482995483 ], [ 24.78602937828796, 59.58579228928749 ], [ 24.785988856223483, 59.585749147615601 ], [ 24.785935582662724, 59.585671217733157 ], [ 24.785926515122267, 59.585605888204022 ], [ 24.785921146076337, 59.58558265352297 ], [ 24.785904610839665, 59.585553408231959 ], [ 24.785881296043108, 59.5855180185468 ], [ 24.785849100172513, 59.585475159480474 ], [ 24.785834294789002, 59.585425016037384 ], [ 24.785829092646026, 59.585390613834534 ], [ 24.785841523618615, 59.585282052334506 ], [ 24.785876128845114, 59.585163359467856 ], [ 24.785877374326677, 59.585130696261039 ], [ 24.785872687087721, 59.585109387428766 ], [ 24.785831338417683, 59.585061834269972 ], [ 24.785788463048906, 59.585020977517601 ], [ 24.785748526324049, 59.584966342613143 ], [ 24.785720381799816, 59.58492566761236 ], [ 24.785678438816966, 59.58485236490688 ], [ 24.785657904795919, 59.584795568113975 ], [ 24.785657398855964, 59.584754504507856 ], [ 24.785664554805663, 59.584695721035253 ], [ 24.785692059525019, 59.584611386796055 ], [ 24.785707176304552, 59.584546299537408 ], [ 24.785699201227743, 59.584500648522216 ], [ 24.785681041284523, 59.584460802847325 ], [ 24.785647788686209, 59.58441275271371 ], [ 24.785622626612113, 59.584388531498163 ], [ 24.785523787089147, 59.584335467167172 ], [ 24.785425370336853, 59.584301465934864 ], [ 24.785280105642858, 59.584269125148182 ], [ 24.785117095482672, 59.584227886306998 ], [ 24.785019893827158, 59.584173644960103 ], [ 24.784924515906148, 59.58410254418505 ], [ 24.784830456592477, 59.58399268478535 ], [ 24.784739909427223, 59.583906914848114 ], [ 24.784642488029117, 59.583806249050504 ], [ 24.784615930929618, 59.583785186605354 ], [ 24.784547727534942, 59.583761839888432 ], [ 24.784416769717069, 59.583720401079127 ], [ 24.784110657431942, 59.58365354751389 ], [ 24.783950583183504, 59.583621239225039 ], [ 24.783874903123529, 59.583611356066037 ], [ 24.783835388343451, 59.583602479307721 ], [ 24.783801387126243, 59.583600140521277 ], [ 24.78377064287584, 59.583602423177176 ], [ 24.78371874762837, 59.583607766417124 ], [ 24.783620916489621, 59.583622833870784 ], [ 24.78344186652464, 59.583679556722068 ], [ 24.783304330609781, 59.583727470061795 ], [ 24.783056282247568, 59.583749377980809 ], [ 24.78280727688351, 59.583729712837858 ], [ 24.782543039446402, 59.583678029101435 ], [ 24.782370747638275, 59.583636976464 ], [ 24.782128935484916, 59.583586011994903 ], [ 24.781981399107345, 59.583534651273951 ], [ 24.781836705826773, 59.5834729777579 ], [ 24.781663895719142, 59.583387718956068 ], [ 24.781490885686431, 59.583294490150465 ], [ 24.781325071730535, 59.58319550069978 ], [ 24.781157480298791, 59.583110012932302 ], [ 24.780945844015299, 59.583031005624072 ], [ 24.780735618553873, 59.582964071762191 ], [ 24.780538526554651, 59.582904851493232 ], [ 24.780339272026666, 59.582835428654469 ], [ 24.7798851934674, 59.582633384456763 ], [ 24.779727855116374, 59.58256947631039 ], [ 24.779566229547569, 59.582499956128558 ], [ 24.779320025884395, 59.582385002405857 ], [ 24.779122608788164, 59.582292785053831 ], [ 24.778906751458919, 59.582173540637896 ], [ 24.778692362697566, 59.581983078170566 ], [ 24.778607065292466, 59.581924713747391 ], [ 24.778484821573183, 59.581870552240396 ], [ 24.778320997088457, 59.581800585702389 ], [ 24.778128056885357, 59.581706518339637 ], [ 24.777880568475005, 59.581548070326207 ], [ 24.777630392620171, 59.581422930785592 ], [ 24.777360486330391, 59.581296291019093 ], [ 24.777137522697885, 59.581207075358179 ], [ 24.776969942359926, 59.581121178330939 ], [ 24.776891669896873, 59.581069512744889 ], [ 24.776835535167876, 59.581032456173595 ], [ 24.776682668299394, 59.580965376462522 ], [ 24.77652573818315, 59.580894711958777 ], [ 24.776375435890618, 59.580825956219591 ], [ 24.776262750574819, 59.580761449710877 ], [ 24.776175164105606, 59.580690090494556 ], [ 24.776105725509012, 59.580608966206896 ], [ 24.775990611923042, 59.580518639897946 ], [ 24.775840691435349, 59.580365620690628 ], [ 24.775749311059172, 59.580237230018206 ], [ 24.775706537032718, 59.580171298588006 ], [ 24.775628995713166, 59.580018007148389 ], [ 24.775607379267818, 59.579958665867899 ], [ 24.77555378019823, 59.579811445557439 ], [ 24.775499378772068, 59.57966238977977 ], [ 24.775440600787668, 59.579577522322332 ], [ 24.775350076990794, 59.579485004652369 ], [ 24.775231614563541, 59.579396573357151 ], [ 24.775084648141181, 59.57930449409475 ], [ 24.775030087972567, 59.579270308925025 ], [ 24.774796152743175, 59.579150732927729 ], [ 24.774520856239299, 59.579030151072942 ], [ 24.774330349861685, 59.57894155745926 ], [ 24.774200691812418, 59.578892094146966 ], [ 24.773991593179254, 59.578812297970742 ], [ 24.773723864028035, 59.57870361765282 ], [ 24.773554705750005, 59.578629897441324 ], [ 24.773388873180192, 59.578559981474505 ], [ 24.773228607243151, 59.578489691651754 ], [ 24.773069303828201, 59.578377620244702 ], [ 24.77297287816798, 59.578295188930788 ], [ 24.772871798612385, 59.578216644562147 ], [ 24.772830292205896, 59.578183567069416 ], [ 24.772758025454003, 59.578105904445913 ], [ 24.772727625018447, 59.578071711789782 ], [ 24.772681982699943, 59.578004557399311 ], [ 24.772644227247017, 59.577975326721429 ], [ 24.772610694084406, 59.577955909404118 ], [ 24.772560533228223, 59.577931338080695 ], [ 24.77249032904647, 59.57789848199954 ], [ 24.772389746439995, 59.577854977892954 ], [ 24.772258824818962, 59.577799434133226 ], [ 24.772137376936978, 59.577754283356747 ], [ 24.772093214651214, 59.577739021174317 ], [ 24.772071366498352, 59.577725280334953 ], [ 24.772026881511504, 59.577677076938699 ], [ 24.771973412009775, 59.577607257154192 ], [ 24.771946596418701, 59.577569560579555 ], [ 24.771926099334252, 59.577536584561244 ], [ 24.771917201848357, 59.577509447241823 ], [ 24.771902197195608, 59.577483000843287 ], [ 24.771863005476852, 59.577436427108928 ], [ 24.771798486675333, 59.577375630110538 ], [ 24.771719987341918, 59.577310112344477 ], [ 24.771659746484755, 59.577252216590232 ], [ 24.771593670439561, 59.577196850273879 ], [ 24.77151531740672, 59.577123701663247 ], [ 24.771478570478902, 59.57709749876355 ], [ 24.771437627523579, 59.577066518004202 ], [ 24.771377160682572, 59.577031396354108 ], [ 24.771332427365614, 59.576995572510107 ], [ 24.771269294071374, 59.576954443256589 ], [ 24.771212571727073, 59.576922118254195 ], [ 24.771051058262916, 59.576832937942882 ], [ 24.771001317287894, 59.576789288910426 ], [ 24.770933695534385, 59.57674918203999 ], [ 24.770866437357753, 59.576711837721291 ], [ 24.770789177353681, 59.576663385133372 ], [ 24.770750323535214, 59.576637086597948 ], [ 24.770694276039148, 59.576597791818919 ], [ 24.770605621972425, 59.576538463403516 ], [ 24.770563382999917, 59.576505667735098 ], [ 24.770527575570934, 59.576483381747977 ], [ 24.770495453944786, 59.57647069688629 ], [ 24.770457041066884, 59.576458380798243 ], [ 24.770434573828432, 59.576450056016128 ], [ 24.770329728969397, 59.576418073852459 ], [ 24.770224287073244, 59.576383159828325 ], [ 24.770068214843437, 59.57633948348829 ], [ 24.770002698390783, 59.576318815564186 ], [ 24.769948908249798, 59.576300233643153 ], [ 24.769874887911143, 59.576283807113604 ], [ 24.769776162159115, 59.57627195867147 ], [ 24.769642343214318, 59.576250584096897 ], [ 24.769551943116198, 59.57623520421658 ], [ 24.769450497515841, 59.576219664143018 ], [ 24.769392176359794, 59.576214321502263 ], [ 24.769323301279119, 59.576198475116563 ], [ 24.769226682443456, 59.576163176845128 ], [ 24.769165482736888, 59.576142429538997 ], [ 24.769101836880111, 59.576119928098585 ], [ 24.769057244435491, 59.576094874252334 ], [ 24.769010069929731, 59.576062052941516 ], [ 24.768976930018944, 59.576041851483012 ], [ 24.76893093686828, 59.576015100240205 ], [ 24.768888080856016, 59.575996400448062 ], [ 24.768843403890418, 59.575981687738626 ], [ 24.768792975270518, 59.575967726561196 ], [ 24.768736205360067, 59.575943299941294 ], [ 24.768681664568522, 59.575929030505378 ], [ 24.768623065575142, 59.575906661070789 ], [ 24.768578448902119, 59.575885206708278 ], [ 24.768510845245522, 59.575864496325522 ], [ 24.768415411643982, 59.575844773463636 ], [ 24.768334255548883, 59.575819447135778 ], [ 24.768231794323782, 59.575790977164289 ], [ 24.768172209080415, 59.575776629112468 ], [ 24.768093645343395, 59.575754815270045 ], [ 24.768023824351147, 59.57573383925498 ], [ 24.767973315404436, 59.57571628770755 ], [ 24.767871206364827, 59.575670464193713 ], [ 24.767812819912464, 59.575638650083924 ], [ 24.767729676648646, 59.575609125035008 ], [ 24.767607099417823, 59.575583105223082 ], [ 24.767500778277615, 59.575556039450717 ], [ 24.767371600062155, 59.575511925578581 ], [ 24.767218389192671, 59.575463750114949 ], [ 24.767151211536603, 59.575443808534331 ], [ 24.767069718838687, 59.575417594734624 ], [ 24.766918178401479, 59.575388537765171 ], [ 24.766842019707735, 59.57537521878325 ], [ 24.766757007923818, 59.575356152396445 ], [ 24.766669502046749, 59.575328483153591 ], [ 24.76655959185064, 59.575295198901209 ], [ 24.766522026022638, 59.575278729711037 ], [ 24.766467393819386, 59.575260312841237 ], [ 24.766372865130784, 59.575238213523775 ], [ 24.766253803459136, 59.575218904327308 ], [ 24.766164824236398, 59.575196494436199 ], [ 24.766089823015179, 59.575164605296308 ], [ 24.766014911925431, 59.575131997486821 ], [ 24.765946246305262, 59.575096202769451 ], [ 24.76588091277344, 59.575040990897499 ], [ 24.765696647987426, 59.574914800869585 ], [ 24.765601075223191, 59.57485567077147 ], [ 24.765545818476042, 59.574814161079011 ], [ 24.765445097747243, 59.574758992301327 ], [ 24.765359165795903, 59.574713073212763 ], [ 24.765287997116502, 59.574673702118311 ], [ 24.765190914726123, 59.574626851087892 ], [ 24.764920880602769, 59.574461705413128 ], [ 24.764859567082834, 59.574416747664031 ], [ 24.764814356571172, 59.574386785929548 ], [ 24.764769456697028, 59.574366705307796 ], [ 24.764723781515031, 59.574358576632775 ], [ 24.764677970896013, 59.574353868702019 ], [ 24.764639404195314, 59.574348679023821 ], [ 24.764608189774627, 59.574334443571765 ], [ 24.764576481338615, 59.574316333200279 ], [ 24.76453702086194, 59.57429971283338 ], [ 24.764499082627459, 59.574290201872252 ], [ 24.764433166584304, 59.574269533523221 ], [ 24.76437809382146, 59.574244772089287 ], [ 24.764257699931676, 59.57419430304418 ], [ 24.76419179425476, 59.574145718170861 ], [ 24.764155851452344, 59.574121833472184 ], [ 24.764082317862691, 59.574074737816716 ], [ 24.763878946163054, 59.573997011108084 ], [ 24.763707734925301, 59.57392363137037 ], [ 24.763590631688974, 59.573869103373248 ], [ 24.763466662328014, 59.573808214806654 ], [ 24.763396404937122, 59.573748550963565 ], [ 24.763350101768559, 59.573702123499075 ], [ 24.763348994354917, 59.573610167520755 ], [ 24.763336039587674, 59.57349015713443 ], [ 24.763328569392762, 59.573433218595063 ], [ 24.76331364893548, 59.573404625398254 ], [ 24.763289826073059, 59.573372771332508 ], [ 24.763267113591603, 59.573353343053242 ], [ 24.763223957473905, 59.573321762156638 ], [ 24.763194030115276, 59.573289108514189 ], [ 24.763141225465798, 59.573187065324618 ], [ 24.763136141144788, 59.573142608034111 ], [ 24.763138329360427, 59.573109212472922 ], [ 24.763132837803727, 59.573074209595759 ], [ 24.763122199369974, 59.573059020232975 ], [ 24.763101232618929, 59.573047202743439 ], [ 24.76307554036304, 59.573040062257391 ], [ 24.763000967691049, 59.573022207869997 ], [ 24.762845049346915, 59.572974814975801 ], [ 24.762706562453658, 59.572936225851841 ], [ 24.762659314694822, 59.572909138581778 ], [ 24.762608843034197, 59.572888990632642 ], [ 24.762404783376358, 59.572845635971376 ], [ 24.762330104764658, 59.572837233883455 ], [ 24.76222939794383, 59.572829583225612 ], [ 24.762127373354993, 59.572824713769883 ], [ 24.762015914033181, 59.572825194641226 ], [ 24.761840195865929, 59.572818765911769 ], [ 24.761757848723953, 59.572819518652047 ], [ 24.76160386679193, 59.572807928657916 ], [ 24.761539016594746, 59.572810959195628 ], [ 24.761370044140183, 59.57282449022307 ], [ 24.761227917047769, 59.572830299437229 ], [ 24.761030846433233, 59.572831945586088 ], [ 24.760908291088718, 59.572839455014389 ], [ 24.760692463233568, 59.572847348408224 ], [ 24.760492166963719, 59.572850340807022 ], [ 24.760272994375761, 59.572859796680277 ], [ 24.760096676807077, 59.572854248760365 ], [ 24.760014826554713, 59.572854279452514 ], [ 24.75993779875472, 59.572849201781082 ], [ 24.759849923309758, 59.572832966014161 ], [ 24.759770727979685, 59.572819561706041 ], [ 24.759672084724432, 59.572810793158773 ], [ 24.759528966350718, 59.572805493563202 ], [ 24.759383997178652, 59.572806057000449 ], [ 24.759225902374773, 59.572793276062079 ], [ 24.759099555373211, 59.572776767015945 ], [ 24.758938871336532, 59.572746254428409 ], [ 24.758825295431301, 59.572727185372578 ], [ 24.758725344141119, 59.572706547965446 ], [ 24.758599350763138, 59.572691589316925 ], [ 24.758459716248542, 59.572667920992743 ], [ 24.758305163163151, 59.572644113680155 ], [ 24.758209326767471, 59.572619134677566 ], [ 24.758079494646932, 59.572568444893555 ], [ 24.757898340444317, 59.572505329756837 ], [ 24.757784500016101, 59.572440948898247 ], [ 24.757703838422135, 59.572414060364359 ], [ 24.757626753475385, 59.572386487038848 ], [ 24.757548664627983, 59.572351962777951 ], [ 24.757439947518076, 59.572286385348555 ], [ 24.757375713438591, 59.572237572106815 ], [ 24.757365919168244, 59.572212485557948 ], [ 24.757365827181136, 59.57218927333691 ], [ 24.757372626665084, 59.572168840200597 ], [ 24.757395182952632, 59.572128972841654 ], [ 24.757464234124146, 59.572030151961364 ], [ 24.757485839203579, 59.572014328604538 ], [ 24.757501528162994, 59.57199185178586 ], [ 24.757504407345824, 59.571970175441308 ], [ 24.75750221294965, 59.5719463110159 ], [ 24.757492639984271, 59.571927192454851 ], [ 24.757461991592983, 59.571904765688238 ], [ 24.75741556393297, 59.571876908879148 ], [ 24.757382405183144, 59.571864110109416 ], [ 24.757361884595529, 59.571848366513649 ], [ 24.757360429863343, 59.571834676989788 ], [ 24.757365206090125, 59.571822711129677 ], [ 24.757377525523783, 59.5717974978973 ], [ 24.757396479507864, 59.571758755253271 ], [ 24.757453584540112, 59.571637562004007 ], [ 24.757478447349808, 59.571583696303399 ], [ 24.757697081298907, 59.571388285736582 ], [ 24.757882018917154, 59.571237060884229 ], [ 24.758059622171135, 59.571115661236014 ], [ 24.757947832718372, 59.57107723723589 ], [ 24.757870841266591, 59.571035462979964 ], [ 24.757844340962585, 59.571029259655539 ], [ 24.757822635374421, 59.571026233388068 ], [ 24.757810797055097, 59.57102773745715 ], [ 24.75777893669003, 59.571040639521762 ], [ 24.757663256863228, 59.571092871473731 ], [ 24.75761604548763, 59.571133876562307 ], [ 24.757494151968093, 59.571223332854167 ], [ 24.75740702679466, 59.571289053941541 ], [ 24.757347079193124, 59.571366773218479 ], [ 24.757303902298553, 59.571414989986529 ], [ 24.757262717225348, 59.571495474164614 ], [ 24.757233278047174, 59.571523388577056 ], [ 24.75716987650874, 59.57160256375429 ], [ 24.757138220141471, 59.571629440612355 ], [ 24.757101546035972, 59.571647890507258 ], [ 24.757062605909152, 59.57166071624119 ], [ 24.756999742991038, 59.571681416526019 ], [ 24.756882217978408, 59.571702546417661 ], [ 24.756542110325594, 59.571768081267855 ], [ 24.756383258512194, 59.571802166626469 ], [ 24.75625418787012, 59.571853944308067 ], [ 24.756118375496051, 59.571915266332546 ], [ 24.756071965956753, 59.571931177457316 ], [ 24.756042817324815, 59.57194356090875 ], [ 24.756012600448404, 59.57196440614878 ], [ 24.755993511617589, 59.571980430304876 ], [ 24.755971725910733, 59.571995320910546 ], [ 24.755964632358346, 59.572000163798165 ], [ 24.755859946420088, 59.572060122692911 ], [ 24.755821421519155, 59.572074956373726 ], [ 24.755701390563775, 59.572121170365037 ], [ 24.755592005272163, 59.572155986323295 ], [ 24.755579531114574, 59.572159953332232 ], [ 24.755489533942487, 59.57218265707219 ], [ 24.755473892065371, 59.572186597290589 ], [ 24.755400822581855, 59.572217857404617 ], [ 24.7553117930747, 59.572254675229381 ], [ 24.755185758103398, 59.572313436018206 ], [ 24.754928247676034, 59.572409095069823 ], [ 24.754640003943663, 59.572512011337828 ], [ 24.754325438650305, 59.572608111525298 ], [ 24.753946410781818, 59.572715251392808 ], [ 24.753666008497511, 59.572783409400515 ], [ 24.753525905142528, 59.572814332449425 ], [ 24.753319335761063, 59.572856863428477 ], [ 24.753084505300304, 59.57290961749549 ], [ 24.752921307146906, 59.572932386311727 ], [ 24.752754362456006, 59.572951370302832 ], [ 24.752612680313678, 59.572954178601918 ], [ 24.75253699188594, 59.572952015275064 ], [ 24.752510527025606, 59.572952237715477 ], [ 24.75247249253465, 59.572952552926409 ], [ 24.752435907184996, 59.572951989177845 ], [ 24.752398573488346, 59.572955190749013 ], [ 24.752342802179314, 59.572955956933079 ], [ 24.752286319995019, 59.572952580086231 ], [ 24.752221284738237, 59.572948084958121 ], [ 24.752096555374113, 59.57292457646993 ], [ 24.751982778377553, 59.572897055999022 ], [ 24.751922400188377, 59.572867095393754 ], [ 24.751891889181728, 59.572860366134179 ], [ 24.751855513191103, 59.572857260741223 ], [ 24.751814591164628, 59.572849405795445 ], [ 24.751792813232957, 59.57283832720438 ], [ 24.751766711318162, 59.572826142181761 ], [ 24.751732415244497, 59.572808258910079 ], [ 24.75168612151359, 59.572789510305995 ], [ 24.751649406298021, 59.57278305862431 ], [ 24.751605020225419, 59.572779379335984 ], [ 24.751561490735007, 59.572778432936033 ], [ 24.751467771456031, 59.572785814564405 ], [ 24.751410493481703, 59.572795260028634 ], [ 24.751306436469228, 59.572821667221362 ], [ 24.751277231640451, 59.572831617344605 ], [ 24.751205618015117, 59.572848109788218 ], [ 24.75109586236518, 59.572854433298119 ], [ 24.750963189137686, 59.572860563741585 ], [ 24.750840553440902, 59.572859716320551 ], [ 24.750727097060707, 59.572869929325975 ], [ 24.750602450200208, 59.572896623419226 ], [ 24.750494972996936, 59.572921227194747 ], [ 24.750367135846435, 59.572965344219348 ], [ 24.750227987818779, 59.573016490908614 ], [ 24.750105581943686, 59.573059804893873 ], [ 24.750058963162999, 59.573071235888882 ], [ 24.749847627417729, 59.573092846739478 ], [ 24.749779934261149, 59.573101022066922 ], [ 24.74968732525268, 59.573113880647377 ], [ 24.74963109242583, 59.573126640637838 ], [ 24.749497231598639, 59.573160046330337 ], [ 24.749466026988795, 59.573160375815469 ], [ 24.749359569672656, 59.573151823405382 ], [ 24.74933773242784, 59.573149272234332 ], [ 24.749295096817292, 59.573142332829057 ], [ 24.749243805419447, 59.57313121462505 ], [ 24.749205547003136, 59.57312136899435 ], [ 24.749172099270336, 59.573119467046617 ], [ 24.749138202738934, 59.573127719826424 ], [ 24.749119869418227, 59.573138855573077 ], [ 24.749116513794419, 59.573155211447911 ], [ 24.749123952550192, 59.573167601473109 ], [ 24.749223350920946, 59.573189855999331 ], [ 24.749269403982886, 59.573204944492204 ], [ 24.749291179426297, 59.573221561883706 ], [ 24.749282076386411, 59.573234036668396 ], [ 24.749259564881264, 59.573241847953057 ], [ 24.749168460339693, 59.573270828046944 ], [ 24.749035155320065, 59.573305414985605 ], [ 24.74886974152605, 59.573332230656682 ], [ 24.748725293352905, 59.573357374495224 ], [ 24.748650352429934, 59.573361066228685 ], [ 24.748582818275551, 59.573357202823999 ], [ 24.748496722553153, 59.57335089519831 ], [ 24.748357074370663, 59.573330420851413 ], [ 24.74831005932645, 59.57332870319231 ], [ 24.748261186564903, 59.573332624167627 ], [ 24.748202233719109, 59.573346449164326 ], [ 24.748171041789266, 59.573357020224464 ], [ 24.748092411401871, 59.573364323037161 ], [ 24.748032915649869, 59.573363115734431 ], [ 24.747958588285094, 59.573356085907392 ], [ 24.747881720408952, 59.573343756442306 ], [ 24.747824551083941, 59.573328443103044 ], [ 24.747780629680264, 59.573309688702125 ], [ 24.747717672601578, 59.573278986775719 ], [ 24.747604478132548, 59.573244044840543 ], [ 24.747548477800731, 59.573215080804061 ], [ 24.747372792756309, 59.573102963717567 ], [ 24.747330526965762, 59.573083032891198 ], [ 24.747292905110434, 59.57307716858741 ], [ 24.747224297925644, 59.573082421497112 ], [ 24.747180256051987, 59.573091188900612 ], [ 24.747147690751998, 59.573101264808763 ], [ 24.747111838127474, 59.573114734371508 ], [ 24.747087294996447, 59.573125931851401 ], [ 24.747066132465594, 59.573143627015028 ], [ 24.747063888891169, 59.573160712604817 ], [ 24.747123672192103, 59.573215327670553 ], [ 24.747137214440752, 59.573227710242193 ], [ 24.747227992706112, 59.573304662654138 ], [ 24.747245751701826, 59.573319714271605 ], [ 24.747272238586721, 59.573354320784702 ], [ 24.747275705792436, 59.573388123865236 ], [ 24.747272399293085, 59.573419649345837 ], [ 24.747224749351378, 59.573484907017757 ], [ 24.747153650661794, 59.573563680833544 ], [ 24.747073341421721, 59.573619464505605 ], [ 24.747038354811998, 59.57363452692902 ], [ 24.746996290666832, 59.573642340579731 ], [ 24.746926013352176, 59.573647333473758 ], [ 24.746894224948974, 59.57365010719883 ], [ 24.74687059161549, 59.573655213567463 ], [ 24.746860908871032, 59.573663894480362 ], [ 24.746858610038341, 59.573679274878629 ], [ 24.746857200883969, 59.573711005055962 ], [ 24.746851670685992, 59.573746842724205 ], [ 24.746842390416461, 59.573783536369412 ], [ 24.746844285780874, 59.573812653730819 ], [ 24.74684405374283, 59.573840777763976 ], [ 24.746819864972078, 59.573902814901551 ], [ 24.746787912449317, 59.573960201283178 ], [ 24.746713057561045, 59.574058995225911 ], [ 24.746620359223989, 59.574159882616385 ], [ 24.74658426152714, 59.574219500513507 ], [ 24.746506493931015, 59.574291238279145 ], [ 24.746480698728519, 59.574347036909735 ], [ 24.746453323039095, 59.574383222288219 ], [ 24.746403728600544, 59.574455958870537 ], [ 24.746345443637583, 59.574514876120801 ], [ 24.746301445638331, 59.574574143564597 ], [ 24.746209475751268, 59.574674478964518 ], [ 24.746099173334063, 59.574772377451815 ], [ 24.746056931573747, 59.574820585084112 ], [ 24.745978662694373, 59.574898626688189 ], [ 24.745759801557075, 59.575057879826794 ], [ 24.745674985829623, 59.57512468394723 ], [ 24.745567190964014, 59.575195028572161 ], [ 24.745537636121394, 59.575215822721781 ], [ 24.745384766312977, 59.575337613084962 ], [ 24.745227678811283, 59.575468053238303 ], [ 24.74511555311587, 59.575574246225862 ], [ 24.744907676038043, 59.575732053615049 ], [ 24.744673468168589, 59.57588265737499 ], [ 24.744561133357681, 59.575969731556938 ], [ 24.744399108408924, 59.576081016149175 ], [ 24.744321827910632, 59.576147112676637 ], [ 24.744184710882347, 59.576244388426552 ], [ 24.744104796727139, 59.576319556705066 ], [ 24.74404173881338, 59.576362028241505 ], [ 24.743994215666621, 59.576409277481638 ], [ 24.743947763202087, 59.576472318927195 ], [ 24.743928581851709, 59.576492578587008 ], [ 24.743884287029065, 59.576539387735224 ], [ 24.743810846404344, 59.576608002623509 ], [ 24.743738916247811, 59.576686635471987 ], [ 24.743708205184646, 59.576720217906015 ], [ 24.743612224177305, 59.576810906441686 ], [ 24.743480653185067, 59.576921686440642 ], [ 24.743413830426917, 59.576980641311678 ], [ 24.743341860794143, 59.577038915954077 ], [ 24.74323765598993, 59.577074823344688 ], [ 24.743221274272738, 59.577080705118149 ], [ 24.743131622484778, 59.577112895234301 ], [ 24.743093760002662, 59.577140745960484 ], [ 24.743055562831881, 59.577171910800715 ], [ 24.743036988683986, 59.577197328264816 ], [ 24.743029150872619, 59.577213601408779 ], [ 24.743020992161394, 59.577230576504569 ], [ 24.743004351122217, 59.577282122007148 ], [ 24.742983607698644, 59.577329758949382 ], [ 24.742895753014356, 59.577428084949069 ], [ 24.742821521855522, 59.577497143518116 ], [ 24.742762309103938, 59.577572445954047 ], [ 24.742703316232365, 59.577656211748028 ], [ 24.74265871471043, 59.577745865905833 ], [ 24.742646588523971, 59.577838901239971 ], [ 24.742619204288783, 59.577888792543163 ], [ 24.742591750185628, 59.57791116298884 ], [ 24.742481348531914, 59.57799573797714 ], [ 24.742390010175853, 59.57805296286778 ], [ 24.742340411949325, 59.578077504166757 ], [ 24.742230073729608, 59.57812123655598 ], [ 24.742162187527391, 59.578150305734908 ], [ 24.742113223695977, 59.578169870532975 ], [ 24.742084642386423, 59.578189267072524 ], [ 24.742053076355599, 59.578225977622537 ], [ 24.741923759857283, 59.578301044679577 ], [ 24.741773961687535, 59.578362026043749 ], [ 24.741741155913797, 59.578372739223788 ], [ 24.74171755757548, 59.578380447559219 ], [ 24.741611391070506, 59.578415116916972 ], [ 24.741449592455819, 59.578458993608926 ], [ 24.741416553513378, 59.578467948666237 ], [ 24.741383120811445, 59.578481573580881 ], [ 24.741355192054513, 59.57850171132786 ], [ 24.741293975386821, 59.578543749264774 ], [ 24.741150201786482, 59.578615745912884 ], [ 24.740943742144712, 59.578704627931629 ], [ 24.740734168005645, 59.578790313562294 ], [ 24.740595635957497, 59.578843053071004 ], [ 24.740508684951511, 59.578877694690895 ], [ 24.740442383744714, 59.578904106087506 ], [ 24.740361120026851, 59.578951264577476 ], [ 24.740278921189322, 59.578979856363929 ], [ 24.740164167036617, 59.579033171420427 ], [ 24.740043334490306, 59.57907310087981 ], [ 24.739814700971461, 59.579137842342625 ], [ 24.739692866618821, 59.579167544143274 ], [ 24.739607750244083, 59.579197112283161 ], [ 24.739422411955086, 59.57928511972402 ], [ 24.739130431016502, 59.579430982553866 ], [ 24.739043493070728, 59.57948102637117 ], [ 24.738940817030983, 59.579544685292682 ], [ 24.738753075538511, 59.579686814069042 ], [ 24.738577471034535, 59.579809297560985 ], [ 24.738535864407108, 59.579831288375296 ], [ 24.73850495352772, 59.579838911293756 ], [ 24.738450496326053, 59.579837967699817 ], [ 24.738425778614481, 59.579828429319711 ], [ 24.738408114272065, 59.579824237304337 ], [ 24.73838673499171, 59.579806251571398 ], [ 24.738366584854177, 59.579782927076003 ], [ 24.738347447172192, 59.579767926890277 ], [ 24.73831654491708, 59.579755559594382 ], [ 24.738290532953563, 59.579749170104776 ], [ 24.738269011485073, 59.579749092777703 ], [ 24.738236898692296, 59.579755968338304 ], [ 24.738222819766118, 59.579765310429359 ], [ 24.738220637946345, 59.579786282166779 ], [ 24.738229033860822, 59.579806198490495 ], [ 24.7382541047382, 59.579824675286289 ], [ 24.738279531744706, 59.579844882506528 ], [ 24.738288257537889, 59.579860425548056 ], [ 24.738292357880795, 59.579880563326604 ], [ 24.738296061263345, 59.579887301771471 ], [ 24.738301881100977, 59.57992224974759 ], [ 24.738242203816593, 59.580021653839033 ], [ 24.73821334472661, 59.580098561920444 ], [ 24.738178066401126, 59.580163468667372 ], [ 24.738146382935412, 59.580195044457682 ], [ 24.738128353580482, 59.580245816045945 ], [ 24.738118850830055, 59.58027847981657 ], [ 24.738121694263697, 59.5803271511462 ], [ 24.738139836326091, 59.580359876043971 ], [ 24.738166344900311, 59.580376082813316 ], [ 24.738176526289838, 59.580386115290793 ], [ 24.73819609393594, 59.580405421700284 ], [ 24.738209745464289, 59.580429383902313 ], [ 24.738213183809904, 59.580481812956805 ], [ 24.738203844790394, 59.580584822710343 ], [ 24.738197007276053, 59.580618791113139 ], [ 24.738188405480606, 59.580653828560962 ], [ 24.738177551180602, 59.580691903577943 ], [ 24.738161903086628, 59.580717034666769 ], [ 24.738155443540688, 59.580739062540069 ], [ 24.73816261093269, 59.580754551449594 ], [ 24.738187009580749, 59.580764854640833 ], [ 24.738223349651236, 59.580769238652074 ], [ 24.738285021238916, 59.580773086282633 ], [ 24.738319815399898, 59.580782891577854 ], [ 24.738397469820637, 59.580836557885824 ], [ 24.738492581233636, 59.580934559112407 ], [ 24.738553217475143, 59.581001569308611 ], [ 24.738771732813298, 59.581162118796691 ], [ 24.73899272218646, 59.581374545806078 ], [ 24.739134197160016, 59.5815283531429 ], [ 24.739227522642405, 59.581682438449988 ], [ 24.739302900650841, 59.581792442916587 ], [ 24.739454980336795, 59.581960103803418 ], [ 24.739623790976008, 59.582149133154019 ], [ 24.739747138053346, 59.58230244876507 ], [ 24.739840413007247, 59.582458867722259 ], [ 24.739947811395805, 59.582663248464151 ], [ 24.739966938807775, 59.582745202296763 ], [ 24.739996012479093, 59.582968700759004 ], [ 24.739997491381057, 59.583031804469648 ], [ 24.739995511036959, 59.58310837394982 ], [ 24.739994219920437, 59.583158046740031 ], [ 24.739976406189644, 59.583219490070363 ], [ 24.739936098753851, 59.583314379065563 ], [ 24.739906104796074, 59.583377308772555 ], [ 24.739917935356484, 59.58347367437721 ], [ 24.739918853823248, 59.583573673571145 ], [ 24.739919114498029, 59.583600304627467 ], [ 24.739963522227775, 59.583659721267793 ], [ 24.740062000222252, 59.583747559324166 ], [ 24.740208450437237, 59.583878807226114 ], [ 24.74036356529011, 59.584028335948766 ], [ 24.740517458282113, 59.584222492227845 ], [ 24.740644600522316, 59.584448780474112 ], [ 24.740688545023609, 59.584557532876985 ], [ 24.740722400314642, 59.58468170011389 ], [ 24.740740914298541, 59.58477517373089 ], [ 24.740706050666198, 59.58488838881955 ], [ 24.740634193521931, 59.585010526979971 ], [ 24.740537862049553, 59.585054044608597 ], [ 24.740332881875283, 59.585115180337468 ], [ 24.740105770207283, 59.585173612071728 ], [ 24.740056157630463, 59.585209435581383 ], [ 24.740047444293616, 59.585236870835416 ], [ 24.740055285024201, 59.585258818745444 ], [ 24.740094237942248, 59.585274048889538 ], [ 24.740159933394519, 59.585290798950702 ], [ 24.740212678196855, 59.585298052789639 ], [ 24.740255610647832, 59.585297686843241 ], [ 24.740295557662431, 59.585291897992938 ], [ 24.740338550191101, 59.585281280820013 ], [ 24.740386330264254, 59.58526455094399 ], [ 24.740421850685614, 59.585249747762937 ], [ 24.740476234447904, 59.585232280738829 ], [ 24.740522487698748, 59.585219410166552 ], [ 24.740628881842834, 59.58520636439421 ], [ 24.740736646124247, 59.585205033829745 ], [ 24.740922797934434, 59.585231871354154 ], [ 24.741084208292225, 59.585291341233422 ], [ 24.741206673515787, 59.585335940115385 ], [ 24.741379771633671, 59.585386691043809 ], [ 24.741633955590565, 59.585459400773033 ], [ 24.741874082247314, 59.585514864725205 ], [ 24.742181444260311, 59.58558940249025 ], [ 24.742424659215196, 59.585657334069062 ], [ 24.742560829658423, 59.585701217463892 ], [ 24.742801454949461, 59.585788282283787 ], [ 24.742884020974483, 59.585809074301217 ], [ 24.74290046589703, 59.585812374953647 ], [ 24.742997250472111, 59.585831776507121 ], [ 24.743219698701051, 59.585883971251747 ], [ 24.743530130793093, 59.585931048290071 ], [ 24.743736099195615, 59.585949952176058 ], [ 24.743988892175324, 59.585988582451563 ], [ 24.744230900320375, 59.586029373371055 ], [ 24.744440380494975, 59.586046936897773 ], [ 24.744568277549437, 59.586054717485709 ], [ 24.744719902905739, 59.586055659099024 ], [ 24.744846587692415, 59.586050107549603 ], [ 24.744993780791877, 59.586050723735305 ], [ 24.745080863730678, 59.586054604638107 ], [ 24.745143063283376, 59.586050565005785 ], [ 24.7453141669906, 59.586040552901544 ], [ 24.745443838100869, 59.586032353912401 ], [ 24.745569938153526, 59.586016284808395 ], [ 24.74569165662383, 59.585987673546271 ], [ 24.745808993357446, 59.585957013365942 ], [ 24.745880037691958, 59.585911569959286 ], [ 24.745987223784184, 59.585867637137468 ], [ 24.746044455219259, 59.585829982137085 ], [ 24.746103502903924, 59.585798474568691 ], [ 24.746161493589327, 59.585777690571476 ], [ 24.74625809588921, 59.585757236252341 ], [ 24.746433543824725, 59.585751686300142 ], [ 24.746748660931328, 59.585762878242932 ], [ 24.746917409603768, 59.585776287396953 ], [ 24.747090174761372, 59.585808963601224 ], [ 24.747204248082262, 59.585824018670273 ], [ 24.747363377605247, 59.585839465349011 ], [ 24.747575131893708, 59.585851750985057 ], [ 24.747743756922205, 59.585849837335047 ], [ 24.748118738540843, 59.585869897200617 ], [ 24.748349542667018, 59.585901660019402 ], [ 24.748876757442961, 59.58596632102212 ], [ 24.749270437256943, 59.586036404108803 ], [ 24.749474740836309, 59.586083197713215 ], [ 24.749729366275751, 59.586138762807188 ], [ 24.750052963236666, 59.586190731790097 ], [ 24.750342557593456, 59.586249086496821 ], [ 24.75059502851537, 59.586297535016314 ], [ 24.750849485424599, 59.586354930079523 ], [ 24.751367727986523, 59.586480841184958 ], [ 24.751706435935958, 59.586581156148725 ], [ 24.751992659502612, 59.586666105099475 ], [ 24.752194654138744, 59.586739414524736 ], [ 24.752308956985782, 59.586779004976798 ], [ 24.752381619292674, 59.586812620635499 ], [ 24.75245543329147, 59.586842280153064 ], [ 24.752532947317988, 59.586857556579638 ], [ 24.752594725093548, 59.586859224742426 ], [ 24.752664756949795, 59.586870737734735 ], [ 24.752794182950584, 59.586930572918817 ], [ 24.753024083567301, 59.587022536481157 ], [ 24.753353295713175, 59.587170771661967 ], [ 24.753646984172448, 59.587310205261964 ], [ 24.753738190118035, 59.587355365695124 ], [ 24.753994503045099, 59.587482310113572 ], [ 24.754138945321859, 59.587557353922371 ], [ 24.754297768009721, 59.587639864594962 ], [ 24.754483982619828, 59.587755413006057 ], [ 24.754635315152704, 59.587872838726419 ], [ 24.754706454086449, 59.587941298630327 ], [ 24.754776689022336, 59.588008892957845 ], [ 24.754811332835008, 59.588035300869862 ], [ 24.754879217890156, 59.588063197629538 ], [ 24.755197643424474, 59.588220690468539 ], [ 24.755367401306586, 59.588354553381109 ], [ 24.755485968652064, 59.588451834032774 ], [ 24.755524770177711, 59.58848367557043 ], [ 24.755585849873462, 59.588555773978769 ], [ 24.755589382120473, 59.58859206272259 ], [ 24.755573475250362, 59.588681628816701 ], [ 24.755537206935575, 59.588775790353935 ], [ 24.755533910295519, 59.588784372522809 ], [ 24.755447329444458, 59.588989965268404 ], [ 24.755449866348648, 59.589006152903103 ], [ 24.755475099685871, 59.589020056614906 ], [ 24.755508888379044, 59.589029414358961 ], [ 24.755546144172289, 59.589029892736697 ], [ 24.755583106372935, 59.589025920563152 ], [ 24.755623025661752, 59.588991708467702 ], [ 24.755720431047077, 59.588853474347687 ], [ 24.755800162066453, 59.58873165097301 ], [ 24.755872733036973, 59.588638942521875 ], [ 24.755914767870916, 59.588585257734074 ], [ 24.756019278969447, 59.588478516309692 ], [ 24.756114832038378, 59.588456166834476 ], [ 24.75627965723255, 59.588422111172548 ], [ 24.756469295185891, 59.58839978000502 ], [ 24.756618842752555, 59.588393556874443 ], [ 24.757180472771086, 59.588367615269618 ], [ 24.75744219306134, 59.588359671899561 ], [ 24.757931956800125, 59.588353760690069 ], [ 24.758137499779973, 59.588357304282994 ], [ 24.758378319475487, 59.588376596108361 ], [ 24.758792387679016, 59.588414982628954 ], [ 24.759048612623328, 59.58844000147753 ], [ 24.759316723272494, 59.588470902859264 ], [ 24.759613045634101, 59.588512655556059 ], [ 24.759792405082525, 59.58854455931035 ], [ 24.759810758706617, 59.588547586598338 ], [ 24.760192240609701, 59.588610481580773 ], [ 24.760478823607805, 59.588668607151256 ], [ 24.760735708045129, 59.588711095723653 ], [ 24.761103747076991, 59.588783005753818 ], [ 24.761388939878362, 59.588831083958773 ], [ 24.761667228680654, 59.58888543980428 ], [ 24.761906905501672, 59.588943536168117 ], [ 24.762291456152692, 59.58907899843792 ], [ 24.762559272938915, 59.589193544705203 ], [ 24.762787622659445, 59.589273678658571 ], [ 24.762980364746142, 59.589354969092845 ], [ 24.763110998707734, 59.589433475872205 ], [ 24.763182669484923, 59.589531038215142 ], [ 24.763201440005766, 59.589572328406391 ], [ 24.763216035626456, 59.589629521677317 ], [ 24.76323456119065, 59.589681979736774 ], [ 24.763248766075801, 59.589758043343181 ], [ 24.763203394654539, 59.589826677253278 ], [ 24.763164349331401, 59.589918478263598 ], [ 24.763136461529776, 59.589969480652393 ], [ 24.763137911235102, 59.589987541486948 ], [ 24.763143243058266, 59.590009780811563 ], [ 24.763171568867911, 59.59004062137295 ], [ 24.763226070605935, 59.590080646206573 ], [ 24.763268101051601, 59.590098194630265 ], [ 24.763320762403438, 59.59010274691574 ], [ 24.763382395998718, 59.590100066400332 ], [ 24.763534221609152, 59.590082269907647 ], [ 24.763601822335584, 59.590068612793829 ], [ 24.76365414190558, 59.590053212707446 ], [ 24.763709409903662, 59.590023271998696 ], [ 24.763793383993221, 59.58995673987971 ], [ 24.763860770900251, 59.589867369156792 ], [ 24.763868806978063, 59.589846049027727 ], [ 24.763863116756649, 59.589821271517565 ], [ 24.763856752603868, 59.589781462671148 ], [ 24.763849418482671, 59.589706346836095 ], [ 24.763866080111477, 59.589647393314657 ], [ 24.763903542779961, 59.589573499867996 ], [ 24.763935245059226, 59.589533973855303 ], [ 24.763995030875332, 59.589499518794675 ], [ 24.764124130210277, 59.589448918972934 ], [ 24.764237726398296, 59.58939536564052 ], [ 24.764269521189895, 59.58937810012543 ], [ 24.764292963480578, 59.589359392803765 ], [ 24.764319577340803, 59.58927070619896 ], [ 24.764389061740761, 59.589181053756946 ], [ 24.76443277063472, 59.589095302308074 ], [ 24.76449443551946, 59.58899407970938 ], [ 24.764595664267002, 59.588914126533957 ], [ 24.764692197159498, 59.588821382322678 ], [ 24.76484257007867, 59.588739268795543 ], [ 24.764956853242172, 59.588688080569469 ], [ 24.765045762143824, 59.588654677355152 ], [ 24.765484103302946, 59.58857473908219 ], [ 24.7657646000103, 59.588529607593948 ], [ 24.765901685384581, 59.588506911830834 ], [ 24.766080026659974, 59.588467406812065 ], [ 24.766302322751432, 59.588424047434039 ], [ 24.766390976324843, 59.588402969190405 ], [ 24.766501294938532, 59.58838181022594 ], [ 24.766578270078792, 59.588354668739321 ], [ 24.766701243811386, 59.588306749827929 ], [ 24.766808593978073, 59.588285069233365 ], [ 24.766900493383847, 59.58826744564147 ], [ 24.76699778056399, 59.588255607364289 ], [ 24.767078636460543, 59.588242805104166 ], [ 24.767130383384039, 59.588229677921468 ], [ 24.767165206056085, 59.588212564675501 ], [ 24.767195753081495, 59.588187182148729 ], [ 24.767228947097685, 59.588163005033316 ], [ 24.767243475475489, 59.588140040259567 ], [ 24.767251321642828, 59.588115121512836 ], [ 24.767258184576487, 59.588097622865114 ], [ 24.767273225572886, 59.588084044289083 ], [ 24.767303711967923, 59.58807010682721 ], [ 24.767320940342714, 59.588060115039895 ], [ 24.767331017077698, 59.588045407316031 ], [ 24.76733331410335, 59.588030313861609 ], [ 24.76731779651238, 59.588003932867352 ], [ 24.767316249097682, 59.587972219714437 ], [ 24.767335068197585, 59.587925245449981 ], [ 24.767376154587936, 59.587892342456612 ], [ 24.767445229304794, 59.587860327288013 ], [ 24.767496559782092, 59.587838190198461 ], [ 24.767520863713742, 59.587824845084718 ], [ 24.767519787462103, 59.587805390779593 ], [ 24.767507908577034, 59.587786510804058 ], [ 24.767487524926036, 59.587768057178295 ], [ 24.767441891486282, 59.587748484396684 ], [ 24.767383988598922, 59.587730374103444 ], [ 24.76733961706298, 59.587706314807811 ], [ 24.767295477213263, 59.587673690793089 ], [ 24.767273878823513, 59.587647677186531 ], [ 24.767253148871966, 59.587618570702425 ], [ 24.767236789070971, 59.587577599167339 ], [ 24.767233009533172, 59.587545019281066 ], [ 24.767227681017559, 59.587508902752589 ], [ 24.76721864183942, 59.587485634835531 ], [ 24.767195975932225, 59.58744917005324 ], [ 24.767202015260224, 59.58741321199328 ], [ 24.767220659865984, 59.587382835867643 ], [ 24.767251114095153, 59.587348666084338 ], [ 24.76729050934518, 59.58732870772058 ], [ 24.767319456892505, 59.587297589615787 ], [ 24.76731939076814, 59.587271118978286 ], [ 24.767302154585327, 59.587247790900989 ], [ 24.767283542756335, 59.58722860886197 ], [ 24.767253037503789, 59.587213427416835 ], [ 24.76723340630533, 59.58719613630052 ], [ 24.767217666189897, 59.587155161162322 ], [ 24.767219145239636, 59.587135979269078 ], [ 24.767233901790693, 59.587109808631396 ], [ 24.767275754636717, 59.58707325684049 ], [ 24.767339361976052, 59.587031211021788 ], [ 24.76741163432547, 59.586991933447017 ], [ 24.767521028207661, 59.58692061057021 ], [ 24.767673268334253, 59.586841902793637 ], [ 24.767872278362777, 59.586764188979615 ], [ 24.768136434964084, 59.586680567156314 ], [ 24.768212642316094, 59.586658060884872 ], [ 24.768544514594726, 59.586560068862184 ], [ 24.768737601156555, 59.586513338332026 ], [ 24.768898668874808, 59.586496469220293 ], [ 24.768926732719667, 59.586496826700248 ], [ 24.768955026848079, 59.586497191811709 ], [ 24.769001548667188, 59.586495808284326 ], [ 24.769051135036054, 59.586503921774714 ], [ 24.769111080443285, 59.586509865586272 ], [ 24.769187703283848, 59.586504994712243 ], [ 24.769250635276958, 59.586498982645018 ], [ 24.769308157189009, 59.586476817788032 ], [ 24.769365759948016, 59.586425362844111 ], [ 24.769427251395943, 59.586313421048324 ], [ 24.769466167850965, 59.586277101044587 ], [ 24.769554585786867, 59.586198214758404 ], [ 24.769596671169865, 59.586167405662657 ], [ 24.769653134265567, 59.586139941828122 ], [ 24.769735054730656, 59.586122823088985 ], [ 24.7698561203736, 59.586113151177372 ], [ 24.769980144131839, 59.586098884031713 ], [ 24.770273837280303, 59.586043603546699 ], [ 24.770429930591945, 59.586029966050795 ], [ 24.770530513277549, 59.58602797983167 ], [ 24.770577540310569, 59.586027050592676 ], [ 24.770622120700445, 59.586027508962779 ], [ 24.770661396238204, 59.586035233109271 ], [ 24.770692539936885, 59.586046541281959 ], [ 24.770724042326613, 59.586054149129751 ], [ 24.770771109844105, 59.586058129611565 ], [ 24.770840101671393, 59.586050097708984 ], [ 24.770948050929807, 59.586041219338028 ], [ 24.770977372007223, 59.5860220905234 ], [ 24.771061265341906, 59.585990534572879 ], [ 24.771115951286024, 59.585972972325344 ], [ 24.771199005154468, 59.585959831532762 ], [ 24.771291512723746, 59.585956132465547 ], [ 24.771369049868998, 59.585954181253797 ], [ 24.771455123660253, 59.58593996348602 ], [ 24.771514201098611, 59.585921711212826 ], [ 24.771545021113937, 59.5859085966108 ], [ 24.771574954865375, 59.585889185821287 ], [ 24.77158679252134, 59.585868722651675 ], [ 24.771583593409883, 59.58583381459907 ], [ 24.771560628363297, 59.585798366582587 ], [ 24.771530398108855, 59.585782071447547 ], [ 24.771517108055239, 59.585756449873578 ], [ 24.771510927197273, 59.585733668216477 ], [ 24.771507981408355, 59.585696658232671 ], [ 24.771521880293403, 59.58566016037836 ], [ 24.771526441456995, 59.585620117501669 ], [ 24.771509099671736, 59.585596126314861 ], [ 24.771497014296479, 59.585577571021624 ], [ 24.771490207936914, 59.585527208891357 ], [ 24.771488332350213, 59.585499922953062 ], [ 24.771470440046908, 59.585466482944796 ], [ 24.771396366366162, 59.58539242080434 ], [ 24.771346502710959, 59.585286953035023 ], [ 24.771341210645375, 59.585212031922829 ], [ 24.771357282080118, 59.585158942209858 ], [ 24.771392132376242, 59.585104252907328 ], [ 24.771420374973161, 59.585046981058106 ], [ 24.771483513717882, 59.584994810640453 ], [ 24.771570967945962, 59.584926071740192 ], [ 24.771648678125025, 59.584885817526938 ], [ 24.771672596064654, 59.584871226159606 ], [ 24.771702903551198, 59.58485272872727 ], [ 24.771744410201727, 59.584828555761604 ], [ 24.771793477326423, 59.584804500237382 ], [ 24.771829875917078, 59.584778310457708 ], [ 24.771848755439308, 59.584757752299588 ], [ 24.771884611788668, 59.58473029103542 ], [ 24.771944679025001, 59.584699600611991 ], [ 24.771982163575338, 59.58468461583287 ], [ 24.772031882158167, 59.58465652606079 ], [ 24.772136311995958, 59.584594177529752 ], [ 24.772268835948715, 59.584518792874739 ], [ 24.77236234232571, 59.584475070904432 ], [ 24.772446391569385, 59.584441251088954 ], [ 24.772531520821833, 59.584403556129367 ], [ 24.772598668319588, 59.584365140337361 ], [ 24.772634218526704, 59.584333596435712 ], [ 24.772675597053055, 59.584295268303649 ], [ 24.772692963665403, 59.584268139233977 ], [ 24.772720870002011, 59.584242941320056 ], [ 24.77279326149683, 59.584189962170434 ], [ 24.772861524383458, 59.584150812640331 ], [ 24.772955032319913, 59.584111910493014 ], [ 24.773028629976004, 59.584084775878551 ], [ 24.773091219232501, 59.584063908187993 ], [ 24.77316843030993, 59.584048583125032 ], [ 24.7732008481339, 59.58404279240272 ], [ 24.773277935339877, 59.584029029874138 ], [ 24.77344332298934, 59.584008934310411 ], [ 24.77357033422501, 59.583996459472019 ], [ 24.77367921951036, 59.583987806730683 ], [ 24.773862827365388, 59.583964166133356 ], [ 24.773951606891544, 59.583948557488384 ], [ 24.774033088255273, 59.583935801075746 ], [ 24.774131471920022, 59.583931571899043 ], [ 24.774256762534627, 59.583927508280311 ], [ 24.774329950815815, 59.583922106936157 ], [ 24.774348786929416, 59.583920722020707 ], [ 24.774379098000676, 59.583920365023943 ], [ 24.774507119047723, 59.583918843424613 ], [ 24.774709735033468, 59.583922396220011 ], [ 24.774875378733856, 59.583931470315932 ], [ 24.77525864289985, 59.583962517353221 ], [ 24.775481540990217, 59.583984620773421 ], [ 24.775583405620552, 59.583999543474199 ], [ 24.775780390277077, 59.584029329464229 ], [ 24.776403239579263, 59.584144215540093 ], [ 24.776553976120837, 59.584194908376318 ], [ 24.776715458736145, 59.584229407528937 ], [ 24.776791249380263, 59.584244168257371 ], [ 24.776852090809797, 59.584256907321532 ], [ 24.776998000075555, 59.58430000713409 ], [ 24.777049893495626, 59.584319288538431 ], [ 24.777069908623439, 59.584337742901809 ], [ 24.77709433874535, 59.584369707577217 ], [ 24.777112307931763, 59.584424115046865 ], [ 24.777139281532822, 59.584532542813577 ], [ 24.777202054511552, 59.584690991558055 ], [ 24.777234471776428, 59.584818417052958 ], [ 24.77724834571092, 59.584846179989306 ], [ 24.777264845159188, 59.584883496224982 ], [ 24.777281225958728, 59.584914843919421 ], [ 24.777374259336838, 59.585031114642611 ], [ 24.777432787137098, 59.585107508971547 ], [ 24.77746517821911, 59.585142353050294 ], [ 24.777483934135802, 59.585150599727569 ], [ 24.777509862083075, 59.585154351999194 ], [ 24.777546718705207, 59.585150697409553 ], [ 24.777576733991008, 59.585145871230615 ], [ 24.777630383905837, 59.585142037100979 ], [ 24.777660382292868, 59.58514344055078 ], [ 24.777701311685298, 59.585151476035648 ], [ 24.777765571056598, 59.585165756463866 ], [ 24.777796101488612, 59.585169526496159 ], [ 24.777826362137183, 59.585170012784118 ], [ 24.777859154921106, 59.585165125325226 ], [ 24.77789754790188, 59.585148158732906 ], [ 24.777916194870858, 59.585135383507371 ], [ 24.777953116126177, 59.585126818390457 ], [ 24.778170333858657, 59.585167136708925 ], [ 24.77843003312703, 59.585216764409203 ], [ 24.778640315754267, 59.585266458124799 ], [ 24.778785937762674, 59.585315275576292 ], [ 24.779005267636919, 59.585416035448375 ], [ 24.779291993975523, 59.585559726458143 ], [ 24.779546900910535, 59.585681172632526 ], [ 24.779646862904322, 59.585740975947957 ], [ 24.779751448895581, 59.585815616673102 ], [ 24.77979349334521, 59.585871461666599 ], [ 24.779824669649631, 59.585920351261763 ], [ 24.779878212973419, 59.586000606786136 ], [ 24.779910621199345, 59.58603450766023 ], [ 24.779945010247665, 59.58606293028236 ], [ 24.779966565248557, 59.58608203937407 ], [ 24.780006460528778, 59.586172267201832 ], [ 24.780045167036896, 59.586229173046419 ], [ 24.780060882998253, 59.586249914338481 ], [ 24.780061917424064, 59.586277267981778 ], [ 24.780071552115494, 59.586330763728249 ], [ 24.780072049445256, 59.58636248301687 ], [ 24.780084595578359, 59.586419435415308 ], [ 24.780100719168622, 59.586448683823541 ], [ 24.780132143839758, 59.586467456411057 ], [ 24.780229667768914, 59.586521914818213 ], [ 24.780253357630066, 59.586551504555416 ], [ 24.780272638644334, 59.586590294068451 ], [ 24.780291009691126, 59.586631081683507 ], [ 24.780314439635131, 59.586667853976678 ], [ 24.78032384350194, 59.586702007149803 ], [ 24.780326198489202, 59.586759908295853 ], [ 24.780307360346633, 59.586809873758867 ], [ 24.780297990452048, 59.58685513364248 ], [ 24.780300166168857, 59.586895190974339 ], [ 24.78031485238267, 59.586917320656724 ], [ 24.780338724488381, 59.58692706271939 ], [ 24.780360863550673, 59.586934561953186 ], [ 24.78037116985238, 59.586944007040664 ], [ 24.780399063563006, 59.586973185965974 ], [ 24.780425249875023, 59.58699275237484 ], [ 24.780447096844693, 59.587019148411656 ], [ 24.780458053083727, 59.587066945350976 ], [ 24.780484544813557, 59.587131445260034 ], [ 24.780484029746614, 59.587159912118551 ], [ 24.780485829870486, 59.587197476250829 ], [ 24.780495873156205, 59.587237110146397 ], [ 24.780507862232735, 59.587284398311049 ], [ 24.780501756995601, 59.587336155723094 ], [ 24.780520542909677, 59.587371851292296 ], [ 24.780547693011464, 59.587428771308396 ], [ 24.780537282982053, 59.587511235080804 ], [ 24.780519049362802, 59.587572121134045 ], [ 24.78049994982921, 59.587606074448068 ], [ 24.780464346994734, 59.587643643841986 ], [ 24.780382948013742, 59.587683422301438 ], [ 24.780299327859979, 59.587727468572908 ], [ 24.780270233238898, 59.587747046940365 ], [ 24.780240224910216, 59.587776953422349 ], [ 24.780235590662027, 59.587804116077869 ], [ 24.780246717237873, 59.587826931003569 ], [ 24.780280138877529, 59.587859389588523 ], [ 24.780297957249246, 59.587884543750732 ], [ 24.780299290372017, 59.587915647700349 ], [ 24.780297818436221, 59.587971084960323 ], [ 24.78027215728806, 59.588035398839395 ], [ 24.780248914915319, 59.588104069895188 ], [ 24.780219356309189, 59.588153595876022 ], [ 24.780198745895216, 59.588181131024875 ], [ 24.780186650030984, 59.588204810003802 ], [ 24.780181533969717, 59.588214830786434 ], [ 24.780172601594312, 59.588247548177542 ], [ 24.78016829530781, 59.588278209641672 ], [ 24.78016583192381, 59.588295835644466 ], [ 24.780154708885096, 59.588357299044986 ], [ 24.780148178720783, 59.588372040715917 ], [ 24.780139208739637, 59.588392317191868 ], [ 24.780133043336882, 59.588421418767425 ], [ 24.780139151252719, 59.588449236158013 ], [ 24.780156413764789, 59.588478702229381 ], [ 24.780189805658235, 59.588532219371942 ], [ 24.780228122523347, 59.588618309322847 ], [ 24.78023915184782, 59.588668511461371 ], [ 24.780245259277134, 59.588696301918816 ], [ 24.780249617566408, 59.588726553218997 ], [ 24.780242895599336, 59.588768431331388 ], [ 24.780206260671822, 59.588844416235631 ], [ 24.780159836566533, 59.588943563816635 ], [ 24.780114327884668, 59.589050219125632 ], [ 24.780077268904304, 59.589149419149301 ], [ 24.78003757076392, 59.589243159187824 ], [ 24.780031399736572, 59.589332231319567 ], [ 24.780011314315381, 59.589452165166414 ], [ 24.779994333959891, 59.589505261258125 ], [ 24.779976171951969, 59.589543113612294 ], [ 24.779941161518689, 59.589571119586175 ], [ 24.779890225494686, 59.589610224364769 ], [ 24.779866432860629, 59.589636449709637 ], [ 24.779860761173925, 59.589659309828384 ], [ 24.779865936566477, 59.589674251752115 ], [ 24.779883489877442, 59.589697854620177 ], [ 24.779909328102985, 59.589719918576598 ], [ 24.779952026150408, 59.589753094493695 ], [ 24.77996850027187, 59.589783669306684 ], [ 24.779975709651449, 59.589837897385898 ], [ 24.779973650107582, 59.58988549278844 ], [ 24.779962774821591, 59.589939270969005 ], [ 24.779935130566187, 59.590003659241702 ], [ 24.779929678880126, 59.590055412704423 ], [ 24.779902059378486, 59.590112395390214 ], [ 24.779903191974842, 59.590133222655211 ], [ 24.779906570631798, 59.590194869768055 ], [ 24.779893029197947, 59.590228116952829 ], [ 24.779869796376726, 59.590251753820141 ], [ 24.779852158543132, 59.590256130511229 ], [ 24.779813856329813, 59.590256419145383 ], [ 24.779793781846962, 59.59024949110642 ], [ 24.779771741655278, 59.590230950371925 ], [ 24.779743489681604, 59.590213263090938 ], [ 24.779715480981093, 59.590200780611241 ], [ 24.779695038909974, 59.590200981888842 ], [ 24.779678845074553, 59.590208886699209 ], [ 24.779678917841359, 59.590221327391433 ], [ 24.779698364043902, 59.590240233505476 ], [ 24.779733226499264, 59.590257468921983 ], [ 24.779774327781386, 59.590271166804712 ], [ 24.77980860290366, 59.590287490083398 ], [ 24.779824807782809, 59.590304000667906 ], [ 24.779831320642515, 59.59032785712467 ], [ 24.779838836406466, 59.590373834192633 ], [ 24.779827000840605, 59.590417403005901 ], [ 24.779804819363882, 59.590444435670022 ], [ 24.779786805313876, 59.590466381143393 ], [ 24.779753402048211, 59.590495041820205 ], [ 24.779699228924112, 59.590535099114057 ], [ 24.779613786414725, 59.590573356917432 ], [ 24.779522049618389, 59.590602702407764 ], [ 24.77943830056698, 59.590632216198856 ], [ 24.779388643097683, 59.590639142247859 ], [ 24.779346453642123, 59.59064440855105 ], [ 24.779329613133985, 59.590646509474958 ], [ 24.779274736587357, 59.590655126839174 ], [ 24.77920651035614, 59.590678355406141 ], [ 24.779163928339447, 59.590691989851408 ], [ 24.779120268753868, 59.590699544714774 ], [ 24.779072587130248, 59.590701540008013 ], [ 24.779012658121815, 59.590710626855753 ], [ 24.778948013431801, 59.590729471726071 ], [ 24.77887306619203, 59.590770764053325 ], [ 24.778788094863927, 59.59081111000809 ], [ 24.778706811772885, 59.590865643636718 ], [ 24.778675494726539, 59.590903519357191 ], [ 24.778643904583465, 59.590933335983209 ], [ 24.778626584697783, 59.590957808511099 ], [ 24.778621376019089, 59.590987792992948 ], [ 24.778626618659217, 59.591038163908372 ], [ 24.778619033093875, 59.59107413160072 ], [ 24.778612223793818, 59.59111230288606 ], [ 24.778612937589582, 59.591175958409671 ], [ 24.778608008541482, 59.591222798681329 ], [ 24.778620427563482, 59.591265721928607 ], [ 24.778648333446338, 59.591304622410888 ], [ 24.778657214697457, 59.59133452395421 ], [ 24.778676507773312, 59.591360657009766 ], [ 24.77869436847266, 59.59138299255477 ], [ 24.778723713141488, 59.591401993132919 ], [ 24.778748097568815, 59.591411570890649 ], [ 24.778785952926128, 59.591444281978376 ], [ 24.778820714342771, 59.591465575500415 ], [ 24.778845330313075, 59.591471345938054 ], [ 24.778890101238069, 59.591475166543205 ], [ 24.778942129503804, 59.59147507562713 ], [ 24.778986418675114, 59.591476412609232 ], [ 24.779035920938263, 59.591480429710856 ], [ 24.779080115132629, 59.591477620183298 ], [ 24.779131303748905, 59.591472561283069 ], [ 24.779199659845215, 59.591468837447309 ], [ 24.779248338664253, 59.591460095060654 ], [ 24.779366460942732, 59.591455683020641 ], [ 24.779419828136753, 59.591451445966584 ], [ 24.779516159077222, 59.591438607795091 ], [ 24.779605978709654, 59.591419748941924 ], [ 24.77969015084728, 59.591394711732157 ], [ 24.77978608795777, 59.591388643793607 ], [ 24.779867703289774, 59.591384904285498 ], [ 24.779930908761543, 59.591382009322906 ], [ 24.780003522499161, 59.591379453812273 ], [ 24.780035376232338, 59.591378915914063 ], [ 24.780068518420993, 59.591375892958744 ], [ 24.78008115010465, 59.591370890497139 ], [ 24.780088281290304, 59.591361486191254 ], [ 24.780086672093201, 59.591349216137758 ], [ 24.780076067315125, 59.591337618493235 ], [ 24.780022854077242, 59.591320769638365 ], [ 24.779920556392145, 59.591304991094574 ], [ 24.779773504314292, 59.591277609654746 ], [ 24.779694529061064, 59.591262905255832 ], [ 24.779685679119808, 59.591248308138304 ], [ 24.779691805953224, 59.591240651170509 ], [ 24.779704572886033, 59.591233053804274 ], [ 24.779735011327528, 59.59121712103321 ], [ 24.779768282065547, 59.591205812293452 ] ] ], [ [ [ 24.754112572736293, 59.448492781942164 ], [ 24.754211921217891, 59.447794592504614 ], [ 24.754359307787279, 59.44778942172087 ], [ 24.754522101804763, 59.447793022590304 ], [ 24.754647353872329, 59.447803346502887 ], [ 24.754755570566253, 59.447832547313581 ], [ 24.754875861241402, 59.447870431195319 ], [ 24.754986249371417, 59.447898955092967 ], [ 24.755115520505729, 59.447937945288501 ], [ 24.755273668204705, 59.447983313856021 ], [ 24.755449017967575, 59.448038054074821 ], [ 24.75574036496339, 59.448138897582744 ], [ 24.755914536786172, 59.448210439485166 ], [ 24.756066040619405, 59.448282883272086 ], [ 24.756109052247208, 59.448310599032887 ], [ 24.756126758767831, 59.448326835119296 ], [ 24.756139589054424, 59.44833861080869 ], [ 24.756169718263344, 59.448371301815087 ], [ 24.756186772525034, 59.44840186856468 ], [ 24.756195817954207, 59.448431404075883 ], [ 24.756198059699742, 59.448438698298411 ], [ 24.756210125962181, 59.448513899254145 ], [ 24.756209348943955, 59.448560816319215 ], [ 24.756200776353545, 59.448612356288713 ], [ 24.756178282677759, 59.448735835217022 ], [ 24.756167112936044, 59.448871529351919 ], [ 24.756175099173365, 59.448932049771194 ], [ 24.756186200562034, 59.448961277242098 ], [ 24.756191358445985, 59.448977657167191 ], [ 24.756184785127285, 59.448994490436817 ], [ 24.756168580934478, 59.44901347955529 ], [ 24.756144366143797, 59.449077551103848 ], [ 24.756131778710564, 59.449105424410405 ], [ 24.756119282077048, 59.449123862610385 ], [ 24.756122414963009, 59.449137166152191 ], [ 24.756130412880356, 59.449148843903473 ], [ 24.756156027997154, 59.449154388139455 ], [ 24.756190291967588, 59.449154029868616 ], [ 24.7562451276258, 59.449145618025916 ], [ 24.756345264933664, 59.449144881522628 ], [ 24.756444166067091, 59.449148030006135 ], [ 24.75649560615339, 59.449168587637892 ], [ 24.7565545870217, 59.449184155741314 ], [ 24.756658397774423, 59.449200417857277 ], [ 24.756741198445276, 59.449206531166304 ], [ 24.756863787689166, 59.449235647697947 ], [ 24.757069331232323, 59.449285741867676 ], [ 24.757323529084477, 59.449346776971495 ], [ 24.757589394134214, 59.449408399793924 ], [ 24.757818896014456, 59.449468705229528 ], [ 24.757980405515216, 59.449514930965073 ], [ 24.758193189155303, 59.449567791463792 ], [ 24.758306719058865, 59.449587371614676 ], [ 24.758365298062166, 59.449590328864723 ], [ 24.758463083444767, 59.449592387097788 ], [ 24.758617560830192, 59.449573561982675 ], [ 24.758826084846941, 59.449560807846368 ], [ 24.75898132573122, 59.449535101697379 ], [ 24.759082521095149, 59.449508943782931 ], [ 24.75920568630228, 59.449478969717767 ], [ 24.759354939224753, 59.449433746224372 ], [ 24.759496237284051, 59.449382024356169 ], [ 24.759628719346768, 59.44932313584593 ], [ 24.759717627832792, 59.449292317528403 ], [ 24.759786140939035, 59.449266356524419 ], [ 24.759828910044472, 59.449239394954496 ], [ 24.759858178468544, 59.449213390971472 ], [ 24.759897817912474, 59.449155118489045 ], [ 24.759969544844246, 59.449076328481809 ], [ 24.75998614840428, 59.44904765064026 ], [ 24.760002581766564, 59.449011325565905 ], [ 24.76002090175664, 59.448975806490132 ], [ 24.760027565532351, 59.44892747299204 ], [ 24.760041357666932, 59.44886356854196 ], [ 24.760041072502649, 59.448799260567021 ], [ 24.760035381398044, 59.448760585546502 ], [ 24.760035689758823, 59.448734039441064 ], [ 24.760040188179531, 59.448691391753535 ], [ 24.760056494278892, 59.448658864576622 ], [ 24.760065536498807, 59.448618389979259 ], [ 24.76010201743544, 59.448541024082928 ], [ 24.760120596709097, 59.448494991682757 ], [ 24.760171854931638, 59.448394847195701 ], [ 24.760191642018828, 59.448329426948924 ], [ 24.760208774249499, 59.448228689420937 ], [ 24.760209833828636, 59.448222435468431 ], [ 24.760211591522211, 59.448191311811634 ], [ 24.760203990740639, 59.448164263760816 ], [ 24.76019826507882, 59.448143048770447 ], [ 24.760191125152392, 59.44811373591066 ], [ 24.760203475353642, 59.448072200896377 ], [ 24.760208416751858, 59.448063725229957 ], [ 24.760229676935111, 59.448027273507719 ], [ 24.760256833789697, 59.447991209656735 ], [ 24.760303140384881, 59.447955358433141 ], [ 24.760373810154338, 59.44789733754132 ], [ 24.7603916943639, 59.447879505163939 ], [ 24.760410047588273, 59.447847899696434 ], [ 24.76042880903011, 59.44781562758746 ], [ 24.760452000557304, 59.447778841509873 ], [ 24.760463732618728, 59.447760236728037 ], [ 24.760488332317582, 59.44771702411532 ], [ 24.760507706825024, 59.447682997980948 ], [ 24.760520974332078, 59.44766447409576 ], [ 24.760569034346567, 59.447597355495844 ], [ 24.760580215711798, 59.447581734172559 ], [ 24.760600398768215, 59.447575315931232 ], [ 24.760663267871948, 59.447562870144857 ], [ 24.76072198027488, 59.44755059196256 ], [ 24.760756855888374, 59.447536611184667 ], [ 24.760774311609065, 59.447520136719227 ], [ 24.760776799737148, 59.447501253154272 ], [ 24.76077333978153, 59.447486003632498 ], [ 24.760765426966284, 59.447470286105336 ], [ 24.760723618338417, 59.447449216687581 ], [ 24.760594590854321, 59.447384170958301 ], [ 24.760708850073954, 59.447296660780069 ], [ 24.760809023568864, 59.447285130550036 ], [ 24.760911460937567, 59.447290553284446 ], [ 24.761022894651276, 59.447282781953291 ], [ 24.761116564195319, 59.447286334153041 ], [ 24.761212440939595, 59.447294002851002 ], [ 24.761343844521633, 59.447323252792131 ], [ 24.761426556007141, 59.447353205990275 ], [ 24.761537447537332, 59.447399387824589 ], [ 24.761554179413675, 59.447409766995037 ], [ 24.761573375998282, 59.44741999726147 ], [ 24.761613314061531, 59.447432262027377 ], [ 24.761667207290333, 59.447444221668036 ], [ 24.76172535734937, 59.447455761691806 ], [ 24.761804040149798, 59.447476572656832 ], [ 24.76185766589067, 59.447490760009359 ], [ 24.761959248547882, 59.44752500235051 ], [ 24.762110884198748, 59.447578416695109 ], [ 24.762328431414339, 59.447650650766356 ], [ 24.762358133789974, 59.44766082011806 ], [ 24.762408097724347, 59.447673628063349 ], [ 24.76245147717578, 59.447686824197632 ], [ 24.762492467954907, 59.447711470566844 ], [ 24.762560000652069, 59.447738162583214 ], [ 24.762684775929948, 59.447770447815714 ], [ 24.762900098122639, 59.447828196329247 ], [ 24.76309108335504, 59.447891031691199 ], [ 24.763181232621712, 59.447928669720078 ], [ 24.7632344670671, 59.447943433298214 ], [ 24.76329200778936, 59.447955254447066 ], [ 24.763346072183598, 59.447970076008708 ], [ 24.763428719722519, 59.448000961917359 ], [ 24.763495427035537, 59.448021428354437 ], [ 24.76354870630707, 59.448038184392871 ], [ 24.763470566279072, 59.44830753685406 ], [ 24.763384831472472, 59.448637831732491 ], [ 24.763263417417413, 59.449075614737836 ], [ 24.763214907295161, 59.449264453055825 ], [ 24.763108708408534, 59.449259026940886 ], [ 24.762875508527586, 59.450114823199634 ], [ 24.762704036631973, 59.450728938743787 ], [ 24.763047396775036, 59.450751385115971 ], [ 24.763191930791844, 59.450217264605428 ], [ 24.763350104935554, 59.449661807747034 ], [ 24.763452016750488, 59.449280592763124 ], [ 24.763350835856684, 59.449271726584243 ], [ 24.763526151605259, 59.448622999916054 ], [ 24.763670579996084, 59.448098295594875 ], [ 24.763801810096876, 59.448118764816122 ], [ 24.763951232653202, 59.448155744426188 ], [ 24.764162666216126, 59.448226224572373 ], [ 24.764286826835434, 59.448281312897514 ], [ 24.764404160267759, 59.448330659676444 ], [ 24.764535165948452, 59.448380420721712 ], [ 24.764649478483229, 59.448416068291536 ], [ 24.764810755519708, 59.448478481164607 ], [ 24.764902529631225, 59.448507437019323 ], [ 24.764979397364034, 59.448535168767812 ], [ 24.765084051816419, 59.448567748130813 ], [ 24.765169148293996, 59.448591347538709 ], [ 24.765332550120064, 59.448642293059734 ], [ 24.765480135766438, 59.448685233231409 ], [ 24.765674026412448, 59.448748272418555 ], [ 24.76588948337573, 59.448813097988229 ], [ 24.766060576409721, 59.448850012065066 ], [ 24.766345524294962, 59.448920860128759 ], [ 24.766448658078389, 59.448942127538061 ], [ 24.766377463584924, 59.449209744724101 ], [ 24.766169446141088, 59.450008739361664 ], [ 24.7659452954404, 59.450835619270251 ], [ 24.765737374050985, 59.451629567090912 ], [ 24.766079864796346, 59.45165362648855 ], [ 24.766292532266089, 59.450843680793916 ], [ 24.766492540300771, 59.450115047967266 ], [ 24.766681798719198, 59.449376001236004 ], [ 24.766787376774591, 59.449016862914824 ], [ 24.766961330291881, 59.449049961999279 ], [ 24.767150442234694, 59.449091545407633 ], [ 24.767320182150272, 59.449124605722915 ], [ 24.767742234557094, 59.449195181708056 ], [ 24.767808679726588, 59.44920925603607 ], [ 24.767905926169721, 59.449223429059749 ], [ 24.768137897539294, 59.449250883093946 ], [ 24.768542088748106, 59.449301623239535 ], [ 24.768680420772299, 59.44932710910799 ], [ 24.768830524311984, 59.449348441690454 ], [ 24.768907544430466, 59.449369401780302 ], [ 24.768968911049488, 59.449384169452799 ], [ 24.769011223167528, 59.449390466526559 ], [ 24.769066702266649, 59.449395088847197 ], [ 24.769144904151577, 59.449395709476349 ], [ 24.769239164922389, 59.449410338816577 ], [ 24.769368882883086, 59.449432956812601 ], [ 24.769520307421054, 59.449452629169144 ], [ 24.769631357053722, 59.449464350419362 ], [ 24.769831050688541, 59.44949177462356 ], [ 24.77019325042151, 59.449553338755848 ], [ 24.770313926552245, 59.449580694575772 ], [ 24.770391388374914, 59.449602414248616 ], [ 24.770478669532281, 59.449617855529468 ], [ 24.770609569805387, 59.449653733090024 ], [ 24.770736623864423, 59.449664471085988 ], [ 24.770798241660184, 59.44967312324939 ], [ 24.770849037565128, 59.449685806452123 ], [ 24.770895789082456, 59.449703639050178 ], [ 24.770944614625687, 59.44972119018999 ], [ 24.771000041042306, 59.449735937826993 ], [ 24.771049703894107, 59.449753089048848 ], [ 24.771099855672201, 59.449760659156084 ], [ 24.771161754933825, 59.449763806731276 ], [ 24.771231182448627, 59.449771649947408 ], [ 24.771340848809945, 59.449782157019754 ], [ 24.771400774091482, 59.449789229921734 ], [ 24.771465317125138, 59.449789219996646 ], [ 24.771577362366429, 59.449790305265388 ], [ 24.771750144965168, 59.449793826401525 ], [ 24.772016759427714, 59.449789777585693 ], [ 24.772281095969433, 59.44980038273038 ], [ 24.772326957863722, 59.449802223399104 ], [ 24.772375643024869, 59.449804173173305 ], [ 24.772450981265159, 59.449808309584526 ], [ 24.772489479905822, 59.449810417793522 ], [ 24.772562639401659, 59.449809423216145 ], [ 24.772624160219593, 59.449800507517274 ], [ 24.772714340749975, 59.449792859964951 ], [ 24.772772393494424, 59.44979118188018 ], [ 24.772978501229257, 59.449799568816722 ], [ 24.773119902343556, 59.449796063932276 ], [ 24.773260876819265, 59.449797076726874 ], [ 24.773276616048221, 59.449796885591596 ], [ 24.773366310605134, 59.449795811456958 ], [ 24.773402001654826, 59.449795386473262 ], [ 24.773503657225739, 59.449788434035376 ], [ 24.773562347940114, 59.449783789514456 ], [ 24.773595517149797, 59.44978255342231 ], [ 24.773631011371812, 59.449781222876268 ], [ 24.773699670788908, 59.44977925767094 ], [ 24.773797463549993, 59.449776896881048 ], [ 24.773897267132032, 59.44977535904696 ], [ 24.774049749329119, 59.44977850270471 ], [ 24.774283740376216, 59.449772558373951 ], [ 24.774329801097366, 59.449766057735445 ], [ 24.774385462407018, 59.449759994367021 ], [ 24.774551194301026, 59.449755788350821 ], [ 24.774594106214352, 59.449754261324458 ], [ 24.77466685911039, 59.449751670098799 ], [ 24.774724914409024, 59.449741561927205 ], [ 24.774876952134274, 59.449696367199145 ], [ 24.775120717906301, 59.449612337570152 ], [ 24.775289689350036, 59.449543514531491 ], [ 24.775334108040173, 59.449512076684762 ], [ 24.775368866418745, 59.449468253843605 ], [ 24.775395622357145, 59.449437700918878 ], [ 24.775413015039451, 59.449394644565373 ], [ 24.775449593105254, 59.449361870402385 ], [ 24.775588115182231, 59.449200621559044 ], [ 24.775641282409325, 59.449138350648795 ], [ 24.775630735332879, 59.44912883450786 ], [ 24.775228104406821, 59.449017522466129 ], [ 24.775169319006181, 59.449000839424514 ], [ 24.7751249594981, 59.448988254659135 ], [ 24.774016470701675, 59.449100465490154 ], [ 24.773101968139581, 59.449192273375907 ], [ 24.772360122772195, 59.449266006751415 ], [ 24.771950251655714, 59.44930428953942 ], [ 24.771580790219751, 59.449156209558964 ], [ 24.771507375505635, 59.449126333906392 ], [ 24.771317237751301, 59.449048945327689 ], [ 24.769829809175018, 59.448455896217475 ], [ 24.769179890840814, 59.448198441313146 ], [ 24.768606958611517, 59.447972679932349 ], [ 24.768246749453418, 59.447827095090275 ], [ 24.768098215461595, 59.447767062746991 ], [ 24.767867759616262, 59.447676403239136 ], [ 24.767510276987394, 59.447535764674264 ], [ 24.766622518862153, 59.447178038685443 ], [ 24.766405078455563, 59.447090415838453 ], [ 24.766234002220795, 59.447022442436769 ], [ 24.765710230145864, 59.446814336180367 ], [ 24.765102694323108, 59.446571579280295 ], [ 24.762662150000534, 59.445598995261733 ], [ 24.762576723707266, 59.445564947184586 ], [ 24.762362684820214, 59.445476364449028 ], [ 24.762425319992772, 59.44544890096774 ], [ 24.762462172216686, 59.445462000224587 ], [ 24.762703228926338, 59.445303232441447 ], [ 24.762664769237652, 59.445287611104071 ], [ 24.762678388218191, 59.445273869567046 ], [ 24.762524391454242, 59.4452145538446 ], [ 24.762724582066653, 59.444948265150494 ], [ 24.76276824347288, 59.444954502527601 ], [ 24.762901156129516, 59.444765166203467 ], [ 24.762866593030044, 59.444756667882928 ], [ 24.762880551126525, 59.444739181033043 ], [ 24.763061534251381, 59.444770107981384 ], [ 24.763110885440238, 59.444778547502658 ], [ 24.766475803095776, 59.445413982917827 ], [ 24.767077860317507, 59.445527077641501 ], [ 24.767089014517044, 59.445515118423621 ], [ 24.767149882480005, 59.445526864644172 ], [ 24.767192048797309, 59.445522974465312 ], [ 24.767327090514357, 59.445336807950248 ], [ 24.767041997327294, 59.445184462560235 ], [ 24.766601577769265, 59.444934922233287 ], [ 24.766573811129369, 59.444904921794837 ], [ 24.766020689126652, 59.444800768368289 ], [ 24.76523438782856, 59.444660031655218 ], [ 24.763654785698119, 59.444372085619506 ], [ 24.763670514975981, 59.444341598985446 ], [ 24.763713974114655, 59.444348277082923 ], [ 24.763855468353285, 59.444161520147745 ], [ 24.763817162032229, 59.444154300537591 ], [ 24.763807373004433, 59.444152454225545 ], [ 24.763821029189256, 59.444123074761215 ], [ 24.76326852101176, 59.444017039133897 ], [ 24.7634035722036, 59.443803793565245 ], [ 24.762907220615045, 59.443700518956312 ], [ 24.762806155115541, 59.443679497183297 ], [ 24.762534126686369, 59.443622893784223 ], [ 24.762005940623261, 59.443524091018134 ], [ 24.760452825070903, 59.443234625103784 ], [ 24.758860662748948, 59.442931065948713 ], [ 24.75827785192735, 59.442822518026283 ], [ 24.757941119764034, 59.442759805450422 ], [ 24.75795309034006, 59.44274163934373 ], [ 24.758037993920652, 59.4426128633283 ], [ 24.757959538981108, 59.442525450001376 ], [ 24.757887701033244, 59.442499516523938 ], [ 24.757867995827269, 59.442492403597477 ], [ 24.757816918214147, 59.442483002570768 ], [ 24.757716674614329, 59.442464557508885 ], [ 24.757669309949392, 59.442454874736654 ], [ 24.758468578734064, 59.441316235337183 ], [ 24.760768124110758, 59.441978386373222 ], [ 24.760467479289183, 59.442361600791848 ], [ 24.760192281574749, 59.442738545373331 ], [ 24.760169326632607, 59.442800079160314 ], [ 24.76105558350935, 59.442982306629986 ], [ 24.762533874697915, 59.443261013569163 ], [ 24.762801357199343, 59.443295721945141 ], [ 24.762877778353381, 59.443305638538696 ], [ 24.763605044446042, 59.443399172358859 ], [ 24.763918161954376, 59.443452680610719 ], [ 24.76481488082117, 59.443616330059797 ], [ 24.765631601268112, 59.443768230058865 ], [ 24.76643550835697, 59.443914364669347 ], [ 24.766983333463831, 59.444015566861204 ], [ 24.76684750242778, 59.444194690945807 ], [ 24.767155253263322, 59.444250422120284 ], [ 24.767292117000043, 59.444079667055533 ], [ 24.767451112698947, 59.444112628132572 ], [ 24.767553827558718, 59.444143322721089 ], [ 24.767646533671048, 59.444171023452022 ], [ 24.767621594575118, 59.444190774142946 ], [ 24.767577476287801, 59.444179891051682 ], [ 24.767450315281732, 59.444148531016154 ], [ 24.767330131032121, 59.444128773126465 ], [ 24.767222943516469, 59.444277990572694 ], [ 24.767432104019115, 59.444319456715753 ], [ 24.767711195040739, 59.444412041783551 ], [ 24.767849112385516, 59.444486642960804 ], [ 24.768084011825241, 59.44437607948521 ], [ 24.76806378933038, 59.444365703530963 ], [ 24.767946377477649, 59.444305444647426 ], [ 24.767842018675459, 59.444267524642335 ], [ 24.767863743818889, 59.444244381461864 ], [ 24.767900428730783, 59.44425470635101 ], [ 24.768021552910128, 59.444288794198656 ], [ 24.768061214024776, 59.444304478794024 ], [ 24.768108327382933, 59.444323118169905 ], [ 24.768153733267624, 59.444341076273524 ], [ 24.76868601865915, 59.444627715113505 ], [ 24.769535437745763, 59.44509150576414 ], [ 24.770099101947892, 59.445392812219701 ], [ 24.770401948637669, 59.445553952218702 ], [ 24.771070366028844, 59.445909597796344 ], [ 24.771454614350436, 59.445828953395356 ], [ 24.771482612414101, 59.445824489518806 ], [ 24.771501246291994, 59.44581127424145 ], [ 24.77146296547027, 59.445761632315183 ], [ 24.771437850446219, 59.445729058963096 ], [ 24.771262422430322, 59.445548916450086 ], [ 24.771104681239724, 59.445386928977321 ], [ 24.771066757044753, 59.445347572264168 ], [ 24.770724901100838, 59.444992772030275 ], [ 24.770572466850716, 59.444837638070396 ], [ 24.770465859480836, 59.444729148656187 ], [ 24.770149985261931, 59.444398782401315 ], [ 24.769936547486893, 59.444175547265637 ], [ 24.769833826988712, 59.444070400916502 ], [ 24.769758135822421, 59.443992924621455 ], [ 24.769821376414541, 59.443981755901142 ], [ 24.769849842308801, 59.443997325861055 ], [ 24.770203113032267, 59.443910160330418 ], [ 24.77020559947098, 59.443903170809925 ], [ 24.770212079443453, 59.443884892040892 ], [ 24.770366795152345, 59.443845225326989 ], [ 24.770535881229769, 59.443801865680022 ], [ 24.770863908380154, 59.443686658846495 ], [ 24.770898954687542, 59.443715745051776 ], [ 24.77132600881988, 59.443581106089049 ], [ 24.77324693181464, 59.445025333648417 ], [ 24.773591840435575, 59.445283357514413 ], [ 24.773681151657954, 59.445344485654076 ], [ 24.773752310024836, 59.445382416578695 ], [ 24.773774988560703, 59.445394500787927 ], [ 24.773851553303185, 59.445435299407741 ], [ 24.773900526356758, 59.445412102852721 ], [ 24.774192828328342, 59.445273641326132 ], [ 24.774218565822913, 59.44526202665535 ], [ 24.77457818993037, 59.445447940528702 ], [ 24.774731176188393, 59.445528559171684 ], [ 24.775315349255212, 59.445836383725208 ], [ 24.775614300655413, 59.44599161909079 ], [ 24.776155699170609, 59.446272761398099 ], [ 24.776414554553345, 59.446146512010955 ], [ 24.776408152155376, 59.446140158261535 ], [ 24.77653185646658, 59.446081582454767 ], [ 24.77654684355948, 59.446098783434607 ], [ 24.776591633414686, 59.446110278985081 ], [ 24.776627526626179, 59.446111162526378 ], [ 24.776668809371898, 59.446110290730765 ], [ 24.776689166179381, 59.446100053828744 ], [ 24.776723578233451, 59.446085416195004 ], [ 24.776756123390761, 59.446079380369568 ], [ 24.776783587961987, 59.446078607979267 ], [ 24.776994211836932, 59.446084978010035 ], [ 24.7771352962367, 59.44608218873055 ], [ 24.777214758957331, 59.446079664179798 ], [ 24.777271723995639, 59.446076177173481 ], [ 24.777337325901431, 59.446068779142152 ], [ 24.777387533698644, 59.44605729771736 ], [ 24.777437779672397, 59.446038975717002 ], [ 24.777490562261992, 59.446000440896213 ], [ 24.777501208907072, 59.445992666968969 ], [ 24.777557826501543, 59.445949145304638 ], [ 24.777575103056797, 59.445940676925083 ], [ 24.777636616114236, 59.445927135810159 ], [ 24.77771267029469, 59.445912736803031 ], [ 24.777760012473532, 59.445899288277779 ], [ 24.777790904590145, 59.445889572484852 ], [ 24.777830329130669, 59.445862849097054 ], [ 24.77784371197491, 59.445847509482476 ], [ 24.777867096453672, 59.445829363895292 ], [ 24.777916821155291, 59.44580598186262 ], [ 24.777967357568002, 59.445786508907062 ], [ 24.778018848111472, 59.445767910023505 ], [ 24.778232354224933, 59.445698146630662 ], [ 24.77837886311335, 59.445650350115557 ], [ 24.7784200940559, 59.445616290788045 ], [ 24.778455331264315, 59.44560078604097 ], [ 24.778531530055336, 59.44556725610439 ], [ 24.77864422663594, 59.445505484746626 ], [ 24.779126425407746, 59.445244666482651 ], [ 24.779206378040886, 59.445245163006732 ], [ 24.779325940408835, 59.445230999376399 ], [ 24.779416917263386, 59.445213261463344 ], [ 24.779587474517271, 59.445181264092291 ], [ 24.779951840802941, 59.44510731933061 ], [ 24.780043513588474, 59.44508380476401 ], [ 24.780139149247219, 59.445060921968199 ], [ 24.780221805067583, 59.44503546782942 ], [ 24.780346736175474, 59.444984753756451 ], [ 24.780469097570432, 59.444935769361699 ], [ 24.78057264591191, 59.444908970348081 ], [ 24.780607303113946, 59.444895928097225 ], [ 24.780670678899419, 59.444883038786649 ], [ 24.780728195574643, 59.444880651149376 ], [ 24.780765151333149, 59.444876401493232 ], [ 24.780805471140109, 59.44486123402276 ], [ 24.780867005230821, 59.444827933265181 ], [ 24.780912729191048, 59.444803191359867 ], [ 24.78095253488739, 59.44476862797876 ], [ 24.780970648610353, 59.444752891928452 ], [ 24.781052162466768, 59.444680638876285 ], [ 24.781246285711767, 59.444539305498537 ], [ 24.781311991676937, 59.444462639672444 ], [ 24.781371562706084, 59.444391351415774 ], [ 24.781413093057875, 59.444309703185787 ], [ 24.781436241235429, 59.444251306257044 ], [ 24.781442834909011, 59.444218223444324 ], [ 24.781453565412505, 59.444188751700267 ], [ 24.781484810875927, 59.444142201009413 ], [ 24.781527601630213, 59.444108732852087 ], [ 24.781560422080386, 59.44409250548842 ], [ 24.781623724131954, 59.444076474252789 ], [ 24.78179361677438, 59.444051201551652 ], [ 24.782068621390813, 59.444012943436427 ], [ 24.782179019165898, 59.444004469042198 ], [ 24.782292099022204, 59.443979954988528 ], [ 24.782334617125226, 59.443962332280314 ], [ 24.782431594957796, 59.443936683923724 ], [ 24.782515448309365, 59.443922145993035 ], [ 24.782663501744988, 59.443897729034859 ], [ 24.782873444207343, 59.44385951471709 ], [ 24.783421330243414, 59.443782144214296 ], [ 24.783573906889913, 59.443760553950646 ], [ 24.78360281188974, 59.443758882849991 ], [ 24.783675459757934, 59.443758441627111 ], [ 24.783719421111815, 59.443756133325678 ], [ 24.783875308530778, 59.443737494340951 ], [ 24.784049405963959, 59.443710479106535 ], [ 24.784255893203017, 59.443670828888628 ], [ 24.784553365588707, 59.443632539479019 ], [ 24.784807211443713, 59.443588898840865 ], [ 24.785072469436898, 59.44356555811752 ], [ 24.785362508355433, 59.443542131929007 ], [ 24.785526226241775, 59.443520688396298 ], [ 24.785720840931852, 59.443502193255362 ], [ 24.785949604510098, 59.443463278216115 ], [ 24.786128561292863, 59.44343921152349 ], [ 24.786319556535968, 59.443409004372739 ], [ 24.78656377876381, 59.443367850500117 ], [ 24.786831880061424, 59.443328797723417 ], [ 24.787139909204999, 59.443288392644426 ], [ 24.78740934809392, 59.443252364860243 ], [ 24.787726257561161, 59.443213395463452 ], [ 24.787795759836186, 59.443208069420749 ], [ 24.787845751905262, 59.443208964301206 ], [ 24.788086528586177, 59.443256977178756 ], [ 24.788132408682799, 59.443272932760514 ], [ 24.788158523823746, 59.443272605922772 ], [ 24.788168894148122, 59.443271331990218 ], [ 24.788179176350646, 59.443270058584147 ], [ 24.788189171158056, 59.443261677240962 ], [ 24.788155933111305, 59.443151918851164 ], [ 24.788138384817096, 59.443109473646594 ], [ 24.788197991835347, 59.443103102452966 ], [ 24.788259869363571, 59.443250122847189 ], [ 24.788287928787824, 59.443265305301495 ], [ 24.788322817218141, 59.443266299376738 ], [ 24.788427435950855, 59.443244353143569 ], [ 24.788507251913501, 59.443223847986168 ], [ 24.788613188704407, 59.443229228188933 ], [ 24.788759855947401, 59.443265702531413 ], [ 24.78919207269351, 59.443354704040786 ], [ 24.789733349716453, 59.443474262713259 ], [ 24.790063171770768, 59.443567582761126 ], [ 24.790358847837247, 59.443638610939317 ], [ 24.790507507112881, 59.443674173713426 ], [ 24.790832143201243, 59.443732136270981 ], [ 24.791164831887901, 59.443780031506677 ], [ 24.791716208763841, 59.443867034230671 ], [ 24.792153691042078, 59.443967314238542 ], [ 24.792226641920223, 59.443986759161895 ], [ 24.792664348933446, 59.444103427850372 ], [ 24.793164977706375, 59.444271279175844 ], [ 24.793185911050873, 59.444277778021203 ], [ 24.793329656965071, 59.44432239792927 ], [ 24.793584399439414, 59.444391950899572 ], [ 24.79382810700714, 59.444460690160057 ], [ 24.793886630686373, 59.444476666295984 ], [ 24.794005416499751, 59.444509083783018 ], [ 24.794375732453446, 59.444625883777633 ], [ 24.794427442517602, 59.444642188028901 ], [ 24.794612938102059, 59.444693834944189 ], [ 24.794785054742341, 59.444746226620751 ], [ 24.794992577919704, 59.444817228810741 ], [ 24.795167222102524, 59.444875161403132 ], [ 24.795309101574688, 59.444934790745826 ], [ 24.79563118531523, 59.445078306305021 ], [ 24.796053466047582, 59.445219491990073 ], [ 24.796559144529255, 59.445398359493048 ], [ 24.797329887395769, 59.445704375655687 ], [ 24.797923406337397, 59.445939996426702 ], [ 24.79830567425094, 59.446101561393647 ], [ 24.798480886446139, 59.446177987471408 ], [ 24.798607317696924, 59.44623914324211 ], [ 24.798758117892007, 59.446309612710529 ], [ 24.79885900976436, 59.446352053788672 ], [ 24.798914429096801, 59.446399366751528 ], [ 24.798952239268242, 59.446424883002997 ], [ 24.798992084740021, 59.446456769434086 ], [ 24.7990136457706, 59.446501298445185 ], [ 24.79907711900761, 59.446542467206037 ], [ 24.799149599161058, 59.446569263223836 ], [ 24.799262389472279, 59.446589019134947 ], [ 24.799351012157587, 59.446611821138532 ], [ 24.799456424594773, 59.44664623596146 ], [ 24.799512032940608, 59.44666918437192 ], [ 24.799698945480937, 59.446742898634028 ], [ 24.800085528027168, 59.446923849209917 ], [ 24.800245543093258, 59.447014916739754 ], [ 24.800331783321848, 59.447067625457173 ], [ 24.8005021928731, 59.44715159167459 ], [ 24.800985263743407, 59.44742713469477 ], [ 24.801161809656676, 59.447535820889243 ], [ 24.801521593868934, 59.447750512660186 ], [ 24.801784387446933, 59.447923907913086 ], [ 24.801924462963999, 59.448014538227767 ], [ 24.802029016899635, 59.448087170665332 ], [ 24.802142227109638, 59.448186070412923 ], [ 24.802245484285326, 59.448273253062752 ], [ 24.802389278470006, 59.448366723851898 ], [ 24.80252399244451, 59.448451111379946 ], [ 24.802718533094513, 59.448575655677537 ], [ 24.802876374114721, 59.448693600981365 ], [ 24.803183987058635, 59.448932574398256 ], [ 24.803394242419685, 59.449081914670884 ], [ 24.803537537510099, 59.449183116321805 ], [ 24.803752997110639, 59.449320485080207 ], [ 24.803863999141139, 59.449406075195419 ], [ 24.804003569308588, 59.449509193187843 ], [ 24.804235319415525, 59.449653149481506 ], [ 24.804421018179301, 59.449785725678822 ], [ 24.804614661164219, 59.449926924721595 ], [ 24.80475315349647, 59.450020775432051 ], [ 24.804892477278251, 59.450125186659719 ], [ 24.805057422296308, 59.450231245508327 ], [ 24.805182218872726, 59.450316893845859 ], [ 24.805293171577862, 59.450418793985826 ], [ 24.805410574843396, 59.450534030086168 ], [ 24.805519225083938, 59.450623466304684 ], [ 24.805669207823865, 59.450777426533016 ], [ 24.805769770145439, 59.450894641467613 ], [ 24.805808763046659, 59.450925768034729 ], [ 24.805869929317922, 59.450974568999094 ], [ 24.80594579257059, 59.451046224822484 ], [ 24.805985735189562, 59.451083961457698 ], [ 24.806057348297749, 59.451174970354856 ], [ 24.806173241417859, 59.451363439087302 ], [ 24.806333920629854, 59.451573653777089 ], [ 24.806476617918332, 59.45178012728573 ], [ 24.806648447795553, 59.452019618598918 ], [ 24.806807923013594, 59.452251752523196 ], [ 24.806917331018322, 59.452451221001645 ], [ 24.807051502281752, 59.452628616250436 ], [ 24.807149017805585, 59.452781352157295 ], [ 24.807292446419027, 59.453012371907121 ], [ 24.807395749454322, 59.453196518007061 ], [ 24.807470926706564, 59.453313384479124 ], [ 24.807588333447164, 59.453511537602687 ], [ 24.80763359838372, 59.453571333091276 ], [ 24.807640496259811, 59.453580438198159 ], [ 24.807740362734481, 59.453712324001465 ], [ 24.807848164583739, 59.453896388280008 ], [ 24.807986711667766, 59.454090605221189 ], [ 24.808128905891749, 59.454339028678604 ], [ 24.808204830162996, 59.454476815120728 ], [ 24.808242522726793, 59.454533936864188 ], [ 24.808254831271345, 59.454552604976541 ], [ 24.808266402953379, 59.454564724521859 ], [ 24.80836004301068, 59.454662778453532 ], [ 24.808435476874415, 59.454764884864048 ], [ 24.808607833244714, 59.454948695986317 ], [ 24.808926666535783, 59.45531483368562 ], [ 24.809065089301033, 59.455499579665791 ], [ 24.809157019883926, 59.455607141006581 ], [ 24.809248370079562, 59.455708045040517 ], [ 24.809275413275326, 59.455740527658286 ], [ 24.809298118230462, 59.455770011727196 ], [ 24.809349263419769, 59.455820237098685 ], [ 24.809425322024996, 59.455900857542936 ], [ 24.809510567706315, 59.456003782774516 ], [ 24.809610860516003, 59.456129246070411 ], [ 24.809680809429256, 59.456202758353719 ], [ 24.809808093992814, 59.456353980697038 ], [ 24.809950753835512, 59.456519974023642 ], [ 24.810040082552081, 59.456640270896074 ], [ 24.810169970038142, 59.456778433552657 ], [ 24.81032950829475, 59.4569496010511 ], [ 24.810359616807471, 59.456990790064971 ], [ 24.810363597405864, 59.456996232841099 ], [ 24.814996946177537, 59.456800622137983 ], [ 24.816397984159419, 59.456750654986742 ], [ 24.816035205388157, 59.454520093065781 ], [ 24.815601110506698, 59.453701399844725 ], [ 24.813310286654385, 59.452299644176286 ], [ 24.813010485191512, 59.451770963015832 ], [ 24.813038501477291, 59.451385144677729 ], [ 24.813158364870148, 59.450717604360477 ], [ 24.81334700552188, 59.450273162773605 ], [ 24.812879108225562, 59.44695524131923 ], [ 24.812447470748165, 59.446158514483663 ], [ 24.81184627896226, 59.445569658366757 ], [ 24.813766897103829, 59.445150345949266 ], [ 24.814580714125629, 59.445103752423329 ], [ 24.814533908593752, 59.444892905794134 ], [ 24.814306149854062, 59.444825910331481 ], [ 24.813978847332681, 59.444709259205858 ], [ 24.81364643223646, 59.444570286440687 ], [ 24.813304980577396, 59.444377417629291 ], [ 24.812980017662088, 59.44417358414573 ], [ 24.812798841951444, 59.444041575289077 ], [ 24.81248137233613, 59.443537417840496 ], [ 24.812188030102622, 59.443128355470542 ], [ 24.809059599959895, 59.44148877384702 ], [ 24.807890305239408, 59.44087762469033 ], [ 24.806927456793996, 59.440408830517129 ], [ 24.803975364227796, 59.439066350498059 ], [ 24.799924627994979, 59.437218173041572 ], [ 24.799940150694066, 59.437203715738875 ], [ 24.798762754309365, 59.436639754006372 ], [ 24.798555022261802, 59.436551245159237 ], [ 24.798272907786654, 59.435391078106385 ], [ 24.797214727505036, 59.435401079826399 ], [ 24.796721979323511, 59.435392661592822 ], [ 24.796230937923127, 59.435366726281607 ], [ 24.795044392276651, 59.435282690898575 ], [ 24.794277861453423, 59.435178696774528 ], [ 24.793832172959959, 59.435109749029486 ], [ 24.793813167758284, 59.434504103927921 ], [ 24.793008784042186, 59.434180486977127 ], [ 24.792417590830706, 59.434090327650331 ], [ 24.792058415671708, 59.43402776548659 ], [ 24.791864558751044, 59.433981443819569 ], [ 24.791679605055066, 59.433931477586711 ], [ 24.791476934300281, 59.433869768120061 ], [ 24.790293256637607, 59.433419326712048 ], [ 24.788283357322069, 59.43225899534913 ], [ 24.786689975870047, 59.431313475675175 ], [ 24.785568627789505, 59.430647444022689 ], [ 24.785245771110763, 59.430491646307324 ], [ 24.784897523007963, 59.430304221038234 ], [ 24.78464020455737, 59.430186182845908 ], [ 24.784156439120807, 59.429874694846156 ], [ 24.783517670357298, 59.429411251410755 ], [ 24.782994425760233, 59.428990655726722 ], [ 24.782016213805754, 59.428174905993863 ], [ 24.780930333731249, 59.427394797607818 ], [ 24.780303740503083, 59.42713656775765 ], [ 24.779986291496588, 59.427114839844783 ], [ 24.779278723543488, 59.427152693634007 ], [ 24.779494588615322, 59.426961375015296 ], [ 24.78032583339699, 59.426537768413098 ], [ 24.780895307164979, 59.426255120551104 ], [ 24.781630467147448, 59.425953263353883 ], [ 24.781763048349926, 59.42601908456875 ], [ 24.784382408962273, 59.424957874032927 ], [ 24.78513032355507, 59.424653945868002 ], [ 24.786682718266849, 59.424523040311215 ], [ 24.786999366036692, 59.424480212841956 ], [ 24.787177936460392, 59.423880744671223 ], [ 24.786975821447445, 59.423603040779348 ], [ 24.786958315506762, 59.423531330197434 ], [ 24.787484105877869, 59.42315402812735 ], [ 24.788610484861028, 59.422344479328522 ], [ 24.788963691014267, 59.422047201726805 ], [ 24.789858115315702, 59.421486345768166 ], [ 24.79070101450883, 59.420960982513755 ], [ 24.791363624915004, 59.420676381587626 ], [ 24.792086293887671, 59.420331090863364 ], [ 24.792460341778206, 59.42014481225695 ], [ 24.792293306533118, 59.419762772426985 ], [ 24.795775644436201, 59.417255869688205 ], [ 24.798938926411559, 59.414555288678493 ], [ 24.801510103714584, 59.412332487077499 ], [ 24.801688043528632, 59.412380596863827 ], [ 24.802512585520475, 59.412290828090605 ], [ 24.803968170665819, 59.412255909382573 ], [ 24.808314541805125, 59.412071735497051 ], [ 24.81106560258749, 59.412076172141283 ], [ 24.813816591787329, 59.412092311572195 ], [ 24.814392827435604, 59.411686307938297 ], [ 24.81492811107131, 59.411528050950878 ], [ 24.81535920636956, 59.411365320523501 ], [ 24.815693229014141, 59.411223209159559 ], [ 24.819052345669327, 59.409516371391732 ], [ 24.81834698620894, 59.409114640726187 ], [ 24.81765271811194, 59.409396974929116 ], [ 24.81465810054679, 59.407852834445187 ], [ 24.816071425212172, 59.407132647687277 ], [ 24.815868984558502, 59.40680274138289 ], [ 24.814982984913392, 59.404927459186744 ], [ 24.813623380318205, 59.402337477417156 ], [ 24.812910407816478, 59.402487941438125 ], [ 24.812784937428638, 59.402438535209228 ], [ 24.81094087040627, 59.40390661638353 ], [ 24.809901013263541, 59.404775533400596 ], [ 24.809461175291439, 59.404350755418356 ], [ 24.807982522561804, 59.405541129308624 ], [ 24.806685190113924, 59.406644553675989 ], [ 24.80707593677208, 59.407100881638009 ], [ 24.80620938997831, 59.407778212225345 ], [ 24.801888506831702, 59.407637287763244 ], [ 24.799062031547422, 59.407585002870185 ], [ 24.796050746652337, 59.40755828279368 ], [ 24.795747246628153, 59.407553565613611 ], [ 24.795290611812511, 59.407195452311107 ], [ 24.794658722062998, 59.406675374239001 ], [ 24.794009570447415, 59.406002519409604 ], [ 24.793386998307763, 59.40522958729386 ], [ 24.792967418411635, 59.404751040712156 ], [ 24.792330146935438, 59.40420037239322 ], [ 24.791428622786416, 59.403482164720387 ], [ 24.790556388140278, 59.402985955199391 ], [ 24.790207483385966, 59.402782119572144 ], [ 24.78997930876244, 59.402614273916164 ], [ 24.789832254609788, 59.402482656683617 ], [ 24.789724936961647, 59.402348736030518 ], [ 24.789628779834359, 59.402172017854156 ], [ 24.789613421615051, 59.402025336362598 ], [ 24.789655264205258, 59.401725433447844 ], [ 24.789865238349151, 59.40115126153372 ], [ 24.789948495173164, 59.400944829473588 ], [ 24.79006448066189, 59.400730121473607 ], [ 24.790222425795367, 59.400502862800309 ], [ 24.790354910590342, 59.400347123966462 ], [ 24.791099208609722, 59.399653575678407 ], [ 24.791891008657394, 59.39893729434155 ], [ 24.792179260939065, 59.398591021371011 ], [ 24.792501773356069, 59.39818134317521 ], [ 24.792554749035393, 59.398066926276826 ], [ 24.792595159126744, 59.397949622660803 ], [ 24.79261457306281, 59.397860633343704 ], [ 24.792630194786749, 59.397744735863974 ], [ 24.792640419093072, 59.397517466286899 ], [ 24.792638476320452, 59.397312084135713 ], [ 24.792601516586764, 59.39675779723455 ], [ 24.792551974614675, 59.396116239743449 ], [ 24.792528160744265, 59.395969698921007 ], [ 24.792495304282948, 59.395858312682421 ], [ 24.792429763386455, 59.395673332345915 ], [ 24.792396403024789, 59.395547765433712 ], [ 24.792366370813848, 59.395406289148283 ], [ 24.792372351110853, 59.39523892163286 ], [ 24.792419586901698, 59.395120050712862 ], [ 24.79251575056804, 59.395000974709248 ], [ 24.792582172074557, 59.394935311556303 ], [ 24.792735442710736, 59.39481203109267 ], [ 24.793058342151625, 59.394587903257289 ], [ 24.793440729472984, 59.394340973159956 ], [ 24.793839455325216, 59.394108126960333 ], [ 24.794321730848338, 59.393850357400204 ], [ 24.795746066003247, 59.393148630899823 ], [ 24.79575577203201, 59.393074062871385 ], [ 24.793627438608333, 59.392055464255918 ], [ 24.791569366301047, 59.391063429495517 ], [ 24.792007685327466, 59.390709788613087 ], [ 24.791298756643165, 59.390541607843666 ], [ 24.791085979745144, 59.389544552906479 ], [ 24.790583512005341, 59.388683176694052 ], [ 24.789222485444693, 59.38672430550114 ], [ 24.788036162037528, 59.385437381298971 ], [ 24.786967403488976, 59.384530099816907 ], [ 24.787744260142059, 59.382740452269204 ], [ 24.784125616772855, 59.381014344343861 ], [ 24.781536128740782, 59.379831328830093 ], [ 24.781302578877362, 59.378109303304257 ], [ 24.781104507169253, 59.376715176989812 ], [ 24.771690534970425, 59.374118423734046 ], [ 24.769897523126563, 59.375285361696818 ], [ 24.769169918223433, 59.376364891462643 ], [ 24.765426264533662, 59.37605585135983 ], [ 24.762006641315782, 59.37575901060503 ], [ 24.760540683041395, 59.374926173859969 ], [ 24.748804947043578, 59.37640474197724 ], [ 24.748178950050708, 59.376731478844668 ], [ 24.747544370808331, 59.377471293335191 ], [ 24.742063014061259, 59.382655230644716 ], [ 24.739908668699549, 59.384723188867184 ], [ 24.73858950720393, 59.385597152292533 ], [ 24.735309042859036, 59.38859902923074 ], [ 24.733362931859077, 59.390417584114076 ], [ 24.733256534592588, 59.390593676565864 ], [ 24.732121638886827, 59.392530399426654 ], [ 24.731218129261098, 59.393802788608241 ], [ 24.730180277417389, 59.394012600451681 ], [ 24.728788371590092, 59.394301208908303 ], [ 24.727900463809718, 59.393179702507531 ], [ 24.727401721613298, 59.392561707991923 ], [ 24.726983655525657, 59.392848857447198 ], [ 24.725486300481762, 59.393782825952556 ], [ 24.725347051730203, 59.394146889755973 ], [ 24.728764395937279, 59.400967054023859 ], [ 24.729202556970652, 59.401519397895214 ], [ 24.729570583189595, 59.402175273763568 ], [ 24.730094382528165, 59.4028370167322 ], [ 24.73015071020135, 59.402902504709836 ], [ 24.729322883903809, 59.40280305475828 ], [ 24.727738163068665, 59.402594321177141 ], [ 24.726518571732356, 59.402434092397506 ], [ 24.726966053480535, 59.402945367468313 ], [ 24.727100818417593, 59.403081699705481 ], [ 24.727799830145319, 59.403787905892408 ], [ 24.728933609600833, 59.404933542205733 ], [ 24.73031491800722, 59.406453477233057 ], [ 24.730203017451064, 59.40648210621913 ], [ 24.730558158976251, 59.406877630790213 ], [ 24.730665483860097, 59.406989155985798 ], [ 24.730789627639432, 59.407080210159982 ], [ 24.730708927867898, 59.407110551392606 ], [ 24.731170485037637, 59.407664912538024 ], [ 24.731251460184627, 59.407647406436062 ], [ 24.731431096675976, 59.407863019906067 ], [ 24.731538012081348, 59.407839893522691 ], [ 24.732073292279015, 59.408456320596649 ], [ 24.733000080405141, 59.40951016238494 ], [ 24.732813979242035, 59.409552492637083 ], [ 24.733180551660872, 59.409985469667227 ], [ 24.733414041989779, 59.410297012324421 ], [ 24.733653459622793, 59.410663011101732 ], [ 24.733889060730235, 59.411072478750206 ], [ 24.733948941290347, 59.411171249376139 ], [ 24.734014485346609, 59.41129593152732 ], [ 24.734166936274327, 59.41148215892153 ], [ 24.734214386166734, 59.4115596338331 ], [ 24.734261277030548, 59.412176890793141 ], [ 24.734281869286644, 59.412528938274079 ], [ 24.7342561538495, 59.413151986272737 ], [ 24.734191065466355, 59.413466988424894 ], [ 24.733988234910196, 59.414109157448607 ], [ 24.733655851552619, 59.414828800408493 ], [ 24.732029472320765, 59.418336134715119 ], [ 24.73083671979613, 59.420925039766324 ], [ 24.729826965628224, 59.422778252895327 ], [ 24.729673510031603, 59.423049575297348 ], [ 24.72886877125784, 59.424212581986694 ], [ 24.728678328451814, 59.42444362236629 ], [ 24.728586314112892, 59.425027804101703 ], [ 24.727289962397087, 59.426397921687837 ], [ 24.726431475227116, 59.427368924945824 ], [ 24.725962805617343, 59.42792349370432 ], [ 24.72533164777353, 59.428675996597605 ], [ 24.724907320373518, 59.429246115963622 ], [ 24.724038705685977, 59.430410966380002 ], [ 24.72343272191457, 59.431549411942093 ], [ 24.723345204014507, 59.431976559211833 ], [ 24.723292759354312, 59.432479098147809 ], [ 24.723333533970681, 59.432807065156688 ], [ 24.7234079019339, 59.433173627223567 ], [ 24.723625047650685, 59.43403312720141 ], [ 24.723703896372204, 59.434319771055797 ], [ 24.724874690491436, 59.43457724795504 ], [ 24.725555961238953, 59.43467708766137 ], [ 24.726305740882651, 59.434783099029396 ], [ 24.726645718853927, 59.434810218256267 ], [ 24.727746474746823, 59.434888430981083 ], [ 24.728147000796643, 59.43492032844739 ], [ 24.729785349646342, 59.43509438454133 ], [ 24.730120623218021, 59.435098989182706 ], [ 24.730646351289359, 59.435093379976848 ], [ 24.73147969535157, 59.435043420434987 ], [ 24.732582610227592, 59.434983069069993 ], [ 24.732669707710901, 59.435042639143283 ], [ 24.732338163751354, 59.436124659587612 ], [ 24.732470444209095, 59.436228413539133 ], [ 24.732489174737779, 59.436485583818992 ], [ 24.732790733437071, 59.43659378176396 ], [ 24.732927043762604, 59.43673736972324 ], [ 24.733381770600683, 59.437248758458225 ], [ 24.73408653447968, 59.438077427501064 ], [ 24.734122778837463, 59.438123814911009 ], [ 24.73423957119843, 59.438203326360927 ], [ 24.734454978118048, 59.438361103819936 ], [ 24.734732613301361, 59.438552286417874 ], [ 24.735144724130773, 59.438759863433013 ], [ 24.735588246392183, 59.438960890205642 ], [ 24.735913809554475, 59.439101981512785 ], [ 24.736049916059358, 59.439194130453849 ], [ 24.736166647155347, 59.439262240046794 ], [ 24.736778187408515, 59.439448948101266 ], [ 24.737058921644824, 59.439562190100574 ], [ 24.738545149153577, 59.44003259490259 ], [ 24.738783358681829, 59.440094185977777 ], [ 24.739140676099449, 59.440259956904406 ], [ 24.739435473422841, 59.440354533028973 ], [ 24.741047867184058, 59.440966662791737 ], [ 24.741291249231232, 59.441063050151648 ], [ 24.741326513258421, 59.441079907526152 ], [ 24.74136201793052, 59.441115973834854 ], [ 24.7421306807605, 59.441408955361325 ], [ 24.74313522581533, 59.441800872228008 ], [ 24.743810836294841, 59.442101101648319 ], [ 24.744173956040182, 59.442272212339574 ], [ 24.744235685617401, 59.442298255243095 ], [ 24.74447615034801, 59.442413594251903 ], [ 24.745224079311789, 59.443579576538532 ], [ 24.745518703481363, 59.44376911440586 ], [ 24.745850115884004, 59.44385924996876 ], [ 24.746081321279991, 59.443921136943466 ], [ 24.746354124021874, 59.443990956663974 ], [ 24.746805115647486, 59.444140466772801 ], [ 24.748592982229404, 59.443945754710313 ], [ 24.749665082529383, 59.443858955586009 ], [ 24.753154265815606, 59.443528991221442 ], [ 24.753012349669866, 59.443943811914046 ], [ 24.753140051950091, 59.444027553262458 ], [ 24.753179072698924, 59.444110455128225 ], [ 24.753138299737287, 59.444403780193369 ], [ 24.753255097077556, 59.444600421913087 ], [ 24.754335450758354, 59.444639481895699 ], [ 24.754151413719349, 59.445964431177707 ], [ 24.755246805332117, 59.446003936422194 ], [ 24.755167583120659, 59.44650296086408 ], [ 24.755020159546792, 59.447361895887923 ], [ 24.753918519808987, 59.447328797801617 ], [ 24.753781340805425, 59.44831801499874 ], [ 24.753779291571817, 59.448480775522903 ], [ 24.753779291045767, 59.448480794910736 ], [ 24.754112572736293, 59.448492781942164 ] ] ] ] } },
{ "type": "Feature", "properties": { "ONIMI": "Tallinn", "OKOOD": "0784", "MNIMI": "Harju maakond", "MKOOD": "0037", "TYYP": "4", "District": "Pirita", "AKOOD": "0596", "TYYP_2": "6", "ONIMI_2": "Tallinn", "OKOOD_2": "0784", "MNIMI_2": "Harju maakond", "MKOOD_2": "0037" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 24.823493857692633, 59.46912940851297 ], [ 24.823475236740983, 59.469127505070936 ], [ 24.823453352155795, 59.46912924864445 ], [ 24.823431906395804, 59.469135289361603 ], [ 24.823256399934138, 59.469208218068829 ], [ 24.822311396473928, 59.469589562865352 ], [ 24.821774268981358, 59.469814630720514 ], [ 24.821609684547056, 59.469879894705521 ], [ 24.821595706394916, 59.469890601387355 ], [ 24.821586807952663, 59.469903475720258 ], [ 24.821586971984203, 59.469916886033289 ], [ 24.821602763854209, 59.469926563331107 ], [ 24.821622952098629, 59.469928340571109 ], [ 24.821645035152425, 59.46992235902021 ], [ 24.822052118162194, 59.469758732137457 ], [ 24.822526768659628, 59.469571315721986 ], [ 24.822939106513573, 59.46940429590812 ], [ 24.823328866277954, 59.46923749659598 ], [ 24.823519652603611, 59.469163251480239 ], [ 24.823524215036471, 59.469153483142442 ], [ 24.823522470016851, 59.469146958932477 ], [ 24.823509902032537, 59.46913638199122 ], [ 24.823493857692633, 59.46912940851297 ] ] ], [ [ [ 24.810363597405864, 59.456996232841099 ], [ 24.810403047546686, 59.457050173947991 ], [ 24.810422188130513, 59.45706982318832 ], [ 24.810548419153875, 59.457199417136437 ], [ 24.810696484603639, 59.457369227447693 ], [ 24.810865967974838, 59.457566877568254 ], [ 24.810957532312781, 59.457671908423279 ], [ 24.811129569679473, 59.457858913921235 ], [ 24.811334777079267, 59.45807329175063 ], [ 24.811498379008018, 59.45824725151919 ], [ 24.811627665733514, 59.45838634103734 ], [ 24.81165588026419, 59.458415243168581 ], [ 24.811712239847218, 59.45844934902965 ], [ 24.811768935053699, 59.458490149508648 ], [ 24.811818597006681, 59.458525876507892 ], [ 24.812038449306613, 59.458654972964567 ], [ 24.812090010704118, 59.458686756308332 ], [ 24.812370154018087, 59.458859404209655 ], [ 24.812586251034372, 59.458980210376446 ], [ 24.812727359909967, 59.459067420674685 ], [ 24.813058494830216, 59.459265354499216 ], [ 24.813287055550482, 59.459418345318248 ], [ 24.81349465941631, 59.459532496745133 ], [ 24.813700064777088, 59.459657936312304 ], [ 24.813858443442861, 59.459744374480181 ], [ 24.814071220812796, 59.459849013531027 ], [ 24.814255931702913, 59.45995101593374 ], [ 24.814535858962053, 59.46010853466732 ], [ 24.8147107927641, 59.460216036829124 ], [ 24.814751009263375, 59.460240743735042 ], [ 24.814949485881801, 59.460355550774892 ], [ 24.815017193679036, 59.46039121883328 ], [ 24.815171112782124, 59.460472296930043 ], [ 24.815304088279269, 59.460549967566337 ], [ 24.815547244922328, 59.460690090177678 ], [ 24.816026947849814, 59.460959145001375 ], [ 24.816203530262175, 59.461093915549114 ], [ 24.816334510736858, 59.461241365358951 ], [ 24.816362975492556, 59.461283415979786 ], [ 24.816364640633719, 59.461295524370833 ], [ 24.816359833443663, 59.461303211379324 ], [ 24.816340545601786, 59.461323376106755 ], [ 24.816287089179266, 59.461363142424744 ], [ 24.816174948969472, 59.461425426991568 ], [ 24.816026856409888, 59.461506498161363 ], [ 24.816019127091973, 59.461510729222752 ], [ 24.815909611274272, 59.461581767653861 ], [ 24.815890853109362, 59.461594594970762 ], [ 24.815883154450436, 59.461606752364233 ], [ 24.815883625376578, 59.461617602421406 ], [ 24.815908877226843, 59.461640345936132 ], [ 24.815946515029985, 59.461662572889878 ], [ 24.816223529896881, 59.461814648194256 ], [ 24.816609793109155, 59.462053246775106 ], [ 24.816960286261043, 59.462265692106243 ], [ 24.817289428029955, 59.462460548699191 ], [ 24.817574158263493, 59.462627905746629 ], [ 24.817770606840586, 59.462749839771234 ], [ 24.817952698691812, 59.462862867847065 ], [ 24.818318120162893, 59.463085226093369 ], [ 24.818441894724597, 59.463162403138384 ], [ 24.818649876104523, 59.463292092023096 ], [ 24.818773965927384, 59.463369114194869 ], [ 24.819038153082573, 59.463533085808827 ], [ 24.819313712904911, 59.46369901495428 ], [ 24.819637844454522, 59.463899291955009 ], [ 24.819966754844426, 59.464097168553131 ], [ 24.820183056443682, 59.464228383192683 ], [ 24.820321114537641, 59.464313162725112 ], [ 24.820625225415352, 59.46450186514992 ], [ 24.820853698632352, 59.464648335308105 ], [ 24.821040165415514, 59.46476127804835 ], [ 24.821114820697137, 59.464806504914236 ], [ 24.821366224734618, 59.464956529766589 ], [ 24.821421609390168, 59.464989578325472 ], [ 24.821630524568267, 59.465126348415041 ], [ 24.821677414291567, 59.465164045957174 ], [ 24.821769197623492, 59.465237837611632 ], [ 24.821837543499083, 59.465301505883517 ], [ 24.821905788579148, 59.465361709698122 ], [ 24.821993156005263, 59.465414684573915 ], [ 24.822098816199837, 59.465484574301875 ], [ 24.822168771859651, 59.465546194624146 ], [ 24.822213076453856, 59.4655900213141 ], [ 24.82229478920733, 59.465684988897806 ], [ 24.822307746035882, 59.465700060916802 ], [ 24.822356475279335, 59.465779641530126 ], [ 24.822438537519915, 59.465892740005096 ], [ 24.822468648654421, 59.465974097467665 ], [ 24.822494133046522, 59.4660713278708 ], [ 24.822506082961116, 59.466106092303605 ], [ 24.822509276007896, 59.46612294878399 ], [ 24.822506688423569, 59.466151717663422 ], [ 24.822489203474614, 59.466223246302576 ], [ 24.822448894916402, 59.466293346425267 ], [ 24.822399032811862, 59.466354942268914 ], [ 24.822352822827749, 59.466404136280374 ], [ 24.822288981548247, 59.466441195909091 ], [ 24.822121600689144, 59.46651120895546 ], [ 24.821977476494805, 59.466572090951651 ], [ 24.821919086291786, 59.466596755329718 ], [ 24.821909712677979, 59.466598645055505 ], [ 24.821887454858206, 59.466603137611806 ], [ 24.82183685341451, 59.466606244944749 ], [ 24.821806168512943, 59.466605053854273 ], [ 24.821787912857499, 59.466604341819028 ], [ 24.821678232286029, 59.466584450820136 ], [ 24.821467181462811, 59.466547686822054 ], [ 24.821376706917405, 59.466531338401516 ], [ 24.821337297764121, 59.466525300235702 ], [ 24.821310821791862, 59.46652015993498 ], [ 24.82129172081034, 59.466520234081578 ], [ 24.821266212015274, 59.466523768319412 ], [ 24.821044184485029, 59.466609946605274 ], [ 24.820721260956233, 59.466745981659557 ], [ 24.820466559438028, 59.466853278369896 ], [ 24.819945334562469, 59.467075259913557 ], [ 24.819907137374674, 59.467091054345545 ], [ 24.819804404979138, 59.467133543263401 ], [ 24.819393402509011, 59.467303507540997 ], [ 24.818946746577826, 59.467496977865572 ], [ 24.818871150622293, 59.467528839442615 ], [ 24.818450419722947, 59.467706168252185 ], [ 24.817978517674074, 59.467905061349455 ], [ 24.817477141687327, 59.46811309436162 ], [ 24.817318251962593, 59.468171603068036 ], [ 24.817141661483429, 59.468246738625076 ], [ 24.816888875824279, 59.468348782868432 ], [ 24.816882249755128, 59.468352737818336 ], [ 24.816884493793989, 59.468358056126725 ], [ 24.816875848771303, 59.468380102855114 ], [ 24.816962694405014, 59.468683976672928 ], [ 24.817066894143725, 59.468981737353268 ], [ 24.817128244244472, 59.469157032869418 ], [ 24.817210892486681, 59.469383408900143 ], [ 24.817257673514703, 59.469511567763604 ], [ 24.817276675026214, 59.469566944471822 ], [ 24.817338267439826, 59.469746376660602 ], [ 24.817468015920753, 59.470108162558908 ], [ 24.817487550765328, 59.470112861940571 ], [ 24.817630447076453, 59.470099919783742 ], [ 24.817641652001495, 59.470091214605262 ], [ 24.817615070156744, 59.470023568820352 ], [ 24.817601949522214, 59.470019314417591 ], [ 24.817553273794093, 59.470019822807103 ], [ 24.817458079881035, 59.469756639197286 ], [ 24.81739567510505, 59.469574249757748 ], [ 24.817309279833733, 59.469333641937887 ], [ 24.817050086882222, 59.468597212843932 ], [ 24.816985630254699, 59.468412196693897 ], [ 24.817025137626899, 59.468394276489128 ], [ 24.817404043479129, 59.468240533490061 ], [ 24.817760949180645, 59.46809487931565 ], [ 24.817967813816455, 59.468010461533915 ], [ 24.818304003964119, 59.467868399451724 ], [ 24.818687987930641, 59.467704755624929 ], [ 24.818873910936308, 59.467625520397895 ], [ 24.819375318583845, 59.467423128146855 ], [ 24.81985884435494, 59.467223358677032 ], [ 24.820281563210376, 59.467044890781608 ], [ 24.820669479195828, 59.466884160932111 ], [ 24.82091145553866, 59.466774366745852 ], [ 24.821146661385708, 59.466670619769083 ], [ 24.821240898181003, 59.466647581599489 ], [ 24.821550438188343, 59.46670166772855 ], [ 24.822237650559064, 59.466822617667837 ], [ 24.822651745635213, 59.466897440636792 ], [ 24.822825717896464, 59.466928877600729 ], [ 24.822931565406481, 59.467231623934282 ], [ 24.823009322313354, 59.467222996439588 ], [ 24.823041111112598, 59.467219467534861 ], [ 24.823059546072574, 59.467269299380412 ], [ 24.822946811251747, 59.467283073578834 ], [ 24.823076794565484, 59.467667878670788 ], [ 24.823080023511295, 59.467677427783606 ], [ 24.823099852214579, 59.467736766328741 ], [ 24.822734516506262, 59.467890916989809 ], [ 24.822469750826397, 59.468002301939833 ], [ 24.821880543882202, 59.468250164449692 ], [ 24.821596039333393, 59.468367640252232 ], [ 24.821569682953559, 59.468378522846081 ], [ 24.820462490331966, 59.468835678057125 ], [ 24.820444464892592, 59.468843124306851 ], [ 24.820424727600187, 59.468851272423159 ], [ 24.820588140773218, 59.46895752789932 ], [ 24.820614167391639, 59.468947572172461 ], [ 24.820628081142821, 59.468942261047609 ], [ 24.820807313909278, 59.468873738215713 ], [ 24.82097344024341, 59.468803779691662 ], [ 24.820992125616904, 59.468795907348344 ], [ 24.821165533157181, 59.46872287803923 ], [ 24.821194608950776, 59.468710632066887 ], [ 24.821461861116266, 59.468598085242434 ], [ 24.821738127868102, 59.468482707825899 ], [ 24.82176222831874, 59.468472638199749 ], [ 24.821785281117041, 59.468463014964456 ], [ 24.8221112656942, 59.468326868610333 ], [ 24.822652961099674, 59.468096976962606 ], [ 24.822907462714841, 59.468152498656075 ], [ 24.823172876076782, 59.468211344945573 ], [ 24.823295226164792, 59.468238471683904 ], [ 24.82330111358673, 59.468252923473578 ], [ 24.823339719984435, 59.468347710783029 ], [ 24.823391899707843, 59.468480860943572 ], [ 24.823472124827425, 59.468685569511514 ], [ 24.823474606246322, 59.468691909586212 ], [ 24.8234897525343, 59.468701150832466 ], [ 24.823914578898282, 59.468949828488093 ], [ 24.824312160708001, 59.469182570791311 ], [ 24.824987310424795, 59.469577778588665 ], [ 24.824689146667335, 59.469693587041881 ], [ 24.824423048536339, 59.469803440114354 ], [ 24.824133232881184, 59.469921771402227 ], [ 24.823138537704278, 59.470325569427736 ], [ 24.82286486645808, 59.470439272719965 ], [ 24.822617879857798, 59.470542557392079 ], [ 24.822339804876464, 59.470656673047927 ], [ 24.822089352476723, 59.470760481023696 ], [ 24.821896028288084, 59.470833860045246 ], [ 24.821758833250076, 59.470890964028413 ], [ 24.821685899855428, 59.470918582476713 ], [ 24.821640224918397, 59.470937538926549 ], [ 24.821618345089355, 59.470943196038775 ], [ 24.821564006212856, 59.470948139829879 ], [ 24.821474063654623, 59.470947784762423 ], [ 24.821239961322394, 59.470933614916447 ], [ 24.820915193433041, 59.470915422202545 ], [ 24.820336695704214, 59.470878897503411 ], [ 24.820129269393288, 59.470869209646814 ], [ 24.81996243784452, 59.47085890089496 ], [ 24.819746014709221, 59.470853613169702 ], [ 24.819549227623256, 59.470857206723885 ], [ 24.819536271456037, 59.470853947912751 ], [ 24.819479387990238, 59.470839615541401 ], [ 24.819440562449426, 59.470832361294967 ], [ 24.819372821935833, 59.4708265166025 ], [ 24.81930006503347, 59.470825550522548 ], [ 24.819233278184484, 59.470816073207651 ], [ 24.819202380193023, 59.470812037171861 ], [ 24.819177038738648, 59.470808720646978 ], [ 24.819099519258856, 59.470786455201058 ], [ 24.81901461485073, 59.470758580218714 ], [ 24.818715867186132, 59.470662301886101 ], [ 24.818631379398536, 59.470634118860794 ], [ 24.818371194757745, 59.470547322112189 ], [ 24.818390867756492, 59.470526131441318 ], [ 24.818383999932024, 59.470516335520372 ], [ 24.818264021526918, 59.47048382137806 ], [ 24.818251615473116, 59.470487309572889 ], [ 24.818182168974896, 59.470542723500259 ], [ 24.81817775388069, 59.47055061457344 ], [ 24.818232672469691, 59.470566916639108 ], [ 24.818500513946972, 59.470646394913523 ], [ 24.818836057544328, 59.470766844068088 ], [ 24.818876272411622, 59.470781280269129 ], [ 24.819000973897992, 59.470841466887904 ], [ 24.819156272409177, 59.470883572503936 ], [ 24.819229804580544, 59.47090580883814 ], [ 24.819269304859169, 59.470911039175149 ], [ 24.819342606716575, 59.470920736302624 ], [ 24.819465207290634, 59.470925692378955 ], [ 24.819626264263164, 59.470920319307709 ], [ 24.819779594789729, 59.47091643037917 ], [ 24.820155344530892, 59.470933886771661 ], [ 24.820460615342515, 59.470952669509266 ], [ 24.820984328110534, 59.470980674812864 ], [ 24.821357400744329, 59.471011448068296 ], [ 24.821529764289149, 59.471020670298159 ], [ 24.821598692071749, 59.471016443500524 ], [ 24.821665051257352, 59.471005913015581 ], [ 24.8217244000924, 59.470988505091832 ], [ 24.821834918285397, 59.470945885466044 ], [ 24.822103840029037, 59.470830239109496 ], [ 24.822610783635895, 59.470633859794361 ], [ 24.8231153251803, 59.470425895454731 ], [ 24.823754324547387, 59.470164852892559 ], [ 24.824163885496723, 59.469999668939643 ], [ 24.824623501498397, 59.469816037389684 ], [ 24.824959570899512, 59.469681050803494 ], [ 24.82510937085042, 59.469628415731471 ], [ 24.825167896825974, 59.469631137368395 ], [ 24.825193225157204, 59.469636840653429 ], [ 24.825226209087571, 59.469653474635138 ], [ 24.825307837776023, 59.46970335932496 ], [ 24.825321978882975, 59.469706008705543 ], [ 24.825351808291988, 59.469699421461712 ], [ 24.825726507411463, 59.469553553475983 ], [ 24.825873695301919, 59.469490000135679 ], [ 24.825932178125523, 59.469452308121056 ], [ 24.82599146165672, 59.469397411499969 ], [ 24.826024169553019, 59.469357439271413 ], [ 24.826053395784427, 59.469317928717452 ], [ 24.826077400256832, 59.469287885507114 ], [ 24.826280275549056, 59.469285509707525 ], [ 24.826537512807814, 59.469294579258658 ], [ 24.826731791103391, 59.469323262569837 ], [ 24.826905946076423, 59.469379774274771 ], [ 24.827172795046327, 59.469484071111587 ], [ 24.827377219541667, 59.469571030082562 ], [ 24.827652737596374, 59.469733719538965 ], [ 24.827887466623761, 59.469883764954439 ], [ 24.828134318928868, 59.470065888713265 ], [ 24.828342483930268, 59.470222410915987 ], [ 24.828545563838631, 59.47039102060878 ], [ 24.8286467257429, 59.470474254986499 ], [ 24.828770070312213, 59.470551990659914 ], [ 24.828897422669552, 59.47066034783478 ], [ 24.828976705108968, 59.470723710245821 ], [ 24.829044235315418, 59.4707876492281 ], [ 24.829100955643518, 59.470843756580848 ], [ 24.829175895644209, 59.470925126729639 ], [ 24.829255482616425, 59.470996620044183 ], [ 24.829337239015082, 59.471062273714011 ], [ 24.82945235307163, 59.471143085721785 ], [ 24.829891381315516, 59.471493209569765 ], [ 24.830224626139454, 59.471747757942566 ], [ 24.830413454423905, 59.47192369881266 ], [ 24.830665644215621, 59.472147822610296 ], [ 24.830846285643929, 59.472352522269958 ], [ 24.830923012253479, 59.4724127667197 ], [ 24.830948532406584, 59.472431295555111 ], [ 24.831038963894517, 59.472508796729912 ], [ 24.831112194950894, 59.472579054289902 ], [ 24.831229122219412, 59.472712924185174 ], [ 24.831320177901947, 59.472810924249352 ], [ 24.831452262274318, 59.472964384484186 ], [ 24.831562718926584, 59.47308745091506 ], [ 24.831706808032227, 59.47324549412231 ], [ 24.831822287314075, 59.473361024037374 ], [ 24.831884518389042, 59.473411960685553 ], [ 24.831993884100754, 59.473515482162519 ], [ 24.832067425321959, 59.473601186307341 ], [ 24.832104907880115, 59.473671785651533 ], [ 24.832234469026055, 59.473814821000559 ], [ 24.832400131691934, 59.474009361565905 ], [ 24.832447052927293, 59.474082800714513 ], [ 24.83249460520382, 59.474137258904697 ], [ 24.832549245560553, 59.474212507465545 ], [ 24.832595397880549, 59.474299964199851 ], [ 24.832653470083226, 59.474390029690305 ], [ 24.832709329160682, 59.474448672381875 ], [ 24.832773154058771, 59.474538037196304 ], [ 24.832899327787761, 59.474680823975461 ], [ 24.832935330630779, 59.474719591714717 ], [ 24.83298911382099, 59.474771209592433 ], [ 24.833088845970114, 59.474868614987628 ], [ 24.833146117230406, 59.474943846668587 ], [ 24.833240820515346, 59.475065036338172 ], [ 24.833301891897264, 59.47516436458659 ], [ 24.833361654893146, 59.47527729194762 ], [ 24.833448662182814, 59.475381653733344 ], [ 24.833514341573899, 59.475491895461175 ], [ 24.833559799176211, 59.47553845794512 ], [ 24.833621608413821, 59.475600024964962 ], [ 24.833687833813361, 59.475680576883775 ], [ 24.833728163258399, 59.475735313693654 ], [ 24.833783005775508, 59.475783235081316 ], [ 24.833849326614647, 59.47585971982538 ], [ 24.833876045361155, 59.475911966336902 ], [ 24.83389677420703, 59.475939304215181 ], [ 24.834179743965919, 59.476306729613739 ], [ 24.834222257700567, 59.476364145485697 ], [ 24.834246781262262, 59.476413165185392 ], [ 24.834268446891574, 59.476468693188195 ], [ 24.834283612276167, 59.476520312532557 ], [ 24.834335851151405, 59.476603825201167 ], [ 24.834369067930123, 59.476659109522004 ], [ 24.834412227048155, 59.476710417024769 ], [ 24.834453602448594, 59.476752992405828 ], [ 24.834483524497806, 59.476803405197955 ], [ 24.834499192519246, 59.476854635329886 ], [ 24.834526190417971, 59.476905237179743 ], [ 24.834542804395245, 59.476967727192942 ], [ 24.834574807659486, 59.477044787863626 ], [ 24.834668421048185, 59.477201764870138 ], [ 24.834761175240036, 59.477346664483285 ], [ 24.83479114695945, 59.477409886767816 ], [ 24.834835911054206, 59.477469756800261 ], [ 24.834874414232395, 59.477534065092989 ], [ 24.834914747162404, 59.47761618071781 ], [ 24.834965880454888, 59.477693317623014 ], [ 24.835012858417564, 59.477741961559389 ], [ 24.835059737727907, 59.477801710394644 ], [ 24.835107996148796, 59.477865893988778 ], [ 24.835146308369247, 59.477958408484625 ], [ 24.835175961928158, 59.478017952205235 ], [ 24.835208008164589, 59.478076484337059 ], [ 24.83523659264538, 59.478135594952413 ], [ 24.835250291995767, 59.478179853507676 ], [ 24.835267697991398, 59.478209743667179 ], [ 24.835290610357099, 59.478241843135692 ], [ 24.835322215957159, 59.478288124710431 ], [ 24.835393665627034, 59.478356102068076 ], [ 24.835437492462813, 59.478412189621537 ], [ 24.835465076621762, 59.478461405123447 ], [ 24.835502374625609, 59.478531887913043 ], [ 24.835528659456031, 59.478598652280667 ], [ 24.835564306694931, 59.478665842053672 ], [ 24.835617882098855, 59.478748187706643 ], [ 24.835641063808783, 59.478801344950831 ], [ 24.835667348528791, 59.478864482664775 ], [ 24.835694685806647, 59.478923762658596 ], [ 24.835743040592536, 59.478995369201854 ], [ 24.835855067824323, 59.479176798054162 ], [ 24.835901364431461, 59.479242151787005 ], [ 24.835937921177305, 59.479316158042508 ], [ 24.835953459647023, 59.47940878075736 ], [ 24.835966842042179, 59.479464531521351 ], [ 24.836032812988481, 59.479588899527826 ], [ 24.836135356101725, 59.4797691225317 ], [ 24.836134322839612, 59.479789605133121 ], [ 24.836145415472188, 59.479847668451335 ], [ 24.836168845071459, 59.479914477707815 ], [ 24.836191864885237, 59.47996960178336 ], [ 24.836213673457355, 59.480030020862614 ], [ 24.836229023874029, 59.480086845299503 ], [ 24.836258680913481, 59.480164378203803 ], [ 24.836270857718858, 59.480199705409767 ], [ 24.836295290233725, 59.480244057338261 ], [ 24.836321247602111, 59.480295904181403 ], [ 24.836351943804186, 59.48040933760555 ], [ 24.836382090692954, 59.480477316070377 ], [ 24.836402952373081, 59.480513476870371 ], [ 24.836463976277347, 59.480625209771915 ], [ 24.836490335660724, 59.480663920992882 ], [ 24.836544053738955, 59.480800090122962 ], [ 24.836565045204892, 59.480845128109735 ], [ 24.836575268765031, 59.480892891541465 ], [ 24.836589105130553, 59.480939743347243 ], [ 24.836618534818797, 59.481008659885333 ], [ 24.836630839997628, 59.481060709980852 ], [ 24.836646361036724, 59.48112012756021 ], [ 24.836664478009258, 59.481199771307026 ], [ 24.836685879319568, 59.481264385029654 ], [ 24.836725531437814, 59.481340687398443 ], [ 24.836762193581976, 59.481434657080456 ], [ 24.836779763189924, 59.481476853148273 ], [ 24.836894878972572, 59.481579319613175 ], [ 24.83691603912807, 59.481656376641155 ], [ 24.836942584840589, 59.481827808274957 ], [ 24.836998985081067, 59.482007703999827 ], [ 24.837025879879519, 59.482062417301265 ], [ 24.837041968621858, 59.482111840058451 ], [ 24.837074656322606, 59.482217809888098 ], [ 24.837093525507932, 59.482261155659998 ], [ 24.837108946854883, 59.482292718854154 ], [ 24.83712907721134, 59.482348624103132 ], [ 24.83714591245851, 59.482442845203543 ], [ 24.837157079711755, 59.482479390580636 ], [ 24.837171305155952, 59.482516868078243 ], [ 24.837186669677102, 59.482545388494415 ], [ 24.837209836656832, 59.482602055426675 ], [ 24.837229007914129, 59.482677832353353 ], [ 24.837241496292037, 59.482712834303541 ], [ 24.837257019783252, 59.482760052327336 ], [ 24.837272688481058, 59.482793050212145 ], [ 24.837298874103929, 59.48286915960923 ], [ 24.837318992034938, 59.482920217439855 ], [ 24.837334919959041, 59.482955143676783 ], [ 24.837346417989249, 59.482984299045199 ], [ 24.837372592118406, 59.483035479989368 ], [ 24.837421514987348, 59.483180180135832 ], [ 24.837452911077609, 59.483247818234581 ], [ 24.83747493086689, 59.483300129683002 ], [ 24.837499121164818, 59.483371844106145 ], [ 24.837510535542453, 59.483422023595601 ], [ 24.837521348243811, 59.483456318013275 ], [ 24.837531010495375, 59.48348328571192 ], [ 24.837541723977616, 59.483517850060807 ], [ 24.837550031722174, 59.483565122848354 ], [ 24.837558321681271, 59.483618859025142 ], [ 24.837562810228242, 59.483683804406958 ], [ 24.837568320212096, 59.483740035828269 ], [ 24.837571331057859, 59.483803733844603 ], [ 24.837585190498881, 59.483867955417089 ], [ 24.837597829867022, 59.483912669231515 ], [ 24.837607901861183, 59.483987229169578 ], [ 24.837617039453598, 59.484030196799715 ], [ 24.837626701106259, 59.484093071581768 ], [ 24.837636600842142, 59.484124669773529 ], [ 24.837647707747429, 59.484146493567351 ], [ 24.837661558319578, 59.484185903382013 ], [ 24.837669213456749, 59.484236052864148 ], [ 24.837688455950417, 59.484330465012448 ], [ 24.837737844562813, 59.484529220145063 ], [ 24.8377620289959, 59.48461285568267 ], [ 24.837775547164561, 59.484727834048769 ], [ 24.837784410648027, 59.484825023092135 ], [ 24.837799293859163, 59.48492802674005 ], [ 24.83780098023519, 59.485018259493287 ], [ 24.837808957590319, 59.485116728855594 ], [ 24.837828878641353, 59.485181244003101 ], [ 24.83782863844522, 59.485256982529357 ], [ 24.837825765678136, 59.485347693075752 ], [ 24.837797698029963, 59.485506975639915 ], [ 24.837794106587864, 59.485606514904212 ], [ 24.837793690460693, 59.485613286030677 ], [ 24.837783344173385, 59.485780570981618 ], [ 24.837777793632714, 59.485885140692702 ], [ 24.837758379945846, 59.485995103739015 ], [ 24.837749704974897, 59.486091748850406 ], [ 24.837749135201541, 59.486115262619315 ], [ 24.83774884975653, 59.48612735613596 ], [ 24.837747173106884, 59.486196353229744 ], [ 24.837756141412051, 59.486272643644178 ], [ 24.83775203650687, 59.486348406687277 ], [ 24.837749844582614, 59.486417263420044 ], [ 24.837743766058189, 59.486518308587222 ], [ 24.837731150818595, 59.486608308971292 ], [ 24.837730449913348, 59.486690450811167 ], [ 24.837701559098782, 59.486784647978979 ], [ 24.837659459650208, 59.486889351058473 ], [ 24.837621562690536, 59.48699486226468 ], [ 24.837593259871685, 59.487051209630813 ], [ 24.837528583015878, 59.487155068605063 ], [ 24.83748542908328, 59.487219014831787 ], [ 24.837439085147189, 59.487304723013501 ], [ 24.837411782108017, 59.487339268406643 ], [ 24.837387361700578, 59.487363983890084 ], [ 24.837344839500169, 59.487411983324563 ], [ 24.837320828719619, 59.48744044847912 ], [ 24.837303650916674, 59.487471177272731 ], [ 24.837303991812597, 59.487508796708916 ], [ 24.837291653151862, 59.487560509369736 ], [ 24.837272496630597, 59.487603387298897 ], [ 24.837253953645074, 59.487629555580085 ], [ 24.837211314403639, 59.487672807000585 ], [ 24.837185967286032, 59.487705804895747 ], [ 24.837175486920295, 59.487736877151619 ], [ 24.837170997645899, 59.487757480380893 ], [ 24.837150154876472, 59.487783438825026 ], [ 24.837122039753762, 59.487814407575613 ], [ 24.837089084787479, 59.487853136028974 ], [ 24.837063160969652, 59.487876308007017 ], [ 24.837038157117536, 59.487906763267915 ], [ 24.837015823888404, 59.487964060032439 ], [ 24.836994065835331, 59.48799085011904 ], [ 24.836954268518596, 59.488034864386194 ], [ 24.836905381662305, 59.488074573618412 ], [ 24.836853261475223, 59.488114868884281 ], [ 24.836800326700715, 59.488151461896351 ], [ 24.836771648029373, 59.488177469993182 ], [ 24.836751165719992, 59.488205194500168 ], [ 24.83674166640089, 59.488250380924704 ], [ 24.836717471244107, 59.488291522302049 ], [ 24.836662154874617, 59.488394387796646 ], [ 24.836639775394382, 59.488474306196643 ], [ 24.836619039671167, 59.488508262153559 ], [ 24.836595440007763, 59.488536993849273 ], [ 24.836529487713982, 59.48862218867535 ], [ 24.836458644830209, 59.488682863059637 ], [ 24.836346166741883, 59.488758020622107 ], [ 24.836304393360631, 59.488809336323669 ], [ 24.836264310436803, 59.488872014837142 ], [ 24.836180127725424, 59.488940807839846 ], [ 24.836089425294325, 59.48902086305786 ], [ 24.836039536948661, 59.489057256888692 ], [ 24.835982632590621, 59.489088255261201 ], [ 24.835815404133569, 59.489146281648509 ], [ 24.835765029949165, 59.489157821825678 ], [ 24.835718800503376, 59.489161310503604 ], [ 24.835617827772804, 59.489158664817218 ], [ 24.835448395817608, 59.489172162056562 ], [ 24.8352762392153, 59.489206304905835 ], [ 24.83521213115862, 59.48921417957002 ], [ 24.83511105040883, 59.489218661718475 ], [ 24.835015159187776, 59.489225319212643 ], [ 24.834885849262299, 59.489248310438882 ], [ 24.834833584312559, 59.489261944832108 ], [ 24.834817694231681, 59.489266084953471 ], [ 24.834774878348796, 59.489281409976741 ], [ 24.834747635026503, 59.489291160604765 ], [ 24.834634861164076, 59.489327772366067 ], [ 24.834554150390581, 59.48934458489812 ], [ 24.83448317600466, 59.48935823879647 ], [ 24.834447140352189, 59.489370064761722 ], [ 24.834407710539452, 59.489383842192019 ], [ 24.834370010165465, 59.489416370072199 ], [ 24.834316465548365, 59.489433270911924 ], [ 24.834270095765742, 59.489449085040675 ], [ 24.834228144329547, 59.489466576786924 ], [ 24.834203009675335, 59.489475954891695 ], [ 24.834167160676927, 59.489491092017616 ], [ 24.834129074591939, 59.48951291300552 ], [ 24.834113899371641, 59.489535468804242 ], [ 24.834107312163443, 59.489555537587819 ], [ 24.83408615819172, 59.489569630215854 ], [ 24.834040249321202, 59.489587703486343 ], [ 24.833967312072168, 59.489614233192164 ], [ 24.83391116179865, 59.489640091178806 ], [ 24.833877900670664, 59.489657851023466 ], [ 24.833804437025336, 59.489698962200499 ], [ 24.833797088930158, 59.489703075122129 ], [ 24.833787453899326, 59.489705335338328 ], [ 24.833709987762749, 59.489723518225752 ], [ 24.833660503523483, 59.489744091543571 ], [ 24.83363510895704, 59.489763794436982 ], [ 24.833616623166971, 59.489780186121465 ], [ 24.83359295025182, 59.489797391565617 ], [ 24.833569992535928, 59.489812806112639 ], [ 24.833559173441905, 59.489820064865746 ], [ 24.833521403789447, 59.48983752994922 ], [ 24.83348585887039, 59.489856471101497 ], [ 24.833442399850966, 59.489872257558474 ], [ 24.833413794373776, 59.489884808278831 ], [ 24.833397413735121, 59.489888394766865 ], [ 24.833349307558187, 59.489898923255474 ], [ 24.83330584371706, 59.489918129836525 ], [ 24.833256199693682, 59.489950248100286 ], [ 24.833244922294259, 59.489957536652291 ], [ 24.833208647602529, 59.489971904198342 ], [ 24.833199625758994, 59.489973980961878 ], [ 24.833170120275835, 59.489980765267269 ], [ 24.833097709135021, 59.489994930178717 ], [ 24.832922781699995, 59.490057543682248 ], [ 24.832907505843082, 59.49006300824724 ], [ 24.832755955234873, 59.490123041192525 ], [ 24.832586828761762, 59.490173157892748 ], [ 24.832560780558087, 59.490183475024011 ], [ 24.832515362234695, 59.490201463861531 ], [ 24.832366834575065, 59.490260292336437 ], [ 24.832264751710962, 59.490301251103986 ], [ 24.832240734318603, 59.490310891076206 ], [ 24.832140017563148, 59.490354318704547 ], [ 24.832056909692334, 59.490395301211429 ], [ 24.832012123939347, 59.490420431365216 ], [ 24.831976337834764, 59.490442569338072 ], [ 24.831958026175037, 59.490473385301172 ], [ 24.831934203538296, 59.490513455135421 ], [ 24.831918769107499, 59.490551515125034 ], [ 24.831864834581538, 59.490592358366484 ], [ 24.831846238295125, 59.490606443464046 ], [ 24.831789248502897, 59.490655914635411 ], [ 24.831755684585971, 59.490681611271199 ], [ 24.831696648806517, 59.490726813397913 ], [ 24.831593601334639, 59.490801422522466 ], [ 24.831420948377158, 59.490918526380149 ], [ 24.831365945699272, 59.490948182146091 ], [ 24.831286490871861, 59.490997920364727 ], [ 24.831211776601311, 59.491040833267149 ], [ 24.831154592466586, 59.491062836556139 ], [ 24.831047212488045, 59.491082148906848 ], [ 24.830937453235766, 59.491119007724585 ], [ 24.830846600562985, 59.491168269895155 ], [ 24.830707931191132, 59.491225571124083 ], [ 24.830605558063969, 59.49126133285931 ], [ 24.830497468983516, 59.491313908022498 ], [ 24.830425761815562, 59.491356424505611 ], [ 24.830274770525456, 59.491414467124841 ], [ 24.830127062958478, 59.491461671935284 ], [ 24.82991076950448, 59.491522306467466 ], [ 24.829635918653, 59.491584152802822 ], [ 24.829383227723898, 59.491634710111057 ], [ 24.829300841529289, 59.491643199490696 ], [ 24.82925489414032, 59.49164030164458 ], [ 24.829222488853251, 59.491638261199888 ], [ 24.829159339252545, 59.491626988459892 ], [ 24.829083587509263, 59.491604870225913 ], [ 24.829049272303688, 59.491589610042986 ], [ 24.82899877211355, 59.491567153498558 ], [ 24.828859803584081, 59.491499013787262 ], [ 24.828659452973422, 59.491406573588208 ], [ 24.828457544685865, 59.491314636592655 ], [ 24.828203708375614, 59.49120168768949 ], [ 24.828035078772736, 59.49112442449961 ], [ 24.827882038314311, 59.491056704253523 ], [ 24.827843289412119, 59.491031812627774 ], [ 24.827775546241138, 59.490988296716161 ], [ 24.827704756820491, 59.490960356562731 ], [ 24.827623514160699, 59.491005741069415 ], [ 24.826970217730128, 59.490706524180688 ], [ 24.826896264244219, 59.490738783440676 ], [ 24.827198263370587, 59.490877941056674 ], [ 24.827557189087742, 59.491049766204533 ], [ 24.827487975283344, 59.491096233145612 ], [ 24.827572213208466, 59.491123361915399 ], [ 24.827676294971187, 59.491159360788693 ], [ 24.827741723827604, 59.491194058171331 ], [ 24.827799136085325, 59.49123008950945 ], [ 24.827902698016683, 59.491282465025051 ], [ 24.828031205148012, 59.491337879570395 ], [ 24.828196947909124, 59.491416157309615 ], [ 24.828294302048054, 59.491466533783914 ], [ 24.828427654019492, 59.491527321512983 ], [ 24.82849554783397, 59.491564606294908 ], [ 24.828530765025057, 59.491594484047937 ], [ 24.828538618593036, 59.491607504832253 ], [ 24.828540217772982, 59.491645726680289 ], [ 24.828531740591497, 59.491697144781938 ], [ 24.828530355576653, 59.491705528771348 ], [ 24.828519681388972, 59.491741350138923 ], [ 24.828491730811773, 59.491791903120152 ], [ 24.828459916122338, 59.491846887939431 ], [ 24.828304838580554, 59.49202328516008 ], [ 24.82828126518589, 59.492046943251111 ], [ 24.828267755501841, 59.492060511134973 ], [ 24.828051287071911, 59.492277832432265 ], [ 24.827920605586392, 59.492397702142227 ], [ 24.827906083500302, 59.492411024991938 ], [ 24.827898703802834, 59.492418979809443 ], [ 24.827833403745633, 59.492489336323068 ], [ 24.827739730180546, 59.492600176362942 ], [ 24.827721006753503, 59.492628049897583 ], [ 24.827708564981283, 59.492660183860629 ], [ 24.827702461242609, 59.492682897371196 ], [ 24.827678691521335, 59.492725416673501 ], [ 24.82763407813562, 59.492786280536052 ], [ 24.827614524013317, 59.492805389005071 ], [ 24.827593701426192, 59.492828966861808 ], [ 24.827565947801926, 59.492846259611753 ], [ 24.827512419414507, 59.492880922445437 ], [ 24.827454183890119, 59.492927482014856 ], [ 24.82743514932141, 59.492944001899254 ], [ 24.827423635178693, 59.492953993409564 ], [ 24.827407885599008, 59.492971309564105 ], [ 24.827399761852725, 59.492996630000839 ], [ 24.827399341775447, 59.493017153425349 ], [ 24.827397361680035, 59.493041771044673 ], [ 24.827382958801291, 59.493062938738419 ], [ 24.827364788271822, 59.493091796185098 ], [ 24.827325798984347, 59.493159689330078 ], [ 24.827308525157704, 59.493201503529946 ], [ 24.827301789290463, 59.493217820576554 ], [ 24.827290988217445, 59.493265222578984 ], [ 24.827285013367565, 59.493313644585598 ], [ 24.827273195512706, 59.493351878752023 ], [ 24.827262342136294, 59.493386938084839 ], [ 24.827247458747518, 59.493463270726338 ], [ 24.827214596126442, 59.493534150554069 ], [ 24.827149701350855, 59.493643202775758 ], [ 24.827099834608354, 59.493702995228318 ], [ 24.827007131569818, 59.493798765635816 ], [ 24.826957257702897, 59.493861924342127 ], [ 24.82690383075893, 59.49391760974089 ], [ 24.826814422031163, 59.494023341456305 ], [ 24.826748333105421, 59.494081763263104 ], [ 24.826717667381711, 59.494112251891437 ], [ 24.826673121093478, 59.494151265620708 ], [ 24.826599670149562, 59.494219877177123 ], [ 24.826460646393013, 59.494321637646927 ], [ 24.826387384672731, 59.4943769623493 ], [ 24.826302518248475, 59.494428383020008 ], [ 24.826237664120981, 59.494473717740448 ], [ 24.826183914587656, 59.494508156954865 ], [ 24.826104199815951, 59.494543440996338 ], [ 24.826004301454574, 59.494590880179956 ], [ 24.825941127538734, 59.49461231718707 ], [ 24.82589751979744, 59.494630373107611 ], [ 24.825858743927196, 59.494641657248259 ], [ 24.82582674275568, 59.494644020991984 ], [ 24.825802899818072, 59.494647213370939 ], [ 24.82578087954769, 59.494655690596545 ], [ 24.825753904344193, 59.494675338939949 ], [ 24.825730416825024, 59.494712900919765 ], [ 24.825670332271223, 59.494764982909281 ], [ 24.825592500025902, 59.494841619543422 ], [ 24.825508454962016, 59.494904461893867 ], [ 24.82547647452316, 59.494937301630287 ], [ 24.825709052119237, 59.49500221949323 ], [ 24.826355781897622, 59.495180486568913 ], [ 24.826835804188136, 59.494984748449198 ], [ 24.829197692375327, 59.498144536800588 ], [ 24.832353385050485, 59.497305987094222 ], [ 24.834292179547656, 59.496964922309886 ], [ 24.841512278247073, 59.49600998725024 ], [ 24.844672762585581, 59.495598670659611 ], [ 24.849141584330635, 59.495049506958637 ], [ 24.855825848291961, 59.494194359508434 ], [ 24.856623140408495, 59.494084873982793 ], [ 24.857833775282693, 59.493992983548509 ], [ 24.866658984779871, 59.493641662248109 ], [ 24.867390505453677, 59.493613244775958 ], [ 24.874467041367389, 59.493338149078028 ], [ 24.877026837957334, 59.49313766734187 ], [ 24.879159034362278, 59.492799230149252 ], [ 24.879859456378046, 59.492654341803373 ], [ 24.884673761414668, 59.491658655955199 ], [ 24.888603127859998, 59.490833482311423 ], [ 24.889188340158722, 59.48795129320203 ], [ 24.887831808192594, 59.485213426749297 ], [ 24.889533897251066, 59.48430848913074 ], [ 24.892195959839622, 59.482898461311137 ], [ 24.894328267257784, 59.483227022535019 ], [ 24.895510848948813, 59.482575354344242 ], [ 24.897615452401688, 59.48117959125404 ], [ 24.899674401085402, 59.479845685660152 ], [ 24.900932409562717, 59.479024865548695 ], [ 24.903568391604527, 59.477309382815605 ], [ 24.905202705584877, 59.476254425420017 ], [ 24.907916869646879, 59.476319874862213 ], [ 24.908345405068033, 59.476616662354935 ], [ 24.914302705457789, 59.473681944420299 ], [ 24.914388390372171, 59.473640235682822 ], [ 24.914935590872901, 59.472847634500546 ], [ 24.914866792636484, 59.471748794644725 ], [ 24.915260912778095, 59.470709489627303 ], [ 24.915269395560635, 59.469920625391737 ], [ 24.908739185595909, 59.469181044583408 ], [ 24.907820816677404, 59.46786084909948 ], [ 24.904608491538571, 59.467256883770986 ], [ 24.904258061601936, 59.465674587166284 ], [ 24.903932207014915, 59.463400535184306 ], [ 24.904031765154745, 59.462764283199085 ], [ 24.905479065018699, 59.460563247235285 ], [ 24.904853686027426, 59.460363316954947 ], [ 24.903860676924836, 59.460119315494666 ], [ 24.902660544392887, 59.45950480964882 ], [ 24.902140731732377, 59.459247498723919 ], [ 24.901493265229689, 59.458703616485188 ], [ 24.90114049430224, 59.458313376700609 ], [ 24.900217289925632, 59.456894502609572 ], [ 24.899638868427942, 59.456418364540774 ], [ 24.899549191523722, 59.456354432353614 ], [ 24.898575174797895, 59.455821113548481 ], [ 24.897511482583631, 59.455443879847849 ], [ 24.894491825497369, 59.454604062417957 ], [ 24.893309038371505, 59.454470161604306 ], [ 24.891040204614473, 59.4545502498489 ], [ 24.887963201345261, 59.454811607142247 ], [ 24.884793745965379, 59.454956902458747 ], [ 24.884402997520411, 59.454963655452623 ], [ 24.884163348071517, 59.454949374736962 ], [ 24.883112391683376, 59.454982277119512 ], [ 24.880277294027561, 59.455000262778611 ], [ 24.877130193001975, 59.454864335430422 ], [ 24.877001683663842, 59.454858817300916 ], [ 24.876382894929517, 59.45486374281969 ], [ 24.869791675345343, 59.455145939322406 ], [ 24.868804772159319, 59.455139703763095 ], [ 24.865648897152823, 59.455038660036408 ], [ 24.864212720726091, 59.454886503874057 ], [ 24.861747186610625, 59.454784517321677 ], [ 24.856756863331128, 59.453601410228906 ], [ 24.854751080722551, 59.453103473109437 ], [ 24.853655679023866, 59.452612536510152 ], [ 24.853187074675866, 59.452310895743572 ], [ 24.852905185528439, 59.452114780119921 ], [ 24.851396209675254, 59.451680183823271 ], [ 24.849954323099851, 59.451467316122162 ], [ 24.847373295987886, 59.45143115930901 ], [ 24.844426070176308, 59.451936542229248 ], [ 24.842732511676907, 59.452233660107986 ], [ 24.841295808899716, 59.452401644082265 ], [ 24.839762660581446, 59.452247507902875 ], [ 24.836255193968167, 59.452641641878508 ], [ 24.834125392029701, 59.452934414803863 ], [ 24.832922098819576, 59.453053161710876 ], [ 24.830888702412306, 59.453100024226309 ], [ 24.828833473232002, 59.452966737138318 ], [ 24.827648798702395, 59.452974184997771 ], [ 24.823279345447293, 59.453261532795018 ], [ 24.820758535130938, 59.453695313646911 ], [ 24.817195448982893, 59.454078960785047 ], [ 24.815601110506698, 59.453701399844725 ], [ 24.816035205388157, 59.454520093065781 ], [ 24.816397984159419, 59.456750654986742 ], [ 24.814996946177537, 59.456800622137983 ], [ 24.810363597405864, 59.456996232841099 ] ] ] ] } },
{ "type": "Feature", "properties": { "ONIMI": "Tallinn", "OKOOD": "0784", "MNIMI": "Harju maakond", "MKOOD": "0037", "TYYP": "4", "District": "Haabersti", "AKOOD": "0176", "TYYP_2": "6", "ONIMI_2": "Tallinn", "OKOOD_2": "0784", "MNIMI_2": "Harju maakond", "MKOOD_2": "0037" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 24.584316430287693, 59.46131275673163 ], [ 24.58428937578503, 59.461302347261849 ], [ 24.58424958731457, 59.461306293801755 ], [ 24.584206754558068, 59.461326088363641 ], [ 24.584201340605961, 59.461332279217295 ], [ 24.584184338042672, 59.461351743822128 ], [ 24.584180498068648, 59.461356132398208 ], [ 24.584142829043515, 59.46144582196677 ], [ 24.58410263238747, 59.461576419632728 ], [ 24.584053555470845, 59.461682766243868 ], [ 24.583987715675235, 59.461738083939487 ], [ 24.583860502218737, 59.461808789949522 ], [ 24.58370861647446, 59.46186302553609 ], [ 24.583703036689563, 59.461889441140016 ], [ 24.583696106898284, 59.461906006534022 ], [ 24.583707768745963, 59.461953727773384 ], [ 24.583698058219106, 59.462018078376481 ], [ 24.58366276335731, 59.462064167414155 ], [ 24.583616107026995, 59.462079077542427 ], [ 24.58360215322271, 59.46210670614532 ], [ 24.583624516774194, 59.462147019271093 ], [ 24.583672339237896, 59.462198251743466 ], [ 24.583731498167868, 59.462231741318426 ], [ 24.583848717228918, 59.462228259797961 ], [ 24.584016200966055, 59.462209456965326 ], [ 24.584096782871075, 59.462199360331972 ], [ 24.584175694707245, 59.462167933930147 ], [ 24.58418739680538, 59.462133448230361 ], [ 24.584248006411872, 59.462090918206272 ], [ 24.584330445760024, 59.462057474311599 ], [ 24.58437295204115, 59.462011344136123 ], [ 24.584372179757807, 59.461967255026167 ], [ 24.584382109159801, 59.461915766710952 ], [ 24.584446175840394, 59.461864029329199 ], [ 24.584470830652936, 59.461829010398098 ], [ 24.584481154892515, 59.461799566589967 ], [ 24.584499774505481, 59.461763299757756 ], [ 24.584450073929922, 59.46172189618823 ], [ 24.584372657058097, 59.461691189820236 ], [ 24.584316856543879, 59.461670495093237 ], [ 24.584292119859633, 59.461651269406005 ], [ 24.584287272373317, 59.461618427953127 ], [ 24.584290350064027, 59.461602472053002 ], [ 24.584296116164253, 59.461572545783476 ], [ 24.584322644936815, 59.461434755652782 ], [ 24.584323568581631, 59.461412471896203 ], [ 24.584326877607026, 59.461331722624337 ], [ 24.584316430287693, 59.46131275673163 ] ] ], [ [ [ 24.574477940998317, 59.464146648954959 ], [ 24.574608372382851, 59.464115623154278 ], [ 24.574777908262135, 59.464067021296117 ], [ 24.574889561866545, 59.464024829585846 ], [ 24.574967600166122, 59.463970109657247 ], [ 24.575064320127154, 59.463899958390002 ], [ 24.575127052014928, 59.463856920711038 ], [ 24.575193969821925, 59.463819484005235 ], [ 24.575263717942899, 59.463780464026392 ], [ 24.575427884150621, 59.463687585840781 ], [ 24.575641225117757, 59.463556297747445 ], [ 24.575802254374373, 59.463449043485817 ], [ 24.575956375339441, 59.463331702719138 ], [ 24.576105866286067, 59.463234776563219 ], [ 24.576262243464463, 59.463124795246713 ], [ 24.576283131129998, 59.463105314712379 ], [ 24.576401841512681, 59.462994609256185 ], [ 24.576465407269836, 59.462924907115287 ], [ 24.576499969252456, 59.462874478656957 ], [ 24.576517288302757, 59.462836899154709 ], [ 24.576519616068985, 59.46280137801201 ], [ 24.576509097570458, 59.462772744169655 ], [ 24.576492549810801, 59.462758867107453 ], [ 24.576438904550237, 59.462746561625181 ], [ 24.576396221523957, 59.462742170313874 ], [ 24.576366475704081, 59.462744634298019 ], [ 24.576318578209019, 59.462757303128619 ], [ 24.57628278457355, 59.462772387544973 ], [ 24.576170401514283, 59.462841028383501 ], [ 24.57605025476369, 59.462925151581402 ], [ 24.575864002823476, 59.463041456975013 ], [ 24.57571126869092, 59.463148944395662 ], [ 24.575527652736337, 59.463244950936506 ], [ 24.575386680351059, 59.463315825403171 ], [ 24.575344474623094, 59.463362813025427 ], [ 24.575336874830786, 59.463371275116643 ], [ 24.575249236960918, 59.463444403061004 ], [ 24.575234164703758, 59.463454235258098 ], [ 24.575153429555982, 59.463506857660462 ], [ 24.575121735839065, 59.463525649096034 ], [ 24.575075426236129, 59.463553103768326 ], [ 24.574947692806525, 59.4636038485746 ], [ 24.574784342112999, 59.463684460738129 ], [ 24.574693484790483, 59.463731992451272 ], [ 24.574667345679675, 59.463752851070318 ], [ 24.574620405690457, 59.463788997554417 ], [ 24.574591172674968, 59.463811512342446 ], [ 24.574547067967345, 59.46384112951614 ], [ 24.574481303330398, 59.463885302195784 ], [ 24.574399429481101, 59.463937462304756 ], [ 24.574311818890767, 59.463979916862264 ], [ 24.574199732230575, 59.46402051215577 ], [ 24.574095846405324, 59.464041179761871 ], [ 24.57394849561868, 59.464065582292143 ], [ 24.573762485938563, 59.464082711472777 ], [ 24.573590607196941, 59.464092732356598 ], [ 24.573446356634069, 59.464109589568345 ], [ 24.573355637782338, 59.464116187019634 ], [ 24.573299867720991, 59.46412024463941 ], [ 24.573243516276136, 59.464124340670267 ], [ 24.573078841420443, 59.464126457183625 ], [ 24.572919278814062, 59.464122124112805 ], [ 24.572655285282835, 59.464137571012017 ], [ 24.572575485943247, 59.464150628418992 ], [ 24.572485822190863, 59.46417178052333 ], [ 24.572448491541003, 59.46418058702254 ], [ 24.572411050849105, 59.464223854662954 ], [ 24.572377480088058, 59.464284403144028 ], [ 24.572378763501082, 59.464359898667709 ], [ 24.572335611949097, 59.464438109555765 ], [ 24.57220864378095, 59.464531791767044 ], [ 24.572125664024263, 59.464595086052306 ], [ 24.572053294345466, 59.464701107105199 ], [ 24.572138478537457, 59.464766166811998 ], [ 24.57229691830323, 59.464778045949693 ], [ 24.572436153816994, 59.46477534973765 ], [ 24.572755875582484, 59.464729859244429 ], [ 24.573084289501207, 59.464676888680088 ], [ 24.573192073427155, 59.46465713848692 ], [ 24.57347003723099, 59.464610068470641 ], [ 24.57352104020497, 59.46460251275154 ], [ 24.573608865878523, 59.464589500631909 ], [ 24.573712306447245, 59.464556914606682 ], [ 24.573780132665092, 59.464521934240743 ], [ 24.573815776567599, 59.464488323741108 ], [ 24.573826496658061, 59.464457703201361 ], [ 24.573826222462458, 59.464427749907379 ], [ 24.573790692629373, 59.464381047416914 ], [ 24.573763043408157, 59.464329957018329 ], [ 24.573751890303107, 59.464297034990295 ], [ 24.573750591883766, 59.464273297872651 ], [ 24.57376408530271, 59.464255528547142 ], [ 24.573794830190675, 59.464244210006321 ], [ 24.573931817226498, 59.46422645049482 ], [ 24.574133901069203, 59.464205498888042 ], [ 24.574299989698936, 59.464179416688822 ], [ 24.574353657955101, 59.464170987286529 ], [ 24.574477940998317, 59.464146648954959 ] ] ], [ [ [ 24.550347602488511, 59.438775653725706 ], [ 24.550365862772985, 59.438780692188317 ], [ 24.550375774870517, 59.43878991454897 ], [ 24.55038911601466, 59.438858834500074 ], [ 24.550419347432452, 59.438886598246484 ], [ 24.550469939398951, 59.438902643288557 ], [ 24.550527933939179, 59.438913253490384 ], [ 24.550556035062289, 59.438914895413006 ], [ 24.550643208925283, 59.43889125477908 ], [ 24.550712995372027, 59.438887749359822 ], [ 24.550776664388259, 59.43889177387382 ], [ 24.550857356703734, 59.438901184884941 ], [ 24.551001955986813, 59.438908604976689 ], [ 24.551055175625702, 59.438917089533007 ], [ 24.551140257083407, 59.438933097712919 ], [ 24.55118689122504, 59.4389364751977 ], [ 24.551317063803008, 59.438919207012219 ], [ 24.551343093562956, 59.438916988551057 ], [ 24.551384596257098, 59.438924552562739 ], [ 24.551418575200607, 59.438936582474732 ], [ 24.551461492317024, 59.438952066822459 ], [ 24.551506864102748, 59.438962397320374 ], [ 24.551568397916299, 59.438965146749567 ], [ 24.551673873001956, 59.438975853800322 ], [ 24.551742821541595, 59.438979388951481 ], [ 24.551771360532502, 59.4389874021225 ], [ 24.551812925440874, 59.439000988995879 ], [ 24.55185434042534, 59.439011838640475 ], [ 24.551882615675183, 59.439026432694853 ], [ 24.551907179686385, 59.439039300848421 ], [ 24.551937141338342, 59.43904809795675 ], [ 24.552008391653423, 59.43905337374725 ], [ 24.552055711878165, 59.439056658284059 ], [ 24.552136381187626, 59.439050422504899 ], [ 24.55216363316304, 59.439037678255033 ], [ 24.552192789583593, 59.439021730366335 ], [ 24.552221443878683, 59.439013971179328 ], [ 24.55223542979591, 59.439014603724807 ], [ 24.552242579606762, 59.439019717298898 ], [ 24.552244745072123, 59.439030480063813 ], [ 24.552234809964784, 59.439044947022794 ], [ 24.552210183708116, 59.439056585139632 ], [ 24.552170236464264, 59.439071133054618 ], [ 24.552119372065974, 59.439093913330566 ], [ 24.552096107116252, 59.439117816645215 ], [ 24.552091792701439, 59.439147475259681 ], [ 24.552090837970102, 59.439163646006243 ], [ 24.552091663058704, 59.43917871415357 ], [ 24.552105082234082, 59.439196619947047 ], [ 24.552127520777741, 59.43921209118961 ], [ 24.552160162663043, 59.439217690339611 ], [ 24.552182000576057, 59.439219986534418 ], [ 24.552200664427701, 59.439218902903754 ], [ 24.552215673533663, 59.439213050101252 ], [ 24.552230557601469, 59.439206003941862 ], [ 24.552253961685544, 59.439195071117489 ], [ 24.55228245694121, 59.439189457977704 ], [ 24.552323884876081, 59.43918473312835 ], [ 24.552354683191201, 59.439180950503861 ], [ 24.552395333072813, 59.439173886018956 ], [ 24.552419169277389, 59.439166793310385 ], [ 24.55244304429743, 59.439155562250626 ], [ 24.552462855785038, 59.439138262140801 ], [ 24.552478788968251, 59.439109999992198 ], [ 24.552497477796113, 59.439076681465828 ], [ 24.552527797476696, 59.439049849057888 ], [ 24.552558540412658, 59.439032844176204 ], [ 24.552588003220261, 59.439028223305236 ], [ 24.552654989663463, 59.439039010279295 ], [ 24.552694905931006, 59.439060691655598 ], [ 24.552725598649346, 59.439090697077596 ], [ 24.552755813127437, 59.439152140336731 ], [ 24.552772534501386, 59.43922053420664 ], [ 24.552798009979984, 59.439255938456029 ], [ 24.552829568268926, 59.439275159397241 ], [ 24.552854388198771, 59.43928638359435 ], [ 24.552992417732575, 59.439318963394598 ], [ 24.553126643688, 59.439353901902848 ], [ 24.553299760406311, 59.439379781134029 ], [ 24.553366423075239, 59.439377050442502 ], [ 24.553484584347402, 59.439364606108761 ], [ 24.553555691685595, 59.43935012430515 ], [ 24.55362093795279, 59.439344823171574 ], [ 24.553710772233135, 59.439330980679422 ], [ 24.553778645361522, 59.439319214291139 ], [ 24.553838777404962, 59.439325675867714 ], [ 24.553888669989625, 59.43933648922232 ], [ 24.553938795528605, 59.439356233234363 ], [ 24.553974584557771, 59.439366773747743 ], [ 24.554014826411084, 59.439371505651465 ], [ 24.554047476994654, 59.439367831534042 ], [ 24.554079791299049, 59.439361896735505 ], [ 24.554105649891827, 59.439345854579457 ], [ 24.554122972883292, 59.439312083762701 ], [ 24.554117617748211, 59.439289422580842 ], [ 24.554125201022835, 59.439274516547073 ], [ 24.554138287817704, 59.43926208283294 ], [ 24.554157446582959, 59.439250224995014 ], [ 24.554180180535834, 59.439244734428883 ], [ 24.554217042953532, 59.439243223843626 ], [ 24.554254468279293, 59.439240561881867 ], [ 24.554307713445102, 59.439225553416549 ], [ 24.554402711586473, 59.439196545291033 ], [ 24.554442483240461, 59.439176656394139 ], [ 24.554476797081055, 59.439163630588197 ], [ 24.554526091166149, 59.439143001401597 ], [ 24.554538987473574, 59.439141645498971 ], [ 24.554560402248146, 59.439141752805902 ], [ 24.554592126635601, 59.439148351614286 ], [ 24.554626072678886, 59.439157185193537 ], [ 24.55466703411463, 59.439174768272849 ], [ 24.554699996987207, 59.439199027670256 ], [ 24.554742053249313, 59.439224334915906 ], [ 24.554801029406867, 59.439246438055442 ], [ 24.554856826807136, 59.439275735793899 ], [ 24.554924968828633, 59.439304424946101 ], [ 24.554973992949105, 59.439330106700858 ], [ 24.55502172034722, 59.439356242729488 ], [ 24.555043696965459, 59.439361634772595 ], [ 24.555061868891865, 59.439363901059174 ], [ 24.555090409033312, 59.439367577840663 ], [ 24.555135001557623, 59.439360217617931 ], [ 24.555147172558609, 59.439350013844972 ], [ 24.555141753761447, 59.439331069258223 ], [ 24.555115170403205, 59.439310068493477 ], [ 24.555107343582566, 59.439291520046758 ], [ 24.555115202920277, 59.439273022174362 ], [ 24.555159325368475, 59.439255179324761 ], [ 24.55528917696148, 59.439230134880681 ], [ 24.555340210416539, 59.439216787005201 ], [ 24.555367484578564, 59.439211977257358 ], [ 24.555376979729044, 59.439221479277343 ], [ 24.555362096367187, 59.439240455614765 ], [ 24.55533897188101, 59.439302141177194 ], [ 24.55536738463967, 59.439342433744592 ], [ 24.555410755674213, 59.439375939791404 ], [ 24.55547685933092, 59.439403874250537 ], [ 24.55557105021256, 59.439446636128054 ], [ 24.555613712657866, 59.439482066076998 ], [ 24.555656919102731, 59.439511928261773 ], [ 24.555704759244595, 59.439518943538026 ], [ 24.555804588122719, 59.439509563665631 ], [ 24.555850439106326, 59.439503723901815 ], [ 24.555882397751148, 59.439501937323371 ], [ 24.555925097686391, 59.439504988632692 ], [ 24.555960434831967, 59.439512622117462 ], [ 24.555997457356145, 59.439531765378774 ], [ 24.556015528612647, 59.439545162869486 ], [ 24.556026677777623, 59.439568562520293 ], [ 24.556045545396394, 59.439589739304971 ], [ 24.556068236702945, 59.439597874981636 ], [ 24.556113632677452, 59.439598724570537 ], [ 24.556152327069, 59.4395928072042 ], [ 24.556183277738587, 59.439582147011315 ], [ 24.556209550102899, 59.439567754376029 ], [ 24.556219770533239, 59.439560081149928 ], [ 24.556229677692702, 59.439552651612708 ], [ 24.556258619036011, 59.43954523144307 ], [ 24.556287018435558, 59.439545659052875 ], [ 24.556338006081294, 59.439569536700937 ], [ 24.556391803964598, 59.439602558527888 ], [ 24.556397760043197, 59.439606833126895 ], [ 24.556420956756732, 59.439611367014976 ], [ 24.556439746985138, 59.439608298434607 ], [ 24.556454413999383, 59.439597706979498 ], [ 24.556472249731982, 59.439579319472323 ], [ 24.556499046428652, 59.439562500897161 ], [ 24.556529972723602, 59.439562253512641 ], [ 24.556569912920427, 59.439575477720268 ], [ 24.556583101509403, 59.439616938482978 ], [ 24.556588917826069, 59.439650495140135 ], [ 24.556605034386354, 59.439674789355763 ], [ 24.556644321451042, 59.43968685832764 ], [ 24.556711502467447, 59.439685631812473 ], [ 24.556806590850236, 59.439664368328344 ], [ 24.55686393162101, 59.439650885478635 ], [ 24.556890083120827, 59.439639904303704 ], [ 24.556915763901873, 59.439619508722352 ], [ 24.556954620703703, 59.439561939414091 ], [ 24.556983614807216, 59.43955127832993 ], [ 24.557016331183725, 59.439552728803015 ], [ 24.557030471894326, 59.43956391659858 ], [ 24.557033643568673, 59.439595222336337 ], [ 24.557062783393931, 59.439702664972117 ], [ 24.557055880401368, 59.439759219449932 ], [ 24.557050458176334, 59.439823361815989 ], [ 24.557067904187004, 59.439853503056064 ], [ 24.557106790066541, 59.439874451386309 ], [ 24.557260382807566, 59.439930461478198 ], [ 24.557560282216379, 59.439973913251841 ], [ 24.557744056959518, 59.439992623141691 ], [ 24.557855270581346, 59.439988552498754 ], [ 24.557910489973764, 59.439988695594465 ], [ 24.557940265653038, 59.439990274992681 ], [ 24.557966866523689, 59.439997101158717 ], [ 24.557984824086109, 59.440011028478693 ], [ 24.558050458500066, 59.440087715235009 ], [ 24.558152742703253, 59.440153043910648 ], [ 24.558246921340665, 59.440232473104935 ], [ 24.558351008027039, 59.440306671784207 ], [ 24.558379014943455, 59.4403240213882 ], [ 24.558398477120928, 59.440348031903298 ], [ 24.558412809602743, 59.440373060562202 ], [ 24.558431908546051, 59.440384550338777 ], [ 24.558473281591098, 59.440388288669944 ], [ 24.558499012086074, 59.440377425659214 ], [ 24.558526432220603, 59.440353485634787 ], [ 24.558552683153383, 59.44034211771978 ], [ 24.558667781931852, 59.440344923901065 ], [ 24.558774628840695, 59.440360439869366 ], [ 24.558864955833794, 59.440397147656192 ], [ 24.558954038068368, 59.440466562201671 ], [ 24.558989579135329, 59.44050378966628 ], [ 24.558986923305159, 59.440524590606437 ], [ 24.558999488217907, 59.440543890695068 ], [ 24.559052538971784, 59.440562237967896 ], [ 24.559092877100692, 59.440573799065149 ], [ 24.559119050586354, 59.440587852929063 ], [ 24.559156056971304, 59.440610137164384 ], [ 24.559210260535256, 59.440635373462293 ], [ 24.55926267342619, 59.440653498945856 ], [ 24.559461475359821, 59.440686585892223 ], [ 24.559494118218609, 59.440681321614342 ], [ 24.559535508561449, 59.44066348891333 ], [ 24.559550679514174, 59.440642805342996 ], [ 24.559532179825244, 59.440613054898066 ], [ 24.559505818391717, 59.440585384501851 ], [ 24.559488610559036, 59.440564560220139 ], [ 24.559485484255621, 59.440551234361955 ], [ 24.559494868086496, 59.440538779940617 ], [ 24.559526531898008, 59.440538313285955 ], [ 24.559590010584362, 59.440558438276895 ], [ 24.559703514575517, 59.440590792128795 ], [ 24.559812224509756, 59.440618821615942 ], [ 24.5598890709974, 59.440669544526791 ], [ 24.559886082432644, 59.440727007092732 ], [ 24.559892442217471, 59.440745121643786 ], [ 24.559923080329348, 59.440763788263155 ], [ 24.559999530501504, 59.44078600330112 ], [ 24.560031037676694, 59.440789944673611 ], [ 24.560058033175434, 59.440787451092348 ], [ 24.560061187397583, 59.440779888407995 ], [ 24.560056144233396, 59.440764685641625 ], [ 24.560044638441276, 59.440746601958217 ], [ 24.560061992497026, 59.440738691638927 ], [ 24.560088192728877, 59.44073284420692 ], [ 24.560142149887096, 59.440733323872117 ], [ 24.560179006427113, 59.440751793429691 ], [ 24.56019910376342, 59.44078220125305 ], [ 24.560211242350473, 59.440808774038878 ], [ 24.560246580896198, 59.440831397196469 ], [ 24.560287287069048, 59.440832040890584 ], [ 24.560344707245335, 59.440831958171607 ], [ 24.560386530253705, 59.440852336496405 ], [ 24.56041516914993, 59.440876962955329 ], [ 24.560493878552553, 59.440910083567523 ], [ 24.560553493820223, 59.440924542388494 ], [ 24.560600709993743, 59.440947052060025 ], [ 24.560735806169898, 59.440981521342955 ], [ 24.560783372142399, 59.441013517663443 ], [ 24.560828593814417, 59.441050864991382 ], [ 24.560864915454456, 59.441061401373766 ], [ 24.560930337360908, 59.441069883849906 ], [ 24.56100463806786, 59.441072745064304 ], [ 24.561056469665573, 59.441071474009519 ], [ 24.561154005759864, 59.441074684978773 ], [ 24.561241030981261, 59.441093595756023 ], [ 24.56128660640708, 59.44111060057913 ], [ 24.5613388856599, 59.441119408160993 ], [ 24.56137010887355, 59.441117838828802 ], [ 24.561398284642344, 59.441109936065182 ], [ 24.561461119975448, 59.441113303699417 ], [ 24.561505220457224, 59.441130637890758 ], [ 24.561514425460224, 59.441150230339687 ], [ 24.561522026207516, 59.44118266608136 ], [ 24.561551608353213, 59.44121528637077 ], [ 24.561578032170651, 59.441224105358266 ], [ 24.561607884877517, 59.441231365753652 ], [ 24.561651220653665, 59.441231118113819 ], [ 24.561674610581882, 59.441234429447789 ], [ 24.561696556401888, 59.441276399921314 ], [ 24.561731242919215, 59.44129786748978 ], [ 24.56178147295963, 59.441310651268346 ], [ 24.561797214681885, 59.441330099118765 ], [ 24.561796932300485, 59.441348295771135 ], [ 24.561791325897818, 59.441361057397415 ], [ 24.561794980152523, 59.44138177761802 ], [ 24.561816544912222, 59.441389845302595 ], [ 24.561854298708724, 59.441389899681432 ], [ 24.561899642507875, 59.441383135402262 ], [ 24.561946054644626, 59.441377030812951 ], [ 24.562012492998711, 59.441390463467648 ], [ 24.562058771790195, 59.441413748601846 ], [ 24.562136019082462, 59.441485303047628 ], [ 24.562195760129857, 59.441538359716148 ], [ 24.562245754526959, 59.4415785586787 ], [ 24.562263990416266, 59.441596389024333 ], [ 24.562307045426181, 59.441606896264567 ], [ 24.562363499851291, 59.4416133786453 ], [ 24.562456670130135, 59.441612522972328 ], [ 24.562513865736015, 59.441595510516954 ], [ 24.562556126874689, 59.441564226293828 ], [ 24.562592517687786, 59.441518963729344 ], [ 24.562643641523309, 59.441458629436234 ], [ 24.562695278739916, 59.441420913811839 ], [ 24.56272965472111, 59.441382823094052 ], [ 24.562747547699701, 59.44135307916978 ], [ 24.562762528939466, 59.44132632789654 ], [ 24.562771871885207, 59.441287913256993 ], [ 24.562754395467493, 59.441248787376352 ], [ 24.562726781421862, 59.441221230689457 ], [ 24.56267933090972, 59.441189952694522 ], [ 24.562665813921541, 59.441170773714617 ], [ 24.562681113259096, 59.441144074948866 ], [ 24.562734199725362, 59.441126963251918 ], [ 24.56281517799858, 59.441112550975262 ], [ 24.562889610642898, 59.441094522148674 ], [ 24.56297311978097, 59.441072199599333 ], [ 24.562983349750102, 59.441067308535828 ], [ 24.563006415666724, 59.441039194141695 ], [ 24.563015665710299, 59.441007997016925 ], [ 24.563030826572728, 59.440986819373414 ], [ 24.56306095071384, 59.440967768031797 ], [ 24.563091206500193, 59.440965259889602 ], [ 24.563128013068322, 59.440973100614414 ], [ 24.563152344206127, 59.440990743201425 ], [ 24.563161051054404, 59.441018551207449 ], [ 24.563158416685898, 59.441049055789577 ], [ 24.563175533838933, 59.441059951929553 ], [ 24.563189663813699, 59.441074604059942 ], [ 24.563212493723576, 59.441085637343051 ], [ 24.563250491595042, 59.441094101297374 ], [ 24.563415245130052, 59.441116286762551 ], [ 24.56348338790486, 59.441122987954458 ], [ 24.563639742032329, 59.441150963041181 ], [ 24.563699858575987, 59.441155104250356 ], [ 24.563725115723337, 59.441156207975688 ], [ 24.563784025665523, 59.441142769262548 ], [ 24.563801402793448, 59.441144840266603 ], [ 24.563823758144746, 59.441155938311901 ], [ 24.563844564606775, 59.441177841766496 ], [ 24.563850490313087, 59.441202627562696 ], [ 24.56384368319878, 59.44121947874833 ], [ 24.563825429793706, 59.441230526230939 ], [ 24.563790029176783, 59.441234214539314 ], [ 24.5637351458589, 59.441235338154186 ], [ 24.563699462499518, 59.441232573513922 ], [ 24.563666000626089, 59.441224512172326 ], [ 24.563627567388234, 59.441220655167086 ], [ 24.563599421375702, 59.441223926385526 ], [ 24.56356941254834, 59.44123607439375 ], [ 24.563546379883739, 59.441251226615542 ], [ 24.563528275313562, 59.441272372030127 ], [ 24.563525785403876, 59.441290246004193 ], [ 24.563567494302713, 59.441369797182659 ], [ 24.563608123686084, 59.441394587081142 ], [ 24.563683422337, 59.441409803216217 ], [ 24.563732210754303, 59.441408867290761 ], [ 24.563826699770818, 59.441389701830722 ], [ 24.563974146374196, 59.44135845152185 ], [ 24.564053183634108, 59.441339630338121 ], [ 24.564086603205556, 59.441323769275428 ], [ 24.564122294674682, 59.441303535872578 ], [ 24.564170537298953, 59.441270627674072 ], [ 24.56420260641951, 59.4412477078262 ], [ 24.56424122961765, 59.441234248097338 ], [ 24.564304066801409, 59.441230226673724 ], [ 24.564359968379861, 59.441241665619579 ], [ 24.564436660117078, 59.441266525139909 ], [ 24.564472772359409, 59.441283506588 ], [ 24.56448328690221, 59.441306513286122 ], [ 24.56448199824078, 59.441335189890033 ], [ 24.564458135396521, 59.441357679666432 ], [ 24.564424628467439, 59.441368245037523 ], [ 24.56436243131165, 59.441381150552679 ], [ 24.564313325246005, 59.441395929902612 ], [ 24.564281011202802, 59.441424299586927 ], [ 24.564274056096934, 59.441446052620606 ], [ 24.564281218036935, 59.441469899539229 ], [ 24.564310809278982, 59.441494359501142 ], [ 24.564345876743058, 59.441513167698162 ], [ 24.56438167167785, 59.441516263818414 ], [ 24.564420739639576, 59.441526509183291 ], [ 24.564448244291547, 59.441537746632555 ], [ 24.564468353571023, 59.441554751783627 ], [ 24.564476033208184, 59.441569386539591 ], [ 24.564464417392173, 59.441581069924354 ], [ 24.564434725941044, 59.441586825469386 ], [ 24.564392654949526, 59.441586844189573 ], [ 24.564356137083802, 59.441584801425378 ], [ 24.564219625191903, 59.4415556919229 ], [ 24.564155987387434, 59.441561431252254 ], [ 24.564114873222579, 59.441577765139243 ], [ 24.564089917607102, 59.441590672578549 ], [ 24.564074698931893, 59.441599786115425 ], [ 24.564060758386159, 59.441611587139121 ], [ 24.564058576031769, 59.441628858376433 ], [ 24.564070263827947, 59.441666357194165 ], [ 24.564099362448381, 59.441732497395591 ], [ 24.564122273008291, 59.441761178056979 ], [ 24.564149890758504, 59.441772854933987 ], [ 24.564422001107349, 59.441797011882898 ], [ 24.564537942010443, 59.441792852526262 ], [ 24.564563252592151, 59.441790760214666 ], [ 24.564591936249091, 59.441780538644089 ], [ 24.564636374502562, 59.44174671303054 ], [ 24.564658236790674, 59.441723675257535 ], [ 24.56467492213125, 59.44171372952291 ], [ 24.564715397441748, 59.441712065871428 ], [ 24.564776951742772, 59.441713552347899 ], [ 24.564827760350543, 59.441707921599956 ], [ 24.564974714035451, 59.441702255438166 ], [ 24.564991307288, 59.441699536175037 ], [ 24.565006505422456, 59.441696652340653 ], [ 24.565071841655989, 59.441645475029134 ], [ 24.56510772315206, 59.441631416407191 ], [ 24.565154880502629, 59.441626689876784 ], [ 24.565202053027939, 59.441630338379312 ], [ 24.565260294054326, 59.4416446304228 ], [ 24.56529528328435, 59.441663995250316 ], [ 24.565314422243937, 59.441723517347356 ], [ 24.565320586992144, 59.441773328653134 ], [ 24.56533412176201, 59.441778566715826 ], [ 24.565389724515143, 59.441798498319933 ], [ 24.565437856930476, 59.44180368658354 ], [ 24.565453162345083, 59.44181580204063 ], [ 24.565453940631535, 59.441833006732054 ], [ 24.565441815949292, 59.44185439603055 ], [ 24.56541533199211, 59.441877175503045 ], [ 24.565385762461108, 59.441891368688104 ], [ 24.565338578336593, 59.441894497589622 ], [ 24.565297952126787, 59.441884923448036 ], [ 24.565287124396928, 59.441878183656613 ], [ 24.565295937400929, 59.441861162210756 ], [ 24.565304556274466, 59.441838800554478 ], [ 24.56529069834923, 59.441824686075002 ], [ 24.565246116924772, 59.441818934948223 ], [ 24.565184687303393, 59.441824997427553 ], [ 24.565137053301896, 59.441841413470932 ], [ 24.565102465790407, 59.44186445206568 ], [ 24.565095458100227, 59.441896851545714 ], [ 24.565083828798301, 59.441923696421796 ], [ 24.565055115142428, 59.441938496264257 ], [ 24.565009641390827, 59.441957658789917 ], [ 24.56498972232184, 59.441977761930673 ], [ 24.564984871310809, 59.442006292237949 ], [ 24.564972152872578, 59.442036561837099 ], [ 24.564976931509957, 59.442047349094061 ], [ 24.564985774570996, 59.442055578553841 ], [ 24.565006336948429, 59.442066908503762 ], [ 24.565020048834935, 59.442084982293132 ], [ 24.565025073917067, 59.442106405692726 ], [ 24.565049163832811, 59.442121077728757 ], [ 24.565111803365408, 59.442148699084633 ], [ 24.565148705251893, 59.442169653566665 ], [ 24.565149588471293, 59.4421953406382 ], [ 24.565124704560805, 59.442213651840348 ], [ 24.565052240881197, 59.442228253407158 ], [ 24.565017532817006, 59.442236570964056 ], [ 24.564983978989403, 59.442243276759406 ], [ 24.5649534694378, 59.442250750446604 ], [ 24.564934647124435, 59.442265777125641 ], [ 24.564934200742481, 59.442285680031389 ], [ 24.564960203602961, 59.442303045816388 ], [ 24.565020715626559, 59.442320308436358 ], [ 24.565062127686318, 59.442341135874202 ], [ 24.565101793248008, 59.442387437238708 ], [ 24.56515063266308, 59.442427855497797 ], [ 24.565198407326953, 59.442451851238836 ], [ 24.565350508635653, 59.442573853870478 ], [ 24.565440307870926, 59.4426172918416 ], [ 24.56552387931395, 59.442643324052121 ], [ 24.565557204871048, 59.442665397889627 ], [ 24.565580335339003, 59.442687021790015 ], [ 24.565589225708845, 59.442706606328585 ], [ 24.565596048196952, 59.442727977101704 ], [ 24.565616696786098, 59.442744477072694 ], [ 24.56566356605417, 59.442757390506799 ], [ 24.565715329527137, 59.442780596946726 ], [ 24.5657339159463, 59.442801351660393 ], [ 24.565740848890275, 59.442815549695908 ], [ 24.565723553369828, 59.442834429838264 ], [ 24.565681839305867, 59.442867804251854 ], [ 24.565636474456202, 59.442872307826448 ], [ 24.5655157948209, 59.442885357301222 ], [ 24.565492860956244, 59.442891586748246 ], [ 24.565479126370985, 59.442903054899652 ], [ 24.565488452896847, 59.442922368274566 ], [ 24.565517801914915, 59.442931003361913 ], [ 24.565579871730446, 59.442931553698429 ], [ 24.565675801275276, 59.442924678609813 ], [ 24.565769604048739, 59.442930936287922 ], [ 24.565833066950979, 59.442957063658397 ], [ 24.565900220332253, 59.44296762777536 ], [ 24.565952505494174, 59.442969099678535 ], [ 24.566037042786657, 59.442967183447138 ], [ 24.566095411955111, 59.442972031254691 ], [ 24.566187867763432, 59.442986337404761 ], [ 24.566260259662741, 59.443002426320589 ], [ 24.566297785084409, 59.443011986570852 ], [ 24.566343459497304, 59.443044222415367 ], [ 24.566347972435118, 59.443066590499498 ], [ 24.566344575111266, 59.443116828846946 ], [ 24.566381721337091, 59.44316304191446 ], [ 24.566425720339868, 59.443195204145688 ], [ 24.56651074593703, 59.443220574192232 ], [ 24.566565747412209, 59.443234888574274 ], [ 24.56663156083432, 59.443254946286679 ], [ 24.566700348225901, 59.443270458017913 ], [ 24.566782795425869, 59.443299752755024 ], [ 24.566821812231623, 59.443306748125238 ], [ 24.566856312181656, 59.443307093355337 ], [ 24.566883508752632, 59.44330387939533 ], [ 24.566910558916778, 59.443304588807592 ], [ 24.566923811105553, 59.443311865598218 ], [ 24.566922638005188, 59.443318710770725 ], [ 24.566900507709821, 59.443330879471823 ], [ 24.566884509552089, 59.443345068426851 ], [ 24.566887557081255, 59.443359830693524 ], [ 24.56691785895347, 59.443383353447899 ], [ 24.566943691411492, 59.443394588598963 ], [ 24.566971995973041, 59.443402904704499 ], [ 24.567000402289047, 59.443403545424935 ], [ 24.567041197144832, 59.443395533279066 ], [ 24.567098893304749, 59.443391640393024 ], [ 24.56719589717197, 59.443409068045867 ], [ 24.567250673837002, 59.443415115718693 ], [ 24.567312276896757, 59.44342040686567 ], [ 24.567376886324105, 59.443422049534533 ], [ 24.567404415616892, 59.443412335019296 ], [ 24.567427543308337, 59.443397612599107 ], [ 24.567454429210844, 59.44339479482754 ], [ 24.567469833543573, 59.443397924106435 ], [ 24.5674837636288, 59.443423716521764 ], [ 24.567499911350367, 59.443465226277539 ], [ 24.567494686385153, 59.443511451039136 ], [ 24.567489638898547, 59.443534407845192 ], [ 24.567493317867498, 59.443556393491384 ], [ 24.567533947405845, 59.443589350830941 ], [ 24.567584837574078, 59.443617129360916 ], [ 24.567705676393292, 59.443668699421295 ], [ 24.567791766997988, 59.443717717235202 ], [ 24.56782757496962, 59.443764914107128 ], [ 24.56783784114112, 59.443797185386877 ], [ 24.567834496368953, 59.443825009123969 ], [ 24.567812441120136, 59.443866082106439 ], [ 24.56777227432412, 59.443924629691395 ], [ 24.567754783871269, 59.443964623752656 ], [ 24.567754256866198, 59.443981699399735 ], [ 24.567773623983033, 59.443994165114105 ], [ 24.567806077204544, 59.443998585293023 ], [ 24.567856692470524, 59.44400023420004 ], [ 24.567972342013721, 59.443995085691093 ], [ 24.568003643967923, 59.44399811044611 ], [ 24.56802597919091, 59.444014162915479 ], [ 24.568024846855188, 59.444038287756925 ], [ 24.568016511962519, 59.444072353758074 ], [ 24.568004476759388, 59.444098015723299 ], [ 24.568006063696942, 59.444116069673775 ], [ 24.568035152908848, 59.444131240243237 ], [ 24.568069582261586, 59.444135768530209 ], [ 24.568099986542478, 59.44413568227575 ], [ 24.568138689230516, 59.444129016323032 ], [ 24.568286886902605, 59.444076743937735 ], [ 24.568477110047677, 59.444051219618821 ], [ 24.568601521306267, 59.444040853278246 ], [ 24.568658268146848, 59.444042349735618 ], [ 24.568679517152741, 59.444049250715516 ], [ 24.568690692373206, 59.444063017357664 ], [ 24.568684021465184, 59.4440741770687 ], [ 24.568659771643617, 59.444085017709504 ], [ 24.568575010897884, 59.444106496454246 ], [ 24.568502703623821, 59.44413170949079 ], [ 24.568464222079626, 59.444155896798925 ], [ 24.568448601790156, 59.444190658534886 ], [ 24.568450009281193, 59.444215921414866 ], [ 24.568462329813045, 59.444236020568113 ], [ 24.568487383601948, 59.444262886930751 ], [ 24.568517355209575, 59.444297164634754 ], [ 24.56852771958151, 59.444329991978933 ], [ 24.568522506271737, 59.444352482755477 ], [ 24.56852645903863, 59.44437498782635 ], [ 24.568546457159837, 59.44439356372564 ], [ 24.568579542610362, 59.444415817366242 ], [ 24.568662951232117, 59.444429541465894 ], [ 24.568864107280501, 59.444456042291129 ], [ 24.569042774545487, 59.444459417668206 ], [ 24.569088599087376, 59.444466535560693 ], [ 24.569106619209702, 59.444478513190333 ], [ 24.569100808706391, 59.444496877363015 ], [ 24.569066187814055, 59.444516856119456 ], [ 24.56902965357774, 59.444534141194794 ], [ 24.56900420157864, 59.444554304735668 ], [ 24.569001165732935, 59.44457628343757 ], [ 24.56901468124995, 59.444598253472201 ], [ 24.569040754951867, 59.444612233991727 ], [ 24.569069155013384, 59.444646293945276 ], [ 24.569072672204168, 59.444702938651112 ], [ 24.569087821129823, 59.444770044624022 ], [ 24.569073968700231, 59.444837895279257 ], [ 24.569065338376522, 59.444908138057428 ], [ 24.569071326432287, 59.444922645198254 ], [ 24.56907453878226, 59.444938771131284 ], [ 24.569099471634136, 59.444947738683922 ], [ 24.569123878891855, 59.444950550599124 ], [ 24.569151936348629, 59.444948158299049 ], [ 24.569177772658836, 59.444938351995795 ], [ 24.569208853852061, 59.444918550089554 ], [ 24.569220049859901, 59.44489862765338 ], [ 24.569218286942757, 59.444870053979862 ], [ 24.569221309839527, 59.444842016168749 ], [ 24.569227724643547, 59.444821836121207 ], [ 24.569244239196451, 59.444814323200177 ], [ 24.569271733040502, 59.444815137933404 ], [ 24.569317816303005, 59.444819768128475 ], [ 24.569375682473868, 59.444831232376323 ], [ 24.569435914646721, 59.444846106430539 ], [ 24.56953311087435, 59.444873692983251 ], [ 24.569580747097458, 59.444900623065855 ], [ 24.569597384416738, 59.444929015717044 ], [ 24.56960433240522, 59.44495667828609 ], [ 24.569679091928357, 59.445009747403944 ], [ 24.569703321344004, 59.445060018929823 ], [ 24.569724147896217, 59.445107064639842 ], [ 24.569751397294386, 59.445122799334548 ], [ 24.569790272424715, 59.445131706447121 ], [ 24.569824744386789, 59.445132374183927 ], [ 24.569862883227596, 59.445128842981134 ], [ 24.569899659260322, 59.445119698351981 ], [ 24.569978261757321, 59.445104071192084 ], [ 24.570000609107137, 59.44509117394378 ], [ 24.570021965605871, 59.445076997334887 ], [ 24.570031426779529, 59.44506193864342 ], [ 24.570032075293767, 59.445044817581099 ], [ 24.570028890245574, 59.445025074007553 ], [ 24.570037035967967, 59.445005128795607 ], [ 24.570052268150807, 59.444995772156666 ], [ 24.570074892877667, 59.444994220037202 ], [ 24.570089135428891, 59.444999499432591 ], [ 24.570095527378811, 59.445013888085157 ], [ 24.570098713906347, 59.445029493467388 ], [ 24.570103900530359, 59.445045592878465 ], [ 24.570118193203289, 59.44505809816598 ], [ 24.570125827044784, 59.445063460021856 ], [ 24.570157921205499, 59.44507480208933 ], [ 24.570212756591307, 59.445091566271039 ], [ 24.570280880493581, 59.445116639028285 ], [ 24.570307569351385, 59.445142070695759 ], [ 24.570304882737137, 59.445160636829911 ], [ 24.570287985724871, 59.44518219084248 ], [ 24.570243788767947, 59.445207175388987 ], [ 24.57018891571143, 59.445257861077643 ], [ 24.570119375911851, 59.445291599690428 ], [ 24.570100246255578, 59.44529873804742 ], [ 24.570074182807002, 59.445319227591085 ], [ 24.570046986379072, 59.445346822472459 ], [ 24.570041166296228, 59.445369872475041 ], [ 24.570067747490782, 59.445430043847367 ], [ 24.570066808839762, 59.445458297066999 ], [ 24.570069600293049, 59.445486139174868 ], [ 24.570091266128784, 59.445504213910638 ], [ 24.570105818791117, 59.445520667748305 ], [ 24.570110433580119, 59.445540521823091 ], [ 24.570131263688847, 59.445569775955654 ], [ 24.570179773951249, 59.445593012681329 ], [ 24.570245808660975, 59.445606918713544 ], [ 24.570277192630872, 59.445609475798697 ], [ 24.57030919427428, 59.445606796881968 ], [ 24.57033508599978, 59.445593965022603 ], [ 24.570357686655509, 59.445577224663097 ], [ 24.570379312855792, 59.445564411265195 ], [ 24.570414440622649, 59.445555767342327 ], [ 24.570477636735546, 59.445550637343089 ], [ 24.570553554705409, 59.445547319326458 ], [ 24.570664964937649, 59.445548739711761 ], [ 24.570724045546061, 59.445557882163413 ], [ 24.570782569472467, 59.44558223325275 ], [ 24.570835280647739, 59.445619490836478 ], [ 24.570898356406641, 59.445653561688985 ], [ 24.570972376184979, 59.44566638249394 ], [ 24.571012977262473, 59.445671978400959 ], [ 24.57102778683549, 59.445679517324194 ], [ 24.571024375851312, 59.445692691712992 ], [ 24.570996612035223, 59.445756967372247 ], [ 24.570978630963641, 59.445775878090465 ], [ 24.570973531912319, 59.445793521132927 ], [ 24.571011212374888, 59.445879577409301 ], [ 24.571013049439518, 59.445906166918526 ], [ 24.571009515818428, 59.445941531831437 ], [ 24.571012023940821, 59.445965030503011 ], [ 24.571058450658867, 59.446011184055294 ], [ 24.571139188266471, 59.446049361283244 ], [ 24.571207260769015, 59.446112314761571 ], [ 24.571221302383602, 59.44613505426576 ], [ 24.57121085029306, 59.446155144193781 ], [ 24.571180896381858, 59.446173837560934 ], [ 24.571120698802506, 59.446212877212567 ], [ 24.571111988213804, 59.446236972091391 ], [ 24.571117996295591, 59.446264207774455 ], [ 24.571120118519712, 59.446295194543737 ], [ 24.571128000365018, 59.446316416800421 ], [ 24.571151184066771, 59.446327159927854 ], [ 24.571191947642262, 59.446329774867998 ], [ 24.571238150841648, 59.446329843799226 ], [ 24.571314458286437, 59.446318094697951 ], [ 24.571365882370035, 59.446296220243589 ], [ 24.5714141520594, 59.446276513821047 ], [ 24.571480539511857, 59.446263981815129 ], [ 24.571523552173794, 59.44624825681155 ], [ 24.571540912462929, 59.44623017455703 ], [ 24.571541091799901, 59.446211394868861 ], [ 24.571540171740537, 59.446194334469034 ], [ 24.571551963761763, 59.446184804081426 ], [ 24.571570457597797, 59.446182847736416 ], [ 24.571618158021781, 59.446190234888867 ], [ 24.571650241768214, 59.446202968002488 ], [ 24.571719121895782, 59.44624774920706 ], [ 24.571754250328393, 59.446279140271884 ], [ 24.571794023790712, 59.44630263874263 ], [ 24.571817048114323, 59.446338597530648 ], [ 24.571823835175614, 59.446373370090058 ], [ 24.571830306027199, 59.44639767738267 ], [ 24.571835325404745, 59.446410043209205 ], [ 24.571854529103369, 59.44642521097024 ], [ 24.571895041098415, 59.446431740554502 ], [ 24.571989204394615, 59.446435721212673 ], [ 24.572063114092636, 59.446447105642484 ], [ 24.572092617347092, 59.446461510514766 ], [ 24.572136597614232, 59.446521444316033 ], [ 24.572158038797784, 59.446538631010988 ], [ 24.572209343264493, 59.446543300359252 ], [ 24.572272224823099, 59.446544553189945 ], [ 24.572319319684802, 59.446535749053545 ], [ 24.572353797487374, 59.44651984542687 ], [ 24.572404218144598, 59.446469671914208 ], [ 24.572467420505927, 59.446419604408369 ], [ 24.572517326913697, 59.446358751007395 ], [ 24.572625337598673, 59.446310741765842 ], [ 24.572683942257388, 59.446265101616824 ], [ 24.572740237214408, 59.446213134049039 ], [ 24.572771989510148, 59.446167179740655 ], [ 24.572838285350393, 59.446170248664949 ], [ 24.572931695676299, 59.446174564053614 ], [ 24.57295878165997, 59.446223503006074 ], [ 24.573011128686051, 59.446269324923591 ], [ 24.573075238315958, 59.446297573446735 ], [ 24.573115079604865, 59.446319805535794 ], [ 24.573130551219013, 59.446341542223074 ], [ 24.573140837059871, 59.446368337309472 ], [ 24.573144106845795, 59.446398268804934 ], [ 24.573144011664212, 59.446425162919667 ], [ 24.573132326555463, 59.44645785244667 ], [ 24.573124577889214, 59.446511880021781 ], [ 24.573119388419119, 59.446611272844073 ], [ 24.573144519830311, 59.446693918168016 ], [ 24.573163509828319, 59.446724669933488 ], [ 24.57319872581958, 59.446749615082418 ], [ 24.573220389568039, 59.446778972817967 ], [ 24.573239025249038, 59.446865281884001 ], [ 24.573250424954036, 59.4469080144343 ], [ 24.573245822104607, 59.446929955161487 ], [ 24.573233059157268, 59.446948843472398 ], [ 24.573234087490164, 59.446971253402005 ], [ 24.573249685708571, 59.447004712873273 ], [ 24.573270660870822, 59.447025590770991 ], [ 24.57330628145791, 59.447043110535844 ], [ 24.573318259651266, 59.447049934426268 ], [ 24.573326799585612, 59.447060911462799 ], [ 24.57332697300459, 59.447076493970044 ], [ 24.573306202831468, 59.447096189120245 ], [ 24.573295989408496, 59.447112588816189 ], [ 24.573296347668919, 59.447138134470514 ], [ 24.573341459086624, 59.447236922354563 ], [ 24.573359952780041, 59.447311589445555 ], [ 24.573352605481613, 59.447366414180976 ], [ 24.573326071616481, 59.447402965416906 ], [ 24.573303059016713, 59.447458163531316 ], [ 24.573273348708206, 59.447496622627789 ], [ 24.573233192751033, 59.447542909827057 ], [ 24.57320113110265, 59.447587276718949 ], [ 24.573188917433065, 59.44762315520321 ], [ 24.573195548594285, 59.447645388145752 ], [ 24.573280905918043, 59.447690608491065 ], [ 24.573380552225757, 59.447785765862207 ], [ 24.573473673993348, 59.447874470506633 ], [ 24.573637913824566, 59.44800436390063 ], [ 24.573933677512706, 59.448270387681696 ], [ 24.57414458405243, 59.448476826433541 ], [ 24.574474205595976, 59.448799585674479 ], [ 24.574814870602399, 59.449131855890464 ], [ 24.574994355512292, 59.449306913776233 ], [ 24.575225927605263, 59.449581913041605 ], [ 24.575427301728947, 59.449867232155249 ], [ 24.575655657622715, 59.450269998589135 ], [ 24.575811333628295, 59.450619600157665 ], [ 24.575922064571611, 59.450903833481277 ], [ 24.575968712976753, 59.451140053187899 ], [ 24.575965033867618, 59.451304510175824 ], [ 24.575953248375345, 59.451830891007653 ], [ 24.575922241893718, 59.452020449069067 ], [ 24.575869511053135, 59.452188917487298 ], [ 24.575801706449202, 59.452375225258507 ], [ 24.575758812736641, 59.452438086768574 ], [ 24.575713219059669, 59.452553104647635 ], [ 24.575493053299478, 59.452794242240252 ], [ 24.575263595148655, 59.452953877344086 ], [ 24.575095534629092, 59.453055363467911 ], [ 24.574877845974154, 59.453210539023182 ], [ 24.574683695083699, 59.453319732573569 ], [ 24.574588821181194, 59.453392030295362 ], [ 24.574529347929243, 59.453537529629749 ], [ 24.574510559351452, 59.453574594790979 ], [ 24.574452463164896, 59.453631148797946 ], [ 24.57436991792931, 59.453710071115225 ], [ 24.574315347369236, 59.453758225625208 ], [ 24.574238876602209, 59.45381152036579 ], [ 24.574097029084154, 59.453899497942707 ], [ 24.573938223091041, 59.454022324235822 ], [ 24.573790382869671, 59.45411786784134 ], [ 24.573613899552676, 59.454223410142269 ], [ 24.573497089721592, 59.454286422082781 ], [ 24.573419283487791, 59.454333833549796 ], [ 24.57331741335577, 59.454397220151087 ], [ 24.573204639138176, 59.454474881874482 ], [ 24.573123686847101, 59.454531346189519 ], [ 24.573075332991337, 59.454589751213881 ], [ 24.573029378413022, 59.454703934769093 ], [ 24.573015486330085, 59.454778652788825 ], [ 24.572996831696212, 59.454844854860426 ], [ 24.572942120831271, 59.454939516647372 ], [ 24.572812161711955, 59.455199565539608 ], [ 24.572781371211182, 59.455286538098612 ], [ 24.572769336568513, 59.455369021648792 ], [ 24.572754995876085, 59.455428607183627 ], [ 24.572707231993796, 59.455515994606948 ], [ 24.572633760179272, 59.455609714458696 ], [ 24.572595632169786, 59.455711314638641 ], [ 24.57255411978127, 59.455834024288073 ], [ 24.572506383376748, 59.455911546325702 ], [ 24.572458651942188, 59.456035656360037 ], [ 24.572393910726031, 59.456117004406728 ], [ 24.572314651592169, 59.456224303705234 ], [ 24.572197751325824, 59.456351918662421 ], [ 24.572158153859192, 59.456408041138083 ], [ 24.572139022581766, 59.456438581474394 ], [ 24.572108699897548, 59.456459978467102 ], [ 24.572030754174104, 59.45651947201609 ], [ 24.571975242328168, 59.456592952257346 ], [ 24.571906923206861, 59.456650312413764 ], [ 24.571840908267315, 59.456730507718596 ], [ 24.57176060719036, 59.45679788370694 ], [ 24.571686780333948, 59.456883355099869 ], [ 24.571486116615429, 59.457092013524679 ], [ 24.571402679191721, 59.457174375676011 ], [ 24.571333224298829, 59.457240914415976 ], [ 24.571269118083208, 59.457300437237343 ], [ 24.571194027857345, 59.45736849940532 ], [ 24.57114596435046, 59.457438374325129 ], [ 24.571048038634469, 59.457577099602382 ], [ 24.571012512003719, 59.457645923740586 ], [ 24.570996513940845, 59.45767623487459 ], [ 24.570965751866297, 59.457698881222534 ], [ 24.570933810607272, 59.457739593376047 ], [ 24.570911482546741, 59.457785757482412 ], [ 24.57088143710655, 59.457833346376702 ], [ 24.570846735298556, 59.457878765171671 ], [ 24.570830056122006, 59.457940075059575 ], [ 24.570803271469142, 59.457973879867176 ], [ 24.570778534424669, 59.458017307574458 ], [ 24.570740995666455, 59.458057595096278 ], [ 24.570680174293052, 59.458137614654788 ], [ 24.570615064143755, 59.458265964227238 ], [ 24.570573084129975, 59.458316979881083 ], [ 24.570449857694388, 59.458540668954683 ], [ 24.570399276809262, 59.458627879078136 ], [ 24.570366734188209, 59.458705522950844 ], [ 24.570356870350697, 59.458756866279586 ], [ 24.570332863550387, 59.458793405757305 ], [ 24.570258233904386, 59.458883017883686 ], [ 24.570191520353124, 59.45897677875184 ], [ 24.57010452850573, 59.459072736791839 ], [ 24.570027429687734, 59.459150779787265 ], [ 24.569975587790822, 59.459210105229872 ], [ 24.569890759845148, 59.459277302069502 ], [ 24.569802787122573, 59.459338884193329 ], [ 24.56977216447547, 59.459385873864143 ], [ 24.569750247351049, 59.459423005610844 ], [ 24.569718794239453, 59.459476246491541 ], [ 24.569699142150991, 59.459502228616813 ], [ 24.569679154903973, 59.459541946209811 ], [ 24.569677098703814, 59.459575132187688 ], [ 24.569644814255248, 59.459668402785319 ], [ 24.569624134492773, 59.459733212634056 ], [ 24.569619330274524, 59.459770808964514 ], [ 24.569587098283129, 59.459812276024167 ], [ 24.569561662762588, 59.459870032946078 ], [ 24.569565765251625, 59.459893964472258 ], [ 24.569597580558519, 59.459953305132018 ], [ 24.569672307920012, 59.460106058065762 ], [ 24.569685704650063, 59.460140595514765 ], [ 24.569677722242396, 59.46016632969453 ], [ 24.569669766304543, 59.46020950506491 ], [ 24.569674970434367, 59.460242381375537 ], [ 24.569705103650062, 59.46027909058666 ], [ 24.569722873219181, 59.460338949734037 ], [ 24.569701078970915, 59.460395111066255 ], [ 24.569697958955505, 59.460547383638016 ], [ 24.569697360594866, 59.460611927125711 ], [ 24.569688765445218, 59.460761593245252 ], [ 24.569671103512722, 59.460810717089217 ], [ 24.569658403459311, 59.460858113948717 ], [ 24.569656786435107, 59.460893317717961 ], [ 24.569665567229123, 59.460922821355304 ], [ 24.569683503760132, 59.460964134364531 ], [ 24.569701533792546, 59.461011048289976 ], [ 24.569714303093139, 59.461049179029651 ], [ 24.569736935038957, 59.461079125460067 ], [ 24.569783058775389, 59.461093315333599 ], [ 24.5698804579024, 59.461129212996788 ], [ 24.570026083877703, 59.461186939241259 ], [ 24.5701385138901, 59.4612622143461 ], [ 24.570233819366432, 59.461344816452865 ], [ 24.570333345877255, 59.461417202941369 ], [ 24.570423981155617, 59.461487276004306 ], [ 24.57047112190488, 59.461508453913119 ], [ 24.570533927671679, 59.461521467200789 ], [ 24.57060992453896, 59.461526021356882 ], [ 24.57069023683858, 59.461518653969335 ], [ 24.570732707592967, 59.461515759139296 ], [ 24.570782437779179, 59.461512482769848 ], [ 24.570828849340103, 59.461520620885608 ], [ 24.570891225895142, 59.461537486772485 ], [ 24.570965362484134, 59.461568942314955 ], [ 24.57107602102958, 59.461626675281472 ], [ 24.571246296748342, 59.461690190907206 ], [ 24.571462102054795, 59.461739640223037 ], [ 24.571671223567471, 59.461778525936573 ], [ 24.571789432376413, 59.461802581709208 ], [ 24.57192987663225, 59.461807492786271 ], [ 24.572168198729763, 59.461810300178534 ], [ 24.572424264524468, 59.461805875811791 ], [ 24.572706598376257, 59.461808275610068 ], [ 24.573147702429338, 59.461817820832032 ], [ 24.573364244612417, 59.461820622045757 ], [ 24.573456276194754, 59.461819045672925 ], [ 24.573567934015799, 59.461832033432295 ], [ 24.573677777636739, 59.461853547675908 ], [ 24.573775096746143, 59.461873814771259 ], [ 24.573904073380398, 59.461905703120912 ], [ 24.574055574601669, 59.461927313896503 ], [ 24.574168380288381, 59.461934066470604 ], [ 24.57428662058734, 59.461930499240388 ], [ 24.574367914366711, 59.461917515035111 ], [ 24.574424259352572, 59.461900680903611 ], [ 24.574476159813599, 59.46188383021704 ], [ 24.574530375424271, 59.461866224368215 ], [ 24.574627976747017, 59.461816257607104 ], [ 24.574634988145377, 59.46181266337009 ], [ 24.574710789795208, 59.461765735888754 ], [ 24.574820405927927, 59.461683625654409 ], [ 24.575011649738581, 59.461566563802627 ], [ 24.575274206987622, 59.461367683884113 ], [ 24.57548021839288, 59.461192631592702 ], [ 24.575658576047523, 59.461031200399944 ], [ 24.575717894253973, 59.460988967207498 ], [ 24.575754697835826, 59.460962756646943 ], [ 24.575791729369261, 59.460915558052768 ], [ 24.575812745691401, 59.460862989676968 ], [ 24.575816635713931, 59.460808180199777 ], [ 24.575799872812055, 59.46073337158704 ], [ 24.575795271854673, 59.460687467979241 ], [ 24.575801899066796, 59.460645429057564 ], [ 24.575818171837891, 59.460606480781344 ], [ 24.575826648498865, 59.460563266831826 ], [ 24.57584697796344, 59.460535862540816 ], [ 24.575875330795405, 59.460509473432438 ], [ 24.575978899398596, 59.460438295026258 ], [ 24.576064141897429, 59.460378435214977 ], [ 24.576159552505025, 59.460334886052095 ], [ 24.576336178042126, 59.460255165441048 ], [ 24.576527840779434, 59.460184579701007 ], [ 24.576735722600556, 59.460117926206323 ], [ 24.576982028832695, 59.460051220833257 ], [ 24.577090255724301, 59.460027372122084 ], [ 24.577229579938169, 59.459996663710925 ], [ 24.577551590839743, 59.459918539647916 ], [ 24.577829588533721, 59.459842753178911 ], [ 24.578099387158886, 59.459774389740716 ], [ 24.578275206181775, 59.459726617361774 ], [ 24.578377492064067, 59.459698826043066 ], [ 24.578831421765482, 59.459595757521122 ], [ 24.578998763587407, 59.459557751923683 ], [ 24.579278974921106, 59.459501584229628 ], [ 24.579592781582917, 59.459441417252819 ], [ 24.579654917146964, 59.459429510395417 ], [ 24.580171715845669, 59.45936126772105 ], [ 24.580277782587117, 59.459348574687318 ], [ 24.580374662950565, 59.459336990232281 ], [ 24.580445074868283, 59.459328565334005 ], [ 24.580640551947013, 59.459311977291939 ], [ 24.58071623526121, 59.459305557708575 ], [ 24.58099713024065, 59.459291922949603 ], [ 24.581280660049604, 59.459279766066608 ], [ 24.581560271857203, 59.459280893130945 ], [ 24.581741875768842, 59.459272928259885 ], [ 24.581753999181359, 59.459272398982051 ], [ 24.581872537338267, 59.459269299337087 ], [ 24.582184080177232, 59.459241191234199 ], [ 24.582572958698584, 59.459202282879211 ], [ 24.582809739704466, 59.459180886095417 ], [ 24.583071120306982, 59.459146067883289 ], [ 24.583250214062069, 59.459127483857095 ], [ 24.583356233058332, 59.459103846191056 ], [ 24.583520377348428, 59.459057680349595 ], [ 24.583669255421079, 59.459025244160763 ], [ 24.583800580257435, 59.459004779439375 ], [ 24.583946065837136, 59.458985382871838 ], [ 24.584133184223081, 59.458976214334783 ], [ 24.584306020760838, 59.458975403152309 ], [ 24.584458473749564, 59.458971594238115 ], [ 24.584578292773561, 59.458960982114235 ], [ 24.5847807173582, 59.458930192163578 ], [ 24.584869802219337, 59.458899133040532 ], [ 24.585018395160059, 59.458861490209316 ], [ 24.585225560224245, 59.458816172490174 ], [ 24.585393606960512, 59.458784941819331 ], [ 24.585482041332892, 59.45877667651505 ], [ 24.585587955722222, 59.458768683553465 ], [ 24.585665599633387, 59.458755376399743 ], [ 24.585742530738152, 59.4587262198582 ], [ 24.585820038434154, 59.45870601028453 ], [ 24.585892809085038, 59.458689268706173 ], [ 24.585999228636414, 59.458644972171776 ], [ 24.58621891975438, 59.4585549569967 ], [ 24.586403203942442, 59.458483608583236 ], [ 24.586515439903415, 59.45842998607489 ], [ 24.58654996170333, 59.458400928050743 ], [ 24.586578803183532, 59.458356347990843 ], [ 24.586622244690258, 59.458295302843226 ], [ 24.586672935351285, 59.458227178843472 ], [ 24.586726038257332, 59.458183487115797 ], [ 24.586772744671119, 59.458122624857388 ], [ 24.586824314159745, 59.458049236643774 ], [ 24.586858584444805, 59.457964543240401 ], [ 24.586890621401352, 59.457891502306488 ], [ 24.586913044234763, 59.457856205532337 ], [ 24.586945102319135, 59.4578162070245 ], [ 24.586990612798129, 59.457776094721019 ], [ 24.587016059964881, 59.457745434286835 ], [ 24.587029569491907, 59.457721972334994 ], [ 24.587037000723953, 59.457693690145483 ], [ 24.587038029081338, 59.457659143946913 ], [ 24.587032175648748, 59.457624323206282 ], [ 24.58703477810052, 59.45758492267742 ], [ 24.587045462342438, 59.45754687750059 ], [ 24.587067022011873, 59.457512679673876 ], [ 24.587114611898894, 59.45745201081828 ], [ 24.587133181356823, 59.4574151782315 ], [ 24.587144671132396, 59.457367587420705 ], [ 24.587146617920638, 59.457324931332359 ], [ 24.587143296285955, 59.457278402913118 ], [ 24.587142173701334, 59.457249009834406 ], [ 24.587137038887171, 59.457173944180219 ], [ 24.587107531789144, 59.45713458797848 ], [ 24.587058014885926, 59.457059558493015 ], [ 24.587018710118478, 59.457000048766666 ], [ 24.587010736854431, 59.456956790560902 ], [ 24.587014933866243, 59.45691372947654 ], [ 24.587026977353371, 59.456869628063778 ], [ 24.587048125150687, 59.456828933067492 ], [ 24.587045266608751, 59.456802150906867 ], [ 24.587040397203356, 59.45677737946788 ], [ 24.587050356514744, 59.456737183143581 ], [ 24.587079046637044, 59.456669560895179 ], [ 24.587138134617376, 59.456573150146738 ], [ 24.587194571879603, 59.456523590640643 ], [ 24.587237216070704, 59.456471749713025 ], [ 24.587242098002523, 59.456443927683729 ], [ 24.58727213670586, 59.456411693776793 ], [ 24.587318393997084, 59.456380276243657 ], [ 24.587360498879676, 59.45635758442566 ], [ 24.587419077857628, 59.456323912680276 ], [ 24.587516043335427, 59.456272816880343 ], [ 24.587697880337352, 59.456175921290644 ], [ 24.587869429455559, 59.456102176902206 ], [ 24.588036209263379, 59.456035150053594 ], [ 24.588199838576653, 59.455972589407381 ], [ 24.588422933214783, 59.455898255098276 ], [ 24.588839020750651, 59.455755716175801 ], [ 24.589248546970719, 59.455631652011945 ], [ 24.589520203302548, 59.45553828383332 ], [ 24.589763176036179, 59.455445249941455 ], [ 24.589833793330289, 59.455403372523186 ], [ 24.589880548559808, 59.455367310969642 ], [ 24.589924380872798, 59.455332510222604 ], [ 24.589972181717421, 59.455310034409756 ], [ 24.590032434703435, 59.455286757764064 ], [ 24.590089744017465, 59.455263539156725 ], [ 24.590223802526101, 59.4551860188433 ], [ 24.590309586545427, 59.455136218757836 ], [ 24.590411907256531, 59.455067942635417 ], [ 24.59055387172717, 59.455008231877422 ], [ 24.590796204140549, 59.45490710207519 ], [ 24.590955725836672, 59.454836010697598 ], [ 24.591019847966137, 59.454793910411503 ], [ 24.591041220185389, 59.4547655115488 ], [ 24.591071747484538, 59.454733274539706 ], [ 24.591128132720723, 59.454697510429071 ], [ 24.591230554445634, 59.454635238489466 ], [ 24.591378032602641, 59.454518402381943 ], [ 24.591499438066332, 59.454446691470054 ], [ 24.59168460357392, 59.454371130433096 ], [ 24.591894700082811, 59.454269146920787 ], [ 24.592059814594375, 59.454193477937977 ], [ 24.592257392528403, 59.454093650594032 ], [ 24.59240096504244, 59.454041677307657 ], [ 24.592425902495648, 59.454015614024875 ], [ 24.592460925472029, 59.453967207680328 ], [ 24.592491476608544, 59.453930365255061 ], [ 24.592517822282673, 59.45388995112264 ], [ 24.592550895341329, 59.453853151199105 ], [ 24.59270421922016, 59.453750533000495 ], [ 24.592765075830819, 59.453717602582046 ], [ 24.592806495817868, 59.453685307160434 ], [ 24.592836412737451, 59.453641420923624 ], [ 24.592888314343487, 59.453571035761826 ], [ 24.592942823361707, 59.45350359212123 ], [ 24.592973967330742, 59.453399404948598 ], [ 24.592977012387756, 59.453325999100919 ], [ 24.592986964440659, 59.453270613973686 ], [ 24.592998504707829, 59.453215311461065 ], [ 24.593029826703415, 59.453179273312763 ], [ 24.593078864409335, 59.453133891622819 ], [ 24.593150411694122, 59.453085159065978 ], [ 24.593326456485091, 59.453000785830376 ], [ 24.593683590143765, 59.452859663636218 ], [ 24.593993665097738, 59.45273280989182 ], [ 24.594267881477442, 59.452616377339204 ], [ 24.594558730025007, 59.452480156662396 ], [ 24.594820728599185, 59.452356884085724 ], [ 24.595049613909552, 59.452288203260046 ], [ 24.595366274500044, 59.452210005222739 ], [ 24.595713518561187, 59.45211841875053 ], [ 24.596148523318618, 59.45201758338046 ], [ 24.596549059354345, 59.451925924070018 ], [ 24.59667913854506, 59.451898719530817 ], [ 24.596845843195222, 59.451863844546509 ], [ 24.597018818827042, 59.451827666261437 ], [ 24.597384001021862, 59.45176018583058 ], [ 24.597806890049185, 59.451682846604093 ], [ 24.598181294960472, 59.451629747404318 ], [ 24.59850081655992, 59.451581752795207 ], [ 24.598784842106532, 59.451527428704345 ], [ 24.598819686274226, 59.451520762328549 ], [ 24.598906675984654, 59.451501740553354 ], [ 24.599102310202319, 59.451458957477264 ], [ 24.599396741126203, 59.451393893665823 ], [ 24.599593008532437, 59.451348081875651 ], [ 24.599801770303618, 59.451336117331181 ], [ 24.599930010238566, 59.451316889103964 ], [ 24.600034213263662, 59.451303299789544 ], [ 24.600139453106429, 59.451292533282277 ], [ 24.600230166906641, 59.451286859743277 ], [ 24.600313213514482, 59.45128430901336 ], [ 24.60036346615864, 59.451284672462286 ], [ 24.600435981116746, 59.451289781764338 ], [ 24.600460699479815, 59.451310880689363 ], [ 24.600562392681081, 59.451355093426585 ], [ 24.600715641099878, 59.451460668096516 ], [ 24.600851884195954, 59.451561634389833 ], [ 24.601002119744638, 59.451665211720425 ], [ 24.601062329019992, 59.45171837442998 ], [ 24.601122034282529, 59.451770937982694 ], [ 24.601166006836731, 59.451807110309971 ], [ 24.60123529230053, 59.451853032330128 ], [ 24.601277593171808, 59.451887372056589 ], [ 24.601306048063567, 59.451927376333956 ], [ 24.601337847942716, 59.451962096102328 ], [ 24.601448551435539, 59.452020207610566 ], [ 24.601909832547527, 59.452331903080534 ], [ 24.602510835565543, 59.452735856305701 ], [ 24.602921110376165, 59.453021525092041 ], [ 24.603221275857255, 59.453227850416155 ], [ 24.603308954734175, 59.453302618537194 ], [ 24.603348604429954, 59.453334375515361 ], [ 24.603385352080672, 59.453360544407637 ], [ 24.603420971290685, 59.453368747412092 ], [ 24.603449395140721, 59.45336986496325 ], [ 24.603639755668326, 59.453380545718844 ], [ 24.603701924253379, 59.453376580474853 ], [ 24.603781829597601, 59.453353647262169 ], [ 24.603869341488863, 59.453325158558904 ], [ 24.604396927208875, 59.453115156600504 ], [ 24.605462508129573, 59.452709577542286 ], [ 24.606579916160374, 59.452273230783732 ], [ 24.607646530337064, 59.451873149510874 ], [ 24.607729332482286, 59.45183798340647 ], [ 24.607745117033655, 59.451820442209986 ], [ 24.607750941902932, 59.451787139240849 ], [ 24.607806451126855, 59.451758876292402 ], [ 24.607910515213337, 59.451756394361794 ], [ 24.60796392389134, 59.451787422225614 ], [ 24.608435057462124, 59.451628669058955 ], [ 24.608339834668481, 59.451545492162332 ], [ 24.608255295990567, 59.451500900976889 ], [ 24.607747290224985, 59.451690918414052 ], [ 24.606969365824202, 59.45199763418502 ], [ 24.606031515336078, 59.452361929549234 ], [ 24.605123037983343, 59.452707438884289 ], [ 24.60474701945261, 59.452850698730678 ], [ 24.604663935563725, 59.452841385410203 ], [ 24.604590366890285, 59.45284171406874 ], [ 24.604527129181246, 59.452840998884582 ], [ 24.604476284981089, 59.452816313277999 ], [ 24.604423324313988, 59.452783612326449 ], [ 24.604381069236016, 59.452762064976007 ], [ 24.604323456150194, 59.452754385018601 ], [ 24.604266352274653, 59.452765517548642 ], [ 24.604252678037231, 59.452781881681034 ], [ 24.604267213565564, 59.452805225853496 ], [ 24.604309500875765, 59.452838559301576 ], [ 24.604348924472053, 59.452867480433817 ], [ 24.604356884234825, 59.452890854760682 ], [ 24.604338553540646, 59.452913092971848 ], [ 24.604307086959089, 59.452945256661792 ], [ 24.604275654946481, 59.452959449134049 ], [ 24.60396779002474, 59.453065688991167 ], [ 24.603871774307549, 59.453096810967118 ], [ 24.603811820697388, 59.453106358531876 ], [ 24.60374592316176, 59.453102818571971 ], [ 24.603471381493051, 59.453058574297842 ], [ 24.603340068263961, 59.453033843906496 ], [ 24.603270376887757, 59.452992188709651 ], [ 24.602855652992979, 59.452715311442319 ], [ 24.602416366687685, 59.452421911643228 ], [ 24.60183799837155, 59.452063428507309 ], [ 24.601653005379362, 59.451926645031854 ], [ 24.601642193023618, 59.451899351840943 ], [ 24.601679753992407, 59.451877780205734 ], [ 24.601997124087646, 59.451761879332693 ], [ 24.602655715376343, 59.451503305780648 ], [ 24.602707403584176, 59.451463114683143 ], [ 24.602857714775539, 59.451400892654966 ], [ 24.603296449258281, 59.451247404867551 ], [ 24.603655859322128, 59.451094781787013 ], [ 24.603977059810013, 59.450990896053611 ], [ 24.60417489371228, 59.45090455003551 ], [ 24.604314123455186, 59.450856012519289 ], [ 24.604376931182294, 59.450842600661936 ], [ 24.604432212321292, 59.450848199748719 ], [ 24.604485579661176, 59.450867191639958 ], [ 24.604830947512383, 59.451094876672336 ], [ 24.605595079177117, 59.450797850712924 ], [ 24.606325551600154, 59.451298766807568 ], [ 24.606392946559335, 59.451265683194464 ], [ 24.60600804585264, 59.451003570470768 ], [ 24.606134338678611, 59.450957343649556 ], [ 24.60650567929239, 59.451214329990243 ], [ 24.606588902210106, 59.451184018972256 ], [ 24.606207428178333, 59.450930248217581 ], [ 24.606289367241118, 59.450902124595046 ], [ 24.606696361679415, 59.451169314239195 ], [ 24.606950200470195, 59.451074016790606 ], [ 24.607033147941006, 59.451065941692683 ], [ 24.607103129480173, 59.451086193480023 ], [ 24.607167825983971, 59.451123525050846 ], [ 24.607230491465359, 59.451156682875329 ], [ 24.607284707359568, 59.451177671501071 ], [ 24.607336241854949, 59.45117576428801 ], [ 24.607389279014576, 59.451166157215425 ], [ 24.607391453616614, 59.451140725573779 ], [ 24.606964168531409, 59.450840184877784 ], [ 24.606476450963079, 59.450520513700951 ], [ 24.606458253571436, 59.450494511591586 ], [ 24.606492079073757, 59.450475298630181 ], [ 24.606855026501957, 59.45034738978984 ], [ 24.606856278865308, 59.450306747127144 ], [ 24.606748819263608, 59.450245869361794 ], [ 24.606277349042802, 59.449916266321551 ], [ 24.605757109483367, 59.449576841089616 ], [ 24.60559539440526, 59.449474399033939 ], [ 24.605543269517366, 59.449423974930717 ], [ 24.605549269575807, 59.449391506050496 ], [ 24.605609573331158, 59.449365313490837 ], [ 24.606055671247589, 59.449184861645072 ], [ 24.606457546590828, 59.449011128840297 ], [ 24.606607558148276, 59.44893302422669 ], [ 24.606669285164031, 59.448875891368296 ], [ 24.606723520146947, 59.448803900862146 ], [ 24.606768034164642, 59.448720590773831 ], [ 24.606809296808727, 59.448657699381776 ], [ 24.606855364211555, 59.448587425054292 ], [ 24.606869950009379, 59.44856795949584 ], [ 24.606899395106321, 59.448538470462609 ], [ 24.606972147725248, 59.448515559186646 ], [ 24.607149062806862, 59.448462545137829 ], [ 24.607389171444634, 59.448406268235743 ], [ 24.607683806111453, 59.448338159588182 ], [ 24.607912518946122, 59.448289474610839 ], [ 24.607999636448724, 59.44827837246779 ], [ 24.608036772071156, 59.448276926385986 ], [ 24.608089653784948, 59.448282615791868 ], [ 24.608188608518077, 59.448296736900623 ], [ 24.60823363632046, 59.448299931117738 ], [ 24.608269560584922, 59.448299756260084 ], [ 24.608311204776804, 59.448295093622853 ], [ 24.608347396292306, 59.448288149175717 ], [ 24.608381977477844, 59.448272944734462 ], [ 24.608428711364351, 59.44824524262571 ], [ 24.608441097840146, 59.448233991629905 ], [ 24.608456242428648, 59.448220223429615 ], [ 24.608464105413411, 59.448196506915657 ], [ 24.608461616453557, 59.448175540166226 ], [ 24.608465051209631, 59.44815855858139 ], [ 24.608472048003247, 59.448152520935835 ], [ 24.608478663071999, 59.448146826162912 ], [ 24.608506292283046, 59.448138206697017 ], [ 24.608697740151996, 59.448095706783526 ], [ 24.608934759978165, 59.448041478987484 ], [ 24.609111974014898, 59.448005534325873 ], [ 24.609232048544889, 59.447981173164841 ], [ 24.609415093315057, 59.447946009015915 ], [ 24.609675736846302, 59.44789593445072 ], [ 24.609965715448634, 59.44786176466723 ], [ 24.610357908104408, 59.447811079526552 ], [ 24.610810819587766, 59.447751942987495 ], [ 24.610942972587473, 59.447736734212533 ], [ 24.611146514615637, 59.44771330369845 ], [ 24.611167315942165, 59.447710334683151 ], [ 24.611428465462573, 59.44767300999235 ], [ 24.611698328694715, 59.447627152474695 ], [ 24.611889757827875, 59.44758398370216 ], [ 24.61210213227956, 59.447541983017608 ], [ 24.612258427447426, 59.447513581853705 ], [ 24.612344350123156, 59.447497967153033 ], [ 24.612515755303662, 59.447457933605527 ], [ 24.612539155794003, 59.447452465773573 ], [ 24.612745124681993, 59.447407244308096 ], [ 24.612900688158668, 59.447375461608587 ], [ 24.613035016758531, 59.447348023473829 ], [ 24.613170414385976, 59.447323192409215 ], [ 24.613302420649298, 59.447298978425351 ], [ 24.613493669809522, 59.447267594386346 ], [ 24.613550721764756, 59.447252283907957 ], [ 24.613594836318384, 59.447245902532146 ], [ 24.613626138922843, 59.447243656193805 ], [ 24.613650711483743, 59.447238685390843 ], [ 24.613659905289765, 59.447230384105531 ], [ 24.613664992367116, 59.447196608462654 ], [ 24.613680206203291, 59.447176044049939 ], [ 24.613702868907641, 59.447163873940191 ], [ 24.61376775981617, 59.447144343756712 ], [ 24.613924127619121, 59.447108390953687 ], [ 24.614162538980004, 59.447051014347124 ], [ 24.614527952305068, 59.446940307433387 ], [ 24.614843510841926, 59.446856995351816 ], [ 24.615025403799375, 59.446800608068799 ], [ 24.615119841141162, 59.446773434825189 ], [ 24.61516556028938, 59.446760277080159 ], [ 24.615301480241143, 59.446737254631927 ], [ 24.61550453268373, 59.446704753269415 ], [ 24.615554125450252, 59.446707008036704 ], [ 24.615574996310045, 59.446707951785555 ], [ 24.615618659880344, 59.446720620146294 ], [ 24.615681224942843, 59.446740372437027 ], [ 24.615713723320912, 59.446741908128949 ], [ 24.615738374470912, 59.446738399748469 ], [ 24.615759808146748, 59.446728515075748 ], [ 24.615776526864448, 59.446709370652357 ], [ 24.615789540449352, 59.446663645926066 ], [ 24.615649923681662, 59.446616498240957 ], [ 24.615680972784322, 59.446592403586543 ], [ 24.615690655911649, 59.446584880830038 ], [ 24.61574777996864, 59.446555107769669 ], [ 24.615825852939675, 59.446501287212534 ], [ 24.615937359552333, 59.44639825322696 ], [ 24.615959628234691, 59.446381901472279 ], [ 24.615999507344984, 59.446366544548034 ], [ 24.616025743618778, 59.446356134680293 ], [ 24.61604447851456, 59.446345194366103 ], [ 24.616065909772797, 59.446328352797039 ], [ 24.616079965831485, 59.446313089692907 ], [ 24.616087658724609, 59.446301761164939 ], [ 24.61615774886479, 59.446250096212871 ], [ 24.616265243322715, 59.446178812936488 ], [ 24.616326282226133, 59.446135511534045 ], [ 24.616363720063845, 59.446108954160678 ], [ 24.616560394515819, 59.445978259154536 ], [ 24.616744122906724, 59.445860900834987 ], [ 24.616925211794729, 59.445768267162471 ], [ 24.61694344632053, 59.445758935868227 ], [ 24.616954886756364, 59.445753092381821 ], [ 24.617094134165196, 59.445693050959648 ], [ 24.617171722205921, 59.445668226226694 ], [ 24.617288094625167, 59.445635831956039 ], [ 24.617468242203504, 59.445580984400166 ], [ 24.617533389566475, 59.445566172810139 ], [ 24.617693527946109, 59.445529766903512 ], [ 24.617955777383155, 59.445462953818819 ], [ 24.617968645385524, 59.445459060441941 ], [ 24.61811327911359, 59.445415321072268 ], [ 24.618198264951531, 59.445385389234055 ], [ 24.618242520390471, 59.445370415021131 ], [ 24.618289726375124, 59.445362635146971 ], [ 24.618346064767195, 59.445358833897032 ], [ 24.618409135521535, 59.445353017187301 ], [ 24.618458133108934, 59.445346835650909 ], [ 24.618506114347635, 59.445335147240513 ], [ 24.618601325089976, 59.445313093456406 ], [ 24.618697061055251, 59.445295713940958 ], [ 24.618783391592959, 59.445286538265194 ], [ 24.618853442549018, 59.445270993827656 ], [ 24.618943824104505, 59.445256781096582 ], [ 24.618998726209107, 59.445242797881001 ], [ 24.619049852486786, 59.445222782163867 ], [ 24.619091454829626, 59.445199561664396 ], [ 24.619122190337176, 59.445169067329978 ], [ 24.619140406163147, 59.44514222247664 ], [ 24.619146841863031, 59.445109257098217 ], [ 24.619152343282558, 59.445092822022154 ], [ 24.619163332280131, 59.445079610677197 ], [ 24.619186539716544, 59.44506947472469 ], [ 24.619203468526305, 59.445062070239835 ], [ 24.619247190972587, 59.445045940208288 ], [ 24.619438166221933, 59.444985859514112 ], [ 24.619621192338961, 59.444925851840239 ], [ 24.619657114128053, 59.444914076109306 ], [ 24.619876333532911, 59.444850504642453 ], [ 24.620134109425692, 59.444763053061557 ], [ 24.620191835044732, 59.444749316367513 ], [ 24.620212738277331, 59.444744334670482 ], [ 24.620267454301398, 59.444716294441299 ], [ 24.620399234012286, 59.444647433694819 ], [ 24.62056814541651, 59.444598532380077 ], [ 24.62073853433899, 59.444540755002478 ], [ 24.620768018654644, 59.444530759738555 ], [ 24.620995142435234, 59.444445551426185 ], [ 24.621233583087196, 59.444359463489739 ], [ 24.621448712103398, 59.444289409595449 ], [ 24.62147501334735, 59.444279842112465 ], [ 24.621677000776408, 59.444206402831256 ], [ 24.621759998908104, 59.444181872971342 ], [ 24.621829410867459, 59.444171123448989 ], [ 24.621915640329103, 59.444163373455737 ], [ 24.62193498027294, 59.444161630439545 ], [ 24.622008884416733, 59.44415074291414 ], [ 24.622115093310839, 59.444121201987386 ], [ 24.622153120997154, 59.444102234335318 ], [ 24.622177072669331, 59.444086591672011 ], [ 24.622187238665841, 59.444068922578886 ], [ 24.622186566838355, 59.444050424975082 ], [ 24.622186889979925, 59.444045908217596 ], [ 24.622188203113001, 59.444028002669214 ], [ 24.622187533088852, 59.444004765410455 ], [ 24.622180360989415, 59.443989386464111 ], [ 24.622161907173304, 59.443976870486239 ], [ 24.62214933923957, 59.443967181262764 ], [ 24.622131312891259, 59.443955875102731 ], [ 24.622126838895088, 59.443939594725144 ], [ 24.622133887222596, 59.443917209691513 ], [ 24.622163026205161, 59.443885402594546 ], [ 24.622247596769903, 59.44378179009297 ], [ 24.622273432458762, 59.443758310909757 ], [ 24.622281339101139, 59.443751119200719 ], [ 24.622292818447917, 59.443736855000573 ], [ 24.62229609275856, 59.443723078396815 ], [ 24.622306826413045, 59.443707552017329 ], [ 24.622327416526325, 59.443694142426658 ], [ 24.622333652647843, 59.443690073488931 ], [ 24.622345549425535, 59.443676471579948 ], [ 24.62234748140256, 59.443670286547523 ], [ 24.622351927556359, 59.44365601069147 ], [ 24.622367956334116, 59.443624436047379 ], [ 24.622403511782984, 59.443584941527618 ], [ 24.622505113270627, 59.443520314542191 ], [ 24.622582828397231, 59.443488439323424 ], [ 24.622669438892228, 59.443447545398662 ], [ 24.622776101241936, 59.44339372005637 ], [ 24.622883570733443, 59.443352278523697 ], [ 24.622993770590693, 59.443321389451626 ], [ 24.623126721912424, 59.443279665602851 ], [ 24.623345755086294, 59.443234823300202 ], [ 24.623644330959532, 59.443180133690589 ], [ 24.623733744044934, 59.443158660135893 ], [ 24.623755653602963, 59.443150953188628 ], [ 24.623781326802927, 59.443138928617536 ], [ 24.62379310491708, 59.443128522805551 ], [ 24.623804717776949, 59.443096520056969 ], [ 24.623819899314963, 59.443081376426804 ], [ 24.623841193432664, 59.443069893227829 ], [ 24.623886612436003, 59.443056195222667 ], [ 24.623957154164206, 59.443043769559495 ], [ 24.624071171467524, 59.443028983476339 ], [ 24.624190301930572, 59.443016219728172 ], [ 24.62443520741763, 59.442971396601692 ], [ 24.62464854188423, 59.442934164362789 ], [ 24.62477794961967, 59.442906082123962 ], [ 24.624848048600239, 59.442880085446177 ], [ 24.624938474908969, 59.442836039059131 ], [ 24.624953621626055, 59.442828660226823 ], [ 24.624972849358581, 59.442808308778837 ], [ 24.625012546950071, 59.442769027260205 ], [ 24.625066319484759, 59.442715819043343 ], [ 24.625135043906081, 59.442642674594332 ], [ 24.625241329877355, 59.44257922606743 ], [ 24.625391119111846, 59.442476477740506 ], [ 24.625521248401029, 59.44240707198275 ], [ 24.625606155495227, 59.442351462525409 ], [ 24.62563669264668, 59.442300007148454 ], [ 24.625661702049658, 59.442270094964265 ], [ 24.625716881639065, 59.442234141805741 ], [ 24.625763330199462, 59.442210357511932 ], [ 24.625827395656724, 59.442150502246434 ], [ 24.625875805679339, 59.442111609926364 ], [ 24.625889336161706, 59.442100701876655 ], [ 24.625946866600248, 59.442078606313686 ], [ 24.626144868924339, 59.442038033359118 ], [ 24.626364245188586, 59.442000051437496 ], [ 24.62650088740898, 59.441978441111104 ], [ 24.62653690615997, 59.441968404466408 ], [ 24.626567320837001, 59.441953609937116 ], [ 24.626588787451595, 59.441939127270949 ], [ 24.626613454238601, 59.441897322511181 ], [ 24.626622131988274, 59.441872667343844 ], [ 24.626631224920491, 59.441853351286568 ], [ 24.626644973658319, 59.441838025623113 ], [ 24.626694013039877, 59.441819102966825 ], [ 24.626715207222713, 59.44181091418276 ], [ 24.626818234781986, 59.441779589198042 ], [ 24.627025007281226, 59.441712842230416 ], [ 24.62716891632552, 59.441648396093932 ], [ 24.627326954509012, 59.441573074648133 ], [ 24.627587439410821, 59.441511690285083 ], [ 24.62786232519068, 59.441442741270784 ], [ 24.628123024577341, 59.441354784020525 ], [ 24.62832812650219, 59.441289479198218 ], [ 24.628663053411426, 59.441173357135618 ], [ 24.628967597304346, 59.441039309373586 ], [ 24.629203210919947, 59.4409341538421 ], [ 24.629440337697147, 59.440806935083558 ], [ 24.629710456903908, 59.440690258268951 ], [ 24.629960424718313, 59.440602016483318 ], [ 24.63014232321472, 59.440546200340968 ], [ 24.630315290205221, 59.440484915043186 ], [ 24.630576167795958, 59.440369151375947 ], [ 24.630690763996849, 59.440300434125632 ], [ 24.630818394171211, 59.440205523379291 ], [ 24.63102649989208, 59.44010502047729 ], [ 24.631240531525986, 59.439993357813819 ], [ 24.631470329741003, 59.439908073315763 ], [ 24.631712334277715, 59.439823403104356 ], [ 24.631915016962502, 59.439725357439613 ], [ 24.63220445072125, 59.439634264838205 ], [ 24.632902711101455, 59.439380364145862 ], [ 24.633147728567256, 59.439274168751936 ], [ 24.633253776523507, 59.439184978690562 ], [ 24.633371974342733, 59.439079078447456 ], [ 24.633390412837969, 59.439054564344346 ], [ 24.633403811739633, 59.439036753093589 ], [ 24.633439106790693, 59.438989859945849 ], [ 24.633483509243337, 59.438912330606719 ], [ 24.633515405192917, 59.438854170156191 ], [ 24.633594670093117, 59.43872167069842 ], [ 24.633655004770564, 59.438666999938512 ], [ 24.633730620798701, 59.438620971924259 ], [ 24.633952442320851, 59.438511242073574 ], [ 24.634177249260091, 59.43844374151535 ], [ 24.6343552080404, 59.438378898917414 ], [ 24.634397139652314, 59.438349738288167 ], [ 24.634426342520133, 59.43832318833347 ], [ 24.63450557803807, 59.438292797678642 ], [ 24.634600213223337, 59.438267925210837 ], [ 24.634711050090708, 59.438246206175492 ], [ 24.635000217360407, 59.438204426220423 ], [ 24.635477224838784, 59.438111954861412 ], [ 24.63584617408079, 59.43804946489584 ], [ 24.636071140269141, 59.438031583162605 ], [ 24.636161127599806, 59.438015582979702 ], [ 24.63628300230387, 59.437969653145402 ], [ 24.636485855261654, 59.437910414435969 ], [ 24.636781193478566, 59.437847299141346 ], [ 24.637033468573559, 59.437801707448514 ], [ 24.637402270279672, 59.437756197609289 ], [ 24.637968784570187, 59.43768410968763 ], [ 24.638265281476674, 59.437630437830208 ], [ 24.638583286808394, 59.437578187029501 ], [ 24.638688266794031, 59.437566771279329 ], [ 24.638785761530659, 59.437548066812099 ], [ 24.638846427892602, 59.437528652256333 ], [ 24.63894287387787, 59.437514189736547 ], [ 24.639099756439862, 59.437493482327561 ], [ 24.639244857485089, 59.437472293324234 ], [ 24.639348763324257, 59.43744870992515 ], [ 24.639436754641217, 59.437422268358823 ], [ 24.639556961856517, 59.437383632587341 ], [ 24.639641923981738, 59.437351667001963 ], [ 24.639711467500554, 59.437322621861661 ], [ 24.639750767583699, 59.437291587175018 ], [ 24.63978205394416, 59.437264577042576 ], [ 24.639831017625259, 59.437225407419362 ], [ 24.639947522344837, 59.437161340473097 ], [ 24.640023160995973, 59.437130317608961 ], [ 24.640121574115806, 59.437077020646115 ], [ 24.640241566681155, 59.437017631263899 ], [ 24.640325444530042, 59.436983372474806 ], [ 24.640452702190473, 59.436967502274349 ], [ 24.640505951469333, 59.436964289961146 ], [ 24.640582117855384, 59.436950185188337 ], [ 24.640672804331992, 59.436926620150658 ], [ 24.640762591429827, 59.436904908616221 ], [ 24.640864350700319, 59.436883533727247 ], [ 24.641000805512519, 59.436869467422945 ], [ 24.641220517084779, 59.436824744270297 ], [ 24.6414834779766, 59.436737960913163 ], [ 24.641689886454177, 59.436652143819565 ], [ 24.641718447037462, 59.436635873593382 ], [ 24.641727998012172, 59.436627397971129 ], [ 24.641744010488136, 59.436615784936322 ], [ 24.641769586298071, 59.436597294058103 ], [ 24.641787876774913, 59.436582177997238 ], [ 24.641812131617428, 59.436567481696159 ], [ 24.641843850723955, 59.436554248109069 ], [ 24.641894428557102, 59.436534576091965 ], [ 24.641958092987128, 59.436504211898814 ], [ 24.642036848564004, 59.436474178043945 ], [ 24.64209148931009, 59.436455033715603 ], [ 24.642134247952868, 59.436431907812974 ], [ 24.64220596210987, 59.436392922560522 ], [ 24.642268288794327, 59.436367358270914 ], [ 24.642362127262459, 59.43631662229614 ], [ 24.642429231056713, 59.436283646837197 ], [ 24.64248031936479, 59.436251799754253 ], [ 24.642571993144937, 59.436206873105952 ], [ 24.642638740493513, 59.436174644338628 ], [ 24.642698335491389, 59.436129434294514 ], [ 24.642727261544223, 59.436088610903099 ], [ 24.642749061681378, 59.436040892308796 ], [ 24.642764325290802, 59.436000746008311 ], [ 24.642782614528684, 59.4359425149256 ], [ 24.642818667918419, 59.435899313810303 ], [ 24.642848896723155, 59.435861248837575 ], [ 24.642853648094974, 59.43585527411765 ], [ 24.642955933111974, 59.435792772947146 ], [ 24.643001306496942, 59.435772883532906 ], [ 24.643151462484717, 59.435740830652975 ], [ 24.643245960275205, 59.43570936368625 ], [ 24.643321815575597, 59.435694854872786 ], [ 24.643410918407262, 59.435686663689026 ], [ 24.643643068046128, 59.435655430399819 ], [ 24.643934399971759, 59.435607740331633 ], [ 24.644414994541346, 59.435532974502209 ], [ 24.644740415438854, 59.435459649017311 ], [ 24.645107770155196, 59.435353361444378 ], [ 24.645501181273904, 59.43527109232231 ], [ 24.645839260288394, 59.435188985697025 ], [ 24.646054345625672, 59.435107391944619 ], [ 24.646307873605132, 59.434973230309502 ], [ 24.646535045621146, 59.434820885185431 ], [ 24.646747660220033, 59.434730927171401 ], [ 24.647057070114368, 59.4346698109713 ], [ 24.647260692852907, 59.434621583942153 ], [ 24.647395217163922, 59.434565760974451 ], [ 24.647528956379261, 59.434488990183645 ], [ 24.64770113507852, 59.434381922425388 ], [ 24.647901397999195, 59.434218908290752 ], [ 24.648028522020788, 59.434110302342091 ], [ 24.64816568133309, 59.433984546304991 ], [ 24.648350790549522, 59.433859882561698 ], [ 24.648459488284686, 59.433806501617219 ], [ 24.648788070775709, 59.433697755366602 ], [ 24.64917637281118, 59.433613291986653 ], [ 24.649368219072347, 59.433553207648743 ], [ 24.649451191244431, 59.433496326162157 ], [ 24.649575009373155, 59.433352330953916 ], [ 24.649731761195358, 59.433228774574054 ], [ 24.649764955056551, 59.433202605341421 ], [ 24.649979281225917, 59.4329747876737 ], [ 24.650296970190286, 59.432740588406773 ], [ 24.650453857897851, 59.432678143665704 ], [ 24.650557150357528, 59.432604069459281 ], [ 24.650684621396415, 59.432516293964284 ], [ 24.65077738268921, 59.432428707857447 ], [ 24.65083825218505, 59.43237083056043 ], [ 24.650944035818789, 59.432282398983695 ], [ 24.651081335205671, 59.432178811439755 ], [ 24.651195337240207, 59.432065859597998 ], [ 24.65121227901469, 59.432037642884751 ], [ 24.651231088015876, 59.432021418756349 ], [ 24.651275898544839, 59.431997471737226 ], [ 24.651322297608406, 59.431973669447558 ], [ 24.651391116107149, 59.431928939298771 ], [ 24.651426120572609, 59.431894358489224 ], [ 24.651444258109084, 59.431868801901246 ], [ 24.651482191139223, 59.431817779256356 ], [ 24.651520466839791, 59.431784735198725 ], [ 24.651582840280795, 59.431746715488202 ], [ 24.651649189838434, 59.431715776623804 ], [ 24.651736920017754, 59.431670441010681 ], [ 24.651846340958958, 59.431622358804333 ], [ 24.651912945657507, 59.43159465912283 ], [ 24.651962178434221, 59.431570743824508 ], [ 24.652019578306014, 59.431533448709139 ], [ 24.652095307134474, 59.431496367995564 ], [ 24.652164415063247, 59.431461366717187 ], [ 24.652243382837447, 59.431419521205797 ], [ 24.652356130425993, 59.431355641070724 ], [ 24.652374242978681, 59.431343594379676 ], [ 24.652502519416977, 59.4312615760768 ], [ 24.652529127440978, 59.431233984523359 ], [ 24.652544289706686, 59.431216925485707 ], [ 24.652562693714344, 59.431190622345333 ], [ 24.652571379542611, 59.431142598906469 ], [ 24.652586354183441, 59.431080657360781 ], [ 24.652593444102454, 59.431025648989511 ], [ 24.652617003358298, 59.430997120997461 ], [ 24.652648674215978, 59.430957861556486 ], [ 24.652707294358805, 59.430886762846853 ], [ 24.652739625236478, 59.430850704786636 ], [ 24.65282551786721, 59.430796993298586 ], [ 24.652865084696526, 59.430780010694257 ], [ 24.653005146973875, 59.430698992672369 ], [ 24.653048950107205, 59.430614188070237 ], [ 24.653112591551103, 59.430482301047533 ], [ 24.653232891950132, 59.4303188042268 ], [ 24.653421415489721, 59.430160920244276 ], [ 24.653576596116853, 59.430059880332493 ], [ 24.653710895420247, 59.429991924279783 ], [ 24.653801928318185, 59.429957504436544 ], [ 24.653838902885951, 59.429938640317985 ], [ 24.653870715311097, 59.429916112406431 ], [ 24.653888090772927, 59.429902444373447 ], [ 24.653928188994879, 59.429844255778043 ], [ 24.653976342916529, 59.429819411473439 ], [ 24.654034686059912, 59.429800369336448 ], [ 24.654073854913612, 59.429778334452465 ], [ 24.654102731780252, 59.429744214201932 ], [ 24.65419766400338, 59.429661354418499 ], [ 24.654267248327876, 59.429634976217812 ], [ 24.654471398913042, 59.429608117606548 ], [ 24.65468558052271, 59.429602725931503 ], [ 24.654778533827184, 59.429603349840377 ], [ 24.654854536386701, 59.429596562459125 ], [ 24.654921113711705, 59.42957879853968 ], [ 24.65495967427508, 59.429559082399471 ], [ 24.6551725115376, 59.429510240436308 ], [ 24.655342782575655, 59.429494590398811 ], [ 24.65558255506356, 59.429510775521891 ], [ 24.655695127615321, 59.429523823575373 ], [ 24.655821244396325, 59.429538446799334 ], [ 24.655986593361753, 59.429542667923187 ], [ 24.65609065045787, 59.429541655679117 ], [ 24.6561089456292, 59.429535568070932 ], [ 24.656130219686499, 59.429525874935372 ], [ 24.656163702381917, 59.429514271748964 ], [ 24.656196901915536, 59.429508109843475 ], [ 24.656228860027955, 59.429507950550907 ], [ 24.656242377453456, 59.429512721611637 ], [ 24.656251870511596, 59.429517117756724 ], [ 24.656310634689198, 59.429521007961803 ], [ 24.656358180125753, 59.429526058206164 ], [ 24.656390899782298, 59.429530688633768 ], [ 24.656404616721161, 59.429535754914234 ], [ 24.656412710451715, 59.429544350142109 ], [ 24.656410291267864, 59.429555996003103 ], [ 24.656384779111434, 59.429568861134662 ], [ 24.656347317047516, 59.429577845067243 ], [ 24.656330664324436, 59.429583215361461 ], [ 24.65632079327397, 59.429589413608447 ], [ 24.656315189833535, 59.429599477489496 ], [ 24.656314135954673, 59.429620748537339 ], [ 24.656303926777834, 59.429629524780381 ], [ 24.656295728306926, 59.429636576450143 ], [ 24.656271727953108, 59.429647432219234 ], [ 24.656247728835435, 59.429652848099728 ], [ 24.65622207832536, 59.4296539274861 ], [ 24.656179671465789, 59.429650395572516 ], [ 24.656136348173046, 59.429650521727943 ], [ 24.656109233508452, 59.429651509645744 ], [ 24.656083176757228, 59.429652537169453 ], [ 24.656053037259465, 59.429656574272897 ], [ 24.656040969002238, 59.429676814023182 ], [ 24.656069086849499, 59.429710641723815 ], [ 24.656131143977436, 59.429725431262703 ], [ 24.656259056170615, 59.429716059391019 ], [ 24.656390164628618, 59.429722658955058 ], [ 24.656474824550994, 59.429718035742191 ], [ 24.656551641660418, 59.429708711891095 ], [ 24.656629845772976, 59.429708303913586 ], [ 24.656703638285233, 59.429732298603376 ], [ 24.65679294606241, 59.429766907096344 ], [ 24.656899819932018, 59.429805153211767 ], [ 24.656944321690805, 59.429815909658615 ], [ 24.656983241880532, 59.429831442616461 ], [ 24.657034475135585, 59.429845908723721 ], [ 24.657068664372858, 59.429849975095017 ], [ 24.65712221907469, 59.429852302065058 ], [ 24.657176741787339, 59.429849076582698 ], [ 24.657233042713226, 59.429841201248379 ], [ 24.657295301189048, 59.429827946034941 ], [ 24.657318787032594, 59.429810503752186 ], [ 24.657351709285834, 59.429799998219515 ], [ 24.657404907504691, 59.429789292689769 ], [ 24.657453889510869, 59.429788392793483 ], [ 24.65751640683267, 59.429797757459305 ], [ 24.657569000103244, 59.429811471501438 ], [ 24.657609808420602, 59.429818897863768 ], [ 24.657672213398087, 59.429827912925944 ], [ 24.657740543515771, 59.429849088712267 ], [ 24.657813264403771, 59.429884174359472 ], [ 24.657935646349575, 59.429946291981629 ], [ 24.658089058691953, 59.430003945571229 ], [ 24.658239472437035, 59.430045850894743 ], [ 24.658364441952532, 59.430062954999187 ], [ 24.658433982021936, 59.430055434851781 ], [ 24.658468139958671, 59.430043226264594 ], [ 24.658500356685021, 59.430026287681294 ], [ 24.658515827387934, 59.43001627309566 ], [ 24.658529222181091, 59.43001465312765 ], [ 24.658545317013665, 59.430020569055614 ], [ 24.658556393473173, 59.430035791990029 ], [ 24.658570838808171, 59.430047542042892 ], [ 24.6585999703947, 59.430068913461213 ], [ 24.658616093835693, 59.43007722601579 ], [ 24.658702275535909, 59.430094846995665 ], [ 24.658835957956644, 59.43009228399918 ], [ 24.6589784063836, 59.430071921080931 ], [ 24.659075485969126, 59.430043409875672 ], [ 24.659143905130311, 59.430022546612548 ], [ 24.659225549282187, 59.43001318805829 ], [ 24.659329691359623, 59.430016508581232 ], [ 24.659478125607357, 59.430040666499771 ], [ 24.659646272634735, 59.430073765020246 ], [ 24.659785083303809, 59.430088599138116 ], [ 24.659889720602646, 59.430082877201166 ], [ 24.66004908009748, 59.430044115889501 ], [ 24.660134707938347, 59.43002117300454 ], [ 24.660208053811683, 59.429981756571998 ], [ 24.660239694996577, 59.429965170535546 ], [ 24.660308314522737, 59.429929188893645 ], [ 24.660398977786496, 59.429912459385449 ], [ 24.660438708861783, 59.429905132581688 ], [ 24.660643815403102, 59.429873196731009 ], [ 24.660705094550288, 59.429854917860453 ], [ 24.660746350344134, 59.429842619195277 ], [ 24.660820810501132, 59.429813502074389 ], [ 24.660895689303857, 59.429801483426012 ], [ 24.660941579745945, 59.42980471805744 ], [ 24.661040025224814, 59.429808685216344 ], [ 24.661089652523629, 59.429810177506482 ], [ 24.661139569349523, 59.429812063298925 ], [ 24.661193201185789, 59.429808346906178 ], [ 24.661226118425034, 59.429797642936435 ], [ 24.661259990476516, 59.429780740237923 ], [ 24.661277718542653, 59.429765417729577 ], [ 24.661279241655212, 59.429753076089945 ], [ 24.661286824876239, 59.429727939139738 ], [ 24.661275412820501, 59.429705446984059 ], [ 24.661278162880098, 59.42966363764274 ], [ 24.661310378296442, 59.429615764648545 ], [ 24.661366874858661, 59.429583469923642 ], [ 24.661423980906065, 59.429551719690366 ], [ 24.661639134753973, 59.429459192900019 ], [ 24.661707275506199, 59.429419541443494 ], [ 24.661754959510468, 59.429392587094831 ], [ 24.66180040653197, 59.429371128739795 ], [ 24.6619525445908, 59.429318218193274 ], [ 24.661996455504813, 59.429302970395611 ], [ 24.662022439335114, 59.429289105377606 ], [ 24.662056900165798, 59.429261714713739 ], [ 24.662101461088042, 59.429240009360171 ], [ 24.662199626608782, 59.42921414746062 ], [ 24.662343475149044, 59.429192678665515 ], [ 24.66248365563326, 59.429174684202103 ], [ 24.662586782963881, 59.429161220804026 ], [ 24.662654071309966, 59.429154930575805 ], [ 24.662744500308232, 59.429128101815486 ], [ 24.662824197712855, 59.429099379077456 ], [ 24.662897524327718, 59.429081029535453 ], [ 24.662973423669829, 59.429068151764334 ], [ 24.663039709029089, 59.429060124907195 ], [ 24.663099533817959, 59.429036950944983 ], [ 24.663146006689495, 59.429023027383224 ], [ 24.663178741261795, 59.429008427944133 ], [ 24.663209069835769, 59.428990600025806 ], [ 24.663228962347187, 59.428980652364515 ], [ 24.663283682764192, 59.428957755379081 ], [ 24.663378122465343, 59.428936094506874 ], [ 24.663431484315968, 59.428928536522236 ], [ 24.663527108204462, 59.428917058138566 ], [ 24.663639814142645, 59.42889789928136 ], [ 24.663719436876104, 59.428879912512528 ], [ 24.663811959202608, 59.42885027172143 ], [ 24.663886112287269, 59.428823730695591 ], [ 24.663933738586056, 59.428803912313683 ], [ 24.663995848175524, 59.4287823152915 ], [ 24.664072350297907, 59.428776031721299 ], [ 24.664184575869623, 59.428792027611436 ], [ 24.664300784596659, 59.428810804041078 ], [ 24.664397725621527, 59.428857137363579 ], [ 24.664515988307869, 59.428971559057544 ], [ 24.664609599323647, 59.429037783468125 ], [ 24.664711959604691, 59.429109951102312 ], [ 24.664802080922868, 59.429177889593916 ], [ 24.664921132480629, 59.429222252700789 ], [ 24.664979246506366, 59.429232542716896 ], [ 24.664996648261056, 59.429240390229623 ], [ 24.665005869368283, 59.429250675844138 ], [ 24.665011544540199, 59.429275216430362 ], [ 24.665005068463252, 59.429300060766707 ], [ 24.664974653204339, 59.429333320500504 ], [ 24.664906206582213, 59.429415910811358 ], [ 24.664891364460992, 59.429459029087091 ], [ 24.664890325233426, 59.429490740069262 ], [ 24.664882402043364, 59.429511848417917 ], [ 24.664863007982536, 59.429526578395979 ], [ 24.664821983952116, 59.429538922075231 ], [ 24.664782335103347, 59.429551312657736 ], [ 24.664744183554582, 59.429564593338682 ], [ 24.664714538162471, 59.429572238572241 ], [ 24.66468245213774, 59.429570353880941 ], [ 24.664635989999049, 59.429563999544847 ], [ 24.664594774461744, 59.429561972410426 ], [ 24.664556072041002, 59.429566862595713 ], [ 24.664475095953385, 59.429600485137968 ], [ 24.66444192444661, 59.429617986577512 ], [ 24.664428770190959, 59.429647361914 ], [ 24.664426393941, 59.429706386778122 ], [ 24.664411127160953, 59.42982194915637 ], [ 24.664402781190528, 59.429871174649392 ], [ 24.664401009733798, 59.429900591278731 ], [ 24.664376710156343, 59.42993127958988 ], [ 24.664318312476574, 59.429967175888684 ], [ 24.664275314058131, 59.429988568897031 ], [ 24.664165041190685, 59.430012509560321 ], [ 24.664051039620698, 59.430061280550539 ], [ 24.663963445231396, 59.430101677650278 ], [ 24.663895853014004, 59.430128688304059 ], [ 24.663859630767313, 59.430140657369684 ], [ 24.663843856317669, 59.430150377890698 ], [ 24.663838321048047, 59.430168377163454 ], [ 24.663840206207048, 59.430250001807266 ], [ 24.6638240853562, 59.430291779882168 ], [ 24.66380673898966, 59.430322172695995 ], [ 24.663706895812592, 59.430413259981655 ], [ 24.663704489852869, 59.430440884450228 ], [ 24.663715403187592, 59.43047298398492 ], [ 24.663737397263183, 59.430499533913739 ], [ 24.663765499382563, 59.430528494707666 ], [ 24.663832271858976, 59.430577152893889 ], [ 24.663979489225472, 59.430660809484074 ], [ 24.664015787337252, 59.430712565621967 ], [ 24.664046090588915, 59.430726021465219 ], [ 24.664100392331214, 59.430728611138491 ], [ 24.664155438910502, 59.430719661968681 ], [ 24.664241873592541, 59.430720150514929 ], [ 24.664361619942845, 59.430751072494687 ], [ 24.664670804659092, 59.430842151523983 ], [ 24.664834740847244, 59.430924609648933 ], [ 24.665076286314402, 59.430975903410307 ], [ 24.665326318124571, 59.431019586485377 ], [ 24.665597352182328, 59.431037866562399 ], [ 24.665746157237173, 59.431057985034755 ], [ 24.665869999503421, 59.431044881136771 ], [ 24.665982812827789, 59.431038170464255 ], [ 24.666155823982919, 59.431066218132294 ], [ 24.666377469148134, 59.431122403767183 ], [ 24.666466980678702, 59.431124302580784 ], [ 24.666515075121652, 59.431126590545411 ], [ 24.666550626053173, 59.431146141363364 ], [ 24.666558845458489, 59.43117819146979 ], [ 24.666554658344651, 59.431248078358756 ], [ 24.666598266534091, 59.431326583271932 ], [ 24.666620725462884, 59.431355141152515 ], [ 24.666668546487326, 59.431365976269248 ], [ 24.666712910526183, 59.431365796505212 ], [ 24.66675196371601, 59.431356218082122 ], [ 24.666787844975037, 59.43134572213426 ], [ 24.666832389230262, 59.431343943563746 ], [ 24.666883577183786, 59.431348531716921 ], [ 24.666928893464739, 59.431361031140078 ], [ 24.666984996174033, 59.431389023556605 ], [ 24.667019432997165, 59.431404728886584 ], [ 24.667048142378547, 59.431411360847186 ], [ 24.667072899820344, 59.431407806311711 ], [ 24.667113253455504, 59.431400761599832 ], [ 24.667193109535983, 59.431403543680197 ], [ 24.667265349899676, 59.431428689374073 ], [ 24.667367490455693, 59.431462785850712 ], [ 24.667426142675609, 59.431498592844143 ], [ 24.667459748133133, 59.431502390187248 ], [ 24.667521861440211, 59.431505199179227 ], [ 24.667587078776076, 59.431509958301298 ], [ 24.667647241304131, 59.431504006885049 ], [ 24.667700232568745, 59.431488145620399 ], [ 24.667753320900438, 59.431471843983246 ], [ 24.667802609321981, 59.43145044487617 ], [ 24.667875422405469, 59.431447167083597 ], [ 24.667960274823709, 59.431459456615578 ], [ 24.668038125727605, 59.431493954081212 ], [ 24.668067003804904, 59.431516410896528 ], [ 24.668080637216605, 59.431545013314597 ], [ 24.668095064319097, 59.431558333351283 ], [ 24.668139235316982, 59.431574401924067 ], [ 24.668201960427005, 59.43158325779077 ], [ 24.668252629010642, 59.43158288393181 ], [ 24.668302753899209, 59.431570026215766 ], [ 24.668322323423549, 59.431563382862144 ], [ 24.668344973243627, 59.431558375582995 ], [ 24.668371395839355, 59.431561580786706 ], [ 24.668386804731981, 59.431572840142294 ], [ 24.668402622353177, 59.431589680931964 ], [ 24.668437759733671, 59.431606926330907 ], [ 24.668467310449078, 59.431614209011904 ], [ 24.668490723551926, 59.431613991393355 ], [ 24.668527279144946, 59.431609155848626 ], [ 24.668566480661266, 59.431599064461658 ], [ 24.668599059691296, 59.431586412425339 ], [ 24.668634271434474, 59.431582238974478 ], [ 24.668664788477166, 59.431584875746871 ], [ 24.66869965788268, 59.431598325290949 ], [ 24.66873407970067, 59.431619515019769 ], [ 24.668750210818427, 59.431632557029467 ], [ 24.66883555069678, 59.431719224509116 ], [ 24.668876288085926, 59.431740786069511 ], [ 24.668960723208869, 59.431758705475374 ], [ 24.668988125565338, 59.431769643498413 ], [ 24.669012005639139, 59.43179417219266 ], [ 24.669052998226185, 59.43189265379182 ], [ 24.669072266816674, 59.431919234643637 ], [ 24.669093843132753, 59.431938586493821 ], [ 24.669120616945879, 59.431955209938138 ], [ 24.669141392960736, 59.431976881837699 ], [ 24.669175692082092, 59.431990872757297 ], [ 24.669221958686979, 59.431990628621875 ], [ 24.669248273941815, 59.431980234488208 ], [ 24.669261028765533, 59.431957620169968 ], [ 24.66928408726201, 59.431891604962026 ], [ 24.66931761934406, 59.431835850739773 ], [ 24.669325557944983, 59.431789553416536 ], [ 24.669306299686195, 59.431736518194008 ], [ 24.669302051827867, 59.43172025604283 ], [ 24.669301124496922, 59.431692702268336 ], [ 24.669322024441943, 59.431669427142161 ], [ 24.669366032020928, 59.431644732912368 ], [ 24.669430803878054, 59.431619529141116 ], [ 24.669468470520091, 59.431601258531657 ], [ 24.669458483166068, 59.431571749018765 ], [ 24.669441178224783, 59.431559063154303 ], [ 24.669365323271904, 59.43153928714009 ], [ 24.669303572902162, 59.43151903543 ], [ 24.6692714589873, 59.431490643773799 ], [ 24.669279581407274, 59.431475324129167 ], [ 24.669296667440179, 59.431466942795844 ], [ 24.669335843016061, 59.431465442018052 ], [ 24.669389723944221, 59.431474351258366 ], [ 24.66943694705456, 59.431483393072824 ], [ 24.669502879091954, 59.431486845990577 ], [ 24.669604858354951, 59.431490106572952 ], [ 24.669692153683719, 59.431493944462879 ], [ 24.669732386332175, 59.431505921205286 ], [ 24.66977140697259, 59.431526180625582 ], [ 24.669783152563308, 59.431555483684569 ], [ 24.669775089786867, 59.431624178559574 ], [ 24.6697568122005, 59.431670654184657 ], [ 24.669734361006341, 59.431725346769902 ], [ 24.669723831803907, 59.43174821911817 ], [ 24.669681448071618, 59.431790131507661 ], [ 24.669667256124061, 59.431805930882483 ], [ 24.669661335307051, 59.431819399126852 ], [ 24.66966746042397, 59.431836136460618 ], [ 24.669690029901076, 59.431847709335997 ], [ 24.669718771539998, 59.431854143066055 ], [ 24.669742541006851, 59.431861311242386 ], [ 24.669755963342922, 59.431876368714818 ], [ 24.669757998246919, 59.431890290211669 ], [ 24.669745736634251, 59.431904787129042 ], [ 24.669721551880638, 59.431920484736445 ], [ 24.669697335089115, 59.431934548741637 ], [ 24.669685271990094, 59.431949287014284 ], [ 24.669690751277258, 59.431959142497732 ], [ 24.669708294638276, 59.431968694245789 ], [ 24.669748767547254, 59.431983910352606 ], [ 24.66977642677104, 59.432004335284432 ], [ 24.669790317042253, 59.432022586079739 ], [ 24.669795262883579, 59.432044778266395 ], [ 24.669797508914098, 59.432076661076088 ], [ 24.66982151117563, 59.43212715861246 ], [ 24.669876162128141, 59.432151018931549 ], [ 24.669919480116022, 59.432156750086627 ], [ 24.669972886782581, 59.432156684799622 ], [ 24.670011876488239, 59.432149260144939 ], [ 24.670038067847631, 59.432143336883442 ], [ 24.670076952938803, 59.432123390236924 ], [ 24.670091642979262, 59.432113225655996 ], [ 24.670108957715428, 59.432101225443382 ], [ 24.670120181360161, 59.432075135871415 ], [ 24.670130817473041, 59.432045144421267 ], [ 24.67014378679551, 59.432036191470644 ], [ 24.670154096353592, 59.432029083304819 ], [ 24.670197033029545, 59.43201987009293 ], [ 24.670263364431541, 59.432018473180442 ], [ 24.670292938088423, 59.432025988731866 ], [ 24.670307070248757, 59.43205386127191 ], [ 24.670313343441642, 59.432092500985739 ], [ 24.670318476447619, 59.432124216486407 ], [ 24.670347605976339, 59.432181842260995 ], [ 24.670353948783259, 59.432194386332441 ], [ 24.670415241506081, 59.432249595177588 ], [ 24.670468071603253, 59.43227671409204 ], [ 24.670485989095042, 59.432279270978093 ], [ 24.670516300457432, 59.432274978354904 ], [ 24.670538266825712, 59.432268358374102 ], [ 24.670561368939843, 59.432254066498196 ], [ 24.670579131108553, 59.432240627672208 ], [ 24.670592543148864, 59.432231789120209 ], [ 24.670613987275772, 59.432219211253319 ], [ 24.67063866722523, 59.432217981411206 ], [ 24.670653405190986, 59.432224629894336 ], [ 24.670679409680865, 59.432245996644447 ], [ 24.670717185363639, 59.432271881545617 ], [ 24.670744298863301, 59.43228331439628 ], [ 24.670769749152971, 59.432284522269512 ], [ 24.670780913530148, 59.432285057853932 ], [ 24.670791052486951, 59.432283623786198 ], [ 24.670829165977896, 59.432278241066165 ], [ 24.670856937180382, 59.432272911031831 ], [ 24.670879846288987, 59.432272130026149 ], [ 24.670913868195939, 59.432281822066585 ], [ 24.670942614881927, 59.432296558959244 ], [ 24.670975991149906, 59.432340670966418 ], [ 24.670979192885557, 59.432344891655326 ], [ 24.670990951601475, 59.432349670175284 ], [ 24.671017630767317, 59.432360512720948 ], [ 24.671087838306036, 59.432364445758942 ], [ 24.671159844015985, 59.432365730441639 ], [ 24.671180908141505, 59.432366107807148 ], [ 24.671197476775497, 59.432364578094671 ], [ 24.671290398103327, 59.432355989423478 ], [ 24.671349219891258, 59.432345429130336 ], [ 24.671383530475254, 59.432348279327563 ], [ 24.671411737194944, 59.432361528738781 ], [ 24.671433414856249, 59.432399407616593 ], [ 24.67147472441583, 59.432458927954734 ], [ 24.67148332009398, 59.432471316856059 ], [ 24.671507428119398, 59.432478635593519 ], [ 24.671541944664153, 59.432474922727017 ], [ 24.67157518450238, 59.432462733375615 ], [ 24.67158528949733, 59.432462259927789 ], [ 24.671628649724212, 59.432460243409899 ], [ 24.67167783901823, 59.432465225902682 ], [ 24.671729232965834, 59.432469416155868 ], [ 24.671804518121796, 59.4324636190875 ], [ 24.671945856239304, 59.432483445529762 ], [ 24.671966913802692, 59.432485277030025 ], [ 24.672102032458856, 59.432497064987174 ], [ 24.672242875005313, 59.432506911542902 ], [ 24.672482190874494, 59.432512709228433 ], [ 24.672591172354707, 59.43254031248793 ], [ 24.672650075603269, 59.432548216286129 ], [ 24.672725697530723, 59.432573844425491 ], [ 24.672747873531783, 59.432603838996194 ], [ 24.672779584695501, 59.432637528127813 ], [ 24.672822994003994, 59.432654999399404 ], [ 24.672880908370626, 59.432660098439008 ], [ 24.672962328814005, 59.432680400552144 ], [ 24.673020089620785, 59.432683067627792 ], [ 24.673036264165212, 59.432681207560719 ], [ 24.673055779260025, 59.432678971346832 ], [ 24.673088203613819, 59.43266297068039 ], [ 24.673114038515319, 59.432643430963438 ], [ 24.673123053959571, 59.432635054487967 ], [ 24.673139667993539, 59.432629565570984 ], [ 24.673151342639713, 59.432628267097805 ], [ 24.67319129000257, 59.432623843803405 ], [ 24.673204424019193, 59.432622385259791 ], [ 24.673224562408979, 59.432622273313619 ], [ 24.673237019072658, 59.432622209620007 ], [ 24.673255389166638, 59.432626253950161 ], [ 24.673268918136603, 59.432643052005581 ], [ 24.673278146016212, 59.432696434186518 ], [ 24.673290011878098, 59.432788007745501 ], [ 24.673281919410002, 59.432873633059316 ], [ 24.673315606098356, 59.432930633446418 ], [ 24.673364910017714, 59.432954797950501 ], [ 24.673422217274407, 59.432980025637505 ], [ 24.673456199992117, 59.432993020654365 ], [ 24.673521760546418, 59.433002487868748 ], [ 24.673541383700037, 59.433011983587953 ], [ 24.673540952133585, 59.433024059467179 ], [ 24.673514672742037, 59.43304250638019 ], [ 24.673490149488995, 59.433066016144437 ], [ 24.673492538665244, 59.433087996856173 ], [ 24.673520789679532, 59.433110572391243 ], [ 24.673571968036448, 59.433139520791364 ], [ 24.67361604955553, 59.433172266719971 ], [ 24.673691490490903, 59.433211845037903 ], [ 24.673764004904641, 59.433228107826892 ], [ 24.673821170975749, 59.43324435038015 ], [ 24.673844863807748, 59.433264516485387 ], [ 24.673860584237367, 59.433292335636033 ], [ 24.673862714441771, 59.433311893954922 ], [ 24.673836402012178, 59.433347432748604 ], [ 24.673807171450648, 59.433438731816906 ], [ 24.673806245729349, 59.433519571793823 ], [ 24.673820619998423, 59.433536311603881 ], [ 24.673834196830896, 59.433549249369996 ], [ 24.673855785194061, 59.433557406432584 ], [ 24.673888106469235, 59.433559386446817 ], [ 24.673950000858653, 59.433557130684065 ], [ 24.673985844457668, 59.433557225490247 ], [ 24.674016511870757, 59.433567373764703 ], [ 24.674054292729615, 59.433574559210037 ], [ 24.674094147052912, 59.433568942222763 ], [ 24.674126496241346, 59.433560724497255 ], [ 24.674172206952889, 59.433558991346075 ], [ 24.674251208744955, 59.433561055409641 ], [ 24.674380561794521, 59.433580446914419 ], [ 24.674566373381374, 59.43361767302671 ], [ 24.674593638367249, 59.433637551395712 ], [ 24.674630145610859, 59.43365786714714 ], [ 24.674672182022487, 59.433674411270395 ], [ 24.674703299004527, 59.433679646823869 ], [ 24.674750062517592, 59.433678572354331 ], [ 24.674786008421851, 59.43366956403829 ], [ 24.674816992259991, 59.433675195222456 ], [ 24.674838483873504, 59.433686467538266 ], [ 24.674845094680947, 59.433703561212646 ], [ 24.674857226867275, 59.433731658960056 ], [ 24.674884991094594, 59.433777692833651 ], [ 24.674910214811664, 59.433893156593584 ], [ 24.674918911156013, 59.433957843045867 ], [ 24.674919492221626, 59.433979231557871 ], [ 24.674935002712104, 59.434000821820888 ], [ 24.674986058704235, 59.434017202913246 ], [ 24.675046002796993, 59.434023224183662 ], [ 24.675108974558153, 59.43402818860536 ], [ 24.675140814083004, 59.434039650186072 ], [ 24.675151907928655, 59.434055311514079 ], [ 24.675142735165899, 59.434078015753833 ], [ 24.675111404422072, 59.434102117273554 ], [ 24.675062121417668, 59.434135350368585 ], [ 24.675059229613151, 59.434150419129836 ], [ 24.675073324667228, 59.434169036366505 ], [ 24.675106444999361, 59.434181101804782 ], [ 24.675157476128934, 59.434206908499604 ], [ 24.675192796139974, 59.434237544417584 ], [ 24.675221254437094, 59.434283879864481 ], [ 24.675234358894311, 59.434312107238135 ], [ 24.675244077097528, 59.4343330539146 ], [ 24.675262433278807, 59.434360410542595 ], [ 24.675318682896769, 59.434384538698893 ], [ 24.675356082213288, 59.434395540830948 ], [ 24.675440795304635, 59.434410968140668 ], [ 24.675531264050864, 59.434418924178765 ], [ 24.675697755585098, 59.434451956965404 ], [ 24.675884649210641, 59.434478538287735 ], [ 24.676013869232989, 59.434502560866576 ], [ 24.676119197988822, 59.434542801233214 ], [ 24.676156410238757, 59.43456035709513 ], [ 24.676180785475971, 59.434583723961111 ], [ 24.676211673602531, 59.434588026759563 ], [ 24.676255799831736, 59.434580071195732 ], [ 24.676282547689947, 59.434557707495379 ], [ 24.676317666465483, 59.434539825053058 ], [ 24.676383783428232, 59.434538040185494 ], [ 24.676468890735229, 59.434534173870716 ], [ 24.676557819893624, 59.434539102988957 ], [ 24.676641052951865, 59.434559833391639 ], [ 24.676711622488913, 59.434609363261721 ], [ 24.676718708856519, 59.434636176170251 ], [ 24.67672228375605, 59.434667199246888 ], [ 24.676734236437781, 59.434688807584685 ], [ 24.676763237125442, 59.434706351619504 ], [ 24.676798556886816, 59.434723531030009 ], [ 24.676836022428898, 59.434731605998103 ], [ 24.676886131081066, 59.434728529730386 ], [ 24.676929347196538, 59.434717329052368 ], [ 24.676938463776469, 59.434714966193589 ], [ 24.677018597814683, 59.434685092645275 ], [ 24.677085761583566, 59.434666757974242 ], [ 24.677146913434406, 59.434650061016278 ], [ 24.677261918056796, 59.434629828502587 ], [ 24.677311896603289, 59.434625531908836 ], [ 24.677369275373433, 59.434623854332195 ], [ 24.677426478484207, 59.434617761094373 ], [ 24.677464002297498, 59.434603887535438 ], [ 24.677515973631149, 59.434581007803317 ], [ 24.677551337568111, 59.43455330324948 ], [ 24.677587965070341, 59.434506678260107 ], [ 24.677616081327219, 59.434466282012217 ], [ 24.677658294841635, 59.434408111137074 ], [ 24.677680768825759, 59.434369332788606 ], [ 24.67773189795361, 59.434310236282201 ], [ 24.677802944935639, 59.434241647586546 ], [ 24.677865141125547, 59.434189711327164 ], [ 24.677946407596185, 59.434134418303096 ], [ 24.678004887509886, 59.434117231997661 ], [ 24.678040407947645, 59.434116106534688 ], [ 24.678078642717381, 59.434125667321894 ], [ 24.678142360251691, 59.434167287337907 ], [ 24.678165002340116, 59.434190285750063 ], [ 24.678194262536053, 59.434220000556316 ], [ 24.678193967422352, 59.434245971696946 ], [ 24.678191278117506, 59.434252529569847 ], [ 24.678183680928004, 59.434271060721393 ], [ 24.678154114124972, 59.434294265193863 ], [ 24.678015883448861, 59.434372993635854 ], [ 24.677906999037653, 59.434422746631675 ], [ 24.677825548890329, 59.434482142922533 ], [ 24.677782513156405, 59.434521063044514 ], [ 24.677736260040717, 59.434570134414756 ], [ 24.677656718645046, 59.434636064789508 ], [ 24.677625811047491, 59.434684105609115 ], [ 24.677606251226642, 59.434720561931719 ], [ 24.677608574871837, 59.434763090588099 ], [ 24.677602532303837, 59.434794279833184 ], [ 24.677607170881572, 59.43480603340177 ], [ 24.677633200700178, 59.434818583533094 ], [ 24.677675439322527, 59.434832737860283 ], [ 24.677725014079108, 59.434856046562729 ], [ 24.677758427666152, 59.434885479757 ], [ 24.677771969594648, 59.434910769248617 ], [ 24.677785337915015, 59.434946849633555 ], [ 24.677785409079057, 59.434966427463941 ], [ 24.677710596277525, 59.435012261601706 ], [ 24.677690286155613, 59.435047321427227 ], [ 24.677685397692091, 59.435086942836406 ], [ 24.67769933306284, 59.435126736665197 ], [ 24.677729067196346, 59.435174106306739 ], [ 24.677754937950507, 59.435199054062934 ], [ 24.677816272059069, 59.435255471129381 ], [ 24.677849136423497, 59.435276738323779 ], [ 24.677851184985798, 59.435292059990758 ], [ 24.677823008916501, 59.435316136931675 ], [ 24.677780150200451, 59.435366698909959 ], [ 24.677753987529222, 59.43538656437326 ], [ 24.677731366309686, 59.435397515688912 ], [ 24.677690272403126, 59.435405572831868 ], [ 24.677642943120105, 59.435406669198294 ], [ 24.677610081655796, 59.435405096828596 ], [ 24.677572371886484, 59.435396305198502 ], [ 24.677521066441397, 59.435379953314438 ], [ 24.677461122757457, 59.435362541719257 ], [ 24.677416777282708, 59.435355031946912 ], [ 24.677383035030573, 59.435354361722595 ], [ 24.677363546085687, 59.435359758234554 ], [ 24.67734264727823, 59.435373151262823 ], [ 24.677332569001813, 59.435389235499343 ], [ 24.677331447958263, 59.435406467561393 ], [ 24.677341280776407, 59.435427781537626 ], [ 24.677390832480317, 59.435455228742931 ], [ 24.677486679542966, 59.435502303189601 ], [ 24.67750335833092, 59.435518707579853 ], [ 24.677516908860976, 59.435560424415833 ], [ 24.677538320147708, 59.435581759591749 ], [ 24.677542351173017, 59.435603749728045 ], [ 24.677545392599896, 59.435620269166733 ], [ 24.677567373670225, 59.435638342862418 ], [ 24.677583438492775, 59.435643332034367 ], [ 24.677619318109734, 59.435644242557416 ], [ 24.677676795370189, 59.43563589465834 ], [ 24.677730312946288, 59.435626041074627 ], [ 24.677785077279093, 59.435615965610609 ], [ 24.677847886047385, 59.435611656640035 ], [ 24.677886771680075, 59.435616636016015 ], [ 24.677946398154724, 59.435634901839187 ], [ 24.678039436695997, 59.435651388761642 ], [ 24.678094873112805, 59.435676956200957 ], [ 24.678145265840083, 59.435702334151664 ], [ 24.678183474504916, 59.435722918439133 ], [ 24.678198097444607, 59.435747591898512 ], [ 24.678221872600709, 59.435778789164061 ], [ 24.678253845049095, 59.435796811284057 ], [ 24.67830533694347, 59.43581362871317 ], [ 24.678345196635636, 59.435825936904514 ], [ 24.678371032407501, 59.435830435786443 ], [ 24.678421435963205, 59.43583148670804 ], [ 24.67847667225854, 59.435821211038743 ], [ 24.678503433206686, 59.435806629630875 ], [ 24.678544777006973, 59.435764998074099 ], [ 24.678565307936861, 59.435748177650453 ], [ 24.678590480591392, 59.435744134083571 ], [ 24.678655417960684, 59.435753826395349 ], [ 24.678710132691805, 59.435767871208398 ], [ 24.678772619393978, 59.435781050079072 ], [ 24.678825983879449, 59.43579631365975 ], [ 24.678882643116335, 59.43582484575559 ], [ 24.678931399673996, 59.435853867409151 ], [ 24.67895298235619, 59.435879339735308 ], [ 24.678959631389194, 59.435916855007356 ], [ 24.678963643116116, 59.435960883011283 ], [ 24.67896979571524, 59.435976075777617 ], [ 24.6789941568518, 59.435992377531299 ], [ 24.67904964607056, 59.436008123784859 ], [ 24.67909284007148, 59.436018816703438 ], [ 24.679121950627707, 59.436017831737075 ], [ 24.679165941039159, 59.436026033977186 ], [ 24.679205783624617, 59.43603745330438 ], [ 24.679258558879773, 59.436060278135855 ], [ 24.679299764873402, 59.436084428345453 ], [ 24.679340123193498, 59.436119121566563 ], [ 24.679378062947396, 59.436154482552944 ], [ 24.679399763835946, 59.436180555634913 ], [ 24.679411122084545, 59.436211152476361 ], [ 24.679394128287967, 59.436231258206973 ], [ 24.679361701228888, 59.436248032412209 ], [ 24.679324117216105, 59.436257158131887 ], [ 24.67929102953746, 59.436259321653459 ], [ 24.679236448455402, 59.436265348299571 ], [ 24.679206397080211, 59.436278411809447 ], [ 24.679192870926471, 59.436299208786039 ], [ 24.67919966321811, 59.436351005237881 ], [ 24.679190980697964, 59.436372558203274 ], [ 24.679187853763807, 59.436397879658379 ], [ 24.679213875059606, 59.436424980758588 ], [ 24.679255158192461, 59.436452990565328 ], [ 24.679329322295477, 59.436479062321517 ], [ 24.679405061546049, 59.436492172526471 ], [ 24.679472575708008, 59.436510836825931 ], [ 24.679499127013308, 59.436525583191688 ], [ 24.679509933805868, 59.436555024872483 ], [ 24.679494045366372, 59.43661476602913 ], [ 24.679466003097001, 59.436671266469453 ], [ 24.679466068949488, 59.43669939013904 ], [ 24.679485330322333, 59.436718859961481 ], [ 24.679571909932431, 59.436743770852466 ], [ 24.682668606171397, 59.435704903596125 ], [ 24.68537302343578, 59.43294146546981 ], [ 24.686509995701488, 59.43186587535255 ], [ 24.687140800214141, 59.431164464316723 ], [ 24.687934360065274, 59.430330510878377 ], [ 24.688135365619029, 59.430285025572751 ], [ 24.688463874658488, 59.429947847420692 ], [ 24.690068370719857, 59.429994743914811 ], [ 24.690149043212777, 59.429993513137198 ], [ 24.691102404424768, 59.43000072041054 ], [ 24.691352496877631, 59.429995547345804 ], [ 24.693099618403743, 59.429959341140389 ], [ 24.693114966092615, 59.429890767776783 ], [ 24.695863795369455, 59.429727163576267 ], [ 24.696562842986388, 59.429688188217092 ], [ 24.696725028494381, 59.429683829077341 ], [ 24.697221653102332, 59.429654718520965 ], [ 24.69983812180055, 59.429465160831846 ], [ 24.699116836626164, 59.42814115114782 ], [ 24.698861926534768, 59.427634330226233 ], [ 24.698752227444597, 59.427465072219256 ], [ 24.698437053969617, 59.427121049658773 ], [ 24.69719767221472, 59.427417030149719 ], [ 24.696147266686751, 59.427554996162335 ], [ 24.694505184134407, 59.425766073539407 ], [ 24.693972001999573, 59.425762063126889 ], [ 24.693097495426827, 59.425745575087838 ], [ 24.692743141145389, 59.425733436609327 ], [ 24.692374783866452, 59.425725769442295 ], [ 24.692025952758836, 59.425725359538333 ], [ 24.690085548259713, 59.425688777146782 ], [ 24.689319077342667, 59.425551857949344 ], [ 24.689301542620441, 59.425512272511952 ], [ 24.689076999325792, 59.425262637576395 ], [ 24.688891323753822, 59.425149425074487 ], [ 24.688489874893968, 59.425064091778509 ], [ 24.688251512378447, 59.424827633780168 ], [ 24.687979912518564, 59.424550145752299 ], [ 24.687657369791676, 59.424183335259798 ], [ 24.687076254480047, 59.423712756060887 ], [ 24.686243380923241, 59.424003459361145 ], [ 24.685212197650255, 59.424350217236878 ], [ 24.68395770429937, 59.423046494505471 ], [ 24.679902263601697, 59.424432688664247 ], [ 24.676183712263416, 59.421234038456113 ], [ 24.674976497450388, 59.420205035781137 ], [ 24.673925743905183, 59.419308884820701 ], [ 24.67369560922279, 59.418964368157212 ], [ 24.6728226159135, 59.418218825940606 ], [ 24.672782485780107, 59.418184470573713 ], [ 24.671318548942867, 59.417229188815718 ], [ 24.669993040891786, 59.41636833982276 ], [ 24.669260711834067, 59.416080231660025 ], [ 24.66806932721278, 59.415552886049092 ], [ 24.663021403739553, 59.413320311971503 ], [ 24.662489083699455, 59.413132326606323 ], [ 24.661600345003801, 59.412763095425177 ], [ 24.660950290643378, 59.412469051847928 ], [ 24.660283305002977, 59.412127513203863 ], [ 24.655285095176165, 59.409899781100734 ], [ 24.651443085820972, 59.408195680136458 ], [ 24.649718715673334, 59.407454198165581 ], [ 24.649361472294544, 59.407445907884465 ], [ 24.648608734213802, 59.407120712325785 ], [ 24.647251864948537, 59.406486722981505 ], [ 24.646533575869029, 59.406072296641234 ], [ 24.646068435803635, 59.405729514809821 ], [ 24.645213693600166, 59.405061439534649 ], [ 24.644295568591975, 59.40408316271423 ], [ 24.644150720161804, 59.403852091860792 ], [ 24.644010460626262, 59.403509327348807 ], [ 24.643866078222963, 59.402919362535307 ], [ 24.643821508047647, 59.40267469431484 ], [ 24.643831368993432, 59.402225897123131 ], [ 24.643932126561751, 59.401780066106632 ], [ 24.644122280855701, 59.401341876309843 ], [ 24.64469582258242, 59.400173521396184 ], [ 24.644829591802228, 59.399864243882256 ], [ 24.644844876660834, 59.399433103744968 ], [ 24.644690676570857, 59.398723706026587 ], [ 24.644459367584545, 59.398340543253518 ], [ 24.643936764662943, 59.397556825806625 ], [ 24.643225393244133, 59.396908412056717 ], [ 24.642439643895788, 59.396318976219085 ], [ 24.641666393710523, 59.395944020076719 ], [ 24.640739293362572, 59.39561451346178 ], [ 24.640230667109527, 59.395429734486939 ], [ 24.639891649732203, 59.395325637740505 ], [ 24.639199515335264, 59.395191478943708 ], [ 24.638855652658297, 59.395119809314103 ], [ 24.636320088560637, 59.394642327744002 ], [ 24.630281289282248, 59.393523545114284 ], [ 24.628658920935159, 59.392961284467638 ], [ 24.627594596588384, 59.392470041413894 ], [ 24.62668676321162, 59.391863410565854 ], [ 24.625994360423633, 59.391169211885 ], [ 24.625530829605534, 59.390526255694219 ], [ 24.625132668901525, 59.389886667535023 ], [ 24.62213025856499, 59.390589066988454 ], [ 24.620078035555679, 59.388530508092295 ], [ 24.598182504315027, 59.394178314363231 ], [ 24.598236526335043, 59.394197727712822 ], [ 24.596894527108613, 59.394514151337617 ], [ 24.594322032202012, 59.39517652530273 ], [ 24.594715281447346, 59.39539117681317 ], [ 24.594228649147933, 59.395566178562298 ], [ 24.594110058869344, 59.395599120330239 ], [ 24.59584253921135, 59.396385005987142 ], [ 24.596140000254238, 59.396515616079135 ], [ 24.597391791849599, 59.39701586517635 ], [ 24.597648056381981, 59.396964072192461 ], [ 24.59841923386244, 59.397277264994798 ], [ 24.600533705560846, 59.39808325395088 ], [ 24.604786234216238, 59.399628235173466 ], [ 24.607326470637069, 59.400819593195486 ], [ 24.608456586369666, 59.40151778749258 ], [ 24.609351406133676, 59.402247672911251 ], [ 24.610038877294247, 59.402810020697054 ], [ 24.6102953441154, 59.4029950982803 ], [ 24.610615565711726, 59.403171441323963 ], [ 24.611631198733527, 59.403681807950463 ], [ 24.612278651140993, 59.404011294207237 ], [ 24.61563608840202, 59.405751473609008 ], [ 24.617557556660845, 59.406708902634783 ], [ 24.619219389179712, 59.407236912434925 ], [ 24.618839953681285, 59.407431966612336 ], [ 24.618169033615228, 59.407778929173055 ], [ 24.617341989687159, 59.408354270487742 ], [ 24.61704543472943, 59.408547583883774 ], [ 24.616867805591486, 59.408637555630435 ], [ 24.616581573817815, 59.408739616080915 ], [ 24.616056421091205, 59.408862723113174 ], [ 24.615436123797451, 59.40902191066246 ], [ 24.614600501719377, 59.409261365623891 ], [ 24.613512286177453, 59.409585117592464 ], [ 24.612217453286689, 59.410044113522396 ], [ 24.610904058652579, 59.410540435975314 ], [ 24.610441315057574, 59.410768886322707 ], [ 24.610042463317164, 59.410995961587233 ], [ 24.609776752903702, 59.411194411355048 ], [ 24.609564371643533, 59.411391446546489 ], [ 24.609304182854046, 59.411702437533243 ], [ 24.609021964284725, 59.412081932345586 ], [ 24.608329405336839, 59.41244079261552 ], [ 24.608226994696473, 59.412534444117938 ], [ 24.608047766974494, 59.41256669085967 ], [ 24.607628959378498, 59.412564765468858 ], [ 24.607602275582021, 59.412589305291341 ], [ 24.606761844227155, 59.413528285221204 ], [ 24.606260520175962, 59.414087599355121 ], [ 24.605534327269218, 59.414898036721915 ], [ 24.604493815686009, 59.416948605558694 ], [ 24.60488007028167, 59.419646833943595 ], [ 24.605829208751221, 59.421421463809004 ], [ 24.606905820103432, 59.423117401474016 ], [ 24.607737091140191, 59.425197047516022 ], [ 24.610579494081769, 59.42541980054159 ], [ 24.610764808565051, 59.425901974540487 ], [ 24.610778886984253, 59.426019413590581 ], [ 24.610755605164119, 59.426126343760679 ], [ 24.610709322269244, 59.426198192159795 ], [ 24.610622981199693, 59.426295899577859 ], [ 24.610455134268641, 59.426505654443197 ], [ 24.610362304196283, 59.42660509732584 ], [ 24.610307371430647, 59.426646465063058 ], [ 24.610209981900972, 59.426678065505506 ], [ 24.610105984769127, 59.426734292514269 ], [ 24.610031223845105, 59.426799899207829 ], [ 24.610003525543302, 59.42691618537836 ], [ 24.60997895438538, 59.426970787481835 ], [ 24.609943655106438, 59.427085672586422 ], [ 24.609830564806213, 59.427244544558107 ], [ 24.609819203915119, 59.427328618591282 ], [ 24.609787846611567, 59.427388009705012 ], [ 24.609722489301916, 59.427437683976805 ], [ 24.609645055025457, 59.427442441151108 ], [ 24.609578107569536, 59.427412858937174 ], [ 24.609511244842182, 59.427417656847879 ], [ 24.609494901171903, 59.427459473916336 ], [ 24.609440004398049, 59.427502995494798 ], [ 24.609405954276745, 59.42754974190715 ], [ 24.609402646922032, 59.427571749996133 ], [ 24.609386679497561, 59.427605037493983 ], [ 24.609372560363202, 59.427623415204565 ], [ 24.609333286103819, 59.427653848284635 ], [ 24.609291812822679, 59.427689408216921 ], [ 24.609262964406479, 59.42771171404145 ], [ 24.609230219481081, 59.427732870927862 ], [ 24.609208448230593, 59.4277469752424 ], [ 24.609152740549696, 59.427774701552678 ], [ 24.609060588149955, 59.427823601799197 ], [ 24.608995557518941, 59.427862053378043 ], [ 24.608953853087691, 59.427894562208976 ], [ 24.608927009539222, 59.427910754565488 ], [ 24.608885741249001, 59.427918306290145 ], [ 24.608853415923637, 59.427923482714831 ], [ 24.608800776920582, 59.427925701014942 ], [ 24.608762719870715, 59.427925607704388 ], [ 24.608734984477078, 59.427931301473585 ], [ 24.608716359937628, 59.427944044657458 ], [ 24.60866278313787, 59.427972748332714 ], [ 24.608612743999036, 59.428002243528475 ], [ 24.608574724183732, 59.428033927303126 ], [ 24.608542008177707, 59.428076538052089 ], [ 24.608490064970514, 59.428137639777979 ], [ 24.608461260018981, 59.428172332982719 ], [ 24.60839037371418, 59.428228315765004 ], [ 24.608369844574927, 59.428242952793362 ], [ 24.608362188987179, 59.428248374155352 ], [ 24.608299412027737, 59.428354139765453 ], [ 24.608265557310251, 59.428382303308311 ], [ 24.608234369500259, 59.428401926712425 ], [ 24.608195598796662, 59.428411262004687 ], [ 24.608143706807109, 59.428415900279717 ], [ 24.608084261835465, 59.428421830151017 ], [ 24.608029704396028, 59.428435098266682 ], [ 24.607995209597792, 59.428447106742681 ], [ 24.607964287687999, 59.42846188146801 ], [ 24.607919777181248, 59.428495210811995 ], [ 24.607892059255221, 59.42852173012205 ], [ 24.60787700853988, 59.428567041879155 ], [ 24.607852131058298, 59.428604679096473 ], [ 24.607846386052373, 59.42860856557683 ], [ 24.607834024758237, 59.428616881157829 ], [ 24.607780363154159, 59.428640916997615 ], [ 24.607712726451247, 59.428661785736537 ], [ 24.60763388391679, 59.428676691813621 ], [ 24.607589185765541, 59.42867968086901 ], [ 24.607544862934596, 59.428674050604371 ], [ 24.607477846543773, 59.428660446068072 ], [ 24.60740895362585, 59.428640386976255 ], [ 24.607354531671351, 59.428631571653177 ], [ 24.607315383313349, 59.428629597861281 ], [ 24.607266772013048, 59.428640414534243 ], [ 24.607213435204354, 59.428662922614087 ], [ 24.60714432904857, 59.428710189151523 ], [ 24.607107116189898, 59.42873782927532 ], [ 24.607093503149478, 59.42875503742885 ], [ 24.607089719067254, 59.428770135629605 ], [ 24.607086426825099, 59.428783166935524 ], [ 24.607098224037863, 59.428802591837005 ], [ 24.607123070785722, 59.428822674689528 ], [ 24.607152022915283, 59.428835826593378 ], [ 24.607176102833396, 59.428852412083842 ], [ 24.607192675922789, 59.428872892146664 ], [ 24.607199266186438, 59.428896739606088 ], [ 24.607207253305038, 59.42895953922676 ], [ 24.60720304120396, 59.42898029468504 ], [ 24.607188428586102, 59.429020577413098 ], [ 24.607188433370442, 59.429020846690129 ], [ 24.607163178616606, 59.429076977376724 ], [ 24.607131361386422, 59.429100912196667 ], [ 24.607105975622268, 59.429110005900526 ], [ 24.60704703386909, 59.4291145864795 ], [ 24.606988340821371, 59.429113331071896 ], [ 24.606894094937157, 59.429104250329743 ], [ 24.606828928955654, 59.42909566384872 ], [ 24.606734205635767, 59.429089457716607 ], [ 24.606693436870252, 59.429095480411569 ], [ 24.606596621305385, 59.429130217280751 ], [ 24.606468096402839, 59.429175153984737 ], [ 24.606417805138133, 59.429190645926305 ], [ 24.606312283213803, 59.429221293433486 ], [ 24.606279682526779, 59.429240833100003 ], [ 24.606266110270933, 59.429260374898995 ], [ 24.60626762910756, 59.429286400145187 ], [ 24.606272066487751, 59.429308103166413 ], [ 24.606271156516478, 59.429326329924955 ], [ 24.606258722483116, 59.429350444564299 ], [ 24.606242897086798, 59.429382025425369 ], [ 24.606231734456788, 59.429408288599035 ], [ 24.60622121657152, 59.429441101745383 ], [ 24.606222579905566, 59.429468294668595 ], [ 24.606235637164573, 59.429489239866676 ], [ 24.606259242710514, 59.42950887976059 ], [ 24.60631571897002, 59.429534113317622 ], [ 24.606348378065437, 59.429557571452726 ], [ 24.606376409448615, 59.429588321967593 ], [ 24.606385558129382, 59.429617364122933 ], [ 24.606366955914641, 59.429661255773176 ], [ 24.606359475486343, 59.42968651456578 ], [ 24.606368295714631, 59.42971690472902 ], [ 24.606392251277967, 59.429746327516519 ], [ 24.606406216711051, 59.429768794541012 ], [ 24.606407251741764, 59.429797335466709 ], [ 24.606388895465269, 59.429815373281166 ], [ 24.606359755697088, 59.42983139613311 ], [ 24.6063123602957, 59.429851273238334 ], [ 24.606257701159123, 59.429858975554787 ], [ 24.606165198226183, 59.429858862891294 ], [ 24.606112804943773, 59.429855244158006 ], [ 24.606059147664688, 59.4298399616005 ], [ 24.606001470770419, 59.429806654492218 ], [ 24.605911808152456, 59.429787857182021 ], [ 24.605812561340596, 59.42978499255269 ], [ 24.605757462334179, 59.429787759537973 ], [ 24.605689958121715, 59.429796418382985 ], [ 24.605656305427143, 59.429806357730165 ], [ 24.605628336528959, 59.429818874145823 ], [ 24.605618891723864, 59.429832741605978 ], [ 24.605614931081256, 59.429857804628035 ], [ 24.605621675122425, 59.429900322839714 ], [ 24.605629866784362, 59.429944988782772 ], [ 24.60562682241666, 59.429981986508899 ], [ 24.605618360229975, 59.430041360968353 ], [ 24.605613548076196, 59.430087971819745 ], [ 24.605606624167375, 59.430114843796005 ], [ 24.605590835608503, 59.430138614726673 ], [ 24.605574243693116, 59.430156823839248 ], [ 24.605534672444922, 59.430180793919199 ], [ 24.605440954566781, 59.43023149405893 ], [ 24.605365663449817, 59.430268105860549 ], [ 24.605313178310567, 59.430289172962041 ], [ 24.605215332672422, 59.43032570883922 ], [ 24.60516599050251, 59.430365073757024 ], [ 24.605110526638292, 59.430416944306067 ], [ 24.605078006767258, 59.430441151133245 ], [ 24.605019740369222, 59.430454165717926 ], [ 24.604988982003594, 59.430468400369342 ], [ 24.604964510914325, 59.430499392419463 ], [ 24.604965695719212, 59.430536460452032 ], [ 24.604966127138141, 59.430600641363924 ], [ 24.604966445539088, 59.430648485330138 ], [ 24.604966835247652, 59.430690404393204 ], [ 24.604953933416784, 59.430727985981584 ], [ 24.604925329251149, 59.430774347044796 ], [ 24.604898080629162, 59.430827613875891 ], [ 24.604858387553008, 59.430854726119932 ], [ 24.604811650110182, 59.430872086188103 ], [ 24.604722003335155, 59.43089413164661 ], [ 24.604619613972588, 59.430903039920324 ], [ 24.604489300261871, 59.430907139182118 ], [ 24.604389604319593, 59.43090894337498 ], [ 24.604289265532852, 59.430894323216187 ], [ 24.604224387245431, 59.430892197289971 ], [ 24.604148720451576, 59.430907715025732 ], [ 24.604002818202204, 59.430937019889761 ], [ 24.60390075517633, 59.430974382009914 ], [ 24.603818099250475, 59.431003217017555 ], [ 24.603807943889915, 59.431016818296534 ], [ 24.603816260738814, 59.431038772886744 ], [ 24.603824596764625, 59.431051840529115 ], [ 24.603807169984865, 59.431082710276911 ], [ 24.603682097520903, 59.431303749721117 ], [ 24.603258501605936, 59.431313052118277 ], [ 24.598137988591834, 59.432019086010413 ], [ 24.595502685782449, 59.432483722148845 ], [ 24.590409474022184, 59.433718141732449 ], [ 24.586075229934064, 59.434850333409507 ], [ 24.581415098316224, 59.436058415363831 ], [ 24.577656373813923, 59.436951789831419 ], [ 24.577640528658776, 59.436921428752683 ], [ 24.574895192212253, 59.437190352290727 ], [ 24.574897306775323, 59.437221761054012 ], [ 24.574034510615572, 59.437265111635199 ], [ 24.571769832408481, 59.437162757213095 ], [ 24.568715102379059, 59.436317287227212 ], [ 24.568755757754641, 59.436282372323411 ], [ 24.566567963076849, 59.435582924368681 ], [ 24.566519859406316, 59.435615087910207 ], [ 24.564953986468741, 59.434881338594707 ], [ 24.562355318337147, 59.432729648382505 ], [ 24.558808188263775, 59.43445810743524 ], [ 24.555539191274086, 59.436070844101501 ], [ 24.551768926516157, 59.437927695916194 ], [ 24.55146486197641, 59.438076633693633 ], [ 24.550665646846095, 59.438590743180193 ], [ 24.550347602488511, 59.438775653725706 ] ] ] ] } },
{ "type": "Feature", "properties": { "ONIMI": "Tallinn", "OKOOD": "0784", "MNIMI": "Harju maakond", "MKOOD": "0037", "TYYP": "4", "District": "Nõmme", "AKOOD": "0524", "TYYP_2": "6", "ONIMI_2": "Tallinn", "OKOOD_2": "0784", "MNIMI_2": "Harju maakond", "MKOOD_2": "0037" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 24.760540683041395, 59.374926173859969 ], [ 24.761059448217932, 59.373414849936204 ], [ 24.761682778026064, 59.371608828977386 ], [ 24.762172132794888, 59.370124061197131 ], [ 24.762970211204273, 59.36775173234664 ], [ 24.764962202249659, 59.366083434670202 ], [ 24.766294932096041, 59.365116918427702 ], [ 24.768302616467572, 59.363762313838144 ], [ 24.770547728577448, 59.362189488106551 ], [ 24.77356199082271, 59.360089718150732 ], [ 24.770877857545909, 59.359973890503397 ], [ 24.767928146003751, 59.359893494124698 ], [ 24.767403905308203, 59.359866753367136 ], [ 24.763391097710823, 59.359699098898382 ], [ 24.758802856998567, 59.359532851449146 ], [ 24.749762809667896, 59.359201252543862 ], [ 24.740967593932641, 59.358864420214992 ], [ 24.732198191236108, 59.358533768496208 ], [ 24.723430781916182, 59.358197314686961 ], [ 24.714671278818198, 59.357868582323725 ], [ 24.716485480692377, 59.360383396976005 ], [ 24.71911227045311, 59.364023407508796 ], [ 24.717377037482819, 59.364062606547975 ], [ 24.715939424493563, 59.364096929637064 ], [ 24.714446620189118, 59.364132704094501 ], [ 24.714074450666413, 59.363295198204582 ], [ 24.7075183184411, 59.364895084319826 ], [ 24.70716132973083, 59.364554893083536 ], [ 24.702341917081682, 59.362128499044132 ], [ 24.697081403895542, 59.35795268031967 ], [ 24.696038740324383, 59.356264608683034 ], [ 24.689980267324081, 59.358400954439787 ], [ 24.671600794277158, 59.364875215593607 ], [ 24.659488663448379, 59.362675096647223 ], [ 24.655340629721511, 59.35840478908905 ], [ 24.651640477067417, 59.356386507402675 ], [ 24.651669945257233, 59.356299733769788 ], [ 24.651645048383383, 59.356294112031428 ], [ 24.651488181192335, 59.356236539971697 ], [ 24.651157169137537, 59.356063310375497 ], [ 24.651013923896254, 59.355991576438413 ], [ 24.650653543002765, 59.355734377204392 ], [ 24.650289737309457, 59.355528003414733 ], [ 24.650210781405086, 59.35548081646494 ], [ 24.650000947353732, 59.355355549369641 ], [ 24.649687444641394, 59.355196233480292 ], [ 24.649572251540448, 59.355147878822173 ], [ 24.649315457184851, 59.355039899027624 ], [ 24.649062025436397, 59.354951561617952 ], [ 24.648767461589113, 59.354863516640464 ], [ 24.648427728061183, 59.35479463532284 ], [ 24.648222833261055, 59.354771857673548 ], [ 24.648153065603754, 59.354764032918133 ], [ 24.647564502610866, 59.354710291147235 ], [ 24.647452607668647, 59.354696659137417 ], [ 24.647333073053709, 59.354688091765567 ], [ 24.647157896127979, 59.354684197120442 ], [ 24.6469821398607, 59.354686858259832 ], [ 24.646877536741957, 59.354696260577313 ], [ 24.646789336806766, 59.35471779068228 ], [ 24.646724584926172, 59.354780050353348 ], [ 24.646704995365308, 59.354831943721997 ], [ 24.646673687052164, 59.354887036704135 ], [ 24.646626214292606, 59.354942658093691 ], [ 24.646594069063855, 59.354962745121725 ], [ 24.646527832064027, 59.354983628320838 ], [ 24.64644770363266, 59.354995064295423 ], [ 24.646230433791608, 59.355005918178627 ], [ 24.646156674360594, 59.355019477107412 ], [ 24.6460699816787, 59.355065057493313 ], [ 24.645888087663863, 59.355133907355679 ], [ 24.645797390029706, 59.355181571943014 ], [ 24.64571153947292, 59.35520659076564 ], [ 24.645526516303008, 59.355240086377869 ], [ 24.645206916102467, 59.355250543679738 ], [ 24.645110976967764, 59.355244012851266 ], [ 24.644976926265159, 59.355230218225834 ], [ 24.644885028817637, 59.35520490559805 ], [ 24.644796824929756, 59.355160992524688 ], [ 24.644732419149424, 59.355120463597409 ], [ 24.644653556317351, 59.355077761352852 ], [ 24.64448918273991, 59.355031438999077 ], [ 24.644370538797322, 59.355004820924762 ], [ 24.644287781885843, 59.354979373275789 ], [ 24.644005375381244, 59.354873393955096 ], [ 24.64370009487838, 59.354794367151115 ], [ 24.643631407650382, 59.354759783444152 ], [ 24.643623370324267, 59.354715656312692 ], [ 24.643641408650293, 59.354683879355626 ], [ 24.643760186357206, 59.354558697478168 ], [ 24.643762293044599, 59.354493065765588 ], [ 24.643753324693286, 59.354436824338563 ], [ 24.643718535676481, 59.354381786676072 ], [ 24.643648004986535, 59.354351969785526 ], [ 24.643552782321681, 59.354336636924351 ], [ 24.643325583021745, 59.354361538684351 ], [ 24.642985736579607, 59.354416884673476 ], [ 24.642893459195754, 59.354417964706748 ], [ 24.642828047240215, 59.354389199559755 ], [ 24.64279693666407, 59.35435209752572 ], [ 24.642789375992479, 59.354192793949814 ], [ 24.642772742545905, 59.354158942531527 ], [ 24.642705145327501, 59.3541354844076 ], [ 24.642578424062055, 59.35411850956713 ], [ 24.642465335523173, 59.354116010562201 ], [ 24.642353106710821, 59.354121855804159 ], [ 24.642272697801189, 59.354118299131379 ], [ 24.642201961147311, 59.354086776860044 ], [ 24.642118799772874, 59.354011418081448 ], [ 24.642095735204361, 59.353962696305722 ], [ 24.641932313192555, 59.353863491985223 ], [ 24.641780434964708, 59.353817374512246 ], [ 24.641719643327992, 59.353819287357283 ], [ 24.641626606159544, 59.353845415865145 ], [ 24.641507097235667, 59.353894295459156 ], [ 24.641335353176881, 59.353951419450937 ], [ 24.64112939331886, 59.35400269567225 ], [ 24.640932816677807, 59.354029328990229 ], [ 24.640526207962065, 59.354030594146955 ], [ 24.640289853781692, 59.354007778024226 ], [ 24.639911606616945, 59.353936818276836 ], [ 24.639508079080116, 59.353839408808497 ], [ 24.639142334329662, 59.353741096121531 ], [ 24.639015296425235, 59.353706973460177 ], [ 24.63875911696303, 59.353648433085951 ], [ 24.638677018713349, 59.353629621651734 ], [ 24.638530919873187, 59.353600887589124 ], [ 24.638436697735376, 59.353582404247334 ], [ 24.638324873948442, 59.353553323826773 ], [ 24.638178013705854, 59.353540302701738 ], [ 24.6379461553373, 59.353550944488475 ], [ 24.637401269450347, 59.353565528949623 ], [ 24.637121361791312, 59.353563744955281 ], [ 24.637033434393025, 59.353562286243978 ], [ 24.636986183534876, 59.353592767633337 ], [ 24.636938255006744, 59.353615173055374 ], [ 24.636494467317128, 59.353706195765298 ], [ 24.636012073480181, 59.353791658657684 ], [ 24.635791342236903, 59.3538052051384 ], [ 24.635763216473304, 59.353824282582011 ], [ 24.635580536116617, 59.353879925859282 ], [ 24.634903860943705, 59.353955103384301 ], [ 24.634721774692871, 59.353967024961513 ], [ 24.634575151863402, 59.353966835663066 ], [ 24.634384049096305, 59.353947560632818 ], [ 24.634215045795006, 59.353925485551542 ], [ 24.634174959769062, 59.353801887302431 ], [ 24.634125656403594, 59.353750148929464 ], [ 24.634094333155716, 59.353701016822285 ], [ 24.634102175200269, 59.353649810496648 ], [ 24.634183234773825, 59.353593672470268 ], [ 24.634420656980822, 59.353484713127187 ], [ 24.634445306329368, 59.35341080374527 ], [ 24.634427231554191, 59.353365108721775 ], [ 24.634376172706158, 59.35332298422535 ], [ 24.634275044830172, 59.353301658645442 ], [ 24.634081333177711, 59.353302683638589 ], [ 24.633685375252266, 59.353337539717444 ], [ 24.633504421272292, 59.353353493777675 ], [ 24.633211695193403, 59.353371422746157 ], [ 24.633084351327316, 59.35335839162687 ], [ 24.632985099655397, 59.353324398551756 ], [ 24.632936314597053, 59.353291059932729 ], [ 24.632903297013417, 59.353245257169732 ], [ 24.632895897320573, 59.353177786350088 ], [ 24.632967619008696, 59.352987309489706 ], [ 24.632972363095874, 59.352920588072514 ], [ 24.632935554975933, 59.352888448505759 ], [ 24.632686136342588, 59.352842880315031 ], [ 24.632585506670136, 59.352838876509026 ], [ 24.632358629410501, 59.352862052157384 ], [ 24.632269372043417, 59.352883667366115 ], [ 24.632126312937007, 59.352952490752706 ], [ 24.63201707355703, 59.353072589034127 ], [ 24.631722618185876, 59.353292144708561 ], [ 24.631607483338112, 59.353245210445962 ], [ 24.631502130474928, 59.353175786748835 ], [ 24.631202451592049, 59.352940415904619 ], [ 24.631089813598354, 59.352876233500304 ], [ 24.630928722687671, 59.352796574124248 ], [ 24.630743319802274, 59.35266631167854 ], [ 24.630647508989114, 59.352570718630041 ], [ 24.630421751561432, 59.352425478471545 ], [ 24.630227032827623, 59.352275959621998 ], [ 24.630041064405258, 59.352152880439704 ], [ 24.629844671208183, 59.352055524987527 ], [ 24.629764211611288, 59.351981940916716 ], [ 24.629701378518494, 59.351958722612437 ], [ 24.629659301120661, 59.351945728246598 ], [ 24.629608403581255, 59.351931070529744 ], [ 24.629527773476397, 59.351915118988252 ], [ 24.629457293011097, 59.351906210529464 ], [ 24.629299083028002, 59.351887219370319 ], [ 24.629246606058807, 59.3518822641594 ], [ 24.628964238779613, 59.351803183316946 ], [ 24.628912161334465, 59.351800829368031 ], [ 24.628114770883808, 59.351925384497001 ], [ 24.628082209438322, 59.351923206200262 ], [ 24.627935409446906, 59.351912866457241 ], [ 24.6277831302402, 59.351939268350655 ], [ 24.627681225147644, 59.351980600354189 ], [ 24.627592998964296, 59.352010555998788 ], [ 24.6274596206034, 59.352022773437085 ], [ 24.627336249932537, 59.352034314558551 ], [ 24.627141040566013, 59.352049160962515 ], [ 24.627044279375824, 59.352054649990713 ], [ 24.626948045486262, 59.35206013643063 ], [ 24.626603318142056, 59.352098676520875 ], [ 24.626394073529077, 59.352134325449484 ], [ 24.6262024536619, 59.35218209863055 ], [ 24.626031053597053, 59.352258950148979 ], [ 24.625751715910297, 59.352403193728101 ], [ 24.625663690603915, 59.352434673178116 ], [ 24.625524362160501, 59.352467743428988 ], [ 24.625356352313847, 59.352489728935033 ], [ 24.625201201312041, 59.352493878725461 ], [ 24.624986803280574, 59.352488166330147 ], [ 24.624708745941309, 59.352471624983728 ], [ 24.624558089268877, 59.35248122855883 ], [ 24.624346576325784, 59.352518051984113 ], [ 24.62413777333094, 59.352578202170498 ], [ 24.623984646171387, 59.352654874380733 ], [ 24.623801364917188, 59.352745783368313 ], [ 24.623703149671442, 59.352797059091181 ], [ 24.623567410560216, 59.352882625150052 ], [ 24.62347708609693, 59.352923180711258 ], [ 24.623043564431057, 59.353038795655486 ], [ 24.622760761751756, 59.353109528268163 ], [ 24.622502084239752, 59.353143879213363 ], [ 24.622249496983287, 59.353184484634085 ], [ 24.621994179989699, 59.353239375779744 ], [ 24.621813177930616, 59.353291670427041 ], [ 24.621690935675932, 59.353326989680703 ], [ 24.621543852105052, 59.353349679151421 ], [ 24.621383358177738, 59.35335995399025 ], [ 24.62117956751867, 59.353357775945234 ], [ 24.620752923698195, 59.353329809952477 ], [ 24.620345313581677, 59.353323835884815 ], [ 24.619953139775227, 59.353325687379531 ], [ 24.619408173536367, 59.353335978419793 ], [ 24.618902713749282, 59.353382527305165 ], [ 24.618622518379517, 59.35343277131571 ], [ 24.618197633512807, 59.35358010770679 ], [ 24.618125457015896, 59.353603877086776 ], [ 24.618020710654132, 59.353634801652717 ], [ 24.617865076029844, 59.353680687628447 ], [ 24.617692643528201, 59.353711481439944 ], [ 24.617522806822425, 59.353710753888315 ], [ 24.617368609168963, 59.353719378365064 ], [ 24.616770297570739, 59.353706210109806 ], [ 24.616196192024667, 59.353641128858001 ], [ 24.615917638560603, 59.353577711783771 ], [ 24.615796441453327, 59.353544437082654 ], [ 24.615485717312669, 59.353373357312677 ], [ 24.615406099887359, 59.35330595473075 ], [ 24.615386776529323, 59.353238269644848 ], [ 24.615403221071485, 59.353145730504025 ], [ 24.615454162221031, 59.352957605247255 ], [ 24.615464786351726, 59.352883585780923 ], [ 24.615432071334499, 59.352735889452035 ], [ 24.615376205228266, 59.352649883136493 ], [ 24.615265179869883, 59.352585679746227 ], [ 24.615089124677908, 59.352492875160358 ], [ 24.61496025443623, 59.352452992717623 ], [ 24.614788724898929, 59.352416361967336 ], [ 24.614556545246582, 59.352389440519183 ], [ 24.614300154827419, 59.35237502001884 ], [ 24.614135449775471, 59.352375790193037 ], [ 24.613843048918547, 59.352402112766214 ], [ 24.613471992197088, 59.352453668084891 ], [ 24.613058379859041, 59.352524182698787 ], [ 24.612621455357115, 59.352647768445912 ], [ 24.612378864901419, 59.35272681913596 ], [ 24.612227186382899, 59.352768011999345 ], [ 24.612160738567479, 59.352778016739542 ], [ 24.612083093719438, 59.352958275642784 ], [ 24.611674662022757, 59.354254556964662 ], [ 24.61181720854519, 59.354243031082142 ], [ 24.613723096819534, 59.354105860623108 ], [ 24.614508553504219, 59.354407941716588 ], [ 24.613880994841494, 59.355735596314233 ], [ 24.610872567109659, 59.35603418776671 ], [ 24.611776644333172, 59.358290809185164 ], [ 24.612711725388092, 59.36063237926718 ], [ 24.612913525301085, 59.361188991695322 ], [ 24.613712508026289, 59.363159365823485 ], [ 24.613854913556899, 59.363510683585332 ], [ 24.613881284415051, 59.363578964109948 ], [ 24.613928607303539, 59.363687901709461 ], [ 24.614628708487363, 59.365335653700384 ], [ 24.614935920754153, 59.366078936777662 ], [ 24.616636202931005, 59.370142511506025 ], [ 24.617601377393637, 59.372341256423674 ], [ 24.618420100765174, 59.374216259303068 ], [ 24.619178462872917, 59.375968378412935 ], [ 24.619965437743442, 59.377809856396063 ], [ 24.620369677925559, 59.378691179177402 ], [ 24.62142784402306, 59.381198075428735 ], [ 24.622253154539269, 59.383063772989459 ], [ 24.623042862068548, 59.38500827003908 ], [ 24.623490759816054, 59.386084350932464 ], [ 24.627006294749869, 59.386346901921087 ], [ 24.625273376623671, 59.38718586758754 ], [ 24.620078035555679, 59.388530508092295 ], [ 24.62213025856499, 59.390589066988454 ], [ 24.625132668901525, 59.389886667535023 ], [ 24.625530829605534, 59.390526255694219 ], [ 24.625994360423633, 59.391169211885 ], [ 24.62668676321162, 59.391863410565854 ], [ 24.627594596588384, 59.392470041413894 ], [ 24.628658920935159, 59.392961284467638 ], [ 24.630281289282248, 59.393523545114284 ], [ 24.636320088560637, 59.394642327744002 ], [ 24.638855652658297, 59.395119809314103 ], [ 24.639199515335264, 59.395191478943708 ], [ 24.639505797019535, 59.394975803729281 ], [ 24.641278617891906, 59.393680796276819 ], [ 24.64337709215663, 59.393471980168563 ], [ 24.648637046363493, 59.392962471952259 ], [ 24.651677511353324, 59.393461812123775 ], [ 24.651812506059976, 59.393498666206881 ], [ 24.651818952519829, 59.393485976947531 ], [ 24.652172075348016, 59.393516542815618 ], [ 24.652897224108706, 59.39285269989508 ], [ 24.655680127746106, 59.393584558882509 ], [ 24.655920719093128, 59.393629948846424 ], [ 24.655663084525507, 59.393916696382739 ], [ 24.656244886079019, 59.394059218991309 ], [ 24.657268896985158, 59.394406804524792 ], [ 24.658630828525304, 59.394294962233772 ], [ 24.658981442237351, 59.394130545882213 ], [ 24.659163323420888, 59.394124697152641 ], [ 24.658878571920525, 59.393367851796242 ], [ 24.660047560093389, 59.393256242924451 ], [ 24.66062474742354, 59.393195893098515 ], [ 24.661280653614828, 59.393128861147225 ], [ 24.662704225323061, 59.393294858302021 ], [ 24.663917009544917, 59.391223530721994 ], [ 24.673407921596311, 59.392539842541055 ], [ 24.674178794414935, 59.392637331293791 ], [ 24.674402375070308, 59.392515266781714 ], [ 24.674710578005875, 59.392382623903423 ], [ 24.675154658274327, 59.392250359986285 ], [ 24.676575059149915, 59.391907235976625 ], [ 24.678665688681324, 59.391398606973162 ], [ 24.678925710869954, 59.391517734222766 ], [ 24.681194507736041, 59.391006259529497 ], [ 24.684178846618327, 59.391851468318876 ], [ 24.684441518585121, 59.391961145267196 ], [ 24.684732403137858, 59.39207336785897 ], [ 24.685044526441235, 59.392112317739539 ], [ 24.687328482708402, 59.392366654179476 ], [ 24.687583824518924, 59.392461192311025 ], [ 24.688940469778057, 59.392916849652508 ], [ 24.689600468298146, 59.393073178693136 ], [ 24.68970263491563, 59.393068603332715 ], [ 24.692023546201412, 59.392786925238212 ], [ 24.692129982488609, 59.39278465950413 ], [ 24.696184619970648, 59.393080859410148 ], [ 24.696072308052582, 59.392552177276166 ], [ 24.696473697395568, 59.392712892674041 ], [ 24.696699138866538, 59.392772471999848 ], [ 24.696941317971877, 59.392832859928831 ], [ 24.697110021856279, 59.392863295068445 ], [ 24.697822493488811, 59.392932678004705 ], [ 24.698519767439876, 59.392964434925339 ], [ 24.699321840230983, 59.392991232244057 ], [ 24.699880228155106, 59.393047778477204 ], [ 24.700416826341701, 59.393131728120757 ], [ 24.701556145765412, 59.393330866778228 ], [ 24.70193962025342, 59.393405931970229 ], [ 24.702437432517733, 59.393503366214745 ], [ 24.703806656711581, 59.393703142450789 ], [ 24.704778472624355, 59.393843363607424 ], [ 24.705437202947166, 59.393925202010763 ], [ 24.707250656802323, 59.393986020497408 ], [ 24.708377921026813, 59.394016130361621 ], [ 24.708793006214169, 59.394027179714598 ], [ 24.70902838939859, 59.394454557398035 ], [ 24.70933892787853, 59.394491034631969 ], [ 24.709865677000572, 59.394505248609555 ], [ 24.710401880949817, 59.394525513694198 ], [ 24.712532661952682, 59.394701424238256 ], [ 24.713238644865736, 59.394734761794666 ], [ 24.713621067228424, 59.39476581203219 ], [ 24.714223295289127, 59.394845668960642 ], [ 24.714613685883169, 59.394617689597901 ], [ 24.71578447830559, 59.393992814338795 ], [ 24.716446623589782, 59.393639558510294 ], [ 24.717078994494216, 59.393302350699358 ], [ 24.717326761059642, 59.393417429942161 ], [ 24.718384196450526, 59.393961403589138 ], [ 24.71886142487822, 59.394257637036254 ], [ 24.72007480237799, 59.395066461067913 ], [ 24.721562052949356, 59.394461614848062 ], [ 24.723764003296793, 59.393913863073557 ], [ 24.725486300481762, 59.393782825952556 ], [ 24.726983655525657, 59.392848857447198 ], [ 24.727401721613298, 59.392561707991923 ], [ 24.727900463809718, 59.393179702507531 ], [ 24.728788371590092, 59.394301208908303 ], [ 24.730180277417389, 59.394012600451681 ], [ 24.731218129261098, 59.393802788608241 ], [ 24.732121638886827, 59.392530399426654 ], [ 24.733256534592588, 59.390593676565864 ], [ 24.733362931859077, 59.390417584114076 ], [ 24.735309042859036, 59.38859902923074 ], [ 24.73858950720393, 59.385597152292533 ], [ 24.739908668699549, 59.384723188867184 ], [ 24.742063014061259, 59.382655230644716 ], [ 24.747544370808331, 59.377471293335191 ], [ 24.748178950050708, 59.376731478844668 ], [ 24.748804947043578, 59.37640474197724 ], [ 24.760540683041395, 59.374926173859969 ] ] ] ] } },
{ "type": "Feature", "properties": { "ONIMI": "Tallinn", "OKOOD": "0784", "MNIMI": "Harju maakond", "MKOOD": "0037", "TYYP": "4", "District": "Lasnamäe", "AKOOD": "0387", "TYYP_2": "6", "ONIMI_2": "Tallinn", "OKOOD_2": "0784", "MNIMI_2": "Harju maakond", "MKOOD_2": "0037" }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ 24.905479065018699, 59.460563247235285 ], [ 24.905492996438117, 59.460323825322767 ], [ 24.905578847603181, 59.458853075458798 ], [ 24.904430232132967, 59.457280297509008 ], [ 24.904932491022208, 59.456935633599379 ], [ 24.905206662293708, 59.456747926583745 ], [ 24.906096823772284, 59.456953844354956 ], [ 24.907105340950636, 59.457187039362061 ], [ 24.907964399452812, 59.457137583810564 ], [ 24.907957043396788, 59.457166001797781 ], [ 24.90823643437237, 59.457155188955383 ], [ 24.908569751895541, 59.457103337883595 ], [ 24.908566041387189, 59.457089808210313 ], [ 24.908731828739029, 59.457044408292631 ], [ 24.908897618039155, 59.45699909791901 ], [ 24.909254466164427, 59.456784688920848 ], [ 24.909290790175291, 59.456585956853758 ], [ 24.909247997898898, 59.456389476400872 ], [ 24.908928931110445, 59.455061915061258 ], [ 24.908960093837155, 59.454722279619915 ], [ 24.909117477569854, 59.454547129601856 ], [ 24.909521540394834, 59.454325571146832 ], [ 24.910250390849551, 59.454037851489204 ], [ 24.910832921955329, 59.453850873941185 ], [ 24.911167745782549, 59.453704836729138 ], [ 24.911372770862339, 59.453458884787821 ], [ 24.911494405358582, 59.453226615911156 ], [ 24.911441114620839, 59.452524802595541 ], [ 24.911396279142291, 59.45231810281247 ], [ 24.911028568854334, 59.451885171176919 ], [ 24.910426873629156, 59.451563644712458 ], [ 24.910134061676395, 59.451260988081103 ], [ 24.910121668056718, 59.451245812777508 ], [ 24.910116838648936, 59.451216760607224 ], [ 24.910104081217145, 59.451141621339602 ], [ 24.910115353357675, 59.451008863142015 ], [ 24.91041233203444, 59.450422945695166 ], [ 24.910612799055389, 59.450013195367866 ], [ 24.910855795965979, 59.449757196293042 ], [ 24.911274590045949, 59.449416314811174 ], [ 24.911651770756166, 59.449022665667314 ], [ 24.911744194513741, 59.448839343457976 ], [ 24.91179002545973, 59.448607957498332 ], [ 24.911742533499471, 59.448499753941313 ], [ 24.911480481904743, 59.448146885060119 ], [ 24.91116685112932, 59.447558725371159 ], [ 24.91110421038449, 59.447166682805801 ], [ 24.911082516058915, 59.446921759845054 ], [ 24.911242333898819, 59.446773520693981 ], [ 24.911790618877927, 59.446589647051425 ], [ 24.912816028215818, 59.446356682450698 ], [ 24.91335208544238, 59.446190751226169 ], [ 24.914446271344143, 59.445933238321679 ], [ 24.914733338369491, 59.445817595600865 ], [ 24.914911422212263, 59.445601807317423 ], [ 24.915040627583359, 59.445450095375108 ], [ 24.915050410007606, 59.445433599417768 ], [ 24.915068804069758, 59.445376646919428 ], [ 24.915082809667293, 59.445333549559024 ], [ 24.915098621880553, 59.445239450015407 ], [ 24.915139716112865, 59.444996784200463 ], [ 24.915159207237945, 59.444882101631507 ], [ 24.915176622007522, 59.444834671578853 ], [ 24.91525066332693, 59.444591417663297 ], [ 24.915429765269373, 59.444407400193562 ], [ 24.915785548919764, 59.444041356223593 ], [ 24.915960074648737, 59.443831515857475 ], [ 24.916204591654861, 59.44368788786791 ], [ 24.916438461511508, 59.443554836644168 ], [ 24.916548537202974, 59.443465915963408 ], [ 24.916751552109634, 59.44334582629601 ], [ 24.91678921205774, 59.443336946174078 ], [ 24.917090237082384, 59.443033221223622 ], [ 24.917212016340887, 59.442881379033125 ], [ 24.917375409860565, 59.442677628753032 ], [ 24.91726410541828, 59.442228294163748 ], [ 24.91729565182456, 59.441991798159464 ], [ 24.917182161890729, 59.441677583438697 ], [ 24.91709565071374, 59.441521266829376 ], [ 24.917057301602291, 59.441451871858945 ], [ 24.91702109078231, 59.441350503637338 ], [ 24.916994992274478, 59.441212169254136 ], [ 24.917000375211757, 59.440992462980724 ], [ 24.917030500137308, 59.440808043814691 ], [ 24.917036810788797, 59.44070826465957 ], [ 24.917087992327058, 59.439909757096984 ], [ 24.917123029400106, 59.43975079849325 ], [ 24.917219366488563, 59.439583961953595 ], [ 24.917277497215608, 59.439523679989122 ], [ 24.917367661147281, 59.439455185255227 ], [ 24.917484937885991, 59.439392157114881 ], [ 24.91759498399097, 59.439342106233461 ], [ 24.917678540654936, 59.439322941444324 ], [ 24.917906630357123, 59.439258153177889 ], [ 24.918129963364954, 59.439213147236103 ], [ 24.918357076299142, 59.439144863926565 ], [ 24.918566122535029, 59.439079848217617 ], [ 24.918957413478239, 59.438969123990603 ], [ 24.919145734265545, 59.438919961855881 ], [ 24.919250328665655, 59.438903252563449 ], [ 24.919324155666231, 59.438909021136574 ], [ 24.91934665365379, 59.438932922638998 ], [ 24.919341560929908, 59.438972906187075 ], [ 24.919325506887859, 59.439031279494962 ], [ 24.919353067746592, 59.439105776364279 ], [ 24.919394542139468, 59.439153693711219 ], [ 24.919467098675671, 59.439217014139246 ], [ 24.919583468897937, 59.439303817832347 ], [ 24.919716921264218, 59.439409713065182 ], [ 24.919777476780816, 59.439472398973187 ], [ 24.920178253815063, 59.439510983446809 ], [ 24.920354832345321, 59.439523125318473 ], [ 24.920515902475355, 59.439509162294705 ], [ 24.921390058249226, 59.439343885307679 ], [ 24.922082236043327, 59.439097287650597 ], [ 24.922369601424634, 59.43889131585177 ], [ 24.922859541999767, 59.438563990018061 ], [ 24.92298446846377, 59.438432318320764 ], [ 24.923029408318904, 59.438360455949201 ], [ 24.92311302713015, 59.438226380522757 ], [ 24.9232129532393, 59.437867494195253 ], [ 24.923233192735793, 59.437807744428682 ], [ 24.923312801487338, 59.437629709209531 ], [ 24.923381931770475, 59.437474818524095 ], [ 24.923611587488839, 59.43703853715224 ], [ 24.923798550551339, 59.43683551036073 ], [ 24.923934869802903, 59.436688855699472 ], [ 24.924126555182482, 59.436517215083775 ], [ 24.924547088190732, 59.436140726068039 ], [ 24.924876005722563, 59.435902228736303 ], [ 24.925161593403953, 59.435794921088615 ], [ 24.925262953821299, 59.435756864138661 ], [ 24.925862142549548, 59.435587742952215 ], [ 24.926283826078741, 59.43550182876276 ], [ 24.926001501593518, 59.435293660591427 ], [ 24.92367838428769, 59.433433763644885 ], [ 24.922858469128588, 59.432750071529277 ], [ 24.921949027448058, 59.43217580382651 ], [ 24.919219605697542, 59.431571338923341 ], [ 24.918862460280469, 59.431493935793057 ], [ 24.915764926093463, 59.430930749615897 ], [ 24.911968325161659, 59.430215954531462 ], [ 24.907605902585715, 59.429413745709084 ], [ 24.904438447514249, 59.428823935739885 ], [ 24.90139258220227, 59.428471020585192 ], [ 24.897618600039802, 59.428038348227489 ], [ 24.89735651822873, 59.428008266367847 ], [ 24.893953968705588, 59.427619727665281 ], [ 24.888460826881214, 59.426985790681087 ], [ 24.887343466123664, 59.426768990448707 ], [ 24.884712707554954, 59.426184785663203 ], [ 24.881387683325428, 59.42544197634691 ], [ 24.879540465293935, 59.42487387560562 ], [ 24.874794225834385, 59.423858151395578 ], [ 24.873019138613046, 59.423430240814241 ], [ 24.869886583683204, 59.422663603871335 ], [ 24.868787723455419, 59.421403931053078 ], [ 24.868970157032237, 59.420763919856611 ], [ 24.869235343972871, 59.41983178697361 ], [ 24.869406953645477, 59.419286284799128 ], [ 24.869506316202799, 59.418961466834418 ], [ 24.869694568393697, 59.418398887444916 ], [ 24.870740840775611, 59.41833649095593 ], [ 24.870578431086354, 59.41718114001651 ], [ 24.870575311003488, 59.415986582210763 ], [ 24.870431989742624, 59.415974603289357 ], [ 24.870439681343058, 59.415951571114867 ], [ 24.870485272724114, 59.415812214726067 ], [ 24.870923497677918, 59.41439569541204 ], [ 24.870771298594253, 59.413613093566745 ], [ 24.870594600797755, 59.412704974311097 ], [ 24.870253595424817, 59.410966214409676 ], [ 24.870183465029768, 59.410736236408113 ], [ 24.867755877451224, 59.410693379238054 ], [ 24.86262497085686, 59.41058752437408 ], [ 24.858759621575622, 59.410512445558872 ], [ 24.855038750043146, 59.410425542025976 ], [ 24.851708296667944, 59.410366349185402 ], [ 24.84371131723595, 59.410202781866374 ], [ 24.839783812414343, 59.410123953897426 ], [ 24.836105055123898, 59.410552426007527 ], [ 24.832773644618154, 59.41100211647035 ], [ 24.829261545910743, 59.411447833711961 ], [ 24.827557410811195, 59.411659647794465 ], [ 24.824788997746914, 59.411993469676183 ], [ 24.820379704097558, 59.410263606614564 ], [ 24.820231547740701, 59.41018041575672 ], [ 24.819282468304092, 59.409647349487422 ], [ 24.819052345669327, 59.409516371391732 ], [ 24.815693229014141, 59.411223209159559 ], [ 24.81535920636956, 59.411365320523501 ], [ 24.81492811107131, 59.411528050950878 ], [ 24.814392827435604, 59.411686307938297 ], [ 24.813816591787329, 59.412092311572195 ], [ 24.81106560258749, 59.412076172141283 ], [ 24.808314541805125, 59.412071735497051 ], [ 24.803968170665819, 59.412255909382573 ], [ 24.802512585520475, 59.412290828090605 ], [ 24.801688043528632, 59.412380596863827 ], [ 24.801510103714584, 59.412332487077499 ], [ 24.798938926411559, 59.414555288678493 ], [ 24.795775644436201, 59.417255869688205 ], [ 24.792293306533118, 59.419762772426985 ], [ 24.792460341778206, 59.42014481225695 ], [ 24.792086293887671, 59.420331090863364 ], [ 24.791363624915004, 59.420676381587626 ], [ 24.79070101450883, 59.420960982513755 ], [ 24.789858115315702, 59.421486345768166 ], [ 24.788963691014267, 59.422047201726805 ], [ 24.788610484861028, 59.422344479328522 ], [ 24.787484105877869, 59.42315402812735 ], [ 24.786958315506762, 59.423531330197434 ], [ 24.786975821447445, 59.423603040779348 ], [ 24.787177936460392, 59.423880744671223 ], [ 24.786999366036692, 59.424480212841956 ], [ 24.786682718266849, 59.424523040311215 ], [ 24.78513032355507, 59.424653945868002 ], [ 24.784382408962273, 59.424957874032927 ], [ 24.781763048349926, 59.42601908456875 ], [ 24.781630467147448, 59.425953263353883 ], [ 24.780895307164979, 59.426255120551104 ], [ 24.78032583339699, 59.426537768413098 ], [ 24.779494588615322, 59.426961375015296 ], [ 24.779278723543488, 59.427152693634007 ], [ 24.779986291496588, 59.427114839844783 ], [ 24.780303740503083, 59.42713656775765 ], [ 24.780930333731249, 59.427394797607818 ], [ 24.782016213805754, 59.428174905993863 ], [ 24.782994425760233, 59.428990655726722 ], [ 24.783517670357298, 59.429411251410755 ], [ 24.784156439120807, 59.429874694846156 ], [ 24.78464020455737, 59.430186182845908 ], [ 24.784897523007963, 59.430304221038234 ], [ 24.785245771110763, 59.430491646307324 ], [ 24.785568627789505, 59.430647444022689 ], [ 24.786689975870047, 59.431313475675175 ], [ 24.788283357322069, 59.43225899534913 ], [ 24.790293256637607, 59.433419326712048 ], [ 24.791476934300281, 59.433869768120061 ], [ 24.791679605055066, 59.433931477586711 ], [ 24.791864558751044, 59.433981443819569 ], [ 24.792058415671708, 59.43402776548659 ], [ 24.792417590830706, 59.434090327650331 ], [ 24.793008784042186, 59.434180486977127 ], [ 24.793813167758284, 59.434504103927921 ], [ 24.793832172959959, 59.435109749029486 ], [ 24.794277861453423, 59.435178696774528 ], [ 24.795044392276651, 59.435282690898575 ], [ 24.796230937923127, 59.435366726281607 ], [ 24.796721979323511, 59.435392661592822 ], [ 24.797214727505036, 59.435401079826399 ], [ 24.798272907786654, 59.435391078106385 ], [ 24.798555022261802, 59.436551245159237 ], [ 24.798762754309365, 59.436639754006372 ], [ 24.799940150694066, 59.437203715738875 ], [ 24.799924627994979, 59.437218173041572 ], [ 24.803975364227796, 59.439066350498059 ], [ 24.806927456793996, 59.440408830517129 ], [ 24.807890305239408, 59.44087762469033 ], [ 24.809059599959895, 59.44148877384702 ], [ 24.812188030102622, 59.443128355470542 ], [ 24.81248137233613, 59.443537417840496 ], [ 24.812798841951444, 59.444041575289077 ], [ 24.812980017662088, 59.44417358414573 ], [ 24.813304980577396, 59.444377417629291 ], [ 24.81364643223646, 59.444570286440687 ], [ 24.813978847332681, 59.444709259205858 ], [ 24.814306149854062, 59.444825910331481 ], [ 24.814533908593752, 59.444892905794134 ], [ 24.814580714125629, 59.445103752423329 ], [ 24.813766897103829, 59.445150345949266 ], [ 24.81184627896226, 59.445569658366757 ], [ 24.812447470748165, 59.446158514483663 ], [ 24.812879108225562, 59.44695524131923 ], [ 24.81334700552188, 59.450273162773605 ], [ 24.813158364870148, 59.450717604360477 ], [ 24.813038501477291, 59.451385144677729 ], [ 24.813010485191512, 59.451770963015832 ], [ 24.813310286654385, 59.452299644176286 ], [ 24.815601110506698, 59.453701399844725 ], [ 24.817195448982893, 59.454078960785047 ], [ 24.820758535130938, 59.453695313646911 ], [ 24.823279345447293, 59.453261532795018 ], [ 24.827648798702395, 59.452974184997771 ], [ 24.828833473232002, 59.452966737138318 ], [ 24.830888702412306, 59.453100024226309 ], [ 24.832922098819576, 59.453053161710876 ], [ 24.834125392029701, 59.452934414803863 ], [ 24.836255193968167, 59.452641641878508 ], [ 24.839762660581446, 59.452247507902875 ], [ 24.841295808899716, 59.452401644082265 ], [ 24.842732511676907, 59.452233660107986 ], [ 24.844426070176308, 59.451936542229248 ], [ 24.847373295987886, 59.45143115930901 ], [ 24.849954323099851, 59.451467316122162 ], [ 24.851396209675254, 59.451680183823271 ], [ 24.852905185528439, 59.452114780119921 ], [ 24.853187074675866, 59.452310895743572 ], [ 24.853655679023866, 59.452612536510152 ], [ 24.854751080722551, 59.453103473109437 ], [ 24.856756863331128, 59.453601410228906 ], [ 24.861747186610625, 59.454784517321677 ], [ 24.864212720726091, 59.454886503874057 ], [ 24.865648897152823, 59.455038660036408 ], [ 24.868804772159319, 59.455139703763095 ], [ 24.869791675345343, 59.455145939322406 ], [ 24.876382894929517, 59.45486374281969 ], [ 24.877001683663842, 59.454858817300916 ], [ 24.877130193001975, 59.454864335430422 ], [ 24.880277294027561, 59.455000262778611 ], [ 24.883112391683376, 59.454982277119512 ], [ 24.884163348071517, 59.454949374736962 ], [ 24.884402997520411, 59.454963655452623 ], [ 24.884793745965379, 59.454956902458747 ], [ 24.887963201345261, 59.454811607142247 ], [ 24.891040204614473, 59.4545502498489 ], [ 24.893309038371505, 59.454470161604306 ], [ 24.894491825497369, 59.454604062417957 ], [ 24.897511482583631, 59.455443879847849 ], [ 24.898575174797895, 59.455821113548481 ], [ 24.899549191523722, 59.456354432353614 ], [ 24.899638868427942, 59.456418364540774 ], [ 24.900217289925632, 59.456894502609572 ], [ 24.90114049430224, 59.458313376700609 ], [ 24.901493265229689, 59.458703616485188 ], [ 24.902140731732377, 59.459247498723919 ], [ 24.902660544392887, 59.45950480964882 ], [ 24.903860676924836, 59.460119315494666 ], [ 24.904853686027426, 59.460363316954947 ], [ 24.905479065018699, 59.460563247235285 ] ] ] ] } }
]
}
# json.dump(districts_geo, open('./districts_geo.json', 'w'))
print(os.listdir('.'))

[]


In [11]:
#Drawing Tallinn map with district layers, weighed by 'Share of population living in families'
data_map = folium.Map(location=[latitude, longitude], zoom_start=12)

df_map=df_districts[df_districts.Indicator=='Share of population living in families']

data_map.choropleth(geo_data=districts_geo,
                   data=df_map,
                   columns=['District','Value'],
                   key_on='feature.properties.District',
                   fill_color='BuPu',
                   fill_opacity=0.5,
                   line_opacity=0.7,
                   legend_name='Share of population living in families'
                   )
folium.LayerControl().add_to(data_map)

In [12]:
# creating marker with additional information for each District
for i in range(df_district_area.shape[0]):
    location = [df_district_area['Latitude'][i],df_district_area['Longitude'][i]]
    tooltip = "District:{}<br> Click for more".format(df_district_area["District"][i])

    folium.Marker(location, # adding more details to the popup screen using HTML
                  popup="""
                  <i>Share of population living in families: </i> <br> <b>{}</b> <br> 
                  <i>Number of families with children aged under 18: </i><b><br>{}</b><br>
                  <i>Average size of family: </i><b><br>{}</b><br>""".format(
                    round(df_districts[(df_districts.District==df_district_area["District"][i]) & (df_districts.Indicator=='Share of population living in families')].Value.values[0],1), 
                    round(df_districts[(df_districts.District==df_district_area["District"][i]) & (df_districts.Indicator=='Number of families with children aged under 18')].Value.values[0],1), 
                    round(df_districts[(df_districts.District==df_district_area["District"][i]) & (df_districts.Indicator=='Average size of family')].Value.values[0],1)),
                  icon=folium.Icon(color='red', icon='info-sign')).add_to(data_map)

In [13]:
data_map

## Define Foursquare Credentials and Version

In [14]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
CLIENT_ID = 'R2DI0ES2KM4GH4BJFR1AY102DHGGAF4WPBGTITH5BCRE0HV5' # your Foursquare ID
CLIENT_SECRET = 'HYUY25VXAZXK4XHSQCBOVQLR50APA4DCIIDJMFH0EPQ2ICC4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [41]:
LIMIT = 1000
radius = 10000

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID, 
CLIENT_SECRET, 
VERSION, 
latitude, 
longitude,
radius, 
LIMIT)

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
if not venues:
    continue
nearby_venues = json_normalize(venues) # flatten JSON
    
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.city','venue.location.lat', 'venue.location.lng','venue.postalCode']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]


SyntaxError: 'continue' not properly in loop (cell_name, line 22)

In [21]:
"""import math

LIMIT = 1000
df_venues = pd.DataFrame()

for i in range(df_district_area.shape[0]):
    radius = math.sqrt(df_district_area['Area'][i] * 1000/3.14)
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df_district_area['Latitude'][i], 
    df_district_area['Longitude'][i], 
    radius, 
    LIMIT)

    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    if not venues:
        continue
    nearby_venues = json_normalize(venues) # flatten JSON
    
    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.city','venue.location.lat', 'venue.location.lng','venue.postalCode']
    nearby_venues =nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#     nearby_venues['District']=df_district_area['District'][i]
    #df_venues=nearby_venues
  
    df_venues=df_venues.append(nearby_venues, ignore_index=True)
"""

In [24]:
del df_venues

In [94]:
df_district_area

,District,Latitude,Longitude,Area
0,Haabersti,59.4267,24.6313,18.60
1,Kesklinna,59.4328,24.7629,30.60
2,Kristiine,59.4164,24.7100,9.40
3,Lasnamäe,59.4293,24.8352,30.00
4,Mustamäe,59.4010,24.6945,8.15
5,Nõmme,59.3807,24.6995,28.70
6,Pirita,59.4749,24.8725,18.70
7,Põhja-Tallinna,59.4550,24.6894,17.30


In [126]:
!conda install -c conda-forge geopandas --yes
import geopandas as gpd

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopandas-0.6.2            |             py_0         906 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.0 MB

The following packages will be UPDATED:

    ca-certificates: 2019.11.27-0      --> 2019.11.28-hecc5488_0 conda-forge
    certifi:         2019.11.28-py36_0 --> 2019.11.28-py36_0     conda-forge
    geopandas:       0.6.1-py_0        --> 0.6.2-py_0            conda-forge

The following packages will be DOWNGRADED:

    openssl:         1.1.1d-h7b6447c_3 --> 1.1.1d-h516909a_0     conda-forge


geopandas-0.6.2      | 906 KB   

In [128]:
df_venues['geometry'] = gpd.points_from_xy(df_venues['lat'], df_venues['lng'])
gdf_venues = gpd.GeoDataFrame(df_venues)

pointInPolys = gpd.tools.sjoin(gdf_venues, districts_gdf, how='left')
pointInPolys

TypeError: 'module' object is not callable

In [39]:
#del df_venues
# Shopping Mall, Supermarket, Grocery Store, Market, Playground, Park, Trail, Beach, Bus Station, Movie Theater, Zoo

In [22]:
df=df_venues.groupby(['categories'])
df.first() 


,name,city,lat,lng,District
categories,,,,,
Airport,Tallinn Airport (TLL) (Lennart Meri Tallinna L...,Tallinn,59.416381,24.799472,Lasnamäe
Airport Terminal,Check In,Tallinn,59.416289,24.799117,Lasnamäe
American Restaurant,Mack Bar-B-Que,Tallinn,59.426536,24.652412,Haabersti
Animal Shelter,Loomade Hoiupaik,Tallinn,59.372065,24.741690,Nõmme
Arcade,Happy Fly batuudikeskus,Tallinn,59.401472,24.656382,Haabersti
Art Gallery,Fotografiska,Tallinn,59.438230,24.728266,Kristiine
Art Museum,Kumu kunstimuuseum (Kumu Kunstimuuseum),Tallinn,59.436186,24.796192,Lasnamäe
Arts & Crafts Store,Kangas ja Nööp,Tallinn,59.412599,24.667962,Nõmme
Asian Restaurant,Chow king,Tallinn,59.410471,24.682543,Haabersti


In [34]:
# find out corallation for parks and number of families in District
fam_venues = pd.DataFrame()

# first lets filter out posible important venues for families 
for i in range(df_district_area.shape[0]):
    fam_venues = fam_venues.append(df_venues[df_venues['categories']=='Park'], ignore_index=True)

In [35]:
fam_venues

,name,categories,city,lat,lng,District
0,Glehni park,Park,Tallinn,59.387104,24.653071,Haabersti
1,Sütiste parkmets,Park,Tallinn,59.395993,24.691840,Haabersti
2,Männi park,Park,Tallinn,59.398258,24.676693,Haabersti
3,Parditiigi park,Park,NaN,59.405276,24.704353,Haabersti
4,Löwenruh' park,Park,Tallinn,59.417502,24.695426,Haabersti
5,Kopli Kalmistupark,Park,Tallinn,59.453659,24.684674,Haabersti
6,Sütiste parkmets,Park,Tallinn,59.395993,24.691840,Kristiine
7,Schnelli Park,Park,Tallinn,59.439098,24.736645,Kristiine
8,Harju tänava park,Park,Tallinn,59.435215,24.743514,Kristiine
9,Kadrioru park,Park,Tallinn,59.440523,24.794699,Lasnamäe


In [33]:
del fam_venues

In [23]:
df_venues[df_venues['name']=='Glehni park']

,name,categories,city,lat,lng,postalCode
